In [ ]:
################## DEFINIZIONI FUNZIONI E VARIABILI UTILI ###################
%matplotlib qt

import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from astropy.io import fits
from scipy.optimize import curve_fit
import os
import re
from os.path import exists, isfile, join
import subprocess
import time
import urllib.request
import urllib.parse
import json
import logging
import multiprocessing
from numpy import linspace, round as around
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import sympy as sp
import sys
import random
import numpy as np
from scipy.special import wofz
from scipy.integrate import quad
import concurrent.futures
from scipy.optimize import fsolve
import pandas as pd
from zero_point import zpt
from astropy.coordinates import SkyCoord
from concurrent.futures import ThreadPoolExecutor
from scipy.interpolate import interp1d
from scipy.fft import fft, ifft, fftfreq
import pexpect
import matplotlib.patches as patches
from collections import defaultdict
from io import StringIO

#Non avendo le griglie correttive su portatile nuovo (==2) ho dovuto fare un po' di modifiche al codice. NLTE_AnalysisSingle ha nella sua definizione 'WL_disponibilis' ma dovrebbe essere WL_disponibilis senza cioè gli apici. Anche WL_disponibilis definito in testa al codice è stato commentato perché va a leggere dati che al momento qua non ci sono.
#Same goes per def NLTE_wlCorrection con 'full_array_corr.

pc=2 #0 sul fisso, 1 sul portatile, 2 sul nuovo portatile

####
if pc==1:
    path_to_pythonEnv = f"/home/starcat/Scrivania/ANALISIATM/.venv/bin/python" #path to the virtual environment
    path_to_linemake = f"/home/starcat/LINEE"                                  #path to linelist folder
    path_to_ARES = f"/home/starcat/Modelli/ARES-master"                        #path to ARES folder
    path_to_abundance = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c/"    #path to abundance folder
    path_to_abundance_0 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_0/"
    path_to_abundance_1 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_1/"
    path_to_abundance_2 = f"/home/starcat/Scrivania/ANALISIATM/spectrum277c_2/"
    path_to_mainfolder = f"/home/starcat/Scrivania/ANALISIATM/"                #path to main folder
    path_to_atmosphere = f"/home/starcat/Scrivania/ANALISIATM/ATMOSFERE/"      #path to atmospheres folder, this is somewhat useless today
    path_to_rnd = f"/home/starcat/Scrivania/ANALISIATM/rnd"                    #just random
    path_to_output = f"/home/starcat/Scrivania/ANALISIATM/ABBONDANZE_OUT/"     #path to output, this is where all the results will be saved
    path_to_interpol = f"/home/starcat/Scrivania/ANALISIATM/interpol/PyKMOD-main/"  #path to the pykmod interpolator
    path_to_correctionGrids = f"/home/starcat/Scrivania/ANALISIATM/correctionGrids/"#path to NLTE correction grids (deprecated)
    path_to_correctionGrids1 = f"/home/starcat/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/"#path to NLTE correction grids
    path_to_nlteInterpol = f"/home/starcat/Scrivania/ANALISIATM/1L-3NErrors-main/"                      #path to NLTE correction grids (deprecated)
    path_to_calibrationFolder = f"/home/starcat/Scrivania/ANALISIATM/Calibrazione"                      #path to the calibration folder, this was used during the calibration of the program
    path_to_fitsFiles = f"/home/starcat/Documenti/Reduced_AOT41/Reduced_AOT41"                          #path to the fits files to analyze
    path_to_spectrum = "/home/starcat/Documenti/spectrum277c1"                                          #path to spectrum
    ### Note! spectrum and abundance come in the same folder if you download them from the official site. In this work tho they should sit in two different folders! So make sure to download the two from the git page!
elif pc == 0:
    path_to_linemake = f"/home/starcat8/Scrivania/ANALISIATM/LINEE"
    path_to_ARES = f"/home/starcat8/Scrivania/ARES-master"
    path_to_abundance = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c/"
    path_to_abundance_0 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_0/"
    path_to_abundance_1 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_1/"
    path_to_abundance_2 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_2/"
    path_to_mainfolder = f"/home/starcat8/Scrivania/ANALISIATM/"
    path_to_atmosphere = f"/home/starcat8/Scrivania/ANALISIATM/ATMOSFERE/"
    path_to_rnd = f"/home/starcat8/Scrivania8/ANALISIATM/rnd"
    path_to_output = f"/home/starcat8/Scrivania/ANALISIATM/ABBONDANZE_OUT/"
    path_to_interpol = f"/home/starcat8/Scrivania/ANALISIATM/interpol/PyKMOD-main/"
    path_to_correctionGrids = f"/home/starcat8/Scrivania/ANALISIATM/correctionGrids/"
    path_to_correctionGrids1 = f"/home/starcat8/Scrivania/ANALISIATM/iron_abundancecorr/correctionGrids/"
    path_to_nlteInterpol = f"/home/starcat8/Scrivania/ANALISIATM/1L-3NErrors-main/"
    path_to_calibrationFolder = f"/home/starcat8/Scrivania/ANALISIATM/Calibrazione"
    path_to_pythonEnv = f"/home/starcat8/Scrivania/ANALISIATM/.venv/bin/python"
    path_to_fitsFiles = f"/home/starcat8/Documenti/Reduced_AOT41"
    path_to_spectrum = "/home/starcat8/Scrivania/ANALISIATM/spectrum277c1"
elif pc == 2:
    path_to_linemake = f"/home/starcat8/Documenti/sbundance/ANALISIATM/line_lists"
    path_to_ARES = f"/home/starcat8/Documenti/sbundance/ANALISIATM/ARES-master"
    path_to_abundance = f"/home/starcat8/Documenti/sbundance/ANALISIATM/spectrum277c/"
    # path_to_abundance_0 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_0/"
    # path_to_abundance_1 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_1/"
    # path_to_abundance_2 = f"/home/starcat8/Scrivania/ANALISIATM/spectrum277c_2/"
    path_to_mainfolder = f"/home/starcat8/Documenti/ANALISIATM/"
    path_to_atmosphere = f"/home/starcat8/Documenti/sbundance/ANALISIATM/ATMOSFERE/"
    path_to_rnd = f"/home/starcat8/Documenti/sbundance/ANALISIATM/rnd"
    path_to_output = f"/home/starcat8/Documenti/sbundance/ANALISIATM/ABBONDANZE_OUT/"
    path_to_interpol = f"/home/starcat8/Documenti/sbundance/ANALISIATM/PyKMOD-main/"
    path_to_correctionGrids = f"/home/starcat8/Documenti/sbundance/ANALISIATM/correctionGrids/"
    path_to_correctionGrids1 = f"/home/starcat8/Documenti/sbundance/ANALISIATM/iron_abundancecorr/correctionGrids/"
    path_to_nlteInterpol = f"/home/starcat8/Documenti/sbundance/ANALISIATM/1L-3NErrors-main/"
    path_to_calibrationFolder = f"/home/starcat8/Documenti/sbundance/ANALISIATM/Calibrazione"
    #path_to_pythonEnv = f"/home/starcat8/Documenti/sbundance/.venv/python"
    path_to_pythonEnv = f"python"
    path_to_fitsFiles = f"/home/starcat8/Documenti/sbundance/ANALISIATM/Reduced_AOT41"
    path_to_spectrum = "/home/starcat8/Documenti/sbundance/ANALISIATM/spectrum277c1"
    interpol_folder = "interpolated_atmospheres"
    path_to_turbospectrum = '/home/starcat8/Documenti/Turbospectrum2019-master'
    path_to_turbospectrumNLTE = '/home/starcat8/Documenti/Turbospectrum_NLTE-master/'
    path_to_turboModels = '/home/starcat8/Documenti/x/xdeimer/'
    path_to_ATLAScom = '/home/starcat8/Documenti/atlas9/atlas9sc'
    path_to_param_GaiaDR3AM_giants_A0 =' /home/starcat8/Scaricati/xLapo/'
    path_to_moogModels='/home/starcat8/Scaricati/pymoogi-master/example/atlas9_models/'
    path_to_moogBatch = '/home/starcat8/Scaricati/pymoogi-master/batcher/'
    path_to_moogOutput = '/home/starcat8/Scaricati/pymoogi-master/example/output/'
    path_to_turboHeiter = '/home/starcat8/Scaricati/xturb/xlapo_turbo/lines/'
    path_to_turbosynthscript = '/home/starcat8/Scaricati/xturb/xlapo_turbo/turbosynth_script/'
    ####


NEW_MATRIX = np.array([[-2.73752381e-04,  2.22800000e-03,  1.25531429e-03,  3.30684003e-03],
            [-3.53622447e-02, -4.34489796e-01, -1.34193878e-01, -6.97848163e+00],
            [-1.50857143e-02, -4.02285714e-01, -1.15394286e+00, -2.93710716e+00],
            [-4.89264069e-03, -2.08303030e-01, -4.36747186e-01, -2.23545660e+01]])
MATRIX = NEW_MATRIX.T
#crea_MatriceRicerca(T, G, M, v): #il fatto che alcune grandezze richiedano un fit quadratico implica una dipendenza dalle grandezze stesse.

N_stelle = 14
trials = 30
temp_log = np.zeros(trials)
logg_log = np.zeros(trials)
T_slope = np.zeros((trials, 2))
Logg_slope = np.zeros((trials, 2))
temp_best_guess1 = 0
temp_best_guess2 = 0
temp_best_guess3 = 0
temperatura = [3500, 3750, 4000, 4250, 4500, 4750, 5000, 5250, 5500, 5750, 6000, 6250, 6500, 6750, 7000, 7250, 7500, 7750, 8000, 8250, 8500, 8750, 9000, 9250, 9500, 9750, 10000, 10250, 10500, 10750, 11000, 11250, 11500, 11750, 12000, 12250, 12500, 12750, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000]
logg_arr = [0, 0.50000, 1.00000, 1.50000, 2.00000, 2.50000, 3.00000, 3.50000, 4.00000, 4.50000, 5.00000]
LOGG_arr = [1.50000, 2.00000, 2.50000, 3.00000]
MH = ["05", "02", "00", "-05", "-10", "-15", "-20"]
metallicity = ["0.5", "0.2", "0.0", "-0.5", "-1.0", "-1.5", "-2.0", "-2.5"]
metallic = [0.5, 0.2, 0.0, -0.5, -1.0, -1.5, -2.0, -2.5]
METAL = [4.25, 4.50, 4.75, 5.00, 5.25, 5.50, 5.75, 6.00, 6.25, 6.50, 6.75, 7.00, 7.25, 7.50, 7.75]
metallic_header = [-2.5, 2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5]
T_EFF_int = np.arange(3500, 5500 + 1, 25)
LOGG_int = np.arange(0.5, 5 + 0.06, 0.05)

MH_int = np.arange(0.0, -2.5 - 0.06, -0.05)
T_EFF_Nint = np.arange(3500, 6000 , 250)
LOGG_Nint = np.arange(0.5, 5 + 0.06, 0.5)
MH_Nint = np.arange(0.0, -2.5 - 0.06, -0.5)
MICROTURB = [0.75, 1.5, 3.0]
ABUNDANCE_SCALE = [3.16228, 1.58489, 1.00000, 0.31620, 0.10000, 0.03162, 0.01000, 0.00316]
ABUNDANCE_SCALE_header = [0.00316, 0.01000, 0.03162, 0.10000, 0.31620, 1.00000, 1.58489, 3.16228]
shortLinelist = False
############### 
if shortLinelist == True:
    lineList = "IronS1" #cambia per usare diverse linelists spectrum       #'Iron_lines'
    lineListT = "IronS1" #cambia per usare diverse linelists turbospectrum #'turbo_iron_lines.ew'
else:
    lineList = "Iron_lines" #cambia per usare diverse linelists spectrum       #'Iron_lines'
    lineListT = "turbo_iron_lines.ew" #cambia per usare diverse linelists turbospectrum #'turbo_iron_lines.ew'
###############
script_list = [
    "Turbo_EW1.csh", "Turbo_EW2.csh", "Turbo_EW3.csh", "Turbo_EW4.csh",
    "Turbo_EW5.csh", "Turbo_EW6.csh", "Turbo_EW7.csh", "Turbo_EW8.csh",
    "Turbo_EW9.csh", "Turbo_EW10.csh", "Turbo_EW11.csh", "Turbo_EW12.csh"
]
script_list2 = [
    "Turbo2_EW1.csh", "Turbo2_EW2.csh", "Turbo2_EW3.csh", "Turbo2_EW4.csh",
    "Turbo2_EW5.csh", "Turbo2_EW6.csh", "Turbo2_EW7.csh", "Turbo2_EW8.csh",
    "Turbo2_EW9.csh", "Turbo2_EW10.csh", "Turbo2_EW11.csh", "Turbo2_EW12.csh"
]
LIGHTSPEED = 299792.458
minceData = [["HARPN1", 4746, 1.89, -0.56], # HD 87740
             ["HARPN2", 4610, 1.73, -0.58], # HD 91276
             ["HARPN3", 4458, 1.54, -0.56], # BD+132383
             ["HARPN4", 4045, 1.21, -0.64], # HD 130971
             ["HARPN5", 4444, 4,-0.5],      # 
             ["HARPN6", 4722, 1.89, 0.02],  # BD+242817
             ["HARPN7", 4725, 2.41, -0.19], # HD 138934
             ["HARPN8", 4444, 4, -0.5],     # 
             ["HARPN9", 4458, 1.54, -0.56], # 
             ["HARPN10", 4393, 1.08, -1.99],# HD 115575
             ["HARPN11", 4193, 0.71, -1.95],# BD-073523
             ["HARPN12", 4167, 0.82, -1.45],# BD+062880
             ["HARPN13", 4154, 0.52, -2.09],# HD 238439
             ["HARPN14", 4287, 0.70, -1.71],# HD 139423
             ["HARPN15", 4316, 0.87, -1.46],# HD 142614
             ["HARPN16", 4276, 0.70, -2.27],# BD+25 4520
             ["HARPN17", 4249, 0.79, -1.61]]# HD 208316
NLTE_wavelenght = [3440.610, 
                   436.886,
            3440.990,
            3465.860,
            3475.440,
            3490.570,
            3565.380,
            3570.100,
            3581.190,
            3608.860,
            3618.770,
            3719.920,
            3737.120,
            3727.620,
            3745.560,
            3748.260,
            3758.230,
            3763.790,
            3815.840,
            3820.430,
            3824.440,
            3825.870,
            3840.440,
            3856.370,
            3859.910,
            3922.910,
            4045.810,
            4063.590,
            4071.740,
            4235.940,
            4250.120,
            4250.790,
            4415.120,
            4445.470,
            4494.560,
            4920.500,
            4994.130,
            5044.210,
            5198.700,
            5216.270,
            5225.520,
            5232.940,
            5236.200,
            5242.490,
            5247.040,
            5250.210,
            5269.540,
            5281.790,
            5379.570,
            5383.370,
            5434.521,
            5491.830,
            5586.750,
            5600.220,
            5661.350,
            5662.520,
            5696.090,
            5701.540,
            5705.460,
            5778.450,
            5784.660,
            5855.080,
            5916.250,
            5956.690,
            6065.479,
            6082.710,
            6151.620,
            6173.330,
            6200.310,
            6219.280,
            6240.650,
            6252.560,
            6265.130,
            6297.790,
            6311.500,
            6430.850,
            6498.940,
            6518.370,
            6574.229,
            6593.870,
            6609.110,
            6699.140,
            6726.670,
            6739.521,
            6750.150,
            6793.260,
            6810.260,
            6837.010,
            6854.820,
            6945.200,
            6978.850,
            7401.680,
            7912.870,
            8293.500,
            4233.170,
            4491.390,
            4508.290,
            4576.330,
            4582.840,
            4583.830,
            4620.520,
            4923.930,
            5018.440,
            5169.020,
            5197.580,
            5234.620,
            5264.810,
            5284.110,
            5325.550,
            5414.070,
            5425.260,
            6239.950,
            6247.560,
            6369.460,
            6432.680,
            6456.380,
            6516.080,
            7222.390,
            7224.490,
            7515.830,
            7711.720]
nomi_fileFits = ["HARPN.2020-05-10T21-18-51.039_s1d_A.fits",
"HARPN.2020-05-10T22-12-22.186_s1d_A.fits",
"HARPN.2020-05-10T23-06-53.246_s1d_A.fits",
"HARPN.2020-05-11T00-59-19.289_s1d_A.fits",
"HARPN.2020-05-11T02-03-44.593_s1d_A.fits",
"HARPN.2020-05-11T03-06-48.862_s1d_A.fits",
"HARPN.2020-05-11T03-59-13.184_s1d_A.fits",
"HARPN.2020-05-11T04-36-55.934_s1d_A.fits",
"HARPN.2020-05-11T20-54-51.023_s1d_A.fits",
"HARPN.2020-06-09T21-12-39.990_s1d_A.fits",
"HARPN.2020-06-09T22-15-01.629_s1d_A.fits",
"HARPN.2020-06-09T23-18-01.289_s1d_A.fits",
"HARPN.2020-06-10T00-12-36.574_s1d_A.fits",
"HARPN.2020-06-10T01-16-38.066_s1d_A.fits",
"HARPN.2020-06-10T01-39-10.823_s1d_A.fits",
"HARPN.2020-06-10T02-23-08.791_s1d_A.fits",
"HARPN.2020-06-10T03-26-11.908_s1d_A.fits",
'H1.fits',
'H2.fits',
'H3.fits',
'H4.fits',
'H5.fits',
'H6.fits',
'H7.fits',
'H8.fits',
'H9.fits',
'H10.fits',
'H11.fits',
'H12.fits',
'H13.fits',
'FIES1.fits',
'FIES2.fits',
'FIES3.fits',
'FIES4.fits',
'FIES5.fits',
'FIES6.fits',
'FIES7.fits',
'FIES8.fits',
'FIES9.fits',
'FIES10.fits',
'FIES11.fits',
'FIES12.fits',
'FIES13.fits',
'FIES14.fits',
'FIES15.fits',
'FIES16.fits',
'FIES17.fits',
'FIES18.fits',
'FIES19.fits',
"FIIa180033_step011_merge.fits",
"FIIa180034_step011_merge.fits",
"FIIa180035_step011_merge.fits",
"FIIa180036_step011_merge.fits",
"FIIa180037_step011_merge.fits",
"FIIa180038_step011_merge.fits",
"FIIa180039_step011_merge.fits",
"FIIa180040_step011_merge.fits",
"FIIa180041_step011_merge.fits",
"FIIa190033_step011_merge.fits",
"FIIa190034_step011_merge.fits",
"FIIa190035_step011_merge.fits",
"FIIa190036_step011_merge.fits",
"FIIa190037_step011_merge.fits",
"FIIa190038_step011_merge.fits",
"FIIa190039_step011_merge.fits",
"FIIa190040_step011_merge.fits",
"FIIa190041_step011_merge.fits",
"FIIa190042_step011_merge.fits",
"FIIa190043_step011_merge.fits"]
periodica_table = {
    'H': 1,
    'He': 2,
    'Li': 3,
    'Be': 4,
    'B': 5,
    'C': 6,
    'N': 7,
    'O': 8,
    'F': 9,
    'Ne': 10,
    'Na': 11,
    'Mg': 12,
    'Al': 13,
    'Si': 14,
    'P': 15,
    'S': 16,
    'Cl': 17,
    'Ar': 18,
    'K': 19,
    'Ca': 20,
    'Sc': 21,
    'Ti': 22,
    'V': 23,
    'Cr': 24,
    'Mn': 25,
    'Fe': 26,
    'Co': 27,
    'Ni': 28,
    'Cu': 29,
    'Zn': 30,
    'Ga': 31,
    'Ge': 32,
    'As': 33,
    'Se': 34,
    'Br': 35,
    'Kr': 36,
    'Rb': 37,
    'Sr': 38,
    'Y': 39,
    'Zr': 40,
    'Nb': 41,
    'Mo': 42,
    'Tc': 43,
    'Ru': 44,
    'Rh': 45,
    'Pd': 46,
    'Ag': 47,
    'Cd': 48,
    'In': 49,
    'Sn': 50,
    'Sb': 51,
    'Te': 52,
    'I': 53,
    'Xe': 54,
    'Cs': 55,
    'Ba': 56,
    'La': 57,
    'Ce': 58,
    'Pr': 59,
    'Nd': 60,
    'Pm': 61,
    'Sm': 62,
    'Eu': 63,
    'Gd': 64,
    'Tb': 65,
    'Dy': 66,
    'Ho': 67,
    'Er': 68,
    'Tm': 69,
    'Yb': 70,
    'Lu': 71,
    'Hf': 72,
    'Ta': 73,
    'W': 74,
    'Re': 75,
    'Os': 76,
    'Ir': 77,
    'Pt': 78,
    'Au': 79,
    'Hg': 80,
    'Tl': 81,
    'Pb': 82,
    'Bi': 83,
    'Po': 84,
    'At': 85,
    'Rn': 86,
    'Fr': 87,
    'Ra': 88,
    'Ac': 89,
    'Th': 90,
    'Pa': 91,
    'U': 92,
    'Np': 93,
    'Pu': 94,
    'Am': 95,
    'Cm': 96,
    'Bk': 97,
    'Cf': 98,
    'Es': 99,
    'Fm': 100,
    'Md': 101,
    'No': 102,
    'Lr': 103,
    'Rf': 104,
    'Db': 105,
    'Sg': 106,
    'Bh': 107,
    'Hs': 108,
    'Mt': 109,
    'Ds': 110,
    'Rg': 111,
    'Cn': 112,
    'Nh': 113,
    'Fl': 114,
    'Mc': 115,
    'Lv': 116,
    'Ts': 117,
    'Og': 118
}
table_periodica = {'1': 'H', '2': 'He', '3': 'Li', '4': 'Be', '5': 'B', '6': 'C', '7': 'N', '8': 'O', '9': 'F', '10': 'Ne', '11': 'Na', '12': 'Mg', '13': 'Al', '14': 'Si', '15': 'P', '16': 'S', '17': 'Cl', '18': 'Ar',
                  '19': 'K', '20': 'Ca', '21': 'Sc', '22': 'Ti', '23': 'V', '24': 'Cr', '25': 'Mn', '26': 'Fe', '27': 'Co', '28': 'Ni', '29': 'Cu', '30': 'Zn', '31': 'Ga', '32': 'Ge', '33': 'As', '34': 'Se',
                  '35': 'Br', '36': 'Kr', '37': 'Rb', '38': 'Sr', '39': 'Y', '40': 'Zr', '41': 'Nb', '42': 'Mo', '43': 'Tc', '44': 'Ru', '45': 'Rh', '46': 'Pd', '47': 'Ag', '48': 'Cd', '49': 'In', '50': 'Sn',
                  '51': 'Sb', '52': 'Te', '53': 'I', '54': 'Xe', '55': 'Cs', '56': 'Ba', '57': 'La', '58': 'Ce', '59': 'Pr', '60': 'Nd', '61': 'Pm', '62': 'Sm', '63': 'Eu', '64': 'Gd', '65': 'Tb', '66': 'Dy',
                  '67': 'Ho', '68': 'Er', '69': 'Tm', '70': 'Yb', '71': 'Lu', '72': 'Hf', '73': 'Ta', '74': 'W', '75': 'Re', '76': 'Os', '77': 'Ir', '78': 'Pt', '79': 'Au', '80': 'Hg', '81': 'Tl', '82': 'Pb',
                  '83': 'Bi', '84': 'Po', '85': 'At', '86': 'Rn', '87': 'Fr', '88': 'Ra', '89': 'Ac', '90': 'Th', '91': 'Pa', '92': 'U'}
datiTrovati=[['HD87740', 4712, 1.93, -0.49, 3.62],
             ['HD91276', 4533, 1.62, -0.66, 2.46],
             ['BD+132383', 4347, 1.37, -0.75, 2.25],
             ['HD130971', 4042, 0.88, -0.71],
             ['BD+242817', 4572, 1.90, -0.17, 3.91],
             ['HD138934', 4654, 2.33, -0.47, 4.04],
             ['HD115575', 4328, 0.71, -1.96, 1.19],
             ['BD-073523', 4162, 0.52, -1.91, 2.01],
             ['BD+062880', 4194, 0.45, -1.45, 1.46],
             ['HD238439', 4144, 0.18, -1.99, 2.85],
             ['HD139423', 4282, 1.01, -1.76],
             ['HD142614', 4325, 1.01, -1.34, 2.12],
             ['BD+25 4520', 4192, 0.16, -2.27],
             ['HD208316', 4235, 0.72, -1.55, 2.39],
             ['BD+27_279', 3958, 0.44, -1.34, 2.80],
             ['TYC_749-2025-1', 4290, 0.57, -1.55, 1.82],
             ['TYC_1924-1531-1a', 4354, 0.59, -2.28, 3.02],
             ['TYC_1924-1531-1b', 4367, 0.58, -2.20, 2.52],
             ['TYC_810-1846-1', 4082, 0.77, -1.29, 2.80],
             ['HD77662', 4394, 0.42, -2.14, 3.40],
             ['BD+21_2046', 3971, 0.39, -1.20, 3.42],
             ['TYC_4386-1860-1', 4365, 0.90, -1.83, 4.65],
             ['BD+13_2153', 4466, 1.05, -1.88, 3.45],
             ['HD233868', 4464, 0.95, -1.90, 4.38],
             ['BD+21_2364', 4206, 0.60, -1.36, 1.55],
             ['TYC_2474-577-1', 4146, 1.08, -0.94, 2.79]]
zpt.load_tables()

notData = [['BD-02  2610', 4076, 1.01,-1.08, 1.57, 'FIES5.fits', 0.4, 75], #buona #
         ['BD+17  2086',4725, 2.28,-0.36, 1.46, 'FIES14.fits', 0, 80],   #buona #
         ['BD+38  1878',4730, 2.49,-0.60, 1.72, 'FIES13.fits', 0.2, 65],   #buona #
         #['BD+40  2102',4649, 2.23,-0.10, 1.12, 'FIES7.fits', 0, 75], 
         ['BD+40  2102',4560, 2.58,0.09, 1.04, 'FIES7.fits', 0.0, 75], #buona con questi valori #
         ['BD+42  1911',4463, 1.67,-0.21, 1.69, 'FIES15.fits', 0.17, 75],#buona # ricercare logg perché fei-feii è no skif
         ['BD+47  1535',4752, 2.30,-0.77, 1.30, 'FIES6.fits', 0.2, 75], #buona #
         ['BD+55  1336',4166, 1.36,-0.47, 1.91, 'FIES18.fits', 0.0, 75],#buona   
         #['HD 237941',4577, 2.24,-0.51, 1.35, 'FIES19.fits'],  #buona # o
         ['HD 237941',4630, 2.27,-0.17, 1.38, 'FIES19.fits', 0.24, 75],  #buona # o
         ['TYC 1961-1226-1', 4242, 0.99, -1.13, 2.02, 'FIES8.fits', 0.4, 80],#buona # o
         ['TYC  249-1043-1', 5093, 2.39, -1.09, 2.23,'FIES17.fits', 0.4, 75], #buona #
         ['TYC 2973-1090-1', 4302, 0.61, -1.74, 2.59,'FIES11.fits', 0.2, 75], #buona # o
         ['TYC 3430-571-1',  4674, 2.25, -0.32, 1.24,'FIES16.fits', 0.2, 80],  #buona #
         #['TYC 3803-1525-1', 4113, 2.07, -0.83, 1.35,'FIES12.fits', 0.2, 70], #buona # o
         ['TYC 3803-1525-1', 4437, 1.77, -0.49, 1.35,'FIES12.fits', 0.2, 70], #buona # buona con questi di più
         ['TYC 5333-1917-1', 4544, 1.73, -1.48, 1.39,'FIES1.fits', 0.2, 65],  #buona # 
         ['HD 245454', 4394, 1.56, 0.11, 0.73,'FIES4.fits', 0, ],         #buona # ricontrollare logg x mmmmm forse logg molto più alto
         ['HD 286195', 3898, 1.06, -0.75, 1.53,'FIES3.fits', 0, ],        #buona # x
         ['HD 287410', 4046, 0.78, -1.00, 1.59,'FIES9.fits', 0.4, 75]] # buona con questi valori.. #
         #['HD 287410', 3781, 1.61, -0.83, 1.50,'FIES9.fits', 0.4], valori fotometrici

otherMinceData_old = [['BD+27_279', 3929, 0.23, -1.34, 2.11, 'H1.fits'],        #-1.23
             ['TYC_749-2025-1', 4129, 0.96, -1.55, 1.67, 'H3.fits'],        #-1.56
             #['TYC_1924-1531-1a', 4354, 0.59, -2.28, 3.02, 'H4.fits'],     #
             ['TYC_1924-1531-1b', 4531, 1.36, -2.20, 1.78, 'H5.fits'],      #-2.01
             ['TYC_810-1846-1', 4041,  0.79, -1.29, 1.71, 'H6.fits'],       #-1.24
             ['HD77662', 4595, 1.45, -2.14, 1.77, 'H7.fits'],               #-1.95
             ['BD+21_2046', 3889, 0.72, -1.20, 1.59, 'H8.fits'],            #-0.92
             ['TYC_4386-1860-1', 4001, 1.22, -1.83, 1.26, 'H9.fits'],       #-1.77
             ['BD+13_2153', 4689, 1.76, -1.88, 1.62, 'H10.fits'],           #-1.50
             ['HD233868', 4631, 1.57, -1.90, 1.71, 'H11.fits'],             #-1.68
             ['BD+21_2364', 4178, 0.97, -1.36, 1.71, 'H12.fits'],           #-1.23
             ['TYC_2474-577-1', 4149, 1.33, -0.94, 1.39, 'H13.fits']]       #-0.62

otherMinceData = [['BD+27_279', 3929, 0.23, -1.23, 2.11, 'H1.fits'],        #-1.23
             ['TYC_749-2025-1', 4129, 0.96, -1.56, 1.67, 'H3.fits'],        #-1.56
             #['TYC_1924-1531-1a', 4354, 0.59, -2.28, 3.02, 'H4.fits'],     #
             ['TYC_1924-1531-1b', 4531, 1.36, -2.01, 1.78, 'H5.fits'],      #-2.01
             ['TYC_810-1846-1', 4041,  0.79, -1.24, 1.71, 'H6.fits'],       #-1.24
             ['HD77662', 4595, 1.45, -2.14, 1.95, 'H7.fits'],               #-1.95
             ['BD+21_2046', 3889, 0.72, -0.92, 1.59, 'H8.fits'],            #-0.92
             ['TYC_4386-1860-1', 4001, 1.22, -1.77, 1.26, 'H9.fits'],       #-1.77
             ['BD+13_2153', 4689, 1.76, -1.50, 1.62, 'H10.fits'],           #-1.50
             ['HD233868', 4631, 1.57, -1.68, 1.71, 'H11.fits'],             #-1.68
             ['BD+21_2364', 4178, 0.97, -1.23, 1.71, 'H12.fits'],           #-1.23
             ['TYC_2474-577-1', 4149, 1.33, -0.62, 1.39, 'H13.fits']]       #-0.62

otherMinceData = [['BD+27_279', 4125, 1.06, -1.13, 2.33, 'H1.fits'],        #-1.23
             ['TYC_749-2025-1', 4129, 0.96, -1.56, 1.67, 'H3.fits'],        #-1.56
             #['TYC_1924-1531-1a', 4354, 0.59, -2.28, 3.02, 'H4.fits'],     #
             ['TYC_1924-1531-1b', 4531, 1.36, -2.01, 1.78, 'H5.fits'],      #-2.01
             ['TYC_810-1846-1', 4041,  0.79, -1.24, 1.71, 'H6.fits'],       #-1.24
             ['HD77662', 4595, 1.45, -2.14, 1.95, 'H7.fits'],               #-1.95
             ['BD+21_2046', 3889, 0.72, -0.92, 1.59, 'H8.fits'],            #-0.92
             ['TYC_4386-1860-1', 4001, 1.22, -1.77, 1.26, 'H9.fits'],       #-1.77
             ['BD+13_2153', 4689, 1.76, -1.50, 1.62, 'H10.fits'],           #-1.50
             ['HD233868', 4631, 1.57, -1.68, 1.71, 'H11.fits'],             #-1.68
             ['BD+21_2364', 4178, 0.97, -1.23, 1.71, 'H12.fits'],           #-1.23
             ['TYC_2474-577-1', 4149, 1.33, -0.62, 1.39, 'H13.fits']]       #-0.62

snotData = [#['BD+27_279', 4125, 1.06, -1.13, 2.33, 'H1.fits'],        #-1.23
             ['TYC_749-2025-1', 4350, 1.06, -1.56, 2.22, 'H3.fits', 0.3, 75],        #-1.56
             #['TYC_1924-1531-1a', 4354, 0.59, -2.28, 3.02, 'H4.fits'],     #
             ['TYC_1924-1531-1b', 4531, 1.36, -2.12, 1.78, 'H5.fits', 0.3, 70],      #-2.01
             ['TYC_810-1846-1', 4041,  0.79, -1.21, 1.71, 'H6.fits', 0.32, 70],       #-1.24
             ['HD77662', 4595, 1.45, -2.07, 1.95, 'H7.fits', 0.29, 75],               #-1.95
             ['BD+21_2046', 3889, 0.72, -0.86, 1.59, 'H8.fits', 0.19, 70],            #-0.92
             #['TYC_4386-1860-1', 4001, 1.22, -1.77, 1.26, 'H9.fits', 0.],       #-1.77
             ['BD+13_2153', 4689, 1.76, -1.59, 1.62, 'H10.fits', 0.33, 70],           #-1.50
             ['HD233868', 4631, 1.57, -1.68, 1.71, 'H11.fits', 0.39, 75],             #-1.68
             ['BD+21_2364', 4178, 0.97, -1.25, 1.71, 'H12.fits', 0.31, 75],           #-1.23
             ['TYC_2474-577-1', 4149, 1.33, -0.62, 1.39, 'H13.fits', 0.1, 70]]       #-0.62




gotData = [['BD-02_2610.out', 4076, 1.01,-1.08, 1.57, 'FIES5.fits', 0.4, 75], #buona #
         ['BD+13_2153.out', 4689, 1.76, -1.59, 1.62, 'H10.fits', 0.33, 70],           #-1.50
         ['BD+17_2086.out',4725, 2.28,-0.36, 1.46, 'FIES14.fits', 0, 80],   #buona #
         ['BD+21_2046.out', 3889, 0.72, -0.86, 1.59, 'H8.fits', 0.19, 70],            #-0.92
         #['TYC_4386-1860-1', 4001, 1.22, -1.77, 1.26, 'H9.fits', 0.],       #-1.77
         ['BD+21_2364.out', 4178, 0.97, -1.25, 1.71, 'H12.fits', 0.31, 75],           #-1.23
         #['BD+38_1878.out',4730, 2.49,-0.60, 1.72, 'FIES13.fits', 0.2, 65],   #buona #
         #['BD+40  2102',4649, 2.23,-0.10, 1.12, 'FIES7.fits', 0, 75], 
         #['BD+40  2102.out',4560, 2.58,0.09, 1.04, 'FIES7.fits', 0.0, 75], #buona con questi valori #
         ['BD+42_1911.out',4463, 1.67,-0.21, 1.69, 'FIES15.fits', 0.17, 75],#buona # ricercare logg perché fei-feii è no skif
         ['BD+47__1535.out',4752, 2.30,-0.77, 1.30, 'FIES6.fits', 0.2, 75], #buona #
         ['BD+55_1336.out',4166, 1.36,-0.47, 1.91, 'FIES18.fits', 0.0, 75],#buona   
         #['HD 237941',4577, 2.24,-0.51, 1.35, 'FIES19.fits'],  #buona # o
         ['TYC_249-1043-1.out', 5093, 2.39, -1.09, 2.23,'FIES17.fits', 0.4, 75], #buona #
         ['TYC_749-2025-1.out', 4350, 1.06, -1.56, 2.22, 'H3.fits', 0.3, 75],        #-1.56
         ['TYC_810-1846-1.out', 4041,  0.79, -1.21, 1.71, 'H6.fits', 0.32, 70],       #-1.24
         ['TYC_1924-1531-1.out', 4531, 1.36, -2.12, 1.78, 'H5.fits', 0.3, 70],      #-2.01
         ['TYC_1961-1226-1.out', 4242, 0.99, -1.13, 2.02, 'FIES8.fits', 0.4, 80],#buona # o
         ['TYC_2474-577-1.out', 4149, 1.33, -0.62, 1.39, 'H13.fits', 0.1, 70], # buona con questi valori.. #
         ['TYC_2973-1090-1.out', 4302, 0.61, -1.74, 2.59,'FIES11.fits', 0.2, 75], #buona # o
         ['TYC_3430-571-1.out',  4674, 2.25, -0.32, 1.24,'FIES16.fits', 0.2, 80],  #buona #
         ['TYC_3803-1525-1.out', 4113, 2.07, -0.83, 1.35,'FIES12.fits', 0.2, 70], #buona # o
         ['TYC_5333-1917-1.out', 4544, 1.73, -1.48, 1.39,'FIES1.fits', 0.2, 65],  #buona # 
         #['TYC 3803-1525-1', 4437, 1.77, -0.49, 1.35,'FIES12.fits', 0.2, 70], #buona # buona con questi di più
         ['HD77662.out', 4595, 1.45, -2.07, 1.95, 'H7.fits', 0.29, 75],               #-1.95
         ['HD233868.out', 4631, 1.57, -1.68, 1.71, 'H11.fits', 0.39, 75],             #-1.68
         ['HD287410.out', 4046, 0.78, -1.00, 1.59,'FIES9.fits', 0.4, 75]]
        #  ['HD 237941.out',4630, 2.27,-0.17, 1.38, 'FIES19.fits', 0.24, 75],  #buona # o
        #  ['HD 245454.out', 4394, 1.56, 0.11, 0.73,'FIES4.fits', 0, ],         #buona # ricontrollare logg x mmmmm forse logg molto più alto
        #  ['HD 286195.out', 3898, 1.06, -0.75, 1.53,'FIES3.fits', 0, ],        #buona # x
         #['TYC_1924-1531-1a', 4354, 0.59, -2.28, 3.02, 'H4.fits'],     #
         #['HD 287410', 3781, 1.61, -0.83, 1.50,'FIES9.fits', 0.4], valori fotometrici
def NLTE_caricaWLdisponibili():
    filename = f"{path_to_correctionGrids1}4500-5000/correctionGrid"

    # Lista per i valori univoci
    unique_values = set()
    with open(filename, "r") as file:
        for line in file:
            # Ignora righe vuote o non formattate correttamente
            if line.strip():
                # Divide la riga in colonne
                columns = line.split()
                try:
                    # Estrae il valore della sesta colonna e lo aggiunge al set
                    unique_values.add(round(float(columns[5])*10, 3))
                except IndexError:
                    # Gestisce righe che non hanno abbastanza colonne
                    print(f"Riga non valida (meno di 6 colonne): {line.strip()}")
    # Lettura del file
    for i in range(7):
        filename = f"{path_to_correctionGrids1}{5000+250*i}-{5250+250*i}/correctionGrid"
        with open(filename, "r") as file:
            for line in file:
                # Ignora righe vuote o non formattate correttamente
                if line.strip():
                    # Divide la riga in colonne
                    columns = line.split()
                    try:
                        # Estrae il valore della sesta colonna e lo aggiunge al set
                        unique_values.add(round(float(columns[5])*10, 3))
                    except IndexError:
                        # Gestisce righe che non hanno abbastanza colonne
                        print(f"Riga non valida (meno di 6 colonne): {line.strip()}")

    # Converte il set in una lista ordinata
    unique_values_list = sorted(unique_values)

    # Stampa dei risultati
    print("Valori univoci nella sesta colonna:")
    return unique_values_list

lineList='eso_fe'


####MIDAS SCRIPT####
def broaden_withMIDAS(spectral_data, broadening, midas_id=-1, MIDAS_folder='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/'):
    os.chdir(MIDAS_folder)

    # Avvia il processo inmidas
    if midas_id == -1:
        child = pexpect.spawn('inmidas')
    else:
        child = pexpect.spawn(f'inmidas {midas_id:02d}')

    # Aspetta il prompt di inmidas (cambia 'Midas...' con il prompt reale se diverso)
    #child.expect('Midas>')  # oppure usa child.expect('@@') se compare subito quello
    child.sendline('y')
    # Invia il primo comando
    child.sendline(f'@@ mktbl {spectral_data}')

    # Aspetta di nuovo il prompt
    child.expect('Midas 001>')

    # Invia il secondo comando
    child.sendline(f'@@ broadtbld {spectral_data} :ri {broadening:.1f}')

    # Attendi che finisca o che appaia di nuovo il prompt, in base a cosa fa il comando
    child.expect('Midas 002>')

    # Invia il terzo comando
    child.sendline(f'ASSIGN/PRIN file {spectral_data}.dat')
    # Attendi di nuovo il prompt
    child.expect('Midas 003>')
    # Invia il quarto comando
    child.sendline(f'PRINT/TABL {spectral_data}.tbl :wave :hbr')
    # Attendi di nuovo il prompt
    child.expect('Midas 004>')
    # Puoi fare ulteriori comandi o chiudere
    child.sendline('exit')
    child.expect(pexpect.EOF)

    print(child.before.decode(errors='ignore'))
def differenza_un_carattere(s1, s2):
    """Ritorna True se le due stringhe differiscono per un solo carattere"""
    if len(s1) != len(s2):
        return False
    differenze = sum(c1 != c2 for c1, c2 in zip(s1, s2))
    return differenze == 1
def elimina_file_cartella(percorso_cartella):
    """
    Elimina tutti i file nella cartella che terminano con .dat, .dat_ o .tbl
    """
    estensioni_da_eliminare = ('.dat', '.dat_', '.tbl', 'ba1')

    for nome_file in os.listdir(percorso_cartella):
        percorso_completo = os.path.join(percorso_cartella, nome_file)
        if os.path.isfile(percorso_completo) and nome_file.endswith(estensioni_da_eliminare):
            os.remove(percorso_completo)
            print(f"Eliminato: {nome_file}")
def trova_coppie_cartella(percorso_cartella):
    """Trova tutte le coppie di file che differiscono per un solo carattere nel nome"""
    files = sorted(os.listdir(percorso_cartella))
    coppie = []

    for i in range(len(files)):
        for j in range(i + 1, len(files)):
            f1, f2 = files[i], files[j]
            if differenza_un_carattere(f1, f2):
                coppie.append([f1, f2])
    
    return coppie
def combina_spettri(sp, sp_out, output_folder='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/o'):
    '''Combina spettri broadenati con MIDAS e combinali\n
    sp: lista contenente i nomi degli spettri da combinare\n
    sp_out: stringa con nome output'''
    #os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/')
    tutte_le_righe = []
    for filename in sp:
        with open(filename, 'r') as f:
            tutte_le_righe.extend(f.readlines())
    # Assicura che la cartella di output esista
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, sp_out)
    with open(output_path, 'w') as f:
        f.writelines(tutte_le_righe)
def formatta(sp):
    with open(sp, 'r') as f:
        lines = f.readlines()
    lint=lines[3:]
    lin=lint[0:len(lint)-1]
    with open(sp+'_', 'w') as f:
        for i in range(len(lin)):
            f.write(f'{lin[i]}')
    return sp+'_'
def spectralBroadening(path, files, broadening, output, outpath='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/o'):
    '''files è una lista di file da allargare in quanto lo spettro può essere contenuto in diversi file (eg. produco spettri in pezzi da 2000A quindi spesso produco 2 file per stella), broadening è il valore del broadening'''
    olddir=os.getcwd()
    os.chdir(path)
    if len(files) == 1:
        broaden_withMIDAS(files[0], broadening=broadening)
        formatta(files[0]+'.dat')
    elif len(files) > 1:
        fils=[]
        for i in range(len(files)):
            broaden_withMIDAS(files[i], broadening=broadening)
            fils.append(formatta(files[i]+'.dat'))
        combina_spettri(fils, output, outpath)

def print_mkgrid_metallicity(data):
    '''data deve essere formattata come \'notData\' o \'snotData\', ovvero una lista di liste, dove ogni lista contiene i dati di una stella.\n
    Indipendently from the data structure, the first element of each sublist must be the star name, and the fourth element must be the metallicity value.\n'''
    for I in range(len(data)):
        print(f'{data[I][0]}: {(data[I][3]-1):.2f},{(data[I][3]-0.8):.2f},{(data[I][3]-0.6):.2f},{(data[I][3]-0.4):.2f},{(data[I][3]-0.2):.2f},{(data[I][3]):.2f},{(data[I][3]+0.2):.2f},{(data[I][3]+0.4):.2f},{(notData[I][3]+0.6):.2f},\n     &{(data[I][3]+0.8):.2f},{(data[I][3]+1):.2f}')


def moltiplica_prima_colonna(input_file, output_file, fattore, sep=None):
    t=Table.read(input_file, format='ascii')
    x=t['col1']
    y=t['col2']
    y=y*fattore
    with open(output_file, 'w') as f:
        for i in range(len(x)):
            f.write(f'{x[i]} {y[i]}\n')
def normalizzas_y(y, x, modo='dividi', coeff=[]):
    """
    Normalizza y usando una parabola di coefficienti a, b, c valutata su x.
    
    Parametri:
        y (array-like): vettore dei dati da normalizzare
        x (array-like): vettore delle ascisse
        a, b, c (float): coefficienti della parabola
        modo (str): 'dividi' per y/f(x), 'moltiplica' per y*f(x)
        
    Ritorna:
        y_n (np.ndarray): vettore y normalizzato
    """
    if len(coeff) == 3:
        func = coeff[0] * np.array(x)**2 + coeff[1] * np.array(x) + coeff[2] 
        print('pigninc')
    elif len(coeff) == 2:
        func = coeff[0] * np.array(x) + coeff[1]
        print('EEE')
    elif len(coeff) == 4:
        func = coeff[0] * np.array(x)**3 + coeff[1] * np.array(x)**2 + coeff[2] * np.array(x) + coeff[3]
    elif len(coeff) == 5:
        func = coeff[0] * np.array(x)**4 + coeff[1] * np.array(x)**3 + coeff[2] * np.array(x)**2 + coeff[3] * np.array(x) + coeff[4]
    if modo == 'dividi':
        y_n = np.array(y) / func
    elif modo == 'moltiplica':
        y_n = np.array(y) * func
    else:
        raise ValueError("modo deve essere 'dividi' o 'moltiplica'")
    return y_n
def somma_prima_colonna(input_file, output_file, fattore, sep=None):
    t=Table.read(input_file, format='ascii')
    x=t['col1']
    y=t['col2']
    y=y+fattore
    with open(output_file, 'w') as f:
        for i in range(len(x)):
            f.write(f'{x[i]} {y[i]}\n')
def plot_mygisfos(path='/home/starcat8/Documenti/MyGisFOS/test/STAR_BD+38_1878/', file='BD+38_1878_FIESta', region=1, datibooni='', path_datibooni='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/', coeff=[-1.40816852e-02 , 1.09519180e+00]):
    '''Plot per MyGisFOS, path è il percorso della cartella contenente i file di MyGisFOS, file è il nome del file senza estensione, region è la regione da plottare (1,2,3,4 o 5), datibooni è il nome del file con i dati buoni da plottare (se non si vuole plottare lasciare vuoto), path_datibooni è il percorso del file con i dati buoni.
    Qualora per region si scelga 4 si plottano le tre regioni insieme, in questo caso datibooni deve essere un file con i dati buoni di tutte e tre le regioni. Scegliendo 5 si plottano direttamente le features oltre ai best fit.'''
    os.chdir(path)
    region_a=None
    region_b=None
    def truncate_lines(input_file, output_file, max_chars=175):
        with open(input_file, 'r') as infile:
            lines = infile.readlines()

        # Rimuove le prime 6 righe
        lines = lines[7:]

        with open(output_file, 'w') as outfile:
            for line in lines[:]:
                outfile.write(line[:max_chars] + '\n')
    if region == 1:
        region_a = '_b'
        region_b = '_01'
    elif region == 2:
        region_a = '_l'
        region_b = '_02'
    elif region == 3:
        region_a = '_u'
        region_b = '_03'
    elif region == 4 or region == 5:
        region_a1 = '_b'
        region_b1 = '_01'
        region_a2 = '_l'
        region_b2 = '_02'
        region_a3 = '_u'
        region_b3 = '_03'
    if region !=4 and region !=5:
        truncate_lines('line_by_line.out', 'lbl2.out')
        T1=Table.read(f'{path}{file}{region_a}.dat.flu', format='ascii')
        x1=T1['col1']
        y1=T1['col2']
        olddir=os.getcwd()
        os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/')
        #T2=Table.read(f'best_fit_synthe{region_b}.dat', format='ascii')
        #T2=Table.read(f'ao.dat', format='ascii')
        T2=Table.read(f'tyc53331.dat', format='ascii')
        x2=T2['col2']
        y2=T2['col3']
        print(x2)
        os.chdir(olddir)
        T3=Table.read(f'best_fit_synthe{region_b}.dat', format='ascii')
        #T3=Table.read(f'{file}{region_a}.dat.con', format='ascii')
        x3=T3['col1']
        y3=T3['col2']
        fig, ax = plt.subplots()
        plt.plot(x1*10, y1, label='flu')
        plt.plot(x2, y2*1.015625, label='best')
        plt.plot(x3*10, y3, label='con')
        TAB=Table.read('lbl2.out', format='ascii')
        dati=TAB
        lunghezze_d_onda = (TAB['col4']+TAB['col5'])/2

        # Crea il grafico
        #plt.figure(figsize=(10, 4))
        # Altezza fissa dei rettangoli
        y_min = -0.1
        y_max = 1.5
        height = y_max - y_min

        # Iterazione sui dati per creare i rettangoli
        for i, (wini, wend, rect_type, label) in enumerate(zip(dati['col4'], dati['col5'], dati['col8'], dati['col1'])):
            # Determina il colore del bordo in base al valore di 'type'
            if rect_type == -1:
                edge_color = 'red'
            elif rect_type == 1:
                edge_color = 'blue'
            else:
                edge_color = 'black'  # Default per tipi non specificati

            # Creazione del rettangolo
            if rect_type==1 and label == 'Mn':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='yellow',
                    alpha=0.5
                )
            elif rect_type==1 and label != 'Mn':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='green',
                    alpha=0.5
                )
            elif rect_type<0 and label == 'Mn':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='cyan',
                    alpha=0.5
                )
            elif rect_type<0 and label != 'Mn':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='red',
                    alpha=0.5
                )

            # Aggiunta del rettangolo agli assi
            ax.add_patch(rect)

            # Aggiunta del testo all'interno del rettangolo
            ax.text(wini*10 +  0.05, 0.3, label,  # Posizione (x, y)
                    color='black', ha='center', va='center', fontsize=10)

        # Disegna una riga verticale per ogni lunghezza d'onda
        # for lw in lunghezze_d_onda:
        #     plt.axvline(x=lw, color='blue', linestyle='--', linewidth=1)

        # Imposta etichette e titolo
        plt.xlabel("Lunghezza d'onda (Å)")
        plt.ylabel("Intensità (arbitraria)")
        plt.title(f"{path}")

        # Opzionalmente imposta i limiti dell'asse x per una vista più chiara
        plt.xlim(min(lunghezze_d_onda)*10 - 50, max(lunghezze_d_onda)*10 + 50)

        plt.grid(True)
        plt.tight_layout()
        plt.show()
        plt.legend()
        plt.show()
    elif region == 4:
        truncate_lines('line_by_line.out', 'lbl2.out')
        T11=Table.read(f'{path}{file}{region_a1}.dat.flu', format='ascii')
        x11=T11['col1']
        y11=T11['col2']
        T12=Table.read(f'{path}{file}{region_a2}.dat.flu', format='ascii')
        x12=T12['col1']
        y12=T12['col2']
        T13=Table.read(f'{path}{file}{region_a3}.dat.flu', format='ascii')
        x13=T13['col1']
        y13=T13['col2']
        olddir=os.getcwd()
        os.chdir(path_datibooni)
        os.chdir(olddir)
        T31=Table.read(f'{path}best_fit_synthe{region_b1}.dat', format='ascii')
        #T3=Table.read(f'{file}{region_a}.dat.con', format='ascii')
        x31=T31['col1']
        y31=T31['col2']
        T32=Table.read(f'{path}best_fit_synthe{region_b2}.dat', format='ascii')
        #T3=Table.read(f'{file}{region_a}.dat.con', format='ascii')
        x32=T32['col1']
        y32=T32['col2']
        T33=Table.read(f'{path}best_fit_synthe{region_b3}.dat', format='ascii')
        #T3=Table.read(f'{file}{region_a}.dat.con', format='ascii')
        x33=T33['col1']
        y33=T33['col2']
        fig, ax = plt.subplots()
        plt.plot(x11*10, y11, label='flu', color='blue')
        plt.plot(x12*10, y12, label='flu', color='blue')
        plt.plot(x13*10, y13, label='flu', color='blue')
        # plt.plot(x2, y2/0.984, label='best', color='red')
        #plt.plot(x2, y2*1.015625, label='best', color='red')
        # plt.plot(x2, y2, label='best', color='red')
        plt.plot(x31*10, y31, label='best_fit_synthe', color='cyan')
        plt.plot(x32*10, y32, label='best_fit_synthe', color='cyan')
        plt.plot(x33*10, y33, label='best_fit_synthe', color='cyan')
        TAB=Table.read('lbl2.out', format='ascii')
        dati=TAB
        lunghezze_d_onda = (TAB['col4']+TAB['col5'])/2

        # Crea il grafico
        #plt.figure(figsize=(10, 4))
        # Altezza fissa dei rettangoli
        y_min = -0.1
        y_max = 1.5
        height = y_max - y_min


        for i, (wini, wend, rect_type, label) in enumerate(zip(dati['col4'], dati['col5'], dati['col8'], dati['col1'])):

            if rect_type == -1:
                edge_color = 'red'
            elif rect_type == 1:
                edge_color = 'blue'
            else:
                edge_color = 'black' 

            if rect_type==1 and (label == 'Mn' or label == 'Na' or label == 'Co'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='yellow',
                    alpha=0.5
                )
            elif label == 'Y':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='purple',
                    alpha=0.5
                )
            elif label == 'Sm':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='magenta',
                    alpha=0.5
                )
            elif  (label == 'Ba'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='pink',
                    alpha=0.8
                )
            elif  (label == 'O'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='blue',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Mg'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='red',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Ca'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='cyan',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Si'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='gold',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Ti'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='black',
                    alpha=0.8
                )
            elif  (label == 'Eu'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='blue',
                    alpha=0.5
                )
            elif (label == 'Zr'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.8,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='brown',
                    alpha=0.5
                )
            elif (label == 'La'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='red',
                    alpha=0.5
                )
            elif rect_type==1 and label != 'Y' and label != 'Ba':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='green',
                    alpha=0.5
                )
            elif rect_type<0:
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='grey',
                    alpha=0.3
                )

            ax.add_patch(rect)

            ax.text(wini*10 +  0.05, 0.3, label,  # Posizione (x, y)
                    color='black', ha='center', va='center', fontsize=10)

        plt.xlabel("Lunghezza d'onda (Å)")
        plt.ylabel("Intensità (arbitraria)")
        plt.title(f"{path}")

        # Opzionalmente imposta i limiti dell'asse x per una vista più chiara
        plt.xlim(min(lunghezze_d_onda)*10 - 50, max(lunghezze_d_onda)*10 + 50)

        plt.grid(True)
        plt.tight_layout()
        plt.show()
        plt.legend()
        plt.show()
    elif region == 5:
        truncate_lines('line_by_line.out', 'lbl2.out')
        T11=Table.read(f'{path}{file}{region_a1}.dat.flu', format='ascii')
        x11=T11['col1']
        y11=T11['col2']
        T12=Table.read(f'{path}{file}{region_a2}.dat.flu', format='ascii')
        x12=T12['col1']
        y12=T12['col2']
        T13=Table.read(f'{path}{file}{region_a3}.dat.flu', format='ascii')
        x13=T13['col1']
        y13=T13['col2']
        olddir=os.getcwd()
        os.chdir(path_datibooni)
        os.chdir(olddir)
        T31=Table.read(f'{path}best_fit_synthe{region_b1}.dat', format='ascii')
        x31=T31['col1']
        y31=T31['col2']
        T32=Table.read(f'{path}best_fit_synthe{region_b2}.dat', format='ascii')
        x32=T32['col1']
        y32=T32['col2']
        T33=Table.read(f'{path}best_fit_synthe{region_b3}.dat', format='ascii')
        x33=T33['col1']
        y33=T33['col2']
        fig, ax = plt.subplots()
        plt.plot(x11*10, y11, label='flu', color='blue')
        plt.plot(x12*10, y12, label='flu', color='blue')
        plt.plot(x13*10, y13, label='flu', color='blue')
        plt.plot(x31*10, y31, label='best_fit_synthe', color='cyan')
        plt.plot(x32*10, y32, label='best_fit_synthe', color='cyan')
        plt.plot(x33*10, y33, label='best_fit_synthe', color='cyan')
        TAB=Table.read('lbl2.out', format='ascii')
        dati=TAB
        lunghezze_d_onda = (TAB['col4']+TAB['col5'])/2

        # Crea il grafico
        #plt.figure(figsize=(10, 4))
        # Altezza fissa dei rettangoli
        y_min = -0.1
        y_max = 1.5
        height = y_max - y_min

        # Iterazione sui dati per creare i rettangoli
        for i, (wini, wend, rect_type, label) in enumerate(zip(dati['col4'], dati['col5'], dati['col8'], dati['col1'])):
            # Determina il colore del bordo in base al valore di 'type'
            if rect_type == -1:
                edge_color = 'red'
            elif rect_type == 1:
                edge_color = 'blue'
            else:
                edge_color = 'black'  # Default per tipi non specificati

            # Creazione del rettangolo
            if rect_type==1 and (label == 'Mn' or label == 'Na' or label == 'Co'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='yellow',
                    alpha=0.5
                )
            elif label == 'Y':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='purple',
                    alpha=0.5
                )
            elif label == 'Sm':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='magenta',
                    alpha=0.5
                )
            elif  (label == 'Ba'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='pink',
                    alpha=0.8
                )
            elif  (label == 'O'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='blue',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Mg'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='red',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Ca'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='cyan',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Si'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='gold',
                    alpha=0.8
                )
            elif rect_type==1 and  (label == 'Ti'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='black',
                    alpha=0.8
                )
            elif  (label == 'Eu'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='blue',
                    alpha=0.5
                )
            elif (label == 'Zr'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.8,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='brown',
                    alpha=0.5
                )
            elif (label == 'La'):
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height + 0.5,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='red',
                    alpha=0.5
                )
            elif rect_type==1 and label != 'Y' and label != 'Ba':
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='green',
                    alpha=0.5
                )
            elif rect_type<0:
                rect = patches.Rectangle(
                    (wini*10, y_min),  # Coordinate dell'angolo inferiore sinistro
                    wend*10 - wini*10,    # Larghezza
                    height,         # Altezza
                    linewidth=2,    # Spessore del bordo
                    edgecolor=edge_color,  # Colore del bordo
                    color='grey',
                    alpha=0.3
                )

            # Aggiunta del rettangolo agli assi
            ax.add_patch(rect)

            # Aggiunta del testo all'interno del rettangolo
            ax.text(wini*10 +  0.05, 0.3, label,  # Posizione (x, y)
                    color='black', ha='center', va='center', fontsize=10)

        # Disegna una riga verticale per ogni lunghezza d'onda
        # for lw in lunghezze_d_onda:
        #     plt.axvline(x=lw, color='blue', linestyle='--', linewidth=1)

        # Imposta etichette e titolo
        plt.xlabel("Lunghezza d'onda (Å)")
        plt.ylabel("Intensità (arbitraria)")
        plt.title(f"{path}")

        # Opzionalmente imposta i limiti dell'asse x per una vista più chiara
        plt.xlim(min(lunghezze_d_onda)*10 - 50, max(lunghezze_d_onda)*10 + 50)

        plt.grid(True)
        plt.tight_layout()
        plt.show()
        plt.legend()
        plt.show()


        
    os.chdir(olddir)
def format_to_results(path_to_abundFiles='/home/starcat8/Scrivania/abundances/', data=gotData):
    elementi = ['O', 'Na', 'Mg', 'Al', 'Si', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Sr', 'Y', 'Zr', 'Mo', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Lu']
    ioni = [0, 1]

    # Directory di lavoro
    path = path
    os.chdir(path)
    files = os.listdir()

    # Apertura file di output
    with open('results.txt', 'w') as f:
        # Intestazione
        intestazione = """Fe        Festd    FeII        FeIIstd        O        Ostd    Na    Nastd    Mg    Mgstd    Al    Alstd    Si    Sistd    Ca    Castd    Sc    Scstd    TiI    TiIstd    TiII    TiIIstd    V    Vstd    Cr    Crstd    CrII    CrIIstd    Mn    Mnstd    Co    Costd    Ni    Nistd    Cu    Custd    Sr    Srstd    Y    Ystd    ZrI    ZrIstd    ZrII    ZrIIstd    Mo    Mostd    Ba    Bastd    La    Lastd    Ce    Cestd    Pr    Prstd    Nd    Ndstd    Sm    Smstd    Eu    Eustd moh   id    selection    Nome\n"""
        f.write(intestazione)

        # Analisi dei file
        for filename in files:
            alef=0
            for J in range(len(data)):
                if filename == data[J][0] and 1==2:
                    print('AOAOAOAO')
                    alef=data[J][6]
            filepath = os.path.join(path, filename)
            try:
                df = estrai_tabella_da_file(filepath)
            except Exception as e:
                print(f"Errore nel file {filename}: {e}")
                continue

            valori = {key: 0.0 for key in intestazione.split() if key not in ['id', 'selection', 'Star']}

            for el in elementi:
                for ion in ioni:
                    sub_df = df[(df['#'] == el) & (df['Ion'] == ion)]
                    if sub_df.empty:
                        continue
                    row = sub_df.iloc[0]
                    xfe = row['[X/Fe]']
                    sigma = row['Sigma']

                    if el == 'Fe':
                        if ion == 0:
                            valori['Fe'] = row['[X/H]']
                            valori['Festd'] = sigma
                        else:
                            valori['FeII'] = row['[X/H]']
                            valori['FeIIstd'] = sigma
                    elif el == 'Ti':
                        if ion == 0:
                            valori['TiI'] = xfe+alef
                            valori['TiIstd'] = sigma
                        else:
                            valori['TiII'] = xfe+alef
                            valori['TiIIstd'] = sigma
                    elif el == 'Cr':
                        if ion == 0:
                            valori['Cr'] = xfe
                            valori['Crstd'] = sigma
                        else:
                            valori['CrII'] = xfe
                            valori['CrIIstd'] = sigma
                    elif el == 'Zr':
                        if ion == 0:
                            valori['ZrI'] = xfe
                            valori['ZrIstd'] = sigma
                        else:
                            valori['ZrII'] = xfe
                            valori['ZrIIstd'] = sigma
                    else:
                        if el in valori:
                            if el == 'O' or el == 'Ne' or el == ' Mg' or el == 'Si' or el == 'Ca':
                                valori[el] = xfe+alef
                                print(f'{xfe} {alef} {xfe+alef}')
                                valori[el + 'std'] = sigma
                            else:
                                valori[el] = xfe
                                valori[el + 'std'] = sigma

            # Scrivi riga
            try:
                line = ' '.join(f"{valori[k]:.2f}" if 'std' not in k else f"{valori[k]:.3f}" for k in valori)
                line += f" 1 H {filename}\n"
                f.write(line)
            except Exception as e:
                print(f"Errore scrittura riga per {filename}: {e}")

# stella = 'TYC749-2025-1'
# plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{stella}/', f'{stella}_{stella}', region=4)#, datibooni=s[I].nome)

def piu_simile(s, L):
    """
    Ritorna una tupla (stringa, indice) in cui:
    - stringa è l'elemento di L più simile a s
    - indice è la sua posizione in L

    La similarità è data dal numero di caratteri uguali nella stessa posizione.
    """
    def similarità(s1, s2):
        return sum(c1 == c2 for c1, c2 in zip(s1, s2))

    max_score = -1
    migliore = None
    indice_migliore = -1

    for i, candidata in enumerate(L):
        score = similarità(s, candidata)
        if score > max_score:
            max_score = score
            migliore = candidata
            indice_migliore = i

    return migliore, indice_migliore



# def broadenaCartella(path):
#     '''Passa il path della cartella con i file da broadare. IMPORTANTE CHE I FILE SIANO NOMINATI CON UN CARATTERE DI DIFFERENZA (solitamente per come calcolo gli spettri, ovvero a pezzi di 2000A sono due file spettrali con nomi \'nomestella1\' \'nomestella2\')\n
#     Inoltre nella cartella ci devono essere SOLO i file da allargare e non altri file. Alla fine restituisce uno spettro unico e broadenato'''
    


################# AUX & EXC################
def crea_lista_primo_elemento(file_path):
    # Lista per memorizzare il primo elemento di ogni riga
    lista_primi_elementi = []

    with open(file_path, 'r') as file:
        for line in file:
            # Dividi la riga in segmenti separati da spazi o altri caratteri di spaziatura
            valori = line.strip().split()  # Usa split(',') se i valori sono separati da virgole
            if valori:
                # Aggiungi il primo elemento della riga alla lista
                lista_primi_elementi.append(float(valori[0]))

    return lista_primi_elementi
def trova_dodicesimo_elemento(file_path, parametro1, parametro2, parametro3, parametro4, tolleranza1=125, tolleranza2=0.25, tolleranza3=0.125, tolleranza4 = 0.01, tolleranza5 = 0.0001):
    '''file_path è il path al file contenente le correzioni;
     parametri: temperatura, logg, metallicità, wavelenght, microturbolenza\n Restituisce una lista con i valori più vicini alla griglia: \n 0: correzione \n 1: T \n 2: G \n 2: M \n 3: v \n 4: WL'''
    U = find_closest(vturbo(parametro1, parametro2, parametro3-7.5), MICROTURB)
    # Compiliamo una regex per identificare ogni segmento non vuoto separato da spazi variabili
    pattern = re.compile(r'\S+')
    stampa = False
    with open(file_path, 'r') as file:
        for line in file:
            # Trova tutti i segmenti della riga che corrispondono al pattern
            valori = pattern.findall(line)
            if len(valori) < 12:
                continue  # Salta righe con meno di 12 elementi
            try:
                # Estrai e converti i primi cinque valori
                primo_valore = float(valori[0])
                secondo_valore = float(valori[1])
                terzo_valore = float(valori[2])
                quarto_valore = float(valori[3])
                quinto_valore = float(valori[5])*10
            except ValueError:
                continue  # Salta la riga se i primi tre valori non sono numerici
            # Controlla se i valori soddisfano le tolleranze specificate
            if parametro2 <=1.5:
                parametro2 = 1.5
            if stampa == True:
                print(f"{abs(primo_valore - parametro1) <= tolleranza1}  {primo_valore} - {parametro1}")
                print(f"{abs(secondo_valore - parametro2) <= tolleranza2} {secondo_valore}-{parametro2}")
                print(f"{abs(terzo_valore - parametro3) <= tolleranza3}   {terzo_valore} - {parametro3}")
                print(f"{abs(quarto_valore - U) <= tolleranza5}    {quarto_valore} - {U}")
                print(f"{abs(quinto_valore - parametro4) <= tolleranza4}    {quinto_valore} - {parametro4}")
            if (abs(primo_valore - parametro1) <= tolleranza1 and   #temperatura
                abs(secondo_valore - parametro2) <= tolleranza2 and #logg
                abs(terzo_valore - parametro3) <= tolleranza3 and   #metal
                abs(quarto_valore - U) <= tolleranza5 and           #vturb
                abs(quinto_valore - parametro4) <= tolleranza4):    #wl
                returner = [float(valori[11]), valori[10], primo_valore, secondo_valore, terzo_valore, quarto_valore, parametro4]
                #print(returner)
                return returner  # l'indice 11 è il dodicesimo elemento (indice 0-based)
    # Se nessuna riga soddisfa i criteri, restituisce None
    return None
def findLine(file_path, parametro1, parametro2, parametro3, parametro4, parametro5, tolleranza1=125, tolleranza2=0.25, tolleranza3=0.125, tolleranza4 = 0.005, tolleranza5 = 0.0001):
    '''file_path è il path al file contenente le correzioni;
     parametri: temperatura, logg, metallicità, wavelenght, microturbolenza\n Restituisce una lista con i valori più vicini alla griglia: \n 0: correzione \n 1: T \n 2: G \n 2: M \n 3: v \n 4: WL'''
    #U = find_closest(vturbo(parametro1, parametro2, parametro3-7.5), MICROTURB)
    # Compiliamo una regex per identificare ogni segmento non vuoto separato da spazi variabili
    pattern = re.compile(r'\S+')
    stampa = False
    #print(f"Ciao!  {parametro1}  -    {parametro2}  -  {parametro3}  -  {parametro4}  -  {parametro5}  -")
    if parametro5 > 3:
        parametro5 = 3
        print("Non ci sono punti sulla griglia con microturbolenza maggiore di 3! Microturbolenza impostata a 3")
    if parametro5 < 0.75:
        parametro5 = 0.75
        print("Non ci sono punti sulla griglia con microturbolenza inferiore a 0.75! Microturbolenza impostata a 0.75")
    if parametro2 < 1.5:
        parametro2 = 1.5
        print("Non ci sono punti sulla griglia con logg inferiore a 1.5! logg impostato a 1.5")
    if parametro1 < 4000:
        parametro1 = 4000
        print("Non ci sono punti sulla griglia con temperatura inferiore a 4000! Teff impostato a 4000")
    if parametro1 > 5500:
        parametro1 = 5500
        print("Non ci sono punti sulla griglia con temperatura superiore a 5500! Teff impostato a 5500")
    if parametro2 <= 1.75 and parametro1 > 4500:
        parametro1 = 4500
    elif parametro2 <= 1.75 and parametro1 < 4000: 
        parametro1 = 4000
    if parametro2 >= 2.75 and parametro1 < 4500:
        parametro1 = 4500 
    with open(file_path, 'r') as file:
        for line in file:
            # Trova tutti i segmenti della riga che corrispondono al pattern
            valori = pattern.findall(line)
            if len(valori) < 12:
                continue  # Salta righe con meno di 12 elementi
            try:
                # Estrai e converti i primi cinque valori
                primo_valore = float(valori[0])
                secondo_valore = float(valori[1])
                terzo_valore = float(valori[2])
                quarto_valore = float(valori[3])
                # print(vturbo(parametro1, parametro2, parametro3-7.5))
                # print(abs(quarto_valore - U) <= tolleranza5)
                quinto_valore = float(valori[5])*10
                # print(f"{primo_valore}   {secondo_valore}   {terzo_valore}   {quarto_valore}   {quinto_valore} \n {parametro1}  {parametro2}  {parametro3}  {parametro4} ")
            except ValueError:
                continue  # Salta la riga se i primi tre valori non sono numerici
            # Controlla se i valori soddisfano le tolleranze specificate
            #print(f"{primo_valore} {secondo_valore} {terzo_valore} {quarto_valore} {quinto_valore} {line}")
            if parametro5 > 1.5 and parametro5 < 3.0:
                tolleranza5 = 0.75
            elif parametro5 > 0.75 and parametro5 < 1.5:
                tolleranza5 = 0.375
            elif parametro5 == 0.75:
                tolleranza5 = 0.0001
            elif parametro5 == 1.5:
                tolleranza5 = 0.0001
            elif parametro5 == 3:
                tolleranza5 = 0.0001
            if stampa == True:
                print(f"{abs(primo_valore - parametro1) <= tolleranza1}  {primo_valore} - {parametro1}")
                print(f"{abs(secondo_valore - parametro2) <= tolleranza2} {secondo_valore}-{parametro2}")
                print(f"{abs(terzo_valore - parametro3) <= tolleranza3}   {terzo_valore} - {parametro3}")
                print(f"{abs(quarto_valore - parametro5) <= tolleranza5}    {quarto_valore} - {parametro5}")
                print(f"{abs(quinto_valore - parametro4) <= tolleranza4}    {quinto_valore} - {parametro4}")
            if (abs(primo_valore - parametro1) <= tolleranza1 and   #temperatura
                abs(secondo_valore - parametro2) <= tolleranza2 and #logg
                abs(terzo_valore - parametro3) <= tolleranza3 and   #metal
                abs(quinto_valore - parametro4) <= tolleranza4 and    #wl
                abs(quarto_valore-parametro5) <= tolleranza5):
                #print("AAAAAAIAAAAAAAAAAA")
                #print("ASSHASSHA")
                # Restituisci il dodicesimo elemento se la riga soddisfa le condizioni
                returner = [float(valori[11]), valori[10], int(primo_valore), secondo_valore, terzo_valore, quarto_valore, parametro4]
                return returner
    # Se nessuna riga soddisfa i criteri, restituisce None
    return None
def findLine1(dati, parametro1, parametro2, parametro3, parametro4, parametro5, tolleranza1=125, tolleranza2=0.25, tolleranza3=0.125, tolleranza4 = 0.01, tolleranza5 = 0.0001):
    '''Dati è la matrice contenente le correzioni. Ad esempio quella che si crea con la funzione Full_Array_Corr();
     parametri: temperatura, logg, metallicità, wavelenght, microturbolenza\n Restituisce una lista con i valori più vicini alla griglia: \n 0: correzione \n 1: clean? \n 2: T \n 3: G \n 4: M \n 5: v \n 6: WL'''
    #U = find_closest(vturbo(parametro1, parametro2, parametro3-7.5), MICROTURB)
    # Compiliamo una regex per identificare ogni segmento non vuoto separato da spazi variabili
    pattern = re.compile(r'\S+')
    stampa = False
    #print(f"Ciao!  {parametro1}  -    {parametro2}  -  {parametro3}  -  {parametro4}  -  {parametro5}  -")
    if parametro5 > 3:
        parametro5 = 3
        print("Non ci sono punti sulla griglia con microturbolenza maggiore di 3! Microturbolenza impostata a 3")
    if parametro5 < 0.75:
        parametro5 = 0.75
        print("Non ci sono punti sulla griglia con microturbolenza inferiore a 0.75! Microturbolenza impostata a 0.75")
    if parametro2 < 1.5:
        parametro2 = 1.5
        print("Non ci sono punti sulla griglia con logg inferiore a 1.5! logg impostato a 1.5")
    if parametro1 < 4000:
        parametro1 = 4000
        print("Non ci sono punti sulla griglia con temperatura inferiore a 4000! Teff impostato a 4000")
    if parametro1 > 5500:
        parametro1 = 5500
        print("Non ci sono punti sulla griglia con temperatura superiore a 5500! Teff impostato a 5500")
    if parametro2 <= 1.75 and parametro1 > 4500:
        parametro1 = 4500
    elif parametro2 <= 1.75 and parametro1 < 4000: 
        parametro1 = 4000
    if parametro2 >= 2.75 and parametro1 < 4500:
        parametro1 = 4500
    if parametro4 >= 4500 and parametro4 <= 5000:
        ARR=dati[0]
    elif parametro4 >= 5000 and parametro4 <= 5250:
        ARR=dati[1]
    elif parametro4 >= 5250 and parametro4 <= 5500:
        ARR=dati[2]
    elif parametro4 >= 5500 and parametro4 <= 5750:
        ARR=dati[3] 
    elif parametro4 >= 5750 and parametro4 <= 6000:
        ARR=dati[4]
    elif parametro4 >= 6000 and parametro4 <= 6250:
        ARR=dati[5]
    elif parametro4 >= 6250 and parametro4 <= 6500:
        ARR=dati[6]
    elif parametro4 >= 6500 and parametro4 <= 6750:
        ARR=dati[7]
    else:
        print("You've been gnomed")
    
    if parametro5 > 1.5 and parametro5 < 3.0:
        tolleranza5 = 0.75
    elif parametro5 > 0.75 and parametro5 < 1.5:
        tolleranza5 = 0.375
    elif parametro5 == 0.75:
        tolleranza5 = 0.0001
    elif parametro5 == 1.5:
        tolleranza5 = 0.0001
    elif parametro5 == 3:
        tolleranza5 = 0.0001
    i=0
    while i < len(ARR):
        if i >= len(ARR):
            break
        if abs(parametro4-ARR[i][5]*10)<=tolleranza4:
            if abs(parametro5-ARR[i][3])<=tolleranza5:
                if abs(parametro3-ARR[i][2])<=tolleranza3:
                    if abs(parametro2-ARR[i][1])<=tolleranza2:
                        if abs(parametro1-ARR[i][0])<=tolleranza1:
                            returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
                            return returner
            # else:
            #     i=i+286
            #     if abs(parametro5-ARR[i][3])<tolleranza5:
            #         if abs(parametro3-ARR[i][2])<tolleranza3:
            #             if abs(parametro2-ARR[i][1])<tolleranza2:
            #                 if abs(parametro1-ARR[i][0])<tolleranza1:
            #                     returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
            #                     return returner
            #     i = i - 1
        else:
            i = i + 858
            if i >= len(ARR):
                break
            if abs(parametro4-ARR[i][5]*10)<=tolleranza4:
                if abs(parametro5-ARR[i][3])<=tolleranza5:
                    if abs(parametro3-ARR[i][2])<=tolleranza3:
                        if abs(parametro2-ARR[i][1])<=tolleranza2:
                            if abs(parametro1-ARR[i][0])<=tolleranza1:
                                returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
                                return returner
                # else:
                #     i=i+286
                #     if abs(parametro5-ARR[i][3])<tolleranza5:
                #         if abs(parametro3-ARR[i][2])<tolleranza3:
                #             if abs(parametro2-ARR[i][1])<tolleranza2:
                #                 if abs(parametro1-ARR[i][0])<tolleranza1:
                #                     returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
                #                     return returner
                    #i = i - 1
            i=i-1
            #     else:
            #         i=i+22
            #         if abs(parametro3-ARR[i][2])<tolleranza3:
            #             if abs(parametro2-ARR[i][1])<tolleranza2:
            #                 if abs(parametro1-ARR[i][0])<tolleranza1:
            #                     returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
            #                     return returner
            # else:
            #     i=i+286
            #     if abs(parametro5-ARR[i][3])<tolleranza5:
            #         if abs(parametro3-ARR[i][2])<tolleranza3:
            #             if abs(parametro2-ARR[i][1])<tolleranza2:
            #                 if abs(parametro1-ARR[i][0])<tolleranza1:
            #                     returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
            #                     return returner
            #     else:
            #         i=i+22
            #         if abs(parametro3-ARR[i][2])<tolleranza3:
            #             if abs(parametro2-ARR[i][1])<tolleranza2:
            #                 if abs(parametro1-ARR[i][0])<tolleranza1:
            #                     returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
            #                     return returner
        # else:
        #     i=i+858
        #     if abs(parametro4-ARR[i][5]*10)<tolleranza4:
        #         if abs(parametro5-ARR[i][3])<tolleranza5:
        #             if abs(parametro3-ARR[i][2])<tolleranza3:
        #                 if abs(parametro2-ARR[i][1])<tolleranza2:
        #                     if abs(parametro1-ARR[i][0])<tolleranza1:
        #                         returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
        #                         return returner
        #             else:
        #                 i=i+22
        #                 if abs(parametro3-ARR[i][2])<tolleranza3:
        #                     if abs(parametro2-ARR[i][1])<tolleranza2:
        #                         if abs(parametro1-ARR[i][0])<tolleranza1:
        #                             returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
        #                             return returner
        #         else:
        #             i=i+286
        #             if abs(parametro5-ARR[i][3])<tolleranza5:
        #                 if abs(parametro3-ARR[i][2])<tolleranza3:
        #                     if abs(parametro2-ARR[i][1])<tolleranza2:
        #                         if abs(parametro1-ARR[i][0])<tolleranza1:
        #                             returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
        #                             return returner
        #                 else:
        #                     i=i+22
        #                     if abs(parametro3-ARR[i][2])<tolleranza3:
        #                         if abs(parametro2-ARR[i][1])<tolleranza2:
        #                             if abs(parametro1-ARR[i][0])<tolleranza1:
        #                                 returner = [float(ARR[i][11]), str(ARR[i][10]), int(ARR[i][0]),  float(ARR[i][1]),  float(ARR[i][2]),  float(ARR[i][3]),  round(float(ARR[i][5]*10), 3)]
        #                                 return returner
        i=i+1
        if i >= len(ARR):
            break
    # Se nessuna riga soddisfa i criteri, restituisce None
    returner = [0, 'no', 0,  0,  0,  0,  0]
    return returner
def crea_cartella(percorso):
    try:
        # Controlla se la cartella esiste già
        if not os.path.exists(percorso):
            os.makedirs(percorso)  # Crea la cartella
            print(f"Cartella '{percorso}' creata con successo!")
        else:
            print(f"La cartella '{percorso}' esiste già.")
    except Exception as e:
        print(f"Errore durante la creazione della cartella: {e}")
def find_indices_with_integer_suffix(lines):
    '''Restituisce l'indice di tutti gli elementi della lista che terminano con un intero seguito da \n.'''
    indices = []
    pattern = re.compile(r'\d+\n$')
    for i, line in enumerate(lines):
        if pattern.search(line):
            indices.append(i)
    return indices
def indici_elementiuguali(lista1, lista2):
    indici = [i for i, (x, y) in enumerate(zip(lista1, lista2)) if x == y]
    return indici
def sposta(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"mv {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def copia(nome_oggetto, path_oggetto, path_destinazione, nome_oggetto_destinazione):
    in_dir = os.getcwd()
    if path_oggetto[-1] == "/":
        path_obj = path_oggetto[:-1]
    else:
        path_obj = path_oggetto 
    if path_destinazione[-1] == "/":
        path_D = path_destinazione[:-1]
    else:
        path_D = path_destinazione
    COMANDO = f"cp {path_obj}/{nome_oggetto} {path_D}/{nome_oggetto_destinazione}"
    os.system(COMANDO)
    os.chdir(in_dir)
def gaussiana(x, A, mu, sigma, off):
    return off - A * np.exp(-(x - mu)**2 / (2 * sigma**2))
def voigt(x, A, mu, sigma, gamma, off):
    z = ((x - mu) + 1j * gamma) / (sigma * np.sqrt(2))
    return off - A * np.real(wofz(z)) / (sigma * np.sqrt(2 * np.pi))
def voigt_upright(x, A, mu, sigma, gamma):
    z = ((x - mu) + 1j * gamma) / (sigma * np.sqrt(2))
    return A * np.real(wofz(z)) / (sigma * np.sqrt(2 * np.pi))
def linear_func(x, a, b):
    return b +a*x
def quad_func(x, a, b, c): #ax^2+bx+c
    return c +b*x + a*x*x
def expo_func(x, a, b):
    return a * np.exp(b*x)
def def_s_Matrix():
    vv=sp.symbols('v')
    vv=0
    dsdv1=-0.015639
    dFdv1=0.1194805
    dYdv1=0.2560606
    dSdv1=1.7464315
    dsdv2=0.0242265
    dFdv2=-0.445342
    dYdv2=-0.939681
    dSdv2=-26.17745
    dsdt=-0.000273752
    dFdt=0.002228
    dYdt=0.001255
    dSdt=0.003307
    dsdg=-0.035362
    dFdg=-0.434498
    dYdg=-0.134194
    dSdg=-6.978472
    dsdm=-0.015086
    dFdm=-0.402286
    dYdm=-1.153942
    dSdm=-2.937107
    s_Matrix=sp.Matrix([[dsdt, dsdg, dsdm, dsdv1*vv+dsdv2],
                        [dFdt, dFdg, dFdm, dFdv1*vv+dFdv2],
                        [dYdt, dYdg, dYdm, dYdv1*vv+dYdv2],
                        [dSdt, dSdg, dSdm, dSdv1*vv+dSdv2]])
    return s_Matrix
def blocca_stampe():
    sys.stdout = open(os.devnull, 'w')
    sys.stderr = open(os.devnull, 'w')
def Full_Array_Corr():
    full_array_corr = []
    cartelle=['4500-5000', '5000-5250', '5250-5500', '5500-5750', '5750-6000', '6000-6250', '6250-6500', '6500-6750']
    for i in range(len(cartelle)):
        file_path = path_to_correctionGrids1 + f"{cartelle[i]}/correctionGrid"
        data_array = read_file_to_array(file_path)
        full_array_corr.append(data_array)
        if data_array is not None:
            print("Dati letti con successo!")
            print(data_array)
    return full_array_corr
def leggi_turboSpectrum_linelist(linelist="GES_582_680.list"):
    '''Questa funzione serve a matchare le righe di una linelist a quelle in formato turbospectrum per creare delle linelist in formato turbospectrum personalizzate.\n
    Nello specifico crea un dizionario a cui le chiavi sono gli elementi chimici e i valori sono delle liste di dizionari contenenti i dati delle righe. \n
    La funzione legge il file di linelist e crea un dizionario con i dati delle righe. \n'''
    dati_per_elemento = {}
    elemento_corrente = None
    numero_isotopo = None

    with open(linelist) as f:
        numero_isotopo = 0
        for riga in f:
            riga = riga.rstrip('\n')
            if riga.startswith("'") and riga[1:7].replace('.', '').strip().isdigit():
                a=riga[1:7]

                continue
            if a is not None:
                numero_isotopo= (a)
            if riga.startswith("'") and len(riga.strip()) <= 10:
                nome_elemento = riga.strip().strip("'")
                if numero_isotopo is not None:
                    chiave = f"{float(numero_isotopo):.3f}{nome_elemento}"
                else:
                    chiave = nome_elemento
                elemento_corrente = chiave
                dati_per_elemento[elemento_corrente] = []
                continue

            if riga.startswith("  "):
                entry = {
                    "wavelength": float(riga[2:10]),
                    "EP": float(riga[12:17]),
                    "loggf": float(riga[19:25]),
                    "damping": float(riga[27:31]),
                    "g_up": float(riga[34:38]),
                    "gamma_rad": (riga[39:47]).strip(),
                    "lower_config": riga[49:52].strip(),
                    "upper_config": riga[54:57].strip(),
                    "J_low": float(riga[60:63]),
                    "J_up": float(riga[66:69]),
                    "description": riga[70:110].strip()
                }
                if elemento_corrente:
                    dati_per_elemento[elemento_corrente].append(entry)
    return dati_per_elemento
def unisci_dati(*file_names):
    dati_totali = {}

    for nome_file in file_names:
        dati_file = leggi_turboSpectrum_linelist(nome_file)
        
        for elemento, righe in dati_file.items():
            if elemento not in dati_totali:
                dati_totali[elemento] = []
            dati_totali[elemento].extend(righe)
    
    return dati_totali
def format_moog_output_to_analysis(pathin, pathout, input_file, output_file):
    """
    Legge un file e rimuove tutte le righe che non iniziano con due spazi.
    
    :param input_file: Il percorso del file di input.
    :param output_file: Il percorso del file di output.
    """
    olddir=os.getcwd()
    os.chdir(pathin)
    try:
        with open(input_file, 'r', encoding='utf-8') as infile:
            lines = infile.readlines()
        
        # Filtra le righe che iniziano con due spazi
        filtered_lines = [line for line in lines if line.startswith('  ')]
        os.chdir(pathout)
        # Scrive le righe filtrate nel file di output
        with open(output_file, "w") as file:
            for i, line in enumerate(filtered_lines):
                if i != 0:  # Indice 2 corrisponde alla terza riga
                    file.write(line)
        
        print(f"Operazione completata. File salvato in: {output_file}")
    except FileNotFoundError:
        print(f"Errore: Il file '{input_file}' non è stato trovato.")
    except Exception as e:
        print(f"Si è verificato un errore: {e}")
    os.chdir(olddir)
def derivata(x, y, showplot=False):
    '''Calcola la derivata prima e seconda di un array y rispetto a un array x.'''
    dx = np.diff(x)
    dy = np.diff(y)
    d2y = np.diff(dy) / dx[:-1]
    dy = dy / dx
    if showplot:
        plt.plot(x[:-1], dy, label='Derivata prima')
        plt.plot(x[:-2], d2y, label='Derivata seconda')
        plt.legend()
        plt.show()
    return dy, d2y
def squareBell(x, filter):
    '''Campana quadrata'''
    return np.where(np.abs(x) < filter, 1.0, 0.0)
def reversesquareBell(x, filter):
    '''Campana quadrata'''
    return np.where(np.abs(x) > filter, 1.0, 0.0)
def filterHighFrequencies(x, y, filter=16, filter_type='expoBell'):
    '''Filtra le alte frequenze da un segnale'''
    y_filtered = np.zeros(len(x))
    if filter_type == 'squareBell':
        for i in range(len(y)):
            y_filtered[i] = y[i] * squareBell(x[i], filter)
    elif filter_type == 'expoBell':
        for i in range(len(y)):
            y_filtered[i] = y[i] * expoBell(x[i], filter)
    return y_filtered
def filterLowFrequencies(x, y, filter=16, filter_type='reversesquareBell'):
    '''Filtra le alte frequenze da un segnale'''
    y_filtered = np.zeros(len(x))
    if filter_type == 'reversesquareBell':
        for i in range(len(y)):
            y_filtered[i] = y[i] * reversesquareBell(x[i], filter)
    return y_filtered
def confronta_array(array1, array2, typ='max', offset=0):
    # Controlla che i due array abbiano la stessa lunghezza
    if len(array1) != len(array2):
        raise ValueError("I due array devono avere la stessa dimensione.")
    
    # Confronta elemento per elemento e crea un nuovo array con i valori maggiori
    if typ=='max':
        risultato = [max(a, b) for a, b in zip(array1, array2)]
    elif typ=='min':
        risultato = [min(a, b) for a, b in zip(array1, array2)]
    return risultato
def aggiustaValori_turbospectrum( alpha, file_mygisfos, path_file_abuxturbo='/home/starcat8/Documenti/', metal_singleMode=0, europio=0, zirconio=0, ossigeno=0, bario=0):
    '''
    Questa funzione serve a correggere le abbondanze di turbospectrum in base ai valori di MyGisFOS. \n
    Prende in input il file di MyGisFOS, il percorso del file di turbospectrum e i valori di alpha, europio, zirconio, ossigeno, bario. \n
    Restituisce una lista con le abbondanze corrette, le abbondanze originali e le differenze tra le due. \n
    '''
    df = estrai_tabella_da_file(file_mygisfos)
    abtur=Table.read(path_file_abuxturbo+'fort.7', format='ascii')
    spec=abtur['col1']
    ab=abtur['col2'].copy()
    #print(abtur)
    
    for i in range(len(df['N'])):
        for j in range(len(spec)):
            if df['N'][i] == spec[j]:
                alpha_corr = 0
                euro=0
                zirco=0
                oxy=0
                bar=0
                #print(table_periodica[f'{spec[j]}'])
                if spec[j] < 26 and spec[j]%2 == 0:
                    alpha_corr = alpha
                if spec[j] == 8:
                    oxy = ossigeno
                if spec[j] == 63:
                    euro = europio
                if spec[j] == 40:
                    zirco=zirconio
                if spec[j] == 56:
                    bar=bario
                if spec[j] > 2:
                    ab[j] = abtur['col2'][j] - alpha_corr + euro + zirco + oxy + bar + metal_singleMode + df['[X/Fe]'][i]
                else:
                    ab[j] = abtur['col2'][j] - alpha_corr + euro + zirco + oxy + bar + metal_singleMode + df['[X/Fe]'][i]
                #print(f'{abtur['col2'][j]} - {alpha_corr} + {df['[X/Fe]'][i]} = {ab[j]}')
    with open(path_file_abuxturbo+'fort.75', 'w') as f:
        for i in range(len(spec)):
            if spec[i] <= 9:
                f.write(f' f{spec[i]:<3}')
            else: 
                f.write(f'{spec[i]:<3} ')
            if spec[i] == 2:
                f.write(f' {ab[i]:.2f}\n')
            else:
                if ab[i] > 0:
                    f.write(f'  {ab[i]:.2f}\n')
                else:
                    f.write(f' {ab[i]:.2f}\n')
    return [[spec, ab], [abtur['col1'], abtur['col2']], [spec, ab-abtur['col2']]]
def concatenate_files_in_groups(S, instrument='FIES'):
    '''S contains a list of names where the files to be concatenated are located. \n'''
    for i in range(len(S)):
        # === Parametri ===
        if instrument == 'HARPS-N':
            cartella = f"/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[i]}(Copia)/"  # <-- Sostituisci con il tuo path
        elif instrument == 'FIES':
            cartella = f"/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[i]}/"  # <-- Sostituisci con il tuo path

        # === Cambio directory ===
        try:
            os.chdir(cartella)
            print(f"✅ Cartella cambiata in: {os.getcwd()}")
        except Exception as e:
            print(f"❌ Errore nel cambiare cartella: {e}")

        # === Raggruppamento file per nome comune (escluso il 7° carattere) ===
        gruppi = defaultdict(dict)

        for filename in os.listdir("."):
            if os.path.isfile(filename) and len(filename) >= 7:
                base = filename[:6] + filename[7:]  # Rimuove il 7° carattere
                chiave = filename[6]  # Il carattere discriminante ('1', '2', '3')
                if chiave in ('1', '2', '3'):
                    gruppi[base][chiave] = filename

        # === Unione dei file in ordine 1-2-3 ===
        for base, gruppo in gruppi.items():
            if all(k in gruppo for k in ('1', '2', '3')):
                try:
                    # Ordina e concatena i contenuti
                    contenuti = []
                    for k in ('1', '2', '3'):
                        with open(gruppo[k], 'r') as f:
                            contenuti.extend(f.readlines())

                    # Nome del file di output
                    nome_output = gruppo['1'] + "_conc"

                    # Scrive il file concatenato
                    with open(nome_output, 'w') as f_out:
                        f_out.writelines(contenuti)

                    print(f"🟢 Creato: {nome_output}")
                except Exception as e:
                    print(f"❌ Errore durante la scrittura del file per gruppo '{base}': {e}")
            else:
                print(f"⚠️  Gruppo incompleto per base '{base}': {list(gruppo.keys())}")
def synthetic_segmentation(S, instrument='FIES'):
    for i in range(len(S)):
        if instrument == 'HARPS-N':
            cartella = f"/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[i]}(Copia)/"  # <-- Sostituisci con il tuo path
        elif instrument == 'FIES':
            cartella = f"/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[i]}/"  # <-- Sostituisci con il tuo path

        os.chdir(cartella)
        intervalli = {
            '1': (4500, 5500),
            '2': (5500, 6500),
            '3': (6500, 7500)
        }

        # === Trova tutti i file che contengono "_conc" nel nome ===
        file_conc = [f for f in os.listdir(".") if "_conc" in f and os.path.isfile(f)]

        for filename in file_conc:
            try:
                # Leggi tutte le righe del file
                with open(filename, 'r') as f:
                    righe = f.readlines()

                # Inizializza i contenitori per ogni intervallo
                divisi = {'1': [], '2': [], '3': []}

                # Filtra le righe per ciascun intervallo (possono andare in più liste)
                for riga in righe:
                    if not riga.strip():
                        continue  # salta righe vuote
                    try:
                        wave = float(riga.split()[0])
                    except ValueError:
                        continue  # salta righe malformate

                    for chiave, (lo, hi) in intervalli.items():
                        if lo <= wave <= hi:
                            divisi[chiave].append(riga)

                # Crea i file output modificando il 7° carattere
                base_name = filename.replace('_conc', '')
                for chiave in ['1', '2', '3']:
                    if len(base_name) < 7:
                        print(f"❌ Nome troppo corto: {base_name}, saltato.")
                        continue
                    nuovo_nome = base_name[:6] + chiave + base_name[7:]
                    with open(nuovo_nome, 'w') as out:
                        out.writelines(divisi[chiave])
                    print(f"🟢 Salvato: {nuovo_nome} ({len(divisi[chiave])} righe)")

            except Exception as e:
                print(f"❌ Errore con il file '{filename}': {e}")


def abbondanzeTurbo(alpha, file_mygisfos, path_file_abuxturbo='/home/starcat8/Documenti/', metal_singleMode=0, europio=0, zirconio=0, ossigeno=0, bario=0):
    !cd /home/starcat8/Documenti && echo 2 | ./abuxturbo.exe
    returner=aggiustaValori_turbospectrum(alpha, file_mygisfos, path_file_abuxturbo, metal_singleMode=metal_singleMode, europio=europio, zirconio=zirconio, ossigeno=ossigeno)
    return returner
def estrai_tabella_da_file(file_path):
    """
    Estrae una tabella delimitata da delimitatori nel file e restituisce un DataFrame.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    delimitatore = "##########################################################################################"
    start_index = None
    end_index = None

    for i, line in enumerate(lines):
        if line.strip() == delimitatore:
            if start_index is None:
                start_index = i
            else:
                end_index = i
                break

    if start_index is None or end_index is None or start_index >= end_index:
        raise ValueError("Delimitatori non trovati correttamente nel file.")

    contenuto_tabella = lines[start_index + 2:end_index]
    tabella_str = ''.join(contenuto_tabella)
    df = pd.read_csv(StringIO(tabella_str), delim_whitespace=True)
    return df
def formattaFile_grafici(path='/home/starcat8/Documenti/MyGisFOS/test/best1/', output='resultis.txt'):
    os.chdir(path)
    files = os.listdir()

    # Apertura file di output
    with open(output, 'w') as f:
        # Intestazione
        intestazione = """Fe        Festd    FeII        FeIIstd        O        Ostd    Na    Nastd    Mg    Mgstd    Al    Alstd    Si    Sistd    Ca    Castd    Sc    Scstd    TiI    TiIstd    TiII    TiIIstd    V    Vstd    Cr    Crstd    CrII    CrIIstd    Mn    Mnstd    Co    Costd    Ni    Nistd    Cu    Custd    Sr    Srstd    Y    Ystd    ZrI    ZrIstd    ZrII    ZrIIstd    Mo    Mostd    Ba    Bastd    La    Lastd    Ce    Cestd    Pr    Prstd    Nd    Ndstd    Sm    Smstd    Eu    Eustd    id    Selection    Nome\n"""
        f.write(intestazione)

        # Analisi dei file
        for filename in files:
            filepath = os.path.join(path, filename)
            try:
                df = estrai_tabella_da_file(filepath)
            except Exception as e:
                print(f"Errore nel file {filename}: {e}")
                continue

            valori = {key: 0.0 for key in intestazione.split() if key not in ['id', 'Selection', 'Nome']}

            for el in elementi:
                for ion in ioni:
                    sub_df = df[(df['#'] == el) & (df['Ion'] == ion)]
                    if sub_df.empty:
                        continue
                    row = sub_df.iloc[0]
                    xfe = row['[X/Fe]']
                    sigma = row['Sigma']

                    if el == 'Fe':
                        if ion == 0:
                            valori['Fe'] = row['[X/H]']
                            valori['Festd'] = sigma
                        else:
                            valori['FeII'] = row['[X/H]']
                            valori['FeIIstd'] = sigma
                    elif el == 'Ti':
                        if ion == 0:
                            valori['TiI'] = xfe
                            valori['TiIstd'] = sigma
                        else:
                            valori['TiII'] = xfe
                            valori['TiIIstd'] = sigma
                    elif el == 'Cr':
                        if ion == 0:
                            valori['Cr'] = xfe
                            valori['Crstd'] = sigma
                        else:
                            valori['CrII'] = xfe
                            valori['CrIIstd'] = sigma
                    elif el == 'Zr':
                        if ion == 0:
                            valori['ZrI'] = xfe
                            valori['ZrIstd'] = sigma
                        else:
                            valori['ZrII'] = xfe
                            valori['ZrIIstd'] = sigma
                    else:
                        if el in valori:
                            valori[el] = xfe
                            valori[el + 'std'] = sigma

            # Scrivi riga
            try:
                line = ' '.join(f"{valori[k]:.2f}" if 'std' not in k else f"{valori[k]:.3f}" for k in valori)
                line += f" 1 H {filename}\n"
                f.write(line)
            except Exception as e:
                print(f"Errore scrittura riga per {filename}: {e}")


s_Matrix=def_s_Matrix()
################# NAMES & FOLDERS MNG #################
def parseFileName(path, nome_file): 
    olddir=os.getcwd()
    os.chdir(path)
    pattern = f"{nome_file[0]}"+r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName2(path, nome_file):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = r'AH_STELLA(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    if match:
        returner.append(float(match.group(1)))
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        return returner
    else:
        return None
def sparseFileName3(path, nome_file, name_suffix='BD+062880'):
    olddir=os.getcwd()
    os.chdir(path)
    pattern = name_suffix + r'_(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)_(-?\d+\.\d+)'
    match = re.match(pattern, nome_file)
    returner = []
    print("nome_file:")
    print(nome_file)
    print(pattern)
    print(os.getcwd())
    if match:
        returner.append(float(match.group(1)))
        print(returner[0])
        returner.append(int(match.group(2)))
        returner.append(float(match.group(3)))
        returner.append(float(match.group(4)))
        #returner.append(1)
        return returner
    else:
        return None
def sparseFileName4(path, nome_file, name_suffix='BD+062880_'):
        os.chdir(path)
        pattern = name_suffix + r'(-?\d+\.\d+)_(\d+)_(-?\d+\.\d+)_(\d+\.\d+)'
        match = re.match(pattern, nome_file)
        returner = []
        if match:
            returner.append(float(match.group(1)))
            returner.append(int(match.group(2)))
            returner.append(float(match.group(3)))
            returner.append(float(match.group(4)))
            return returner
        else:
            return None
def read_file_to_list(file_path):
    '''Legge un file e inserisce tutte le righe in una lista.'''
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines
def make_set(T, G, M, DT=500, DG=1, DM=1):
    set1 = [[T-DT, G+DG, M-DM], [T, G+DG, M-DM], [T+DT, G+DG, M-DM], [T-DT, G, M-DM]]
    set2 = [[T, G, M-DM], [T+DT, G, M-DM], [T-DT, G-DG, M-DM], [T, G-DG, M-DM]]
    set3 = [[T+DT, G-DG, M-DM],[T-DT, G+DG, M], [T, G+DG, M], [T+DT, G+DG, M]]
    set4 = [[T-DT, G, M], [T, G, M], [T+DT, G, M], [T-DT, G-DG, M]]
    set5 = [[T, G-DG, M], [T+DT, G-DG, M], [T-DT, G+DG, M+DM], [T, G+DG, M+DM]]
    set6 = [[T+DT, G+DG, M+DM], [T-DT, G, M+DM], [T, G, M+DM], [T+DT, G, M+DM]]
    set7 = [[T-DT, G-DG, M+DM], [T, G-DG, M+DM], [T+DT, G-DG, M+DM]]
    SET_ALL = [set1, set2, set3, set4, set5, set6, set7]
    return SET_ALL
def update_special_lines(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    processed_lines = []
    special_lines = []
    normal_line_count = 0
    
    for line in lines:
        if line.startswith("'"):
            if special_lines and normal_line_count > 0:
                # Aggiorna il numero nella terza posizione della riga speciale
                elements = special_lines[-1].split()
                if len(elements) >= 3:
                    elements[2] = str(normal_line_count)
                    special_lines[-1] = ' '.join(elements) + '\n'
                normal_line_count = 0
            special_lines.append(line)
        else:
            normal_line_count += 1
            processed_lines.append(line)
    
    # Aggiungi le righe speciali processate a processed_lines
    for special_line in special_lines:
        processed_lines.append(special_line)
    
    # Scrivi le righe processate nel file di output
    with open(output_file, 'w') as file:
        file.writelines(processed_lines)
def checkExistingAtmo(T, G, M, V, path=path_to_atmosphere+interpol_folder):
    atmo_name = f"{path}/T{T:.0f}_G{G:.2f}_M{M:.2f}_{V:.2f}.model"
    if exists(atmo_name):
        return (atmo_name)
    else:
        return False
def crea_dynMatrix(x1, y1, x2, y2): #
    '''Matrice delle derivate, ma creata step by step.\n
      In x ci va (T, G, M, V), in y (s, F, Y, S).\n
      Restituisce la matrice che deve essere invertita per il calcolo della variazione dei parametri.'''
    if len(y1)== 4 and len(x1) == 4 and len(y2)== 4 and len(x2) == 4:
        dx0=x2[0]-x1[0]
        dx1=x2[1]-x1[1]
        dx2=x2[2]-x1[2]
        dx3=x2[3]-x1[3]       
        if dx0<=0.0001:
            dx0=0.0001 
        if dx1<=0.0001:
            dx1=0.0001 
        if dx2<=0.0001:
            dx2=0.0001 
        if dx3<=0.0001:
            dx3=0.0001     
        dsdt=(y2[0]-y1[0])/(dx0)
        dsdg=(y2[0]-y1[0])/(dx1)
        dsdm=(y2[0]-y1[0])/(dx2)
        dsdv=(y2[0]-y1[0])/(dx3)
        dFdt=(y2[1]-y1[1])/(dx0)
        dFdg=(y2[1]-y1[1])/(dx1)
        dFdm=(y2[1]-y1[1])/(dx2)
        dFdv=(y2[1]-y1[1])/(dx3)
        dYdt=(y2[2]-y1[2])/(dx0)
        dYdg=(y2[2]-y1[2])/(dx1)
        dYdm=(y2[2]-y1[2])/(dx2)
        dYdv=(y2[2]-y1[2])/(dx3)
        dSdt=(y2[3]-y1[3])/(dx0)
        dSdg=(y2[3]-y1[3])/(dx1)
        dSdm=(y2[3]-y1[3])/(dx2)
        dSdv=(y2[3]-y1[3])/(dx3)
    else:
        print("Errore nel fornire y!")
    matrice = np.array([
    [dsdt, dsdg, dsdm, dsdv],
    [dFdt, dFdg, dFdm, dFdv],
    [dYdt, dYdg, dYdm, dYdv],
    [dSdt, dSdg, dSdm, dSdv]
    ])
    return matrice
def nome_da_Fits(path_to_fits=path_to_fitsFiles, nome='HARPN1.fits', instrument='HARPS-N'):
    # Apri il file FITS
    hdul = fits.open(path_to_fits+"/"+nome)
    header = hdul[0].header
    if instrument == 'HARPS-N':
        nome_stella = header.get('HIERARCH TNG OBS TARG NAME', 'Nome della stella non trovato')
    elif instrument == 'FIES': 
        nome_stella = header.get('OBJECT', 'Nome della stella non trovato.')
    
    nome_stella = nome_stella.replace(" ", "")
    #nome_stella = nome_stella.replace(" ", "")
    print(f'Nome della stella: {nome_stella}')
    # Chiudi il file FITS
    hdul.close()
    return nome_stella
def riattiva_stampe():
    sys.stdout = sys.__stdout__
    sys.stderr = sys.__stderr__
def read_file_to_array(file_path):
    
    """
    Legge un file di testo con dati strutturati in colonne e li inserisce in un array a 12 dimensioni.
    
    Args:
        file_path (str): Percorso del file da leggere.

    Returns:
        np.ndarray: Un array NumPy con 12 colonne contenenti i dati.
    """
    try:
        data = np.genfromtxt(file_path, dtype=None, encoding=None, comments=None)
    except Exception as e:
        print(f"Errore durante la lettura del file: {e}")
        return None
    
    print(len(data))
    return data
def line_distrib_over_loggf(loggf):
    '''Crea maschera per distribuire le lunghezze d'onda secondo il loggf delle righe (per evitare che ne siano troppe attorno a 3 e poche agli estremi).\n
    Restituisce maschera booleana.\n'''
def correzione(M):
    return -0.15-0.075*M
def crea_ATMinterpol(T, logg, MH_g, nome_Stella, T0=0, logg0=0, MH_g0=0, n_istanze=1, instrument='HARPS-N', lineList=lineList, path_to_output=path_to_output): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
    '''T: temperatura (kelvin)
    logg: gravità superficiale (dex)
    MH_g: [M/H] (dex)'''
    os.chdir(path_to_interpol)
    if n_istanze == 1:
        vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
        #print(f"{T}   {logg}   {vturb}   {MH_g}")
        commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
        os.system(commandInterpol)

        #### FORMATTA KURUCZ ####
        header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
        header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
        def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
            with open(input_file, 'r') as f_in:
                lines = f_in.readlines()

            # Remove the first 3 rows and the last 2 rows
            lines = lines[3:-2]

            with open(output_file, 'w') as f_out:
                f_out.write(header1)
                f_out.write(header2)
                f_out.writelines(lines)
                f_out.write(tail)

        read_and_copy_file('modelatmosphere.txt', 'out.txt')
        #spectrum
        comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
        os.system(comandoATMOS)
        preparaAbundance(nome_Stella, lineList=lineList)
        os.chdir(path_to_abundance)
        creaParamfileAbnd(vturbo(T, logg, MH_g))
        SlanciaAbundance()
        gnome_stella = nome_da_Fits(nome=nome_Stella, instrument=instrument)
        nome_output = f"{gnome_stella}_{MH_g:.2f}_{T:.0f}_{logg:.2f}"
        comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
        os.system(comando_bsh2)
        return nome_output
def crea_ATMinterpol2(T, logg, MH_g, microTurb, nome_Stella, processore=0, T0=0, logg0=0, MH_g0=0, n_istanze=1, path_to_output=path_to_output): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
    '''T: temperatura (kelvin)
    logg: gravità superficiale (dex)
    MH_g: [M/H] (dex)'''
    os.chdir(path_to_interpol)
    if processore==0:
        if n_istanze == 1:
            #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
            vturb=microTurb
            commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
            os.system(commandInterpol)

            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()

                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]

                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)

            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
            os.system(comandoATMOS)
            #preparaAbundance(nome_Stella)
            os.chdir(path_to_abundance)
            creaParamfileAbnd(vturb)
            SlanciaAbundance()
            nome_output = f"{nome_Stella}_{MH_g:.2f}_{T:.0f}_{logg:.2f}_{vturb:.2f}"
            #comando_bsh2 = f"mv PINOLOS {path_to_calibrationFolder}/{nome_output}"# calibrazione
            comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}" # ricerca
            os.system(comando_bsh2)
            return nome_output
        if n_istanze == 2:
            ##### prima atmosfera
            #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg

            commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
            print(vturb)
            os.system(commandInterpol)

            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()

                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]

                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)

            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
            os.system(comandoATMOS)
            preparaAbundance(nome_Stella)
            os.chdir(path_to_abundance)
            creaParamfileAbnd(vturbo(T, logg, MH_g))
            ##### seconda atmosfera
            vturb0 = 0.14 - 0.08 * (MH_g0) + 4.9 * T / 10000 - 0.47 * logg0

            commandInterpol = f"{path_to_pythonEnv} pykmod.py {T0} {logg0} {vturb0} {MH_g0}"
            print(vturb)
            os.system(commandInterpol)

            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T0}.  GRAVITY {logg0} LTE\nTITLE  [{MH_g0}] VTURB={vturb0}  L/H=1.25 NOVER NEW ODF\n"
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g0):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()

                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]

                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)

            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/ModelE"
            os.system(comandoATMOS)
            preparaAbundance(nome_Stella)
            os.chdir(path_to_abundance)
            creaParamfileAbnd(vturbo(T, logg, MH_g))
            SlanciaAbundance(2)
            nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}"
            comando_bsh2 = f"mv PINOLOS {path_to_output}/{nome_output}"
            os.system(comando_bsh2)
            return lista_nome_output
    else:
        #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
        for i in range(processore):
            vturb=microTurb[i]
            #print(f"{type(T)}   {type(logg)}   {type(MH_g)}   {type(vturb)}")
            os.chdir(path_to_abundance)
            password = "Birdos"
            commandInterpol = f"sudo {path_to_pythonEnv} {path_to_interpol}pykmod.py {T} {logg} {vturb} {MH_g}"
            # Usa subprocess per passare la password a sudo
            process = subprocess.run(
                f"echo {password} | sudo -S {commandInterpol}",
                shell=True,
                text=True,  # Necessario per trattare input/output come stringhe
                capture_output=True  # Per catturare stdout e stderr
            )
            #### FORMATTA KURUCZ ####
            header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"
            MH_g=float(MH_g)
            header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 0.92150 2 0.07840\n ABUNDANCE CHANGE  3 -10.94  4 -10.64  5  -9.49  6  -3.52  7  -4.12  8  -3.21\n ABUNDANCE CHANGE  9  -7.48 10  -3.96 11  -5.71 12  -4.46 13  -5.57 14  -4.49\n ABUNDANCE CHANGE 15  -6.59 16  -4.71 17  -6.54 18  -5.64 19  -6.92 20  -5.68\n ABUNDANCE CHANGE 21  -8.87 22  -7.02 23  -8.04 24  -6.37 25  -6.65 26  -4.54\n ABUNDANCE CHANGE 27  -7.12 28  -5.79 29  -7.83 30  -7.44 31  -9.16 32  -8.63\n ABUNDANCE CHANGE 33  -9.67 34  -8.63 35  -9.41 36  -8.73 37  -9.44 38  -9.07\n ABUNDANCE CHANGE 39  -9.80 40  -9.44 41 -10.62 42 -10.12 43 -20.00 44 -10.20\n ABUNDANCE CHANGE 45 -10.92 46 -10.35 47 -11.10 48 -10.27 49 -10.38 50 -10.04\n ABUNDANCE CHANGE 51 -11.04 52  -9.80 53 -10.53 54  -9.87 55 -10.91 56  -9.91\n ABUNDANCE CHANGE 57 -10.87 58 -10.46 59 -11.33 60 -10.54 61 -20.00 62 -11.03\n ABUNDANCE CHANGE 63 -11.53 64 -10.92 65 -11.69 66 -10.90 67 -11.78 68 -11.11\n ABUNDANCE CHANGE 69 -12.04 70 -10.96 71 -11.98 72 -11.16 73 -12.17 74 -10.93\n ABUNDANCE CHANGE 75 -11.76 76 -10.59 77 -10.69 78 -10.24 79 -11.03 80 -10.91\n ABUNDANCE CHANGE 81 -11.14 82 -10.09 83 -11.33 84 -20.00 85 -20.00 86 -20.00\n ABUNDANCE CHANGE 87 -20.00 88 -20.00 89 -20.00 90 -11.95 91 -20.00 92 -12.54\n ABUNDANCE CHANGE 93 -20.00 94 -20.00 95 -20.00 96 -20.00 97 -20.00 98 -20.00\n ABUNDANCE CHANGE 99 -20.00\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
            tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"
            def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                with open(input_file, 'r') as f_in:
                    lines = f_in.readlines()
                # Remove the first 3 rows and the last 2 rows
                lines = lines[3:-2]
                with open(output_file, 'w') as f_out:
                    f_out.write(header1)
                    f_out.write(header2)
                    f_out.writelines(lines)
                    f_out.write(tail)
            #print(f"aa {os.getcwd()}  ehiehi")
            os.chdir(path_to_interpol)
            read_and_copy_file('modelatmosphere.txt', 'out.txt')
            comandoATMOS = f"cp {path_to_interpol}out.txt {path_to_abundance}/Model{i+1}"
            os.system(comandoATMOS)
            creaParamfileAbnd(vturb, i+1)
        os.chdir(path_to_abundance)
        if __name__ == "__main__":
            main_integration()
        crea_cartella(path_to_calibrationFolder + "/" + nome_Stella)
        for i in range(processore):
            vturb=microTurb[i]
            nome_output = f"{nome_Stella}_INT{MH_g:.2f}_{T:.0f}_{logg:.2f}_{vturb:.2f}"
            copia("stellaFes", path_to_abundance, path_to_calibrationFolder + "/" + nome_Stella, "stellaFes")
            sposta(f"PINOLOS{i+1}", path_to_abundance, path_to_calibrationFolder + "/" + nome_Stella, nome_output)
        return nome_output
def getTail(T, L, M):
    OLD_DIR = os.getcwd()
    file_path = path_to_atmosphere + f"MH{M:.1f}/atmoT{int(T)}{L:.1f}.txt"
    # Leggi l'ultima riga del file
    try:
        with open(file_path, 'r') as file:
            last_line = file.readlines()[-2]
    except FileNotFoundError:
        with open('/home/starcat8/Documenti/sbundance/ANALISIATM/ATMOSFERE/MH-1.0/atmoT45001.5.txt', 'r') as file:
            last_line = file.readlines()[-2]
            print('Missing atmosphere, using standard file tail, the one from the atmosphere 4500K, 1.5dex in logg and -1.0 [M/H]')
    # Estrai i numeri dalla riga usando una regex
    numbers = re.findall(r'[-+]?\d*\.\d+|[-+]?\d+', last_line)
    # Converti i numeri estratti in float
    numbers = [float(num) for num in numbers]
    numbers[0]

    if numbers[1] == 0.0:
        grandezza = "E+00"
    if numbers[1] < 0:
        grandezza = f"E-0{int(abs(numbers[1]))}"
    if numbers[1] > 0:
        grandezza = f"E+0{int(numbers[1])}"

    returner = f"{numbers[0]}" + grandezza
    os.chdir(OLD_DIR)
    return returner
def vturbo(T, LG, Met):
    return 0.14 - 0.08 * (Met) + 4.9 * T / 10000 - 0.47 * LG
def creaParamfileAbnd(microturb, numero_abundance=0, path_to_abundance=path_to_abundance, file_linee="stellaFes"):
    if numero_abundance==0:
        with open(f'param.txt', 'w') as paramfile:
            paramfile.write(f'ATMname ModelE\nLINESname {file_linee}\nOUTname PINOLOS\nvturb {str(microturb)}')
            sposta('param.txt', os.getcwd(), path_to_abundance, 'param.txt')
    else:
        with open(f'param{numero_abundance}.txt', 'w') as paramfile:                #{numero_abundance}
            sposta(f'param{numero_abundance}.txt', os.getcwd(), path_to_abundance, f'param{numero_abundance}.txt')#f'param{numero_abundance}.txt')
def creaParamfileAbnd1(microturb, numero_abundance=0, path_to_abundance=path_to_abundance, file_linee="stellaFes"):
    '''Crea paramFile di abundance.'''
    with open(f'param{numero_abundance}.txt', 'w') as paramfile:                #{numero_abundance}
        paramfile.write(f'ATMname Model{numero_abundance}\nLINESname {file_linee}\nOUTname PINOLOS{numero_abundance}\nvturb {str(microturb)}')
        sposta(f'param{numero_abundance}.txt', os.getcwd(), path_to_abundance, f'param{numero_abundance}.txt')#f'param{numero_abundance}.txt')
def SlanciaAbundance(n_istanze=1):
    if (n_istanze >= 1) and (n_istanze <= 4): 
        if n_istanze==1:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            process1.wait()
            os.chdir(DIR_INI)
        if n_istanze==2:
            DIR_INI = os.getcwd()
            os.chdir(path_to_abundance)
            abnd_path = "./abundance"
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            os.chdir(DIR_INI)
        if n_istanze==3:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            os.chdir(DIR_INI)
        if n_istanze==4:
            DIR_INI = os.getcwd()
            abnd_path = "./abundance"
            os.chdir(path_to_abundance)
            process1 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_0)
            process2 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_1)
            process3 = subprocess.Popen([abnd_path, '-t'])
            os.chdir(path_to_abundance_2)
            process4 = subprocess.Popen([abnd_path, '-t'])
            # Attendi che entrambi i processi terminino
            process1.wait()
            process2.wait()
            process3.wait()
            process3.wait()
            os.chdir(DIR_INI)
    else:
        print(f"Numero di istanze assegnate DEVE essere compreso tra 1 e 4 inclusi. Inserito {n_istanze}")
        return None
def stampaAnalisi(path_file, nome):
    A = simpleAnalysisSingle(path_file, nome, 26.0)
    B = simpleAnalysisSingle(path_file, nome, 26.1)
    with open("exit", 'w') as f:
        f.write(f"FeI = {A[0]}+{A[2]}\nFeII = {B[0]}+{B[2]}\n s(FeI) = {A[1]}\nDF = {A[0]-B[0]}")
    sposta("exit", os.getcwd(), path_file, "exit")
def simpleAnalysisSingle_old(path_file, nome_file, elemento=26.0, radSolver='spectrum', noFes=False): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std e la slope EWR.\n
    Poiché MOOG produce output in formati sostanzialmente diversi questa funzione si dirama a seconda del radSolver usato. \'spectrum\' e \'turbospectrum\' vengono trattati allo stesso modo. \'moog\' restituisce nell'output direttamente la slope, le abbondanze medie. Inoltre contiene anche le EW misurate. Questo però non permette di avere totale giurisdizione sull'utilizzo delle righe. In particolare il sigma-clipping non è applicabile."""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(4)
    EW=None
    WL=None
    if radSolver == 'spectrum' or radSolver == 'turbospectrum' or radSolver == 'turbo':
        if noFes==True:
            t1 = Table.read(nome_file, format='ascii')
            WL = t1['col1']
        if radSolver == 'turbo':
            olddir = os.getcwd()
            os.chdir(path_to_output)
            t1 = Table.read('stellaFes_turbo', format='ascii')
            EW = t1['col7']
            WL = t1['col3']
            EWR = EW/WL
            os.chdir(olddir)
        else:
            t1 = Table.read('stellaFes', format='ascii')
            EW = t1['col8']
            WL = t1['col1']
            EWR = EW/WL
        t2 = Table.read(nome_file, format='ascii')
        x=t2['col3'] #potential energy
        y=t2['col7'] #abbundanzia
        fe=t2['col2']
    elif radSolver == 'moog':
        t1 = Table.read(nome_file, format='ascii')
        EW = t1['col5']
        WL = t1['col1']
        EWR = EW/WL
        x=t1['col3'] #potential energy
        y=t1['col8'] #abbundanzia
        fe=t1['col2']

    # for i in range(len(EW)):
    #     EWR.append(EW[i] / WL[i])
    #EWR = EW/WL
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    # EW=EW[limit]
    # WL=WL[limit]
    print(noFes)
    if noFes == False:
        EWR=EWR[limit]
    if len(x) > 5:
        limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
        x=x[limit]
        y=y[limit]
        if noFes == False:
            EWR=EWR[limit]
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    if noFes == False:
        popt2, pcov2 = curve_fit(linear_func, EWR, y,method='lm')
        std2=np.std(y-(linear_func(EWR,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    if noFes:
        print('EWR non disponibile in mancanza di EW (manca il file stellaFes!)')
    elif noFes == False:
        returner[3] = popt2[0]   #slope EWR\A
    return returner
def simpleAnalysisSingle(path_file, nome_file, elemento=26.0, radSolver='spectrum', noFes=False, eqwidth_file='stellaFes_turbo'): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std e la slope EWR.\n
    Poiché MOOG produce output in formati sostanzialmente diversi questa funzione si dirama a seconda del radSolver usato. \'spectrum\' e \'turbospectrum\' vengono trattati allo stesso modo. \'moog\' restituisce nell'output direttamente la slope, le abbondanze medie. Inoltre contiene anche le EW misurate. Questo però non permette di avere totale giurisdizione sull'utilizzo delle righe. In particolare il sigma-clipping non è applicabile."""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(4)
    EW=None
    WL=None
    if radSolver == 'spectrum' or radSolver == 'turbospectrum' or radSolver == 'turbo':
        if noFes==True:
            t1 = Table.read(nome_file, format='ascii')
            WL = t1['col1']
        if radSolver == 'turbo':
            olddir = os.getcwd()
            os.chdir(path_to_output)
            t1 = Table.read(eqwidth_file, format='ascii')
            EW = t1['col7']
            WL = t1['col3']
        #    EWR = EW/WL
            os.chdir(olddir)
        else:
            t1 = Table.read('stellaFes', format='ascii')
            EW = t1['col8']
            WL = t1['col1']
        #    EWR = EW/WL
        t2 = Table.read(nome_file, format='ascii')
        x=t2['col3'] #potential energy
        y=t2['col7'] #abbundanzia
        fe=t2['col2']
    elif radSolver == 'moog':
        t1 = Table.read(nome_file, format='ascii')
        EW = t1['col5']
        WL = t1['col1']
        #EWR = EW/WL
        x=t1['col3'] #potential energy
        y=t1['col8'] #abbundanzia
        fe=t1['col2']
    # for i in range(len(EW)):
    #     EWR.append(EW[i] / WL[i])
    print('AAA')
    print(y)
    print('BBB')
    EWR = EW/WL
    # if elemento == 26.0:
    #     elemento = 1
    # elif elemento == 26.1:
    #     elemento = 2
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    if noFes == False:
        EW=EW[limit]
        WL=WL[limit]
        EWR=EWR[limit]
    print('KKK')
    print(y)
    print('JJJ')
    if len(x) > 5:
        limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
        x=x[limit]
        y=y[limit]
        if noFes == False:
            EW=EW[limit]
            WL=WL[limit]
            EWR=EWR[limit]
    print('CCC')
    print(y)
    print('DDD')
    if len(y)>1:
        popt, pcov = curve_fit(linear_func, x, y,method='lm')
        std1=np.std(y-(linear_func(x,*popt)))
        if noFes == False:
            popt2, pcov2 = curve_fit(linear_func, EWR, y,method='lm')
            std2=np.std(y-(linear_func(EWR,*popt)))
        #print(f"deviazione standard = {std1}")
        os.chdir(DIR_INI)
        returner[0] = np.mean(y) #media
        returner[1] = popt[0]    #slope
        returner[2] = std1       #std
        if noFes:
            print('EWR non disponibile in mancanza di EW (manca il file stellaFes!)')
        elif noFes == False:
            returner[3] = popt2[0]   #slope EWR\A
    elif len(y)==1:
        returner[0] = y[0]
        returner[1] = 0
        returner[2] = 0
        if noFes:
            print('EWR non disponibile in mancanza di EW (manca il file stellaFes!)')
        elif noFes == False:
            returner[3] = 0
    return returner
def simpleAnalysisSingle2(path_file, nome_file, elemento=26.0): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(4)
    t1 = Table.read('stellaFes', format='ascii')
    #print(os.getcwd())
    EW = t1['col8']
    WL = t1['col1']
    EWR = EW/WL
    #print(EWR)
    # for i in range(len(EW)):
    #     EWR.append(EW[i] / WL[i])
    t2 = Table.read(nome_file, format='ascii')
    x=t2['col3'] #potential energy
    y=t2['col7'] #abbundanzia
    fe=t2['col2']
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    popt2, pcov2 = curve_fit(linear_func, EWR, y,method='lm')
    std2=np.std(y-(linear_func(EWR,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    returner[3] = popt2[0]   #slope EWR\A
    #print(nome_file)
    return returner
def NLTE_AnalysisSingle(path_file, nome_file, T, G, M, V, elemento=26.0, WL_disponibilis='WL_disponibilis', radSolver='spectrum'): #passa path e nome file, restituisce abbondanza di un elemento, slope della retta che interpola i punti sul grafico abbondanza-energia potenziale, restituisce anche la standard deviation
    """Restituisce: media, slope, std\n Ogni riga viene corretta per gli effetti NLTE"""
    DIR_INI = os.getcwd()
    MM=M
    M=M+7.5
    os.chdir(path_to_output)
    file_path = 'stellaFes'
    lista_primi_elementi = crea_lista_primo_elemento(file_path)
    #print(lista_primi_elementi)
    correctionList = []
    PATH= path_to_correctionGrids1
    #AA=trova_dodicesimo_elemento(f"{PATH}4500-5000/correctionGrid", T, G, M, lista_primi_elementi[0])
    # print("BUB")
    # print(AA)
    # print("UBU")
    WL_disponibili=NLTE_caricaWLdisponibili()
    print("AHUHU")
    print(len(lista_primi_elementi))
    for i in range(len(lista_primi_elementi)):
        PATH=path_to_correctionGrids1
        if float(lista_primi_elementi[i]) < 5000:
            PATH=PATH+"4500-5000/correctionGrid"
        elif float(lista_primi_elementi[i]) > 5000 and lista_primi_elementi[i] < 5250:
            PATH=PATH+"5000-5250/correctionGrid"
        elif float(lista_primi_elementi[i]) > 5250 and lista_primi_elementi[i] < 5500:
            PATH=PATH+"5250-5500/correctionGrid"
        elif float(lista_primi_elementi[i]) > 5500 and lista_primi_elementi[i] < 5750:
            PATH=PATH+"5500-5750/correctionGrid"
        elif float(lista_primi_elementi[i]) > 5750 and lista_primi_elementi[i] < 6000:
            PATH=PATH+"5750-6000/correctionGrid"
        elif float(lista_primi_elementi[i]) > 6000 and lista_primi_elementi[i] < 6250:
            PATH=PATH+"6000-6250/correctionGrid"
        elif float(lista_primi_elementi[i]) > 6250 and lista_primi_elementi[i] < 6500:
            PATH=PATH+"6250-6500/correctionGrid"
        elif float(lista_primi_elementi[i]) > 6500 and lista_primi_elementi[i] < 6750:
            PATH=PATH+"6500-6750/correctionGrid"
        else:
            print("Nooo!")
        print(lista_primi_elementi[i] in WL_disponibili)
        print(f"{lista_primi_elementi[i]} - {WL_disponibili}")
        for j in range(len(WL_disponibili)):
            if abs(lista_primi_elementi[i] - WL_disponibili[j])<0.01:
                correctionList.append(NLTE_intCorrection(WL_disponibili[j], elemento, T, G, MM, V))
                print("!A!")
        #correctionList.append(trova_dodicesimo_elemento(f"{PATH}", T, G, M, lista_primi_elementi[i]))

        print(f"{lista_primi_elementi[i]}   {i}    {T}   {G}     {MM}")
        #  print(correctionList[i])
    os.chdir(path_to_linemake)
    t=Table.read("Iron_lines3", format='ascii')
    W=t['col1']
    weight=t['col8']
    os.chdir(path_file)
    returner = np.zeros(4)
    t1 = Table.read('stellaFes', format='ascii')
    EW = t1['col8']
    WL = t1['col1']
    EWR = EW/WL
    #t1 = Table.read('stellaFes', format='ascii')
    #print(os.getcwd())
    t2 = Table.read(nome_file, format='ascii')
    x=t2['col3']
    y=t2['col7']
    X=t2['col1']
    #print(f"{len(x)}  {len(lista_primi_elementi)}")
    fe=t2['col2']
    limit=((fe==elemento))
    x=x[limit]
    y=y[limit]
    X=X[limit]
    EWR=EWR[limit]
    correctionList=[el for el in correctionList if el is not None]
    print("pistacchio")
    print(correctionList)
    print("iaccio")
    for i in range(len(correctionList)):
        for j in range(len(X)):
            print(f"{X[j]}=={correctionList[i][3]}")
            if abs(X[j]-correctionList[i][3])<=0.01:
                #print("pistacchio")
                print(y[j])
                y[j]=y[j]+correctionList[i][1]
                print(f"{y[j]} + {correctionList[i][1]}")
                #print("iaccio")
    limit=((y<np.mean(y)+1.5*np.std(y))&(y>np.mean(y)-1.5*np.std(y)))
    x=x[limit]
    y=y[limit]
    EWR=EWR[limit]
    X=X[limit]
    xx = x.tolist()
    yy=y.tolist()
    XX=X.tolist()
    weighting = 0
    if weighting == 1:
        for i in range(len(W)):
            for j in range(len(X)):
                if W[i]==X[j]:
                    if weight[i] == 'aa':
                        for k in range(20):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'am' or 'ma':
                        for k in range(14):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'ax' or 'xa':
                        for k in range(12):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'aA' or 'Aa':
                        for k in range(25):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'aX' or 'Xa':
                        for k in range(2):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'xx':
                        for k in range(4):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'xm' or 'mx':
                        for k in range(6):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'Xx' or 'xX':
                        for k in range(1):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'Ax' or 'xA':
                        for k in range(12):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'mm':
                        for k in range(8):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'mX' or 'Xm':
                        for k in range(5):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'Am' or 'mA':
                        for k in range(12):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'AA':
                        for k in range(35):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'AX' or 'XA':
                        for k in range(10):
                            yy.append(y[j])
                            xx.append(x[j])
                    elif weight[i] == 'X':
                        for k in range(1):
                            yy.append(y[j])
                            xx.append(x[j])
    xx=np.asarray(xx)
    yy=np.asarray(yy)
    popt, pcov = curve_fit(linear_func, x, y,method='lm')
    std1=np.std(y-(linear_func(x,*popt)))
    popt2, pcov2 = curve_fit(linear_func, EWR, y,method='lm')
    std2=np.std(y-(linear_func(EWR,*popt)))
    #print(f"deviazione standard = {std1}")
    os.chdir(DIR_INI)
    returner[0] = np.mean(y) #media
    returner[1] = popt[0]    #slope
    returner[2] = std1       #std
    returner[3] = popt2[0]   #slope EWR\A
    #print(nome_file)
    return returner
def simpleAnalysisDouble(path_file, nome_file, elemento1=26.0, elemento2=26.1): #passa path del file, il nome, e restituisce le abbondanze di due elementi (o diverse ionizzazioni)
    """Restituisce: abbondanza el1, abbondanza el2"""
    DIR_INI = os.getcwd()
    os.chdir(path_file)
    returner = np.zeros(2)
    #t1 = Table.read('stellaFes', format='ascii')
    t2 = Table.read(nome_file, format='ascii')
    t3 = Table.read(nome_file, format='ascii')
    x2=t2['col3']
    y2=t2['col7']
    fe2=t2['col2']
    limit=((fe2==elemento1))
    x2=x2[limit]
    y2=y2[limit]
    limit=((y2<np.mean(y2)+1*np.std(y2))&(y2>np.mean(y2)-1*np.std(y2)))
    x2=x2[limit]
    y2=y2[limit]
    x3=t3['col3']
    y3=t3['col7']
    fe3=t3['col2']
    limit=((fe3==elemento2))
    x3=x3[limit]
    y3=y3[limit]
    limit=((y3<np.mean(y3)+1*np.std(y3))&(y3>np.mean(y3)-1*np.std(y3)))
    x3=x3[limit]
    y3=y3[limit]
    os.chdir(DIR_INI)
    returner[0] = np.mean(y2) #abbondanza elemento1
    returner[1] = np.mean(y3)+correzione(returner[0]) #abbondanza elemento2
    return returner
def creaOptFileARES(specfits='Stello.fits', readlinedat=lineList, fileout='stella.ares', lambdai="4600.", lambdaf="6700.", smoothder="4", space="3.0", rejt="1;5764,5766,6047,6052,6068,6076", lineresol="0.1", miniline="2", plots_flag="0", rvmask="3,6021.8,6024.06,6027.06,6024.06,20"):
    os.chdir(path_to_ARES)
    copia(specfits, path_to_fitsFiles, path_to_ARES, specfits)
    copia(readlinedat, path_to_linemake, path_to_ARES, readlinedat)
    with open('mine.opt', 'w') as paramfile:
        paramfile.write(f'specfits=\'{specfits}\'\nreadlinedat=\'{readlinedat}\'\nfileout=\'{fileout}\'\nlambdai={lambdai}\nlambdaf={lambdaf}\nsmoothder={smoothder}\nspace={space}\nrejt={rejt}\nlineresol={lineresol}\nminiline={miniline}\nplots_flag={plots_flag}\nrvmask=\'{rvmask}\'')
def parameterAnalysis(param_space_directions, T_center, log_center, M_center, T_steps=50, log_steps=0.1, M_steps=0.1, T_tot=8, log_tot=8, M_tot=10):
    if (param_space_directions < 1) or (param_space_directions>4):
        print("Il primo argomento in parameterAnalysis deve essere un int \n1 - muoviti solo in temperatura\n2 - muoviti in temperatura e logg\n3 - muoviti in temperatura logg e metallicità\n4 muoviti solo in logg")
    elif param_space_directions == 1:
        T = T_center - (T_steps*T_tot)/2
        T_temp = T
        ### muovi temperatura
        for i in range(T_tot):
            crea_ATMinterpol(round(T_temp), round(log_center, 1), round(M_center, 2))                
            T_temp = T_temp + T_steps
    elif param_space_directions == 2:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        T_temp = T
        log_temp = log
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_center, 2))
                log_temp = log_temp + log_steps
            T_temp = T_temp + T_steps
            log_temp = log
    elif param_space_directions == 3:
        T = T_center - (T_steps*T_tot)/2
        log = log_center - (log_steps*log_tot)/2
        M = M_center - (M_steps*T_tot)/2
        T_temp = T
        log_temp = log
        M_temp = M
        ### muovi temperatura e al suo interno il logg
        for i in range(T_tot):
            for j in range(log_tot):
                for k in range(M_tot):
                    crea_ATMinterpol(round(T_temp), round(log_temp, 1), round(M_temp, 2))
                    M_temp = M_temp + M_steps
                log_temp = log_temp + log_steps
                M_temp = M
            T_temp = T_temp + T_steps
            log_temp = log
    print("FINITO")
def find_closest(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    return arr[closest_index]
def creaHeader_metal(Fe_H, metallic=metallic_header, ABUNDANCE_SCALE=ABUNDANCE_SCALE_header):
    return 10**Fe_H
def preparaAbundance(stellaFITS, path_to_abundance=path_to_abundance, eqw_low=15, eqw_high=150, lineList=lineList, rad_solver = 'spectrum', outFes='stellaFes', instrument='HARPS-N', HeiterFormat=False):
    # Apre il file in modalità lettura
    """Produce il file "StellaFes" contenente le EQUIVALENT WIDTH della stella.\nNome del file FITS (doppler corretto) contenente la stella da analizzare
    eqw low and high sono gli estremi di equivalent width accettata nel processo di analisi, per trattare la curva di crescita delle eqw e rimanere fuori del rumore e dalla saturazione della riga.\n\n
    Per cambiare il solver di radiazione, cambiare il parametro rad_solver in 'spectrum' ('s') o 'turbospectrum' ('t')"""
    if HeiterFormat==True:
        creaLineList(stellaFITS, rad_solver=rad_solver, line_list='ESO_FE', instrument=instrument, ew_low=eqw_low, ew_high=eqw_high)
    else:
        i=0
        os.chdir(path_to_linemake)
                #Iron_lines#
        with open(lineList, 'r') as file:
            LINEM = repr(file.read())
        os.chdir(path_to_ARES)
        creaOptFileARES(stellaFITS, lineList)
        ! ./ARES
        with open('stella.ares', 'r') as file:
            ARES = repr(file.read())

        vect1 = LINEM.split("\\n")
        vect2 = ARES.split("\\n")
        VECT1 = np.zeros((len(vect1),9))
        VECT2 = np.zeros((len(vect2),9))

        for i in range(1,len(vect2)-1):
            temp2 = vect2[i].split()
            VECT2[i-1][0]= temp2[0]
            VECT2[i-1][1]= temp2[1]
            VECT2[i-1][2]= temp2[2]
            VECT2[i-1][3]= temp2[3]
            VECT2[i-1][4]= temp2[4]
            VECT2[i-1][5]= temp2[5]
            VECT2[i-1][6]= temp2[6]
            VECT2[i-1][7]= temp2[7]
            VECT2[i-1][8]= temp2[8]
        t4 = Table.read('stella.ares', format='ascii')
        for i in range(1, len(vect1)-1):
            temp1 = vect1[i].split()
            VECT1[i-1][0]= float(temp1[0])
            VECT1[i-1][1]= temp1[1]
            VECT1[i-1][2]= float(temp1[2])
            VECT1[i-1][3]= temp1[3]
            VECT1[i-1][4]= temp1[4]
            VECT1[i-1][5]= 1.0
            VECT1[i-1][6]= 99
            VECT1[i-1][7]= 0
        if lineList=='Iron_lines' or lineList=='IronS1' or lineList=='eso_fe':
            for i in range(len(VECT1)):
                for j in range(len(VECT2)):
                    if (VECT1[i][0] == VECT2[j][0]) and (VECT2[j][4]>eqw_low) and (VECT2[j][4]<eqw_high) and (float(VECT1[i][1])==26.0):
                        VECT1[i][7] = VECT2[j][4]
                    elif (VECT1[i][0] == VECT2[j][0]) and (float(VECT1[i][1])==26.1):
                        VECT1[i][7] = VECT2[j][4]
        else:
            for i in range(len(VECT1)):
                for j in range(len(VECT2)):
                    if (VECT1[i][0] == VECT2[j][0]) and (VECT2[j][4]>eqw_low) and (VECT2[j][4]<eqw_high):
                        VECT1[i][7] = VECT2[j][4]
                    elif (VECT1[i][0] == VECT2[j][0]):
                        VECT1[i][7] = VECT2[j][4]

        # for i in range(1, len(vect1)-1): # per ARES linemake
        #     temp1 = vect1[i].split()
        #     VECT1[i-1][0]= float(temp1[0])
        #     VECT1[i-1][1]= temp1[1]
        #     VECT1[i-1][2]= float(temp1[2]) * 8066.01
        #     VECT1[i-1][3]= 0
        #     VECT1[i-1][4]= temp1[3]
        #     VECT1[i-1][5]= 1.0
        #     VECT1[i-1][6]= 99
        #     VECT1[i-1][7]= 0

        ############################## PREPARA FILE PER ABUNDANCE ################################

        os.chdir(path_to_abundance)
        bug = []
        o=0
        T_atmosold=0
        for i in range(len(VECT1)):
            if VECT1[i][7]>eqw_low and VECT1[i][7]<eqw_high: #if VECT1[i][1] == 26.0 and VECT1[i][7]>30:
                bug.append(VECT1[i][7])
                o=o+1
        # indici_righe_da_rimuovere = np.where((VECT1[:, 7] < eqw_low) | (VECT1[:, 7] > eqw_high))[0]
        # VECT1 = np.delete(VECT1, indici_righe_da_rimuovere, axis=0)
        linesout = 'stellantess'
        iron_linesout = outFes
        with open(linesout, 'w') as file:
            for i in range(len(VECT1)):
                #print(VECT1[i][1])
                if VECT1[i][7]!=0:
                    file.write(str(VECT1[i][0]))
                    file.write('  ')
                    file.write(str(VECT1[i][1]))
                    file.write('  ')
                    file.write(str(VECT1[i][2]))
                    file.write('  ')
                    file.write(str(VECT1[i][3]))
                    file.write('  ')
                    file.write(str(VECT1[i][4]))
                    file.write('  ')
                    file.write(str(VECT1[i][5]))
                    file.write('  ')
                    file.write(str(VECT1[i][6]))
                    file.write('  ')
                    file.write(str(VECT1[i][7]))
                    file.write('  ')
                    file.write('a')
                    file.write('\n')

        # with open(iron_linesout, 'w') as file:
        #     for i in range(len(VECT1)):
        #         if VECT1[i][7]!=0 and (VECT1[i][1]==26 or VECT1[i][1]==26.1):
        #             file.write(str(VECT1[i][0]))
        #             file.write('  ')
        #             file.write(str(VECT1[i][1]))
        #             file.write('  ')
        #             file.write(str(VECT1[i][2]))
        #             file.write('  ')
        #             file.write(str(VECT1[i][3]))
        #             file.write('  ')
        #             file.write(str(VECT1[i][4]))
        #             file.write('  ')
        #             file.write(str(VECT1[i][5]))
        #             file.write('  ')
        #             file.write(str(VECT1[i][6]))
        #             file.write('  ')
        #             file.write(str(VECT1[i][7]))
        #             file.write('  ')
        #             file.write('a')
        #             file.write('\n')
        with open(iron_linesout, 'w') as file:
            for i in range(len(VECT1)):
                if VECT1[i][7]!=0:
                    file.write(str(VECT1[i][0]))
                    file.write('  ')
                    file.write(str(VECT1[i][1]))
                    file.write('  ')
                    file.write(str(VECT1[i][2]))
                    file.write('  ')
                    file.write(str(VECT1[i][3]))
                    file.write('  ')
                    file.write(str(VECT1[i][4]))
                    file.write('  ')
                    file.write(str(VECT1[i][5]))
                    file.write('  ')
                    file.write(str(VECT1[i][6]))
                    file.write('  ')
                    file.write(str(VECT1[i][7]))
                    file.write('  ')
                    file.write('a')
                    file.write('\n')
        o=0
        vturb = 1.8
        comando_bash = f"cp {iron_linesout} {path_to_output}{iron_linesout}"
        os.system(comando_bash)
        if rad_solver == 'turbo':
            comando_bash = f"cp {iron_linesout} {path_to_turboModels}{iron_linesout}"
            os.system(comando_bash)
        os.chdir(path_to_abundance)
def avvicina_interpol(path_to_stella, nome_file, TEFF, G, M, nome_stella, nome_cartella=path_to_output):
    temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
    temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    sgn=1
    g=-1
    teff=-1
    met=1
    returner = []
    if abs(temp0[1]) > 0.001:
        if (abs(temp0[1]) < 0.06) and (abs(temp0[1]) >= 0.035):
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 70*sgn
        elif abs(temp0[1] > 0.01) and abs(temp0[1]) < 0.035:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 30*sgn
        elif abs(temp0[1]) < 0.01:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 10*sgn
        elif abs(temp0[1]) < 0.13 and abs(temp0[1]) > 0.06:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 150*sgn
        else:
            if temp0[1] < 0:
                sgn = -1
            teff = TEFF + 300*sgn
        crea_ATMinterpol(teff, G, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{G:.2f}", path_to_output, path_to_output, f"{nome_file1}")
    else:
        nome_file1 = nome_file
        teff = TEFF
    temp0 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.0)
    temp1 = simpleAnalysisSingle(f"{path_to_output}", nome_file1, 26.1)
    sgn1=1
    if (temp0[0]-temp1[0])>0:
        sgn1=-1
    if abs(temp0[0]-temp1[0]) > 0.01:
        if abs(temp0[0]-temp1[0]) < 0.4:
            g = G+(temp0[0]-temp1[0])
        else:
            g = G+((temp0[0]-temp1[0]))
        crea_ATMinterpol(teff, g, temp0[0], nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{temp2:.2f}_{teff:.0f}_{g:.2f}"
        sposta(f"{nome_stella}_INT{temp0[0]:.2f}_{teff:.0f}_{g:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        #copia(f"{nome_file1}", path_to_output, path_to_output, f"{nome_file1}{nome_stella}")
        met=temp2
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    return returner
def avvicina_interpol6(path_to_stella, nome_file, temperatu, G, M, V, nome_stella, nome_cartella=path_to_output, NLTE=0, searchMatrix=s_Matrix):
    #blocca_stampe()
    old_vturb = vturbo(temperatu, G, M)
    if NLTE==0:
        temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
        temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    elif NLTE == 1:
        temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
        temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    v=sp.symbols('v')
    # A = np.array([[dsdg, dsdt, dsdm],
    #      [dfdg, dfdt, dfdm],
    #      [dydg, dydt, dydm]])
    #riattiva_stampe()
    #sp.pprint(s_Matrix)
    A = s_Matrix
    AA=A.subs(v, V)
    A_numpy = np.array(AA.tolist(), dtype=float)
    A_inv = np.linalg.inv(A_numpy)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    ewr = -1*(temp0[3])
    B = np.array([s, f, m, ewr])
    #blocca_stampe()
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DT = (np.dot(A_inv, B))[0]
        DG = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        Dv = (np.dot(A_inv, B))[3]
        if abs(DG) > 0.10:
            DG=0.10*(DG/abs(DG))
        if abs(DT) > 150:
            DT = 150*(DT/abs(DT))
        if abs(DM) > 0.10:
            DM = 0.10*(DM/abs(DM))
        if abs(Dv) > 0.1:
            Dv = 0.1*(Dv/abs(Dv))
        temperatu=temperatu+DT
        G=G+DG
        velturb=V+Dv
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol2(teff, G, temp0[0]+DM, velturb, nome_stella)
        nome_file1 = f"{nome_stella}2_{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}"
        sposta(f"{nome_stella}_{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    returner.append(velturb)
    return returner
def nextParam(path_to_stella, nome_file, temperatu, G, M, V, nome_stella, nome_cartella=path_to_output, NLTE=0, searchMatrix=s_Matrix):
    blocca_stampe()
    if NLTE==0:
        temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
        temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
    elif NLTE == 1:
        temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
        temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
    sgn=1
    g=-1.0
    teff=-1.0
    met=1.0
    DG = 0
    DT = 0
    returner = []
    dsdg = -0.033574
    dsdt = -0.00023335
    dsdm = 0.0104
    dfdg = -0.5227
    dfdt = 0.001455
    dfdm = -0.3574 
    dydg = -0.13522
    dydt = 0.00041
    dydm = -0.8345
    v=sp.symbols('v')
    # A = np.array([[dsdg, dsdt, dsdm],
    #      [dfdg, dfdt, dfdm],
    #      [dydg, dydt, dydm]])
    riattiva_stampe()
    #sp.pprint(s_Matrix)
    A = s_Matrix
    AA=A.subs(v, V)
    A_numpy = np.array(AA.tolist(), dtype=float)
    A_inv = np.linalg.inv(A_numpy)
    nlte_corrF=0#.2
    nlte_corrS=0#-0.027
    nlte_corrM=0#.1
    f=-1*((temp0[0])-(temp1[0]+nlte_corrF))
    s = -1*(temp0[1]+nlte_corrS)
    m = -1*(temp0[0]-(M+nlte_corrM))
    ewr = -1*(temp0[3])
    B = np.array([s, f, m, ewr])
    blocca_stampe()
    if abs(temp0[1]+nlte_corrS) > 0.0001 or abs((temp0[0])-(temp1[0]+nlte_corrF)) > 0.0003 or abs(M - (temp0[0])+nlte_corrM)>0.001:
        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
        DT = (np.dot(A_inv, B))[0]
        DG = (np.dot(A_inv, B))[1]
        DM = (np.dot(A_inv, B))[2]
        Dv = (np.dot(A_inv, B))[3]
        if abs(DG) > 0.5:
            DG=0.5*(DG/abs(DG))
        if abs(DT) > 250:
            DT = 250*(DT/abs(DT))
        if abs(DM) > 0.5:
            DM = 0.5*(DM/abs(DM))
        if abs(Dv) > 0.3:
            Dv = 0.3*(Dv/abs(Dv))
        temperatu=temperatu+DT
        G=G+DG
        velturb=V+Dv
        teff= temperatu
        #temp0[0] = temp0[0]+DM
        crea_ATMinterpol2(teff, G, temp0[0]+DM, velturb, nome_stella)
        temp2 = simpleAnalysisSingle(path_to_output, f"{nome_stella}_INT{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", 26.0)[0]
        nome_file1 = f"AH_STELLA{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}"
        sposta(f"{nome_stella}_INT{(temp0[0]+DM):.2f}_{teff:.0f}_{G:.2f}_{velturb:.2f}", path_to_output, path_to_output, f"{nome_file1}")
        g=G
        met = temp0[0]
    else:
        nome_file1 = nome_file
        teff = temperatu
    stampaAnalisi(path_to_output, nome_file1)
    returner.append(teff)
    returner.append(g)
    returner.append(met)
    returner.append(nome_file1)
    returner.append(velturb)
    return returner
def leggi_log(path_cartella_log):
    '''Di seguito i nomi delle colonne di log.txt. Restituisce la tavola letta con Table.read(). T      G      M   vturb   FeI            FeII           FeI-M/H        FeI-FeII       s(FeI)  EWR'''
    olddir = os.getcwd()
    os.chdir(path_cartella_log)
    log_table = Table.read("log.txt", format='ascii')
    os.chdir(olddir)
    return log_table
def minimo_tavola(tab, nome_colonna, toll=2.0, mode='abs'):
    '''Passata una tabella restituisce l'elemento minimo della colonna specificata e il relativo indice di riga. mode=abs trova il minimo in valore assoluto, mode=n trova il minimo considerando anche il segno'''
    x=tab[nome_colonna]
    if mode=='abs':
        X = abs(x)
    if mode=='n':
        X = x
    returner = []
    min = 0
    i=0
    absmin = X[X.argmin()]
    while min < absmin*toll or min-absmin<=0.05:
        M = X.argmin()
        min = X[M]
        min_index = M
        returner.append([min, int(min_index)])
        X[np.where(X == min)] = 1
        i=i+1
    return returner
def final_atmo(path_cartella_log): #
    '''Passa log file, restituisce la best atmosfera. Nelle cartelle contenenti le atmosfere analizzate è possibile creare un log file attraverso la funzione folderLog. Passa qua la directory del log di quella stella. Si cerca prima l'atmosfera che più minimizza FeI-FeII. Da qua si verifica che l'atmosfera che minimizza la slope sia sufficientemente vicina al minimo (i.e. entro 1.2*minimo_slope).'''
    tab=leggi_log(path_cartella_log)
    dF=minimo_tavola(tab, 'FeI-FeII')
    slope=minimo_tavola(tab, 's(FeI)')
    if (tab[slope[0][1]] == tab[dF[0][1]]) and (tab[dF[0][1]]['FeI-M/H'] <= 0.01):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(dF[0][1], slope):
        print(tab[dF[0][1]])
        print()
        return tab[dF[0][1]]
    elif esiste_in_lista(slope[0][1], dF):
        print(tab[slope[0][1]])
        print()
        return tab[slope[0][1]]
    elif esiste_in_lista(dF[1][1], slope): 
        print(tab[dF[1][1]])
        print()
        return tab[dF[1][1]]
    elif esiste_in_lista(slope[1][1], dF): 
        print(tab[slope[1][1]])
        print()
        return tab[slope[1][1]]
    elif len(dF)>= 3 and esiste_in_lista(dF[2][1], slope): 
        print(tab[dF[2][1]])
        print()
        return tab[dF[2][1]]
    elif len(slope)>= 3:
        if esiste_in_lista(slope[2][1], dF): 
            print(tab[slope[2][1]])
            print()
            return tab[slope[2][1]]
    else:
        print("nessuna atmosfera trovata.. :C")
def esiste_in_lista(x, lista2):
    esiste = any(x == sub2[1] for sub2 in lista2)
    return esiste
def NLTE_Fe_correction(t, e, g, x, wi):
    url = 'http://www.inspect-stars.com/cp/application.py/A_from_e?element_name=Fe'

    # Cambia il valore del campo 't'
    form_data = {
        't': f'{t}', 
        'e': f'{e}',
        'g': f'{g}',
        'x': f'{x}',
        'wi': f'{wi}'
    }

    # Codifica i dati come query string per l'invio con POST
    encoded_data = urllib.parse.urlencode(form_data).encode('utf-8')

    # Fai la richiesta POST con i dati del form
    req = urllib.request.Request(url, data=encoded_data)

    # Aggiungi un User-Agent per simulare un browser reale
    req.add_header('User-Agent', 'chrome')

    with urllib.request.urlopen(req) as response:
        result = response.read().decode('utf-8')

    # 1. Estrazione del blocco <pre>
    pre_block = re.search(r'<pre>(.*?)</pre>', result, re.DOTALL)
    if pre_block:
        pre_content = pre_block.group(1)  # Ottieni il contenuto tra <pre> e </pre>
        # 2. Estrazione dei numeri (inclusi decimali)
        numbers = re.findall(r'\d+\.\d+|\d+', pre_content)
        # Stampa o utilizza i numeri come preferisci
        print("Numeri estratti:", numbers)
        if len(numbers) == 5:
            return numbers
        else:
            returner = [e, 0, 0, 0, 0]
            return returner
    else:
        print("Blocco <pre> non trovato.")
def find_closest2(num, arr):
    arr = np.array(arr)
    closest_index = np.abs(arr - num).argmin()
    if (arr[closest_index]-num)>0.011:
        print("No line found!")
        return 0
    else:
        return arr[closest_index]
def createCorrectionGrid():
    o=0
    OLDDIR = os.getcwd()
    os.chdir(path_to_correctionGrids)
    with open('correction_grid.txt', 'w') as gridder:
        gridder.write("EW               T               G                V                deltaNLTE\n")
        for E in range(20, 200, 10):
            print(f"EW_riga: {E}")
            for i in range(4100, 5500, 50):
                for j in range(10, 26, 1):
                    for k in range(100, 200, 5):
                        #for l in range(120):
                        U=NLTE_Fe_correction(i, E, j/10, k/100, 34)
                        a0 = int(E-20)
                        a1 = int((i-4000)/20)
                        a2 = int((j-10))
                        a3 = int((k-100)/5)
                        gridder.write(f"{U[0]}               {i}               {j/10}                {k/100}                {U[3]}")
                        gridder.write("\n")
                        o=o+1
                        if o == 200:
                            time.sleep(np.random.exponential(10))
    os.chdir(OLDDIR)
def extractSlope(log_path, x, y, p0=0, p1=0, p2=0, p3=0):
    '''Questa funzione restituisce l'andamento di una grandezza y in funzione di x. I dati sono letti dai file log.txt prodotti.\n
    Ad esempio y=s(FeI), x=T. Le quantità devono essere passate come stringhe:\n
    x):\n
    'T'= Temperatura\n
    'G'= log(g)\n
    'M'= [M/H]\n
    'v'= vturb\n
    y):\n
    's'= s(FeI)\n
    'F'= A(FeI)-A(FeII)\n
    'Y'= [Fe/H]_dedotta - [Fe/H]_modello\n
    'S'= S(EWR)\n
    p_i):\n
    p0 = T0\n
    p1 = G0\n
    p2 = M0\n
    p3 = v0\n
    Restituisce dy/dx da fit lineare centrato su tre dei punti (p0, p1 e p2, p3), uno rimane libero per ottenere la variazione.'''
    tab = leggi_log(log_path)
    # Crea i vettori dati
    if True:
        if y == 's':
            Y = tab['s(FeI)']
        elif y == 'F':
            Y = tab['FeI-FeII']
        elif y == 'Y':
            Y = tab['FeI-M/H']
        elif y == 'S':
            Y = tab['EWR']
        else:
            print("'y' inserita non valida\n Opzioni valide: s, F, Y, S")
        
        if x == 'T':
            X = tab['T']
            x1 = tab['G']
            x2 = tab['M']
            x3 = tab['vturb']            
            lim=((x1==p1))
            X=X[lim]
            Y=Y[lim]
            #print(Y)
            x2=x2[lim]
            x3=x3[lim]
            lim=((x2==p2))
            X=X[lim]
            Y=Y[lim]
            x3=x3[lim]         
            lim=(abs(x3-p3)<0.05)
            X=X[lim]
            Y=Y[lim]
        elif x == 'G':
            X = tab['G']
            x0 = tab['T']
            x2 = tab['M']
            x3 = tab['vturb']
            lim=((x0==p0))
            X=X[lim]
            Y=Y[lim]
            x2=x2[lim]
            x3=x3[lim]
            lim=((x2==p2))
            X=X[lim]
            Y=Y[lim]
            x3=x3[lim]
            lim=((x3==p3))
            X=X[lim]
            Y=Y[lim]
        elif x == 'M':
            X = tab['M']
            x0 = tab['T']
            x1 = tab['G']
            x3 = tab['vturb']
            lim=((x0==p0))
            X=X[lim]
            Y=Y[lim]
            x1=x1[lim]
            x3=x3[lim]
            lim=((x1==p1))
            X=X[lim]
            Y=Y[lim]
            x3=x3[lim]
            lim=((x3==p3))
            X=X[lim]
            Y=Y[lim]
        elif x == 'v':
            X = tab['vturb']
            x0 = tab['T']
            x1 = tab['G']
            x2 = tab['M']
            lim=((x0==p0))
            X=X[lim]
            Y=Y[lim]
            x1=x1[lim]
            x2=x2[lim]
            lim=((x1==p1))
            X=X[lim]
            Y=Y[lim]
            x2=x2[lim]
            lim=((x2==p2))
            X=X[lim]
            Y=Y[lim]
        else:
            print("'x' inserita non valida\n Opzioni valide: T, G, M, v")
    if x == 'T' or x== 'G' or x=='M':
        popt, pcov = curve_fit(linear_func, X, Y,method='lm')
        X_fit = np.linspace(min(X), max(X), 100)
        Y_fit = linear_func(X_fit, *popt)
        m, c = popt
        # Plot dei dati originali e della retta di best fit
        plt.figure(figsize=(8, 6))
        plt.scatter(X, Y, color='red', label='Dati originali')
        plt.plot(X_fit, Y_fit, color='blue', label=f'Retta di best fit: ${y}({x}) = {m:.7f}{x} + {c:.7f}$')
        plt.xlabel(f'{x}')
        plt.ylabel(f'{y}({x})')
        plt.title('Fit Lineare')
        plt.legend()
        plt.grid()
        plt.savefig(path_to_calibrationFolder +f'/pt_{y}-{x}.pdf')
        plt.show()
        return popt
    elif x == 'v':
        popt, pcov = curve_fit(quad_func, X, Y,method='lm')
        X_fit = np.linspace(min(X), max(X), 100)
        Y_fit = quad_func(X_fit, *popt)
        m, c , d = popt
        # Plot dei dati originali e della retta di best fit
        plt.figure(figsize=(8, 6))
        plt.scatter(X, Y, color='red', label='Dati originali')
        plt.plot(X_fit, Y_fit, color='blue', label=f'Retta di best fit: ${y}({x}) = {m:.7f}{x}^2 + {c:.7f}{x} + {d:.2f}$')
        plt.xlabel(f'{x}')
        plt.ylabel(f'{y}({x})')
        plt.title('Fit Lineare')
        plt.legend()
        plt.grid()
        plt.savefig(path_to_calibrationFolder +f'/pt_{y}-{x}.pdf')
        plt.show()
        return popt
    #return c
def corrDoppler(x, v):
    '''Dando una lunghezza d'onda osservata restituisce il valore della lunghezza d'onda senza effetto doppler\nv in km/s'''
    return x/((v/LIGHTSPEED)+1)
def NLTE_wlCorrection(wl, el, T, G, M, V, full_array_corr='full_array_corr'):
    DIR_INI = os.getcwd()
    M=M+7.5
    os.chdir(path_to_output)
    file_path = 'stellaFes'
    #lista_primi_elementi = crea_lista_primo_elemento(file_path)
    #print(lista_primi_elementi)
    correctionList = []
    PATH= path_to_correctionGrids1
    PATH=path_to_correctionGrids1
    if wl < 5000:
        PATH=PATH+"4500-5000/correctionGrid"
    elif wl > 5000 and wl < 5250:
        PATH=PATH+"5000-5250/correctionGrid"
    elif wl > 5250 and wl < 5500:
        PATH=PATH+"5250-5500/correctionGrid"
    elif wl > 5500 and wl < 5750:
        PATH=PATH+"5500-5750/correctionGrid"
    elif wl > 5750 and wl < 6000:
        PATH=PATH+"5750-6000/correctionGrid"
    elif wl > 6000 and wl < 6250:
        PATH=PATH+"6000-6250/correctionGrid"
    elif wl > 6250 and wl < 6500:
        PATH=PATH+"6250-6500/correctionGrid"
    elif wl > 6500 and wl < 6750:
        PATH=PATH+"6500-6750/correctionGrid"
    else:
        print("Nooo!")
    #correctionList.append(findLine(f"{PATH}", int(T), G, M, wl, V))
    correctionList.append(findLine1(full_array_corr, T, G, M, wl, V))
        #print(correctionList[i])
    correctionList=[el for el in correctionList if el is not None]
    # print("AIUO")
    #print(correctionList)

    for i in range(len(correctionList)):
        if abs(wl - correctionList[i][6])<0.01:
            return correctionList[i][0]
def checkExist_newParam(nome, T, G, M, V, path, t_fixed=0, g_fixed=-1, m_fixed=5, v_fixed=-5, iter=0):
    iter=iter+1
    file_path = path + f"/{nome}_{M:.2f}_{T:.0f}_{G:.2f}_{V:.2f}"
    if iter >= 10:
        return [T, G, M, V]
    if os.path.isfile(file_path):
        if t_fixed == 0:
            T=T+1
        if g_fixed==-1:
            G=G+0.01
        if m_fixed==5:
            M=M-0.01
        if v_fixed==-5:
            V=V+0.01
        return checkExist_newParam(nome, T, G, M, V, path, t_fixed=0, g_fixed=-1, m_fixed=5, v_fixed=-5, iter=iter)

    else:
        return [T, G, M, V]
#def NLTE_interpola()
def moving_average(X, Y, window_size):
    def interpolate(x):
        dist = np.abs(X-x)
        index = np.argsort(dist)[:window_size]
        y_avg = np.mean(Y[index])
        return y_avg
    return interpolate
def moda_bin(dati, bins, showHist=True):
    hist, bin_edges = np.histogram(dati, bins=bins)
    max_index = np.argmax(hist)
    mode = (bin_edges[max_index]+bin_edges[max_index+1])/2
    if showHist:
        plt.hist(dati, bins=bins)
    return mode
def creaParamfileSpectrum(microturb, atm_model, nome_output, start_wl, end_wl, step, path_to_spectrum=path_to_spectrum, numero_spectrum=-1, file_linee='luke.lst'):
    if numero_spectrum == -1:
        with open(f'parameters', 'w') as paramfile:                #{numero_abundance}
            paramfile.write(f'ATMname {atm_model}\nLINESname {file_linee}\nOUTname {nome_output}.synstar\nvturb {str(microturb)}\nstartwl {start_wl}\nendwl {end_wl}\nstep {step}')
            sposta(f'parameters', os.getcwd(), path_to_spectrum, f'parameters')#f'param{numero_abundance}.txt')
    elif numero_spectrum >= 0:
        with open(f'parameters{numero_spectrum}', 'w') as paramfile:                #{numero_abundance}
            paramfile.write(f'ATMname {atm_model}{numero_spectrum}\nLINESname {file_linee}\nOUTname {nome_output}{numero_spectrum}.synstar\nvturb {str(microturb)}\nstartwl {start_wl}\nendwl {end_wl}\nstep {step}')
            sposta(f'parameters{numero_spectrum}', os.getcwd(), path_to_spectrum, f'parameters{numero_spectrum}')#f'param{numero_abundance}.txt')
def spectrum_compatibile(x): #passa da % --> log_10(), 0.92--->-0.036
    return np.log10(x)
def get_file_names(directory):
    file_names = []
    for file_name in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file_name)):
            file_names.append(file_name)
    return file_names
def creaChem( H=0.92150, He=0.07840, Li=-10.94, Be=-10.64, B=-9.49, C=-3.52, N=-4.12, O=-3.21, F=-7.48, Ne=-3.96, Na=-5.71, Mg=-4.46, Al=-5.57, Si=-4.49, P=-6.59, S=-4.71, Cl=-6.54, Ar=-5.64, K=-6.92, Ca=-5.68, Sc=-8.87, Ti=-7.02, V=-8.04, Cr=-6.37, Mn=-6.65, Fe=-4.54, Co=-7.12, Ni=-5.79, Cu=-7.83, Zn=-7.44, Ga=-9.16, Ge=-8.63, As=-9.67, Se=-8.63, Br=-9.41, Kr=-8.73, Rb=-9.44, Sr=-9.07, Y=-9.80, Zr=-9.44, Nb=-10.62, Mo=-10.12, Tc=-20.00, Ru=-10.20, Rh=-10.92, Pd=-10.35, Ag=-11.10, Cd=-10.27, In=-10.38, Sn=-10.04, Sb=-11.04, Te=-9.80, Io=-10.53, Xe=-9.87, Cs=-10.91, Ba=-9.91, La=-10.87, Ce=-10.46, Pr=-11.33, Nd=-10.54, Pm=-20.00, Sm=-11.03, Eu=-11.53, Gd=-10.92, Tb=-11.69, Dy=-10.90, Ho=-11.78, Er=-11.11, Tm=-12.04, Yb=-10.96, Lu=-11.98, Hf=-11.16, Ta=-12.17, W=-10.93, Re=-11.76, Os=-10.59, Ir=-10.69, Pt=-10.24, Au=-11.03, Hg=-10.91, Tl=-11.14, Pb=-10.09, Bi=-11.33, Po=-20.00, At=-20.00, Rn=-20.00,  Fr=-20.00, Ra=-20.00, Ac=-20.00, Th=-11.95, Pa=-20.00, U=-12.54, path_to_stdatom=path_to_spectrum, nome_stdatom='stdatom.dat', nome_stdatom_out='stdatom.dat', metal=0.0):
    '''Crea composizione chimica, applica la scalatura di spectrum, ovvero nel trattare i metalli corregge per la metallicità dell'atmosfera (che si assume essere data da [Fe/H]). spectrum sottrae il valore della metallicità (l'header dell'atmosfera ATLAS9) dal valore letto in stdatom, questo significa che se volessi avere una stella con [M/H]=-1, ma con la stessa quantità di sodio dovrei modificare stdatom con un sodio +1 rispetto al solare.'''
    H=spectrum_compatibile(H)
    He=spectrum_compatibile(He)
    periodic_Table = [H, He, Li, Be, B, C, N, O, F, Ne, Na, Mg, Al, Si, P, S, Cl, Ar, K, Ca, Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, Ga, Ge, As, Se, Br, Kr, Rb, Sr, Y, Zr, Nb, Mo, Tc, Ru, Rh, Pd, Ag, Cd, In, Sn, Sb, Te, Io, Xe, Cs, Ba, La, Ce, Pr, Nd, Pm, Sm, Eu, Gd, Tb, Dy, Ho, Er, Tm, Yb, Lu, Hf, Ta, W, Re, Os, Ir, Pt, Au, Hg, Tl, Pb, Bi, Po, At, Rn,  Fr, Ra, Ac, Th, Pa, U]
    os.chdir(path_to_stdatom)
    table = Table.read(nome_stdatom, format='ascii')
    for i in range(len(periodic_Table)):
        if i < 2:
            table['Abund'][i]=periodic_Table[i]
        else:
            table['Abund'][i]=periodic_Table[i]
    table.write(nome_stdatom_out, format='ascii', overwrite=True)
def remove_first_two_lines(file_path):
    '''Rimuove le prime due righe di un file di testo.'''
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Escludi le prime due righe
    lines = lines[2:]
    
    with open('no'+file_path, 'w') as file:
        file.writelines(lines)
    return 'no'+file_path
def split_file(output_dir=path_to_turboModels + 'lineliste/output', input_file=path_to_turboModels + 'lineliste/' + lineListT, lines_per_file=1500):
    '''Separa la linelist fornita in input in varie sotto-liste contententi ognuna 35 transizioni. \n
    Copia i file creati in path_to_turboModels'''
    os.makedirs(output_dir, exist_ok=True)
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    # Lista per contenere le righe del file corrente
    current_file_lines = []
    file_count = 1
    last_two_special_lines = []
    previous_special_lines = []

    for line in lines:
        # Aggiungi la riga al file corrente
        current_file_lines.append(line)
        
        # Se la riga inizia con "'", aggiorna le ultime due righe speciali trovate
        if line.startswith("'"):
            last_two_special_lines.append(line)
            if len(last_two_special_lines) > 2:
                last_two_special_lines.pop(0)
        
        # Se la riga non inizia con "'", incrementa il contatore delle righe normali
        if not line.startswith("'"):
            if len([l for l in current_file_lines if not l.startswith("'")]) == lines_per_file:
                # Se il file corrente non inizia con una riga speciale, aggiungi le ultime due righe speciali trovate nel file precedente
                if not current_file_lines[0].startswith("'"):
                    current_file_lines = previous_special_lines + current_file_lines
                
                # Scrivi il file corrente
                output_file = os.path.join(output_dir, f'{os.path.basename(input_file)}_part_{file_count}')
                with open(output_file, 'w') as out_file:
                    out_file.writelines(current_file_lines)
                print(f'Creato file: {output_file}')
                
                # Resetta la lista per il prossimo file
                current_file_lines = []
                file_count += 1
                previous_special_lines = last_two_special_lines.copy()
    
    # Scrivi l'ultimo file se ci sono righe rimanenti
    if current_file_lines:
        # Se il file corrente non inizia con una riga speciale, aggiungi le ultime due righe speciali trovate nel file precedente
        if not current_file_lines[0].startswith("'"):
            current_file_lines = previous_special_lines + current_file_lines
        
        output_file = os.path.join(output_dir, f'{os.path.basename(input_file)}_part_{file_count}')
        with open(output_file, 'w') as out_file:
            out_file.writelines(current_file_lines)
        print(f'Creato file: {output_file}')
    
    returner = []
    for i in range(1, 1 + file_count):
        copia(f'{os.path.basename(input_file)}_part_{i}', output_dir, path_to_turboModels, f'{os.path.basename(input_file)}_part_{i}')
        copia(f'{os.path.basename(input_file)}_part_{i}', output_dir, path_to_turboModels+'lineliste/', f'{os.path.basename(input_file)}_part_{i}')
        returner.append([f'{os.path.basename(input_file)}_part_{i}', f'{os.path.basename(input_file)}_part_', i])
        print('AAII' + f'{os.path.basename(input_file)}_part_{i}')
        update_row_number(f'{(input_file)}_part_{i}')
        copia(f'{os.path.basename(input_file)}_part_{i}', path_to_turboModels+'lineliste/', path_to_turboModels, f'{os.path.basename(input_file)}_part_{i}')
    return returner
def list_files_with_prefix(directory, prefix):
    '''Restituisce una lista di tutti i file in una cartella che iniziano con un prefisso noto.'''
    files_with_prefix = []
    for filename in os.listdir(directory):
        if filename.startswith(prefix):
            files_with_prefix.append(filename)
    return files_with_prefix
def NLTE_intCorrection(wl, el, T, G, M, V):
    #maiuscole inserite, minuscole sulla griglia
    Tcheck = False
    Gcheck = False
    Mcheck = False
    Vcheck = False
    overT = []
    overG = []
    overM = []
    overV = []
    overT.append(NLTE_wlCorrection(wl, el, T, G, M, V)) #dà il valore della correzione nel punto più vicino sulla griglia
    overG.append(overT[0])
    overM.append(overT[0])
    overV.append(overT[0])
    t=find_closest(T, temperatura)
    g=find_closest(G, LOGG_arr)
    m=find_closest(M+7.5, METAL)-7.5
    v=find_closest(V, MICROTURB)
    if True:
        if (g==2.0) or (g==2.5):
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t=T
                t1=t

            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:
                #print("sgryb")
                m=M
                m1=m


            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    v1=v
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    #return 0
                else:
                    v1=v
                    print('ayyy lmao')
            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    v1=v
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    #return 0
                else:
                    v1=v


            if G>g:
                g1=g+0.5
            elif G<g:
                g1=g-0.5
            elif G==g:
                g1=find_closest(G-0.5, logg_arr)
        if g==1.5:
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t1=t


            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:

                m=M+0.01
                m1=find_closest(M-0.25+7.5, METAL)-7.5


            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    v1=v
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    #return 0
                else:
                    v1=v

            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    v1=v
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    #return 0
                else:
                    v1=v
                    print("oyyy vey!")

            if G>g:
                g1=g+0.5
            elif G<g:
                print("Non c'è logg sotto 1.5dex e sopra 3dex!")
                g1=g
            elif G==g:
                g1=find_closest(G-0.5, logg_arr)
        if g==3.0:
            if T>t:
                t1=find_closest(T+250, temperatura)
            elif T<t:
                t1=find_closest(T-250, temperatura)
            elif t==T:
                t=T
                t1=t

            if M>m:
                m1=find_closest(M+0.25+7.5, METAL)-7.5
            elif M<m:
                m1=find_closest(M-0.25+7.5, METAL)-7.5
            elif M==m:
                print("sgryb")
                m=M
                m1=m
            if m1 == 4.25 or m1 == 7.75:
                print("Non ci sono i dati sotto i 4.50dex e sopra i 7.50dex!")
                return 0

            if v==0.75:
                if V>v:
                    v1=1.5
                elif V<v:
                    print("Non ci sono i dati sotto i 0.75km/s e sopra i 3km/s!")
                    v1=v
                    #return 0
                else:
                    v1=v

            elif v==1.5:
                if V>=v:
                    v1=3.0
                elif V<v:
                    v1=0.75
            elif v==3.0:
                if V<v:
                    v1 = 1.5
                elif V>v:
                    print("Non ci sono i dati SOTTO i 0.75km/s e sopra i 3km/s!")
                    v1=v
                    #return 0
                else:
                    v1=v


            if G<g:
                g1=g-0.5
            elif G>=g:
                print(f"Non c'è logg sotto 1.5dex e SOPRA 3dex! {G}, {g}")
                g1=g


        if t1 == 3750 or t1 == 5750:
            print('Non ci sono dati sotto i 4000K o sopra i 5500K!')
            #return 0
        if m1 == 4.25 or m1 == 7.75:
            print("Non ci sono i dati sotto i 4.50dex e sopra i 7.50dex!")
            return 0
        if g1==1.5 or g == 1.5:
            if t1 == 3750 or t1 == 4750:
                print('Non ci sono dati SOTTO i 4000K o sopra i 4500K! (per questo logg)')
        if g1 == 3 or g == 3:
            if t1 == 4250 or t1 == 5750:
                print('Non ci sono dati sotto i 4500K o SOPRA i 5500K! (per questo logg)')
                #return 0
            if g==3:
                if g1 > g:
                    print(f'Non ci sono dati sopra ai 3dex di logg {G}, {g}, {g1}')
        
    #print(v1)
    print(f"true  close next {wl}\n{T}, {t}, {t1}\n{G},  {g},  {g1}\n{M},  {m},  {m1}\n{V},  {v},  {v1}")
    if g==1.5 and t1>4500:
        overT.append(NLTE_wlCorrection(wl, el, 4500, G, M, V))
    else:
        overT.append(NLTE_wlCorrection(wl, el, t1, G, M, V))
    if g==3.0 and t1<4500:
        overT.append(NLTE_wlCorrection(wl, el, 4500, G, M, V))
    else:
        overT.append(NLTE_wlCorrection(wl, el, t1, G, M, V))
    # if G < 1.5:
    #     if t1>4500:    
    #         overG.append(NLTE_wlCorrection(wl, el, 4500, 2, M, V)) #
    overG.append(NLTE_wlCorrection(wl, el, T, g1, M, V))
    overM.append(NLTE_wlCorrection(wl, el, T, G, m1, V))
    overV.append(NLTE_wlCorrection(wl, el, T, G, M, v1))

    #print(f"{overT} --- {overG} --- {overM} --- {overV}")
    if t != t1:
        corrt=(overT[1]-overT[0])/(max(t1, t) - min(t1, t))*(T-t)
    else:
        corrt=0
    if g != g1:
        corrg=(overG[1]-overG[0])/(max(g1, g) - min(g1, g))*(G-g)
    else:
        corrg=0
    if m != m1:
        corrm=(overM[1]-overM[0])/(max(m1, m) - min(m1, m))*(M-m)
    else:
        corrm=0
    if v != v1:
        corrv=(overV[1]-overV[0])/(max(v1, v) - min(v1, v))*(V-v)
    else:
        corrv=0
    corrT=overT[0]+corrt
    corrG=overG[0]+corrg
    corrM=overM[0]+corrm
    corrV=overV[0]+corrv
    totalCorr = corrt + corrg + corrm + corrv
    #print(f"{t}, {t1}, eheh {overT}, {corrT}\n{g}, {g1}, eheh {overG}, {corrG}\n{m}, {m1}, eheh {overM}, {corrM}\n{v}, {v1}, eheh {overV}, {corrV}\n")
    #print(f"originale: {overT[0]}, \ninterpol: {overT[0]+totalCorr}\nDelta:{totalCorr}")
    return [-1*overT[0], -1*(overT[0]+totalCorr), -1*totalCorr, wl]
# from main import *
def delete_files_with_prefix(directory, prefix):
    '''Elimina tutti i file in una cartella il cui nome inizia con un prefisso specificato.'''
    for filename in os.listdir(directory):
        if filename.startswith(prefix):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"File rimosso: {file_path}")
def delete_files(directory):
    """
    Cancella tutti i file che seguono il formato "PINOLOSx" (dove x è un numero) in una cartella specificata.

    Args:
        directory (str): Percorso della cartella in cui cercare i file.
    """
    try:
        # Controlla se la cartella esiste
        if not os.path.isdir(directory):
            print(f"Directory non trovata: {directory}")
            return

        # Itera sui file nella cartella
        for filename in os.listdir(directory):
            # Verifica se il nome del file corrisponde al pattern "PINOLOS" seguito da un numero
            if filename.startswith("PINOLOS") and filename[7:].isdigit():
                file_path = os.path.join(directory, filename)
                # Elimina il file
                os.remove(file_path)
                print(f"File cancellato: {file_path}")
                
    except Exception as e:
        print(f"Errore durante l'eliminazione dei file: {e}")
def delete_allfiles(directory):
    for file in os.listdir(directory):
        try:
            path = os.path.join(directory, file)
            if os.path.isfile(path):
                os.remove(path)
                print(f'File rimosso: {path}')
        except FileNotFoundError:
            continue
def fix_24th_line_35th_char(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    if len(lines) < 24:
        raise ValueError("Il file ha meno di 24 righe.")
    
    if len(lines[23]) < 35:
        raise ValueError("La 24esima riga ha meno di 35 caratteri.")
    
    if lines[23][34] == '-':
        lines[23] = lines[23][:34] + ' ' + lines[23][35:]
    elif lines[23][34] == ' ':
        return
    
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)
def fix_1st_line_35th_char(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    if len(lines) < 1:
        raise ValueError("Il file ha meno di 1 righe.")
    
    if len(lines[0]) < 35:
        raise ValueError("La 24esima riga ha meno di 35 caratteri.")
    
    if lines[0][34] == '-':
        lines[0] = lines[0][:34] + ' ' + lines[0][35:]
    elif lines[0][34] == ' ':
        return
    
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)
def creaTurboSpectrumScript_csh(processore=None, eqW_file=lineListT, linelist_part=1, wl_min = 4610, wl_max = 6730, wl_delta = 0.005, alpha_enhancement=0.1, TURBVEL = 1.5, modelName = 'atmoT475015', outname = None, metalScaling = -0.500, MOLECULE_LIST = 'LISTE_molecules_all_v12.1.dat', PURE_LTE='false', marcs='false', path_to_turboModel=path_to_turboModels, FIX_ABU=[[6, 'Cabu', 7.30]], ISOTOPE1=[[3.006, 0.07], [3.007, 0.93], [6.012, 1.00], [6.013, 0.00]], ISOTOPE2=[[3.006, 0.07], [3.007, 0.93], [6.012, 1.00], [6.013, 0.00], [56.134, 0.02417], [56.135, 0.06592], [56.136, 0.07854], [56.137, 0.1123], [56.138, 0.7170], [63.151, 0.478], [63.153, 0.522]]):
    '''Crea uno script csh per eseguire il programma TurboSpectrum.
    Args: processore: var di supporto per la parallelizzazione, se messa a 0 gira su singolo core\n
    eqW_file: Nome del file contenente le righe di assorbimento e relative larghezze equivalenti\n
    alpha_enhancement: Sovrabbondanza degli elementi alpha\n
    TURBVEL: Velocità di microturbolenza\n
    wl_min: inizio intervallo lunghezze d'onda\n
    wl_max: fine intervallo lunghezze d'onda\n
    wl_delta: passo lunghezze d'onda\n
    modelName: Nome del modello atmosferico\n
    metalScaling: Metallicità\n
    MOLECULE_LIST: Lista delle righe molecolari\n
    PURE_LTE: tieni false perché sì\n
    marcs: true come stringa se si utilizza un modello MARCS\n
    FIX_ABU: Una lista contenente gli elementi che vogliamo fissare nella creazione di un atmosfera. Specificare il numero della specie e una stringa con il nome dell'elemento che sarà il nome della variabile nello script csh e il valore dell'abbondanza, di default un esempio per il carbonio\n'''
    os.chdir(path_to_turboModel)
    if processore == 0 or processore == None:
        with open('Turbo_EW.csh', 'w') as f:
            # Definisce le variabili
            f.write('#!/bin/csh -Vf\n\n')
            f.write(f'set Turbo_HOME = {path_to_turbospectrum}\nset mpath = {path_to_turboModel}\n')
            f.write(r'set dpath = ${Turbo_HOME}/DATA')
            f.write('\nset ewfile = ${mpath}')
            f.write(f'{eqW_file}\n')
            f.write('\n')
            f.write(r'ln -s ${dpath} DATA ')
            f.write('\n\n')
            f.write('echo ${dpath}\n\n')
            f.write('mkdir -p contopac \nmkdir -p outdata\nmkdir -p syntspec\n\n\ndate\n\n')
            for i in range(len(FIX_ABU)):
                f.write(f'set {FIX_ABU[i][1]} = \'{FIX_ABU[i][2]:.2f}\'\n')
            f.write('\n\n')
            if metalScaling < -1.5:
                alpha_enhancement=0.4
            f.write(f'set  alpha = {alpha_enhancement}\n')
            f.write(f'set  TURBVEL = {TURBVEL}\n')
            f.write(f'set  lam_min = {wl_min}\n')
            f.write(f'set  lam_max = {wl_max}\n')
            f.write(f'set   deltalam = \'{wl_delta}\'\nset MODEL      = \'{modelName}\'\n')
            f.write('ln -s ${mpath}/${MODEL}\n')
            f.write(f'set  METALLIC   = \'{metalScaling}\'\n')
            f.write('set SUFFIX     = _${lam_min}_${lam_max}V${TURBVEL}A${alpha}C_${Cabu}'+f'_{linelist_part}'+'.dat\n')
            if outname == None:
                f.write('set result     = ${ewfile}S${MODEL}${SUFFIX}\n')
            elif isinstance(outname, str):
                f.write(f'set result     = {outname}\n')
            else:
                print("Errore: outname deve essere una stringa")
            f.write('set MOLECULE_LIST = ' + f'\'{MOLECULE_LIST}\'\n\n\n')
            #prima run per l'opacità continua
            f.write('${Turbo_HOME}/exec-gf-v19.1/babsma_lu << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'MODELINPUT:\'  \'${MODEL}\'\n')
            f.write('\'MARCS-FILE:\' '+f'\'{marcs}\'\n')
            f.write('\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE1)}\'\n')
            for i in range(len(ISOTOPE1)):
                f.write(f' {ISOTOPE1[i][0]} {ISOTOPE1[i][1]:.2f}\n')      
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n\'XIFIX:\' \'T\'\n$TURBVEL\n\'CONTINUOUS-OPACITIES :\' \'${dpath}/jonabs_vac_v07.3.dat\'\n\'LOGFILE:\' \'outdata/blistaa.dat\'\nEOF\n\n\n##################################################\n')
            #seconda run per le righe
            f.write('${Turbo_HOME}/exec-gf-v19.1/eqwidt_lu << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'INTENSITY/FLUX:\' \'Flux\'\n\'COS(THETA)    :\' \'1.00\'\n\'ABFIND        :\' \'.true.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'RESULTFILE :\' \'${result}\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'R-PROCESS  :\'    \'0.00\'\n\'S-PROCESS  :\'    \'0.00\'\n')
            f.write('\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE2)}\'\n')
            for i in range(len(ISOTOPE2)):
                f.write(f' {ISOTOPE2[i][0]} {ISOTOPE2[i][1]:.2f}\n')
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n')
            f.write('\'NFILES   :\' \'1\'\n${ewfile}\n\'SPHERICAL:\'  \'F\'\n 30\n 300.00 \n 15\n 1.30\n\'LOGFILE:\' \'outdata/blistbb.dat\'\n\'MULTIDUMP:\'  \'F\'\nEOF')
    elif processore > 0:
        print(os.getcwd())
        with open(f'Turbo_EW{processore}.csh', 'w') as f:
            # Definisce le variabili
            f.write('#!/bin/csh -Vf\n\n')
            f.write(f'set Turbo_HOME = {path_to_turbospectrum}\nset mpath = {path_to_turboModel}\n')
            f.write(r'set dpath = ${Turbo_HOME}/DATA'+f'{processore}')
            f.write('\nset ewfile = ${mpath}')
            f.write(f'{eqW_file}\n')
            f.write('\n')
            f.write(r'ln -s ${dpath} DATA ')
            f.write('\n\n')
            f.write('echo ${dpath}\n\n')
            f.write('mkdir -p contopac \nmkdir -p outdata\nmkdir -p syntspec\n\n\ndate\n\n')
            for i in range(len(FIX_ABU)):
                f.write(f'set {FIX_ABU[i][1]} = \'{FIX_ABU[i][2]:.2f}\'\n')
            f.write('\n\n')
            f.write(f'set  alpha = {alpha_enhancement}\n')
            f.write(f'set  TURBVEL = {TURBVEL}\n')
            f.write(f'set  lam_min = {wl_min}\n')
            f.write(f'set  lam_max = {wl_max}\n')
            f.write(f'set   deltalam = \'{wl_delta}\'\nset MODEL      = \'{modelName}{processore}\'\n')
            f.write('ln -s ${mpath}/${MODEL}\n')
            f.write(f'set  METALLIC   = \'{metalScaling}\'\n')
            f.write('set SUFFIX     = _${lam_min}_${lam_max}V${TURBVEL}A${alpha}C_${Cabu}'+f'_{processore}'+f'_{linelist_part}'+'.dat\n')
            if outname == None:
                f.write('set result     = ${ewfile}S${MODEL}${SUFFIX}\n')
            elif isinstance(outname, str):
                f.write(f'set result     = {outname}\n')
            else:
                print("Errore: outname deve essere una stringa")
            f.write('set MOLECULE_LIST = ' + f'\'{MOLECULE_LIST}\'\n\n\n')
            #prima run per l'opacità continua
            f.write('${Turbo_HOME}/exec-gf-v19.1/babsma_lu'+f'{processore}'+' << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'MODELINPUT:\'  \'${MODEL}\'\n')
            f.write('\'MARCS-FILE:\' '+f'\'{marcs}\'\n')
            f.write('\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE1)}\'\n')
            for i in range(len(ISOTOPE1)):
                f.write(f' {ISOTOPE1[i][0]} {ISOTOPE1[i][1]:.2f}\n')      
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n\'XIFIX:\' \'T\'\n$TURBVEL\n\'CONTINUOUS-OPACITIES :\' \'${dpath}/jonabs_vac_v07.3'+f'_{processore}'+'.dat\'\n\'LOGFILE:\' \'outdata/blistaa'+f'{processore}'+'.dat\'\nEOF\n\n\n##################################################\n')
            #seconda run per le righe
            f.write('${Turbo_HOME}/exec-gf-v19.1/eqwidt_lu'+f'{processore}'+' << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'INTENSITY/FLUX:\' \'Flux\'\n\'COS(THETA)    :\' \'1.00\'\n\'ABFIND        :\' \'.true.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'RESULTFILE :\' \'${result}\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'R-PROCESS  :\'    \'0.00\'\n\'S-PROCESS  :\'    \'0.00\'\n')
            f.write('\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE2)}\'\n')
            for i in range(len(ISOTOPE2)):
                f.write(f' {ISOTOPE2[i][0]} {ISOTOPE2[i][1]:.2f}\n')
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n')
            f.write('\'NFILES   :\' \'1\'\n${ewfile}\n\'SPHERICAL:\'  \'F\'\n 30\n 300.00 \n 15\n 1.30\n\'LOGFILE:\' \'outdata/blistbb.dat\'\n\'MULTIDUMP:\'  \'F\'\nEOF')
def creaTurboSpectrumScript_csh2(processore=None, eqW_file=lineListT, linelist_part=1, wl_min = 4610, wl_max = 6730, wl_delta = 0.005, alpha_enhancement=0.1, TURBVEL = 1.5, modelName = 'atmoT475015', outname = None, metalScaling = -0.500, MOLECULE_LIST = 'LISTE_molecules_all_v12.1.dat', PURE_LTE='false', marcs='false', path_to_turboModel=path_to_turboModels, FIX_ABU=[[6, 'Cabu', 7.30]], ISOTOPE1=[[3.006, 0.07], [3.007, 0.93], [6.012, 1.00], [6.013, 0.00]], ISOTOPE2=[[3.006, 0.07], [3.007, 0.93], [6.012, 1.00], [6.013, 0.00], [56.134, 0.02417], [56.135, 0.06592], [56.136, 0.07854], [56.137, 0.1123], [56.138, 0.7170], [63.151, 0.478], [63.153, 0.522]]):
    '''Crea uno script csh per eseguire il programma TurboSpectrum.
    Args: processore: var di supporto per la parallelizzazione, se messa a 0 gira su singolo core\n
    eqW_file: Nome del file contenente le righe di assorbimento e relative larghezze equivalenti\n
    alpha_enhancement: Sovrabbondanza degli elementi alpha\n
    TURBVEL: Velocità di microturbolenza\n
    wl_min: inizio intervallo lunghezze d'onda\n
    wl_max: fine intervallo lunghezze d'onda\n
    wl_delta: passo lunghezze d'onda\n
    modelName: Nome del modello atmosferico\n
    metalScaling: Metallicità\n
    MOLECULE_LIST: Lista delle righe molecolari\n
    PURE_LTE: tieni false perché sì\n
    marcs: true come stringa se si utilizza un modello MARCS\n
    FIX_ABU: Una lista contenente gli elementi che vogliamo fissare nella creazione di un atmosfera. Specificare il numero della specie e una stringa con il nome dell'elemento che sarà il nome della variabile nello script csh e il valore dell'abbondanza, di default un esempio per il carbonio\n'''
    os.chdir(path_to_turboModel)
    if processore == 0 or processore == None:
        with open('Turbo_EW.csh', 'w') as f:
            # Definisce le variabili
            f.write('#!/bin/csh -Vf\n\n')
            f.write(f'set Turbo_HOME = {path_to_turbospectrum}\nset mpath = {path_to_turboModel}\n')
            f.write(r'set dpath = ${Turbo_HOME}/DATA')
            f.write('\nset ewfile = ${mpath}')
            f.write(f'{eqW_file}\n')
            f.write('\n')
            f.write(r'ln -s ${dpath} DATA ')
            f.write('\n\n')
            f.write('echo ${dpath}\n\n')
            f.write('mkdir -p contopac \nmkdir -p outdata\nmkdir -p syntspec\n\n\ndate\n\n')
            for i in range(len(FIX_ABU)):
                f.write(f'set {FIX_ABU[i][1]} = \'{FIX_ABU[i][2]:.2f}\'\n')
            f.write('\n\n')
            if metalScaling < -1.5:
                alpha_enhancement=0.4
            f.write(f'set  alpha = {alpha_enhancement}\n')
            f.write(f'set  TURBVEL = {TURBVEL}\n')
            f.write(f'set  lam_min = {wl_min}\n')
            f.write(f'set  lam_max = {wl_max}\n')
            f.write(f'set   deltalam = \'{wl_delta}\'\nset MODEL      = \'{modelName}\'\n')
            f.write('ln -s ${mpath}/${MODEL}\n')
            f.write(f'set  METALLIC   = \'{metalScaling}\'\n')
            f.write('set SUFFIX     = _${lam_min}_${lam_max}V${TURBVEL}A${alpha}C_${Cabu}'+f'_{linelist_part}'+'.dat\n')
            if outname == None:
                f.write('set result     = ${ewfile}S${MODEL}${SUFFIX}\n')
            elif isinstance(outname, str):
                f.write(f'set result     = {outname}\n')
            else:
                print("Errore: outname deve essere una stringa")
            f.write('set MOLECULE_LIST = ' + f'\'{MOLECULE_LIST}\'\n\n\n')
            #prima run per l'opacità continua
            f.write('${Turbo_HOME}/exec-gf-v19.1/babsma_lu << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'MODELINPUT:\'  \'${MODEL}\'\n')
            f.write('\'MARCS-FILE:\' '+f'\'{marcs}\'\n')
            f.write('\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE1)}\'\n')
            for i in range(len(ISOTOPE1)):
                f.write(f' {ISOTOPE1[i][0]} {ISOTOPE1[i][1]:.2f}\n')      
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n\'XIFIX:\' \'T\'\n$TURBVEL\n\'CONTINUOUS-OPACITIES :\' \'${dpath}/jonabs_vac_v07.3.dat\'\n\'LOGFILE:\' \'outdata/blistaa.dat\'\nEOF\n\n\n##################################################\n')
            #seconda run per le righe
            f.write('${Turbo_HOME}/exec-gf-v19.1/eqwidt_lu << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'INTENSITY/FLUX:\' \'Flux\'\n\'COS(THETA)    :\' \'1.00\'\n\'ABFIND        :\' \'.true.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'RESULTFILE :\' \'${result}\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'R-PROCESS  :\'    \'0.00\'\n\'S-PROCESS  :\'    \'0.00\'\n')
            f.write('\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE2)}\'\n')
            for i in range(len(ISOTOPE2)):
                f.write(f' {ISOTOPE2[i][0]} {ISOTOPE2[i][1]:.2f}\n')
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n')
            f.write('\'NFILES   :\' \'1\'\n${ewfile}\n\'SPHERICAL:\'  \'F\'\n 30\n 300.00 \n 15\n 1.30\n\'LOGFILE:\' \'outdata/blistbb.dat\'\n\'MULTIDUMP:\'  \'F\'\nEOF')
    elif processore > 0:
        print(os.getcwd())
        with open(f'Turbo_EW{processore}.csh', 'w') as f:
            # Definisce le variabili
            f.write('#!/bin/csh -Vf\n\n')
            f.write(f'set Turbo_HOME = {path_to_turbospectrum}\nset mpath = {path_to_turboModel}\n')
            f.write(r'set dpath = ${Turbo_HOME}/DATA'+f'{processore}')
            f.write('\nset ewfile = ${mpath}')
            f.write(f'{eqW_file}\n')
            f.write('\n')
            f.write(r'ln -s ${dpath} DATA ')
            f.write('\n\n')
            f.write('echo ${dpath}\n\n')
            f.write('mkdir -p contopac \nmkdir -p outdata\nmkdir -p syntspec\n\n\ndate\n\n')
            for i in range(len(FIX_ABU)):
                f.write(f'set {FIX_ABU[i][1]} = \'{FIX_ABU[i][2]:.2f}\'\n')
            f.write('\n\n')
            f.write(f'set  alpha = {alpha_enhancement}\n')
            f.write(f'set  TURBVEL = {TURBVEL}\n')
            f.write(f'set  lam_min = {wl_min}\n')
            f.write(f'set  lam_max = {wl_max}\n')
            f.write(f'set   deltalam = \'{wl_delta}\'\nset MODEL      = \'{modelName}{processore}\'\n')
            f.write('ln -s ${mpath}/${MODEL}\n')
            f.write(f'set  METALLIC   = \'{metalScaling}\'\n')
            f.write('set SUFFIX     = _${lam_min}_${lam_max}V${TURBVEL}A${alpha}C_${Cabu}'+f'_{processore}'+f'_{linelist_part}'+'.dat\n')
            if outname == None:
                f.write('set result     = ${ewfile}S${MODEL}${SUFFIX}\n')
            elif isinstance(outname, str):
                f.write(f'set result     = {outname}\n')
            else:
                print("Errore: outname deve essere una stringa")
            f.write('set MOLECULE_LIST = ' + f'\'{MOLECULE_LIST}\'\n\n\n')
            #prima run per l'opacità continua
            f.write('${Turbo_HOME}/exec-gf-v19.1/babsma_lu'+f'{processore}'+' << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'MODELINPUT:\'  \'${MODEL}\'\n')
            f.write('\'MARCS-FILE:\' '+f'\'{marcs}\'\n')
            f.write('\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE1)}\'\n')
            for i in range(len(ISOTOPE1)):
                f.write(f' {ISOTOPE1[i][0]} {ISOTOPE1[i][1]:.2f}\n')      
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n\'XIFIX:\' \'T\'\n$TURBVEL\n\'CONTINUOUS-OPACITIES :\' \'${dpath}/jonabs_vac_v07.3'+f'_{processore}'+'.dat\'\n\'LOGFILE:\' \'outdata/blistaa'+f'{processore}'+'.dat\'\nEOF\n\n\n##################################################\n')
            #seconda run per le righe
        with open(f'Turbo2_EW{processore}.csh', 'w') as f:
            f.write('#!/bin/csh -Vf\n\n')
            f.write(f'set Turbo_HOME = {path_to_turbospectrum}\nset mpath = {path_to_turboModel}\n')
            f.write(r'set dpath = ${Turbo_HOME}/DATA'+f'{processore}')
            f.write('\nset ewfile = ${mpath}')
            f.write(f'{eqW_file}\n')
            f.write('\n')
            f.write(r'ln -s ${dpath} DATA ')
            f.write('\n\n')
            f.write('echo ${dpath}\n\n')
            f.write('mkdir -p contopac \nmkdir -p outdata\nmkdir -p syntspec\n\n\ndate\n\n')
            for i in range(len(FIX_ABU)):
                f.write(f'set {FIX_ABU[i][1]} = \'{FIX_ABU[i][2]:.2f}\'\n')
            f.write('\n\n')
            f.write(f'set  alpha = {alpha_enhancement}\n')
            f.write(f'set  TURBVEL = {TURBVEL}\n')
            f.write(f'set  lam_min = {wl_min}\n')
            f.write(f'set  lam_max = {wl_max}\n')
            f.write(f'set   deltalam = \'{wl_delta}\'\nset MODEL      = \'{modelName}{processore}\'\n')
            f.write('ln -s ${mpath}/${MODEL}\n')
            f.write(f'set  METALLIC   = \'{metalScaling}\'\n')
            f.write('set SUFFIX     = _${lam_min}_${lam_max}V${TURBVEL}A${alpha}C_${Cabu}'+f'_{processore}'+f'_{linelist_part}'+'.dat\n')
            if outname == None:
                f.write('set result     = ${ewfile}S${MODEL}${SUFFIX}\n')
            elif isinstance(outname, str):
                f.write(f'set result     = {outname}\n')
            else:
                print("Errore: outname deve essere una stringa")
            f.write('set MOLECULE_LIST = ' + f'\'{MOLECULE_LIST}\'\n\n\n')
            #Run
            f.write('${Turbo_HOME}/exec-gf-v19.1/eqwidt_lu'+f'{processore}'+' << EOF\n\'PURE-LTE:\' ' + f'   \'.{PURE_LTE}.\'\n\'LAMBDA_MIN:\' ' + ' \'${lam_min}\'\n')
            f.write('\'LAMBDA_MAX:\'  \'${lam_max}\'\n')
            f.write('\'LAMBDA_STEP:\' \'${deltalam}\'\n')
            f.write('\'INTENSITY/FLUX:\' \'Flux\'\n\'COS(THETA)    :\' \'1.00\'\n\'ABFIND        :\' \'.true.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'RESULTFILE :\' \'${result}\'\n\'METALLICITY:\'    \'${METALLIC}\'\n\'ALPHA/Fe   :\'    \'${alpha}\'\n\'HELIUM     :\'    \'0.00\'\n\'R-PROCESS  :\'    \'0.00\'\n\'S-PROCESS  :\'    \'0.00\'\n')
            f.write('\'INDIVIDUAL ABUNDANCES:\'   ' + f'\'{len(FIX_ABU)}\'\n')
            for i in range(len(FIX_ABU)):
                f.write(f'{FIX_ABU[i][0]} $'+'{'+f'{FIX_ABU[i][1]}'+'}\n')
            f.write('\'ISOTOPES : \' ' + f'\'{len(ISOTOPE2)}\'\n')
            for i in range(len(ISOTOPE2)):
                f.write(f' {ISOTOPE2[i][0]} {ISOTOPE2[i][1]:.2f}\n')
            f.write('\'INCLUDE FOLLOWING ATOMS:\' \'10\'  \'(lines to read)\'\n\'H \' \'He\' \'Li\' \'Be\' \'B \' \'C \' \'N \' \'O \' \'F \' \'Ne\' \n\'Na\' \'Mg\' \'Al\' \'Si\' \'P \' \'S \' \'Cl\' \'Ar\' \'K \' \'Ca\' \n\'Sc\' \'Ti\' \'V \' \'Cr\' \'Mn\' \'Fe\' \'Co\' \'Ni\' \'Cu\' \'Zn\' \n\'Ga\' \'Ge\' \'As\' \'Se\' \'Br\' \'Kr\' \'Rb\' \'Sr\' \'Y \' \'Zr\' \n\'Nb\' \'Mo\' \'Tc\' \'Ru\' \'Rh\' \'Pd\' \'Ag\' \'Cd\' \'In\' \'Sn\' \n\'Sb\' \'Te\' \'I \' \'Xe\' \'Cs\' \'Ba\' \'La\' \'Ce\' \'Pr\' \'Nd\' \n\'Pm\' \'Sm\' \'Eu\' \'Gd\' \'Tb\' \'Dy\' \'Ho\' \'Er\' \'Tm\' \'Yb\' \n\'Lu\' \'Hf\' \'Ta\' \'W \' \'Re\' \'Os\' \'Ir\' \'Pt\' \'Au\' \'Hg\' \n\'Tl\' \'Pb\' \'Bi\' \'Po\' \'At\' \'Rn\' \'Fr\' \'Ra\' \'Ac\' \'Th\' \n\'Pa\' \'U \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \'  \' \n')
            f.write('\'MOLECULES :\' \'${dpath}/${MOLECULE_LIST}\'\n')
            f.write('\'NFILES   :\' \'1\'\n${ewfile}\n\'SPHERICAL:\'  \'F\'\n 30\n 300.00 \n 15\n 1.30\n\'LOGFILE:\' \'outdata/blistbb.dat\'\n\'MULTIDUMP:\'  \'F\'\nEOF')
def process_linelist(input_file, output_file, comparison_number, new_value, delta=0.01):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    processed_lines = []
    for line in lines:
        if line.startswith("'"):
            processed_lines.append(line)
        else:
            elements = line.split()
            if len(elements) >= 9:
                try:
                    first_element = float(elements[0])
                    if abs(first_element - comparison_number) <= delta:
                        elements[8] = str(new_value)
                except ValueError:
                    pass
            processed_lines.append(' '.join(elements) + '\n')
    with open(output_file, 'w') as file:
        file.writelines(processed_lines)
def run_script2(script_list, script_list2, processori=12, sub_linelist_number=4, eqW_file=[lineListT], 
                outname=None, alpha_enhancement=0.4, TURBVEL=[1.5], modelName='atmoT475015', 
                metalScaling=[-0.500], path_to_turboModels=path_to_turboModels):#path_to_turboModels="."):
    """Lancia gli script .csh in parallelo e aspetta che thresholdfiniscano prima di continuare."""

    os.chdir(path_to_turboModels)
    
    for j in range(sub_linelist_number):  # j = parti in cui è stata divisa la linelist
        processes = []  # Lista processi per questa iterazione

        # Creazione e lancio degli script
        if processori == 0:
            Outname = f"{outname}pr{0}pt{j+1}" if outname else None
            #wl_min, wl_max = wl_extremes(os.path.join(path_to_turboModels, eqW_file[j]))
            wl_min, wl_max = wl_extremes(path_to_turboModels + eqW_file[j])

            creaTurboSpectrumScript_csh2(0, eqW_file=eqW_file[j], linelist_part=j+1, 
                                        wl_min=wl_min-15, wl_max=wl_max+15, 
                                        alpha_enhancement=alpha_enhancement, 
                                        TURBVEL=TURBVEL[0], modelName=modelName, 
                                        outname=Outname, metalScaling=metalScaling[0])

            print(f"Creato script per il programma 0")
        else:
            for i in range(1, processori + 1):  # i = numero di processori
                Outname = f"{outname}pr{i}pt{j+1}" if outname else None
                print('in run_script2')
                print(path_to_turboModels + eqW_file[j])
                print('in run_script2')
                wl_min, wl_max = wl_extremes(path_to_turboModels + eqW_file[j])

                creaTurboSpectrumScript_csh2(i, eqW_file=eqW_file[j], linelist_part=j+1, 
                                            wl_min=wl_min-15, wl_max=wl_max+15, 
                                            alpha_enhancement=alpha_enhancement, 
                                            TURBVEL=TURBVEL[i-1], modelName=modelName, 
                                            outname=Outname, metalScaling=metalScaling[i-1])

                print(f"Creato script per il programma {i}")

        # Avviare gli script in parallelo
        if processori > 0:
            scripts_to_run = script_list[:processori]
        else:
            scripts_to_run = script_list
        for script in scripts_to_run:
            try:
                proc = subprocess.Popen(["csh", script], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                processes.append(proc)
            except Exception as e:
                print(f"Errore nell'avviare {script}: {e}")

        # Attendere che tutti i processi terminino
        for proc in processes:
            stdout, stderr = proc.communicate()  # Blocca l'esecuzione finché il processo non termina
            print(f"Processo {proc.args} terminato con codice {proc.returncode}")
            if stdout:
                print(f"Output:\n{stdout}")
            if stderr:
                print(f"Errori:\n{stderr}")

        print(f"Tutti gli script per linelist {j+1} hanno terminato.")

        print("Tutti gli script della prima lista hanno terminato l'esecuzione.")

        # Esecuzione degli script della seconda lista
        processes = []  # Lista processi per la seconda lista
        if processori > 0:
            scripts_to_run = script_list2[:processori]
        else:
            scripts_to_run = script_list2
        for script in scripts_to_run:
            try:
                proc = subprocess.Popen(["csh", script], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                processes.append(proc)
            except Exception as e:
                print(f"Errore nell'avviare {script}: {e}")

        # Attendere che tutti i processi terminino
        for proc in processes:
            stdout, stderr = proc.communicate()  # Blocca l'esecuzione finché il processo non termina
            print(f"Processo {proc.args} terminato con codice {proc.returncode}")
            if stdout:
                print(f"Output:\n{stdout}")
            if stderr:
                print(f"Errori:\n{stderr}")

        print("Tutti gli script della seconda lista hanno terminato l'esecuzione.")

def run_script(script_list, processori=12, sub_linelist_number=4, eqW_file=[lineListT], 
                outname=None, alpha_enhancement=0.4, TURBVEL=[1.5], modelName='atmoT475015', 
                metalScaling=[-0.500], path_to_turboModels="."):
    """Lancia gli script .csh in parallelo e aspetta che finiscano prima di continuare."""

    os.chdir(path_to_turboModels)
    
    for j in range(sub_linelist_number):  # j = parti in cui è stata divisa la linelist
        processes = []  # Lista processi per questa iterazione

        # Creazione e lancio degli script
        if processori==0:
            Outname = f"{outname}pr{0}pt{j+1}" if outname else None
            wl_min, wl_max = wl_extremes(os.path.join(path_to_turboModels, eqW_file[j]))

            creaTurboSpectrumScript_csh(0, eqW_file=eqW_file[j], linelist_part=j+1, 
                                        wl_min=wl_min-15, wl_max=wl_max+15, 
                                        alpha_enhancement=alpha_enhancement, 
                                        TURBVEL=TURBVEL[0], modelName=modelName, 
                                        outname=Outname, metalScaling=metalScaling[0])

            print(f"Creato script per il programma 0")
        else:
            for i in range(1, processori + 1):  # i = numero di processori
                Outname = f"{outname}pr{i}pt{j+1}" if outname else None
                wl_min, wl_max = wl_extremes(os.path.join(path_to_turboModels, eqW_file[j]))

                creaTurboSpectrumScript_csh(i, eqW_file=eqW_file[j], linelist_part=j+1, 
                                            wl_min=wl_min-15, wl_max=wl_max+15, 
                                            alpha_enhancement=alpha_enhancement, 
                                            TURBVEL=TURBVEL[i-1], modelName=modelName, 
                                            outname=Outname, metalScaling=metalScaling[i-1])

                print(f"Creato script per il programma {i}")

        # Avviare gli script in parallelo
        if processori > 0:
            script_list2 = script_list[:processori]
        else:
            script_list2 = script_list
        for script in script_list2:
            try:
                proc = subprocess.Popen(["csh", script], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                processes.append(proc)
            except Exception as e:
                print(f"Errore nell'avviare {script}: {e}")

        # Attendere che tutti i processi terminino
        for proc in processes:
            stdout, stderr = proc.communicate()  # Blocca l'esecuzione finché il processo non termina
            print(f"Processo {proc.args} terminato con codice {proc.returncode}")
            if stdout:
                print(f"Output:\n{stdout}")
            if stderr:
                print(f"Errori:\n{stderr}")

        print(f"Tutti gli script per linelist {j+1} hanno terminato.")

    print("Tutti gli script hanno terminato l'esecuzione.")

def remove_lines_with_ninth_element(input_file, output_file, value_to_remove=-1.0):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    processed_lines = []
    
    for line in lines:
        # Dividi la riga in elementi
        elements = line.split()
        if len(elements) >= 9:
            try:
                # Controlla se il nono elemento è uguale a value_to_remove
                ninth_element = float(elements[8])
                if ninth_element != value_to_remove:
                    processed_lines.append(line)
            except ValueError:
                processed_lines.append(line)
        else:
            processed_lines.append(line)
    
    # Scrivi le righe processate nel file di output
    with open(output_file, 'w') as file:
        file.writelines(processed_lines)
def preparaTurboAbundance(stellaFITS, path_to_abundance=path_to_abundance, eq20w_low=15, eqw_high=150, lineList=lineList, turboList=path_to_turboModels+'lineliste/'+lineListT, instrument='FIES'):
    '''Prepara le liste con le equivalent width usando ARES. \n
    Restituisce una lista contenente i nomi delle linelist segmentate nella cartella path_to_turboModels. (ad esempio una linelist da 140 righe diventa 4 linelist da 35 righe)'''
    # preparaAbundance(stellaFITS, path_to_abundance, eqw_low, eqw_high, lineList, rad_solver = 'turbo')
    # os.chdir(path_to_turboModels)
    # ta = Table.read('stellaFes', format='ascii')
    # x=ta['col1']
    # y=ta['col8']
    # output = turboList+'.proc'
    # process_linelist(turboList, output, x[0], y[0])
    # for i in range(1, len(x)):
    #     process_linelist(output, output, x[i], y[i])
    # remove_lines'turbo_lines.ew'_with_ninth_element(output, output)
    # ew_part_number = split_file(input_file=output, lines_per_file=1500)
    print()
    creaLineList(fits=stellaFITS, rad_solver='turbo', line_list='ESO_FE_turbo', instrument=instrument)
    sposta('turbo_lines.ew', path_to_ARES, path_to_turboModels, 'turbo_lines.ew')
    return [['turbo_lines.ew']]
    #return ew_part_number

def crea_photometric_parameters_inputFile(Source, RAdeg, DEdeg, Plx, Gmag, BP_RP, zp, A0, met):
    os.chdir()
    if isinstance(Source, (int, float, complex)):  # Controlla se è un singolo valore numerico
        Source=[Source]
    if isinstance(RAdeg, (int, float, complex)):
        RAdeg=[RAdeg]
    if isinstance(DEdeg, (int, float, complex)):
        DEdeg=[DEdeg]
    if isinstance(Plx, (int, float, complex)):
        Plx=[Plx]
    if isinstance(Gmag, (int, float, complex)):
        Gmag=[Gmag]
    if isinstance(BP_RP, (int, float, complex)):
        BP_RP=[BP_RP]
    if isinstance(zp, (int, float, complex)):
        zp=[zp]
    if isinstance(A0, (int, float, complex)):
        A0=[A0]
    if isinstance(met, (int, float, complex)):
        met=[met]
    
    tmpdir=os.getcwd()
    os.chdir(path_to_param_GaiaDR3AM_giants_A0)

    with open('input_par.dat', mode='w') as f:
        f.write('# Source              RAdeg           DEdeg           Plx    Gmag      BP-RP    zp        A0        met \n')
        if len(Source)==len(RAdeg) and len(Source)==len(DEdeg) and len(Source)==len(Plx) and len(Source)==len(Gmag) and len(Source)==len(BP_RP) and len(Source)==len(zp) and len(Source)==len(A0) and len(Source)==len(met):
            for i in range(len(Source)):
                f.write(f'{Source[i]} {RAdeg[i]} {DEdeg[i]} {Plx[i]} {Gmag[i]} {BP_RP[i]} {zp[i]} {A0[i]} {met[i]}\n')
            os.chdir(os.chdir(tmpdir))
            return 0
        else:
            print(f'Wronge table dimensions!\n\n lenghts:\nSource: {len(Source)}\n RAdeg:  {len(RAdeg)}\nDEdeg:  {len(DEdeg)}\nPlx:  {len(Pls)}\nGmag:  {len(Gmag)}\nBP-RP:  {len(BP_RP)}\nzp:  {len(zp)}\nA0:  {len(A0)}\nmet:  {len(met)}\n')
            os.chdir(os.chdir(tmpdir))
            return 0

def lanciaTurbospectrum2(stellaFITS, processore=12, eqW_file=lineListT, wl_min = 4610, wl_max = 6730, wl_delta = 0.005, alpha_enhancement=0.1, TURBVEL = [1.5], modelName = 'atmoT475015', outname = None, metalScaling=[-0.500], linelist=lineList):
    ewfiles = preparaTurboAbundance(stellaFITS=stellaFITS)
    eqW_files = []
    for i in range(len(ewfiles)):
        eqW_files.append(ewfiles[i][0])
    #print(eqW_files)
    if processore > 0:
        run_script2(script_list=script_list, script_list2=script_list2, processori=processore,sub_linelist_number=(len(ewfiles)), eqW_file=eqW_files, alpha_enhancement=alpha_enhancement, TURBVEL=TURBVEL, modelName=modelName, outname=outname, metalScaling=metalScaling)
    else:
        run_script2(script_list=['Turbo_EW.csh'], script_list2=['Turbo2_EW.csh'], processori=processore,sub_linelist_number=(len(ewfiles)), eqW_file=eqW_files, alpha_enhancement=alpha_enhancement, TURBVEL=TURBVEL, modelName=modelName, outname=outname, metalScaling=metalScaling)

def wl_extremes(file_path, threshold=3500):
    '''Restituisce le lunghezze d'onda minima e massima presenti in un file.'''
    first_entries = []
    print('wl_extremes')
    print(file_path)
    print('wl_extremes')
    data=read_file_to_list(file_path)
    for element in data:
        if len(element) > 0:
            match = re.search(r'[-+]?\d*\.\d+|\d+', element)
            if match:
                try:
                    first_entry = float(match.group())
                    if first_entry > threshold:
                        first_entries.append(first_entry)
                except ValueError:
                    continue

    if first_entries:
        return min(first_entries), max(first_entries)
    else:
        return None, None
def update_integer_suffix(line, new_value):
    '''Aggiorna il valore dell'intero alla fine della riga, lasciando invariato il resto della riga.'''
    pattern = re.compile(r'(\d+)\n$')
    match = pattern.search(line)
    
    if match:
        # Sostituisci il vecchio valore con il nuovo valore
        updated_line = line[:match.start(1)] + str(new_value) + '\n'
        return updated_line
    else:
        # Se la riga non termina con un intero seguito da \n, restituisci la riga originale
        return line
def conta_righe_totali(file_path):
    with open(file_path, 'r') as f:
        return sum(1 for _ in f)
def update_row_number(file_path):
    '''Aggiorna il numero di riga in un file.'''
    lines = read_file_to_list(file_path)
    indices = find_indices_with_integer_suffix(lines)
    if len(indices) > 1:
        for i in range(len(indices)):
            if i+1 < len(indices):
                lines[indices[i]] = update_integer_suffix(lines[indices[i]], indices[i+1]-indices[i]-2)
            else:
                lines[indices[i]] = update_integer_suffix(lines[indices[i]], 35+2*len(indices)-(indices[i]+2))
    elif len(indices) == 1:
        lines[indices[0]] = update_integer_suffix(lines[indices[0]], conta_righe_totali(file_path)-2)
    with open(file_path, 'w') as file:
        file.writelines(lines)
def turbo_to_spectrum_resultConversion(path, nameResult):
    '''Converti i risultati formattati da turbospectrum in una formattazione tipo spectrum'''
    os.chdir(path)
    print(os.getcwd())
    # with open(nameResult, "r") as f:
    #     lines = f.readlines()
    # # Salta le prime due righe
    # lines_to_write = lines[2:]
    # with open(nameResult, "w") as f:
    #     f.writelines(lines_to_write)
    tab = Table.read(nameResult, format='ascii')
    el = tab['col1']
    ion = tab['col2']
    wl = tab['col3']
    pot_en = tab['col4']
    loggf = tab['col5']
    eqw = tab['col6']
    ewerr = tab['col7']
    abund = tab['col12']
    for i in range(len(ion)):
        periodica_table
    with open(nameResult, 'w') as f:
        for i in range(len(el)):
            f.write(f'{wl[i]:.3f}   {(periodica_table[el[i]]+0.1*(ion[i]-1)):.1f}  {pot_en[i]:.3f} 0.00   {abund[i]-12.036:.3f}    {(abund[i]):.3f}  {(abund[i]+4.54-12.036):.3f}\n')
def combine_files(file_list, output_file, file_list_path=os.getcwd(), syn=False):
    '''Unisce i contenuti di vari file in un unico file.'''
    if syn:
        os.chdir(file_list_path)
    with open(output_file, 'w') as outfile:
        for fname in file_list:
            with open(fname, 'r') as infile:
                for line in infile:
                    # Rimuovi tutte le occorrenze di "strong? wings cut?" dalla riga
                    cleaned_line = line.replace("strong? wings cut?", "")
                    outfile.write(cleaned_line)
def retrieveAtmosphere(T, G, M, V=2, fullpath='', TYPE='mucciarelli', alpha=0):
    if TYPE=='castelli':
        if fullpath=='' or fullpath == 'n':
            T_close = find_closest(T, T_EFF_Nint)
            logg_close=find_closest(G, LOGG_Nint)
            M_close=find_closest(M, MH_Nint)
            return f'atmoT{T_close}{logg_close:.1f}.txt'
        elif fullpath=='y':
            T_close = find_closest(T, T_EFF_Nint)
            logg_close=find_closest(G, LOGG_Nint)
            M_close=find_closest(M, MH_Nint)
            return path_to_atmosphere + f'MH{M_close:.1f}/atmoT{T_close}{logg_close:.1f}.txt'
    elif TYPE=='mucciarelli':
        T_array=np.arange(3750, 7000, 125)
        if fullpath=='' or fullpath == 'n':
            T_close = find_closest(T, T_array)
            logg_close=find_closest(G, LOGG_Nint)
            M_close=find_closest(M, [0.5, 0.25, 0.0, -0.25, -0.5, -0.75, -1.0, -1.25, -1.5, -1.75, -2.0, -2.25, -2.5, -2.75, -3.0, -3.5, -4.0, -4.5, -5.0])
            return f'{int(T_close)}_{logg_close:.2f}_{M_close:.2f}.mod'
        elif fullpath=='y':
            T_close = find_closest(T, T_array)
            logg_close=find_closest(G, LOGG_Nint)
            M_close=find_closest(M, [0.5, 0.25, 0.0, -0.25, -0.5, -0.75, -1.0, -1.25, -1.5, -1.75, -2.0, -2.25, -2.5, -2.75, -3.0, -3.5, -4.0, -4.5, -5.0])
            return path_to_atmosphere + f'a_{alpha:.2f}/{int(T_close)}_{logg_close:.2f}_{M_close:.2f}.mod'

def turbo_resultCombine(path_to_turboResult, file_namePrefix, outname, outfold=os.getcwd()):
    '''path_to_turboResult: path alla cartella contenente i risultati di TurboSpectrum\n
    file_namePrefix: base del nome del file di output (escluse le x di ptx), eg: per un file chiamato starpr2pt5 inserire starpr2pt \n
    out_fold: cartella di output'''
    os.chdir(path_to_turboResult)
    files = list_files_with_prefix(path_to_turboResult, file_namePrefix)
    files1 = []
    for i in range(len(files)):
        files1.append(remove_first_two_lines(files[i]))
    os.chdir(path_to_turboResult)
    combine_files(files1, outname)
    lines = read_file_to_list(outname)
    sortedlines = sorted(lines, key=lambda line: float(line.split()[2]))
    with open(outname, 'w') as f:
        f.writelines(sortedlines)
def creaATLAS9com(teff, logg, metal, vturb, modelName, mixing_lenght=1.25, layerI_opticaldepth = -6.875, opticaldepth_step=0.125, outpath=path_to_ATLAScom, outname='brewed_atlas9.com', ncycles=1, k_ros='', path_to_k_ros='/home/starcat8/Documenti/atlas9/ODF-2/ODF/NEW/', odf_bin = '', path_to_odf_bin='/home/starcat8/Documenti/atlas9/ODF-2/ODF/NEW/', molecules=True, path_to_molecules='/home/starcat8/Documenti/atlas9/ODF-2/ODF/NEW/molecules.dat', convergedModel = None, path_to_convergedModel='/home/starcat8/Documenti/atlas9/atlas9sc/', TYPE='mucciarelli', path_to_fortexe='/home/starcat8/Documenti/atlas9/atlas9sc/atlas9mem.exe', alpha_enhancement=0, change_abundance=False, H=0, He=0, Li=0, Be=0, B=0, C=0, N=0, O=0, F=0, Ne=0, Na=0, Mg=0, Al=0, Si=0, P=0, S=0, Cl=0, Ar=0, K=0, Ca=0, Sc=0, Ti=0, V=0, Cr=0, Mn=0, Fe=0, Co=0, Ni=0, Cu=0, Zn=0, Ga=0, Ge=0, As=0, Se=0, Br=0, Kr=0, Rb=0, Sr=0, Y=0, Zr=0, Nb=0, Mo=0, Tc=0, Ru=0, Rh=0, Pd=0, Ag=0, Cd=0, In=0, Sn=0, Sb=0, Te=0, Io=0, Xe=0, Cs=0, Ba=0, La=0, Ce=0, Pr=0, Nd=0, Pm=0, Sm=0, Eu=0, Gd=0, Tb=0, Dy=0, Ho=0, Er=0, Tm=0, Yb=0, Lu=0, Hf=0, Ta=0, W=0, Re=0, Os=0, Ir=0, Pt=0, Au=0, Hg=0, Tl=0, Pb=0, Bi=0, Po=0, At=0, Rn=0,  Fr=0, Ra=0, Ac=0, Th=0, Pa=0, U=0):
                                                                                                                                                                                                        #=kapm25.ros                                                                  = 'm25big2.bdf                                                                                                                                                                     #convergedModel = 'amm23cp00op00t4000g05v20.mod'
    '''Crea il file .com per ATLAS9. \n
    - logg: gravità superficiale \n
    - metal: metallicità della stella \n
    - vturb: velocità di microturbolenza\n
    - modelName: nome del modello che verrà prodotto\n
    Altri argomenti:\n
    - mixing_lenght: mixing lenght\n
    - layerI_opticaldepth: profondità ottica del primo strato del modello\n
    - opticaldepth_step: larghezza strato in profondità ottica\n
    - outpath: path in cui verrà creato il file .com\n
    - outname: nome del file .com\n
    - ncycles: numero di cicli. Ogni volta che ATLAS gira ha un massimo di 15 iterazione che compie. Dopo di che si interrompe, che sia giunto a convergenza o meno. Si può aumentare il numero di iterazioni aggiungendo cicli (a colpi di 15 iterazioni) con questa variabile\n
    - path_to_k_ros: opacità al continuo\n
    - path_to_odf_bin: path al file delle ODF\n
    - molecules: se True, abilita le molecole\n
    - path_to_molecules: path al file delle opacità molecolari\n
    - convergerdModel: nome del modello di partenza, se non viene specificato si utilizzano i modelli in path_to_atmospheres, il più vicino a quelli specificati da teff, logg, metal\n
    - path_to_convergedModel: path a un modello di partenza\n
    - path_to_fortexe: path al file fortran\n
    Elementi chimici\n
    \n idrogeno ed elio vengono definiti in percentuale, tutti gli altri elementi in scala logaritmica\n
     I valori di base inseriti sono quelli del Sole di Grievesse. \n
     Per inserire un abbondanza diversa si specifica il valore in scala logaritmica che verrà aggiunto all'abbondanza di base. \n
    - H: abbondanza di idrogeno, in percentuale\n
    - He: abbondanza di elio, in percentuale\n
    - Li: abbondanza di litio, in log10\n
    - Be: abbondanza di berillio, in log10\n
    ...etc\n
    Nota, lo iodio lo si indica con Io invece di I'''
    olddir = os.getcwd()
    os.chdir(outpath)
    v_closest=vturb
    if molecules:
        molecules = 'ON'
    else:
        molecules = 'OFF'
    if convergedModel == None:
        if TYPE=='castelli':
            convergedModel = retrieveAtmosphere(teff, logg, metal, TYPE=TYPE)
            M_close=find_closest(metal, MH_Nint)
            copia(convergedModel, path_to_atmosphere + f'MH{M_close:.1f}/', path_to_convergedModel, convergedModel)
        elif TYPE=='mucciarelli':
            
            convergedModel = retrieveAtmosphere(teff, logg, metal, TYPE=TYPE, alpha=alpha_enhancement)
            alpha_enhancement=find_closest(alpha_enhancement, [0.4, 0.2, 0.0, -0.2, -0.4])
            M_close=find_closest(metal, [0.5, 0.25, 0.0, -0.25, -0.5, -0.75, -1.0, -1.25, -1.5, -1.75, -2.0, -2.25, -2.5, -2.75, -3.0, -3.5, -4.0, -4.5, -5.0])
            copia(convergedModel, path_to_atmosphere + f'a_{alpha_enhancement:.2f}/', path_to_convergedModel, convergedModel)
            v_closest=find_closest(vturb, [1,2,4,8])
    if TYPE=='castelli':
        k_sign = 'm'
        odf_sign = 'm'
        if metal >= 0:
            k_sign = 'p'
            odf_sign = 'p'
        k_metal = find_closest(metal, [0.5, 0.25, 0.0, -0.5, -1.0, -1.5, -2.0, -2.5, -3.0, -3.5, -4.0])
        k_metal = int(k_metal*10)
        k_metal = abs(k_metal)
        if k_metal < 10:
            k_metal = '0'+str(k_metal)
        else:
            k_metal = str(k_metal)
        if k_ros == '':
            k_ros = 'kap'+k_sign+k_metal+'.ros'
        if odf_bin == '':
            odf_bin = odf_sign  +k_metal+f'big{int(v_closest)}.bdf'
    elif TYPE=='mucciarelli':
        alpha_sign='m'
        if alpha_enhancement>=0:
            alpha_sign='p'
        alph=alpha_enhancement*10
        alph=abs(alph)
        alph=int(alph)
        k_sign = 'm'
        odf_sign = 'm'
        if metal >= 0:
            k_sign = 'p'
            odf_sign = 'p'
        k_metal = find_closest(metal, [0.5, 0.25, 0.0, -0.25, -0.5, -0.75, -1.0, -1.25, -1.5, -1.75, -2.0, -2.25, -2.5, -2.75, -3.0, -3.5, -4.0, -4.5, -5.0])
        k_metal = int(k_metal*100)
        k_metal = abs(k_metal)
        path_to_k_ros=f'/home/starcat8/Documenti/atlas9/ODF-2/ODF/odfbig_cl_a{alpha_sign}0{alph}/'
        path_to_odf_bin=f'/home/starcat8/Documenti/atlas9/ODF-2/ODF/odfbig_cl_a{alpha_sign}0{alph}/'
        if k_metal < 100:
            k_metal = '0'+str(k_metal)
        else:
            k_metal = str(k_metal)
        if k_ros == '':
            k_ros = 'cl_a'+alpha_sign+f'0{alph}_'+k_sign+k_metal+'.ros'
        if odf_bin == '':
            odf_bin = 'cl_a'+alpha_sign+f'0{alph}_'+k_sign+k_metal+f'big{int(v_closest)}.bdf'
    
    with open(outname, mode='w') as f:
        f.write(f'ln -s {path_to_k_ros}{k_ros} fort.1\nln -s {path_to_odf_bin}{odf_bin} fort.9\nln -s {path_to_molecules} fort.2\nln -s {path_to_convergedModel}{convergedModel} fort.3\n{path_to_fortexe}<<EOF>testa.out\n')
        f.write(f'READ KAPPA\nREAD PUNCH\nMOLECULES {molecules}\nREAD MOLECULES\nFREQUENCIES 337 1 337 BIG\nVTURB {vturb:.2f}E+5\nCONVECTION OVER {mixing_lenght:.2f} 0 36\nTITLE  [{metal:.2f}] VTURB = {vturb:.1f} L/H={mixing_lenght:.2f} NOVER NEW ODF\nSCALE 72 {layerI_opticaldepth:.3f} {opticaldepth_step:.3f} {int(teff)} {logg:.2f}\nABUNDANCE SCALE   {(10**metal):.5f} ')
        if change_abundance:
            f.write(f'ABUNDANCE CHANGE 1 0.92156 2 0.07844\nABUNDANCE CHANGE  3 {Li-10.99}  4 {Be-10.66}  5  {B-9.34}  6  {C-3.65}  7  {N-4.26}  8  {O-3.38}\n ABUNDANCE CHANGE  9  {F-7.48} 10  {Ne-4.20} 11  {Na-5.87} 12  {Mg-4.51} 13  {Al-5.67} 14  {Si-4.53}\n ABUNDANCE CHANGE 15  {P-6.68} 16  {S-4.90} 17  {Cl-6.54} 18  {Ar-5.86} 19  {K-6.96} 20  {Ca-5.73}\n ABUNDANCE CHANGE 21  {Sc-8.99} 22  {Ti-7.14} 23  {V-8.04} 24  {Cr-6.40} 25  {Mn-6.65} 26  {Fe-4.59}\n ABUNDANCE CHANGE 27  {Co-7.12} 28  {Ni-5.81} 29  {Cu-7.83} 30  {Zn-7.44} 31  {Ga-9.16} 32  {Ge-8.46}\n ABUNDANCE CHANGE 33  {As-9.75} 34  {Se-8.71} 35  {Br-9.48} 36  {Kr-8.76} 37  {Rb-9.44} 38  {Sr-9.12}\n ABUNDANCE CHANGE 39  {Y-9.83 } 40  {Zr-9.45} 41 {Nb-10.62} 42 {Mo-10.12} 43 {Tc-22.03} 44 {Ru-10.20}\n ABUNDANCE CHANGE 45 {Rh-10.92} 46 {Pd-10.35} 47 {Ag-11.10} 48 {Cd-10.27} 49 {In-10.44} 50 {Sn-10.04}\n ABUNDANCE CHANGE 51 {Sb-11.04} 52  {Te-9.85} 53 {Io-10.53} 54  {Xe-9.77} 55 {Cs-10.97} 56  {Ba-9.87}\n ABUNDANCE CHANGE 57 {La-10.91} 58 {Ce-10.46} 59 {Pr-11.33} 60 {Nd-10.59} 61 {Pm-22.03} 62 {Sm-11.03}\n ABUNDANCE CHANGE 63 {Eu-11.52} 64 {Gd-10.92} 65 {Tb-11.76} 66 {Dy-10.90} 67 {Ho-11.53} 68 {Er-11.11}\n ABUNDANCE CHANGE 69 {Tm-12.04} 70 {Yb-10.96} 71 {Lu-11.98} 72 {Hf-11.16} 73 {Ta-12.21} 74 {W-10.93}\n ABUNDANCE CHANGE 75 {Re-11.81} 76 {Os-10.59} 77 {Ir-10.66} 78 {Pt-10.40} 79 {Au-11.03} 80 {Hg-10.91}\n ABUNDANCE CHANGE 81 {Tl-11.14} 82 {Pb-10.04} 83 {Bi-11.39} 84 {Po-22.03} 85 {At-22.03} 86 {Rn-22.03}\n ABUNDANCE CHANGE 87 {Fr-22.03} 88 {Ra-22.03} 89 {Ac-22.03} 90 {Th-11.98} 91 {Pa-22.03} 92 {U-12.56}\n ABUNDANCE CHANGE 93 -22.03 94 -22.03 95 -22.03 96 -22.03 97 -22.03 98 -22.03\n ABUNDANCE CHANGE 99 -22.03\n')
        f.write('ITERATIONS 15 PRINT 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1\nPUNCH 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1\nBEGIN                    ITERATION  15 COMPLETED\n')
        if ncycles > 1:
            for i in range(ncycles-1):
                f.write(f'\nSCALE 72 {layerI_opticaldepth:.3f} {opticaldepth_step:.3f} {int(teff)}. {logg:.2f}\nITERATIONS 15 PRINT 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1\nPUNCH 5 0 0 0 0 0 0 0 0 0 0 0 0 0 1\nBEGIN                    ITERATION  15 COMPLETED')
        f.write(f'END\nEOF\nmv fort.7 {modelName}\n/bin/rm fort.*\n')
    outso = os.getcwd() + '/'+outname
    os.chdir(olddir)
    return outso
def creaMOOG_BATCH(moog_number, modelName, lineList='AAA_lines_abf.dat'):
    '''Crea il file batch per MOOG. \n
    moog_number: per la parallelizzazione. Specifica per quale numero di MOOG creare il file batch. \n
    '''
    os.chdir(path_to_moogBatch+f'runs{moog_number+1}/')
    with open('batch.par', mode='w') as f:
        f.write('abfind\nstandard_out \'abfind_eqw1\'\nsummary_out \'abfind_eqw2\'\nmodel_in ')
        f.write(f'\'atlas9_models/{modelName}\'\nlines_in \'{lineList}\'\n')
        f.write('freeform 0\natmosphere 1\nlines 1\nmolecules 2\nflux/int 0\ndamping 1\nplot 1')
def creaLineList(fits, rad_solver, line_list, instrument, ew_low=10, ew_high=110, output_linelist='stellaFes', elements='iron', el_number=1):
    '''Crea linelist equipaggiata con le equivalent width misurate con ARES.\n
    Si deve specificare il rad_solver \'spectrum\' per SPECTRUM, \'turbo\' per TURBOSPECTRUM, \'moog\' per MOOG.\n
    ew_low: valore minimo di EW da considerare\n
    ew_high: valore massimo di EW da considerare\n
    output_linelist: nome del file di output'''
    olddir = os.getcwd()
    os.chdir(path_to_linemake)
    MASTERLIST = Table.read(line_list, format='ascii')
    copia(line_list, path_to_linemake, path_to_ARES, line_list)
    if instrument == 'HARPS-N':
        lambdai=4600
        lambdaf=6800
    elif instrument == 'FIES':
        lambdai=4200
        lambdaf=8800
    os.chdir(path_to_ARES)
    creaOptFileARES(specfits=fits, readlinedat=line_list, lambdai=lambdai, lambdaf=lambdaf)
    ! ./ARES
    ARESLIST = Table.read('stella.ares', format='ascii')
    #sel = (ARESLIST['col5'] >= ew_low) & (ARESLIST['col5'] <= ew_high) & (((ARESLIST['col6']/ARESLIST['col5'] <= 0.1) & MASTERLIST['IONM'] == 26.0) | ((ARESLIST['col6']/ARESLIST['col5'] <= 0.2) & MASTERLIST['IONM'] == 26.1))
    sel = (ARESLIST['col5'] >= ew_low) & (ARESLIST['col5'] <= ew_high) & (ARESLIST['col6'] < 8) #Selezione delle righe in base a EW e errore associato all'EW
    ARESLIST = ARESLIST[sel]
    if rad_solver == 'spectrum':
        with open('stellaFes', 'w') as f:
            for i in range(len(MASTERLIST)):
                for j in range(len(ARESLIST)):                     #MASTERLIST Lam IONG IONM LOGGF ELOW Rad Stark VDWM VDWG Bark Flags_lgf_syn GESPIPE
                    if MASTERLIST['Lam'][i] == ARESLIST['col1'][j]:#serve WL SPECIE EPlow EPhigh loggf 1.0 99 EW
                        f.write(f'{MASTERLIST["Lam"][i]} {MASTERLIST["IONM"][i]} {(MASTERLIST["ELOW"][i]*8065.5):.3f} {150000} {MASTERLIST["LOGGF"][i]} 1.0   99   {ARESLIST['col5'][j]}   a\n')
        copia('stellaFes', os.getcwd(), path_to_abundance, 'stellaFes')
    
    elif rad_solver == 'turbo':
        turboFormat_Heiter = HeiterLines() #tutte le righe di turbospectrum da matchare con la linelist che si vuole usare
        species = MASTERLIST['IONM']
        #print(species)
        print(len(os.getcwd()))
        equivalent_width=0
        equivalent_width_error=0
        old_equivalent_width=0
        intext=True
        total=[]
        if elements=='iron':
            fei=0 #FeI
            feii=0#FeII
        if elements=='alpha':
            ci=0  #CI
            oi=0  #OI
            mgi=0  #MgI
            sii=0  #SiI
            siii=0 #SiII
            cai=0  #CaI
            caii=0 #CaII
            tii=0  #TiI
            tiii=0 #TiII

        elementi=[]
        with open('turbo_lines.ew', 'w') as f:
            for j in range(len(species)):
                for k in range(len(ARESLIST)):
                    if MASTERLIST['Lam'][j] == ARESLIST['col1'][k]:
                        equivalent_width = ARESLIST['col5'][k]
                        equivalent_width_error = ARESLIST['col6'][k]
                if True: #nome dell'elemento nel dizionario
                    if (species[j]-int(species[j])) == 0:
                        if len(table_periodica[f'{int(species[j])}']) == 1:
                            turboFormat_speciename = f'{float(int(species[j])):.3f}'+table_periodica[f'{int(species[j])}']+'  I '
                        elif len(table_periodica[f'{int(species[j])}']) == 2:
                            turboFormat_speciename = f'{float(int(species[j])):.3f}'+table_periodica[f'{int(species[j])}']+' I '
                    else:
                        if len(table_periodica[f'{int(species[j])}']) == 1:
                            turboFormat_speciename = f'{float(int(species[j])):.3f}'+table_periodica[f'{int(species[j])}']+'  II'
                        elif len(table_periodica[f'{int(species[j])}']) == 2:
                            turboFormat_speciename = f'{float(int(species[j])):.3f}'+table_periodica[f'{int(species[j])}']+' II'
                for i in range(len(turboFormat_Heiter[turboFormat_speciename])):
                    if abs(MASTERLIST['Lam'][j] - turboFormat_Heiter[turboFormat_speciename][i]['wavelength']) < 0.01 and (old_equivalent_width != equivalent_width):
                        total.append([float(MASTERLIST['Lam'][j]), float(MASTERLIST['ELOW'][j]), float(MASTERLIST['LOGGF'][j]), float(turboFormat_Heiter[turboFormat_speciename][i]['damping']), float(turboFormat_Heiter[turboFormat_speciename][i]['g_up']), (turboFormat_Heiter[turboFormat_speciename][i]['gamma_rad']), (turboFormat_Heiter[turboFormat_speciename][i]['lower_config']), '\'', (turboFormat_Heiter[turboFormat_speciename][i]['upper_config']), float(equivalent_width), float(equivalent_width_error), (turboFormat_Heiter[turboFormat_speciename][i]['description']), turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6]])
                        if elements=='iron':
                            if (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Fe I '):
                                fei += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Fe II'):
                                feii += 1
                        elif elements=='alpha':
                            if (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'C  I '):
                                Ci += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'O  I '):
                                oi += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Mg I '):
                                mgi += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Si I '):
                                sii += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Si II'):
                                siii += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Ca I '):
                                cai += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Ca II'):
                                caii += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Ti I '):
                                tii += 1
                            elif (turboFormat_Heiter[turboFormat_speciename][i]['description'][1:6] == 'Ti II'):
                                tiii += 1
                old_equivalent_width = equivalent_width
                        #row = (float(MASTERLIST['Lam'][j]), float(MASTERLIST['ELOW'][j]), float(MASTERLIST['LOGGF'][j]), float(turboFormat_Heiter[turboFormat_speciename][i]['damping']), float(turboFormat_Heiter[turboFormat_speciename][i]['g_up']), (turboFormat_Heiter[turboFormat_speciename][i]['gamma_rad']), (turboFormat_Heiter[turboFormat_speciename][i]['lower_config']), '\'', (turboFormat_Heiter[turboFormat_speciename][i]['upper_config']), float(equivalent_width), float(equivalent_width_error), (turboFormat_Heiter[turboFormat_speciename][i]['description']))
            if elements=='iron':
                f.write(f'\'26.000              \'    1       {fei}\n\'Fe I \'\n')
                for i in range(fei):
                    row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                    formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                    f.write(formatted_row + '\n')
                f.write(f'\'26.000              \'    2       {feii}\n\'Fe II\'\n')
                for i in range(fei, fei + feii):
                    row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                    formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                    f.write(formatted_row + '\n')
            elif elements=='alpha':
                if ci>0:
                    f.write(f'\'6.000              \'    1       {ci}\n\'C  I \'\n')
                    for i in range(ci):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if oi>0:
                    f.write(f'\'8.000              \'    1       {oi}\n\'O  I \'\n')
                    for i in range(ci,ci+oi):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if mgi>0:
                    f.write(f'\'12.000              \'    1       {mgi}\n\'Mg I \'\n')
                    for i in range(ci+oi,ci+oi+mgi):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if sii>0:
                    f.write(f'\'14.000              \'    1       {sii}\n\'Si I \'\n')
                    for i in range(ci+oi+mgi,ci+oi+mgi+sii):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if siii>0:
                    f.write(f'\'14.000              \'    2       {siii}\n\'Si II\'\n')
                    for i in range(ci+oi+mgi+sii,ci+oi+mgi+sii+siii):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if cai>0:
                    f.write(f'\'20.000              \'    1       {cai}\n\'Ca I \'\n')
                    for i in range(ci+oi+mgi+sii+siii,ci+oi+mgi+sii+siii+cai):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if caii>0:
                    f.write(f'\'20.000              \'    2       {caii}\n\'Ca II\'\n')
                    for i in range(ci+oi+mgi+sii+siii+cai,ci+oi+mgi+sii+siii+cai+caii):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if tii>0:
                    f.write(f'\'22.000              \'    1       {tii}\n\'Ti I \'\n')
                    for i in range(ci+oi+mgi+sii+siii+cai+caii,ci+oi+mgi+sii+siii+cai+caii+tii):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
                if tiii>0:
                    f.write(f'\'22.000              \'    2       {tiii}\n\'Ti II\'\n')
                    for i in range(ci+oi+mgi+sii+siii+cai+caii+tii,ci+oi+mgi+sii+siii+cai+caii+tii+tiii):
                        row = (total[i][0], total[i][1], total[i][2], total[i][3], total[i][4], total[i][5], total[i][6], total[i][7], total[i][8], total[i][9], total[i][10], total[i][11])
                        formatted_row = "{:>10.3f}{:>7.3f}{:>8.2f}{:>6.1f}{:>7.1f}{:>10}{:>4}{:>2}{:>2}{:>6.1f}{:>7.1f}{:>39}".format(*row)
                        f.write(formatted_row + '\n')
            #for i in range(len(turboFormat_Heiter[turboFormat_speciename])):
            return total

    elif rad_solver == 'moog':
        with open('stellaFes', 'w') as f:
            f.write('sun.asc\n')
            for i in range(len(MASTERLIST)):
                for j in range(len(ARESLIST)):                     #MASTERLIST Lam IONG IONM LOGGF ELOW Rad Stark VDWM VDWG Bark Flags_lgf_syn GESPIPE
                    if MASTERLIST['Lam'][i] == ARESLIST['col1'][j]:#serve WL SPECIE EPlow loggf EW ERRORE?
                        row = (MASTERLIST['Lam'][i], MASTERLIST['IONM'][i], (MASTERLIST['ELOW'][i]), MASTERLIST['LOGGF'][i], ((ARESLIST['col5'][j])), ((ARESLIST['col6'][j])))
                        formatted_row = "{:>10}{:>10}{:>10}{:>10}{:>30}{:>10.3e}".format(*row)
                        f.write(formatted_row + '\n')
                        #f.write(f'  {MASTERLIST["Lam"][i]}      {MASTERLIST["IONM"][i]}      {(MASTERLIST["ELOW"][i]):.3f}      {MASTERLIST["LOGGF"][i]}                         {(ARESLIST['col5'][j]):.2f} {(ARESLIST['col6'][j]):.3e}\n')
        for i in range(1, 13):
            copia('stellaFes', os.getcwd(), path_to_moogBatch+f'runs{i}', 'AAA_lines_abf.dat')
    os.chdir(olddir)
def HeiterLines():
    '''Restituisce le righe di HeiterLines'''
    olddir = os.getcwd()
    os.chdir(path_to_turboHeiter)
    returner = unisci_dati('gf0400.list', 'gf0420.list', 'gf0440.list', 'gf0460.list', 'gf0475.list', 'GES_475_582.list', 'GES_582_680.list', 'gf0720.list', 'gf0760.list', 'gf0800.list')
    os.chdir(olddir)
    return returner

def convertATLAS9_to_MOOGcompatible(path, name, outname, T, G, M, V):
    '''Converte i file di ATLAS9 in un formato compatibile con MOOG. \n'''
    olddir = os.getcwd()
    os.chdir(path)
    with open(name, 'r') as f:
        lines = f.readlines()[23:-2]
    with open(name, 'w') as f:
        for line in lines:
            f.write(line)
    fix_1st_line_35th_char(name)
    Tab = Table.read(name, format='ascii')
    RHOX = Tab['col1']
    TEMP = Tab['col2']
    P = Tab['col3']
    XNE = Tab['col4']
    # bashcmd = f'rm {name}us'
    # os.system(bashcmd)
    with open(outname, 'w') as f:
        f.write(f'KURTYPE\n  {T:.0f}/   {G:.2f}/   {M:.2f}      mic = {V:.4f}\n             72\n{T:.0f}\n')
        for i in range(len(RHOX)):
            f.write(f'  {RHOX[i]}  {TEMP[i]} {P[i]} {XNE[i]}\n')
        f.write(f'     {V:.2f}E+05\nnatoms        0          {M:.2f}\nnmol         12\n      101.      106.      107.      108.      606.      607.      608.      707.\n      708.      808.    10108.    60808.')
    
    os.chdir(olddir)

def crea_synthScriptTurbo(path_to_model, modelName, outname='star', outpath=path_to_output, wl_min=3700, wl_max=6000, delta_wl=0.01, TURBVEL=2.00, limit_width=20, abundances=None, scriptNumber=-1, long=True):
    '''Crea script per lanciare il codice TURBOSPECTRUM in spettrosintesi. \n'''
    os.chdir(path_to_turbospectrumNLTE)
    wing=limit_width
    wl_max=wl_max+wing
    wl_iterationNumber=int((wl_max-wl_min)/2000)+1
    if abundances is not None:
        if len(abundances) !=2:
            raise ValueError('abundances deve essere una lista di due elementi: [specie, abbondanza]')
    if scriptNumber==-1:
        with open('synth_script', 'w') as f:
            f.write('#!/bin/csh -f\nset wdir = \"outsynth\"\nset NODE = \"`/bin/uname -a|cut -d \' \' -f 2`\"\n set BASDIR = ./\ncd ${BASDIR}\nmkdir ${wdir}\ncd ${wdir}\npwd \nmkdir syntspec\nmkdir contopac\n')
            f.write(f'ln -s {path_to_turbospectrumNLTE}DATA .\nln -s {path_to_turbospectrumNLTE}lines .\nset mpath = {path_to_model}\nset THOME = {path_to_turbospectrumNLTE}\nset MODEL = {modelName}\n\nset k = 1\nwhile ($k < {wl_iterationNumber+1})\nset ord = \"`printf "%02d" $k `\"\n\n')
            wing=limit_width
            for k in range(1, wl_iterationNumber+1):
                if k == 1:
                    wing=0
                else:
                    wing=limit_width
                f.write(f'if ( $k == {k}) then\n set lam_min = {wl_min+1990*(k-1)-wing}\n set lam_max = {min(wl_max-wing, wl_min+1990*(k)-wing)}\nendif\n')
            f.write(f'\n\nset deltalam   = \'{delta_wl:.2f}\'\nset TURBVEL    = \'{TURBVEL:.2f}\'\n')
            f.write('set VT = "`echo ${TURBVEL} | sed  -e \'')
            f.write(r's/\./_/')
            f.write('\'`\"\nset SUFFIX     = N${ord}_vt${VT}.dat\n')
            f.write(f'set result     = {outname}')
            f.write('${k}')
            f.write('\n\n\n')
            f.write('time ${THOME}/exec-gf/babsma_lu <<EOF\n\'LAMBDA_MIN:\'  \'${lam_min}\'\n\'LAMBDA_MAX:\'  \'${lam_max}\'\n\'LAMBDA_STEP:\' \'${deltalam}\'\n\'MODELINPUT:\' \'$mpath/${MODEL}\'\n\'MARCS-FILE:\' \'.false.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n')
            if abundances is None:
                f.write('\'INDIVIDUAL ABUNDANCES:\'   \'91\'\n 2  10.88\n 3  -1.02\n 4  -0.73\n 5   0.76\n 6   6.45\n 7   5.81\n 8   7.11\n 9   2.39\n10   6.40\n11   4.24\n12   5.89\n13   4.41\n14   5.88\n15   3.41\n16   5.51\n17   3.20\n18   4.85\n19   3.06\n20   4.66\n21   1.02\n22   3.28\n23   1.94\n24   3.60\n25   3.45\n26   5.47\n27   2.85\n28   4.17\n29   2.22\n30   2.60\n31   1.05\n32   1.54\n33   0.27\n34   1.31\n35   0.51\n36   1.23\n37   0.33\n38   0.85\n39   0.15\n40   0.52\n41  -0.63\n42  -0.11\n43 -10.02\n44  -0.27\n45  -0.95\n46  -0.38\n47  -0.83\n48  -0.32\n49  -1.27\n50   0.04\n51  -1.02\n52   0.15\n53  -0.48\n54   0.22\n55  -0.95\n56   0.13\n57  -0.86\n58  -0.45\n59  -1.28\n60  -0.58\n61 -10.02\n62  -1.09\n63  -1.53\n64  -0.96\n65  -1.71\n66  -0.91\n67  -1.56\n68  -1.10\n69  -1.91\n70  -1.11\n71  -1.94\n72  -1.18\n73  -2.19\n74  -1.38\n75  -1.77\n76  -0.69\n77  -0.69\n78  -0.41\n79  -1.23\n80  -0.86\n81  -1.26\n82   0.01\n83  -1.38\n84 -10.02\n85 -10.02\n86 -10.02\n87 -10.02\n88 -10.02\n89 -10.02\n90  -1.97\n91 -10.02\n92  -2.57\n\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n\'XIFIX:\' \'T\'\n$TURBVEL\nEOF\n\n\n')
            elif abundances is not None:
                f.write(f'\'INDIVIDUAL ABUNDANCES:\'   \'{len(abundances[0])}\'\n')
                for i in range(len(abundances[0])):
                    if abundances[0][i] <= 9:
                        f.write(f' {abundances[0][i]:<3}')
                    else: 
                        f.write(f'{abundances[0][i]:<3} ')
                    if abundances[0][i] == 2:
                        f.write(f' {abundances[1][i]:.2f}\n')
                    else:
                        if abundances[1][i] > 0:
                            f.write(f'  {abundances[1][i]:.2f}\n')
                        else:
                            f.write(f' {abundances[1][i]:.2f}\n')
                #f.write('\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n\'XIFIX:\' \'T\'\n$TURBVEL\nEOF\n\n\n')
                f.write( '\'ISOTOPES : \' \'66\'\n3.006   0.076\n3.007   0.924\n5.010   0.1982\n5.011   0.8018\n6.012   0.990\n6.013   0.010\n7.014   1.000\n19.039  0.93132\n19.040  0.00147\n19.041  0.06721\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n22.046  0.08249\n22.047  0.07437\n22.048  0.7372\n22.049  0.05409\n22.050  0.05185\n28.058  0.680769\n28.060  0.262231\n28.061  0.011399\n28.062  0.036345\n28.064  0.009256\n29.063  0.692\n29.065  0.308\n31.069  0.60108\n31.071  0.39892\n56.134  0.024 \n56.135  0.066 \n56.136  0.078 \n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150  \n62.148  0.112\n62.149  0.138 \n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478 \n63.153  0.522\n78.190  0.00014\n78.192  0.00783\n78.194  0.32967\n78.195  0.33832\n78.196  0.25242\n78.198  0.07163\n80.196  0.0015\n80.198  0.0997\n80.199  0.1687\n80.200  0.2310\n80.201  0.1318\n80.202  0.2986\n80.204  0.0687\n\'XIFIX:\' \'T\'\n$TURBVEL\nEOF\n\n\n')
            f.write('time ${THOME}/exec-gf/bsyn_lu <<EOF\n\'NLTE :\'          \'.false.\'\n\'LAMBDA_MIN:\'     \'${lam_min}\'\n\'LAMBDA_MAX:\'     \'${lam_max}\'\n\'LAMBDA_STEP:\'    \'${deltalam}\'\n\'INTENSITY/FLUX:\' \'Flux\'\n\'ABFIND        :\' \'.false.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'RESULTFILE :\' \'syntspec/${result}\'\n')
            if abundances is None:
                f.write('\'INDIVIDUAL ABUNDANCES:\'   \'91\'\n 2  10.88\n 3  -1.02\n 4  -0.73\n 5   0.76\n 6   6.45\n 7   5.81\n 8   7.11\n 9   2.39\n10   6.40\n11   4.24\n12   5.89\n13   4.41\n14   5.88\n15   3.41\n16   5.51\n17   3.20\n18   4.85\n19   3.06\n20   4.66\n21   1.02\n22   3.28\n23   1.94\n24   3.60\n25   3.45\n26   5.47\n27   2.85\n28   4.17\n29   2.22\n30   2.60\n31   1.05\n32   1.54\n33   0.27\n34   1.31\n35   0.51\n36   1.23\n37   0.33\n38   0.85\n39   0.15\n40   0.52\n41  -0.63\n42  -0.11\n43 -10.02\n44  -0.27\n45  -0.95\n46  -0.38\n47  -0.83\n48  -0.32\n49  -1.27\n50   0.04\n51  -1.02\n52   0.15\n53  -0.48\n54   0.22\n55  -0.95\n56   0.13\n57  -0.86\n58  -0.45\n59  -1.28\n60  -0.58\n61 -10.02\n62  -1.09\n63  -1.53\n64  -0.96\n65  -1.71\n66  -0.91\n67  -1.56\n68  -1.10\n69  -1.91\n70  -1.11\n71  -1.94\n72  -1.18\n73  -2.19\n74  -1.38\n75  -1.77\n76  -0.69\n77  -0.69\n78  -0.41\n79  -1.23\n80  -0.86\n81  -1.26\n82   0.01\n83  -1.38\n84 -10.02\n85 -10.02\n86 -10.02\n87 -10.02\n88 -10.02\n89 -10.02\n90  -1.97\n91 -10.02\n92  -2.57\n\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n')
            elif abundances is not None:
                f.write(f'\'INDIVIDUAL ABUNDANCES:\'   \'{len(abundances[0])}\'\n')
                for i in range(len(abundances[0])):
                    if abundances[0][i] <= 9:
                        f.write(f' {abundances[0][i]:<3}')
                    else: 
                        f.write(f'{abundances[0][i]:<3} ')
                    if abundances[0][i] == 2:
                        f.write(f' {abundances[1][i]:.2f}\n')
                    else:
                        if abundances[1][i] > 0:
                            f.write(f'  {abundances[1][i]:.2f}\n')
                        else:
                            f.write(f' {abundances[1][i]:.2f}\n')
                #f.write('\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n')
                f.write('\'ISOTOPES : \' \'66\'\n3.006   0.076\n3.007   0.924\n5.010   0.1982\n5.011   0.8018\n6.012   0.990\n6.013   0.010\n7.014   1.000\n19.039  0.93132\n19.040  0.00147\n19.041  0.06721\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n22.046  0.08249\n22.047  0.07437\n22.048  0.7372\n22.049  0.05409\n22.050  0.05185\n28.058  0.680769\n28.060  0.262231\n28.061  0.011399\n28.062  0.036345\n28.064  0.009256\n29.063  0.692\n29.065  0.308\n31.069  0.60108\n31.071  0.39892\n56.134  0.024 \n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150  \n62.148  0.112\n62.149  0.138 \n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478 \n63.153  0.522\n78.190  0.00014\n78.192  0.00783\n78.194  0.32967\n78.195  0.33832\n78.196  0.25242\n78.198  0.07163\n80.196  0.0015\n80.198  0.0997\n80.199  0.1687\n80.200  0.2310\n80.201  0.1318\n80.202  0.2986\n80.204  0.0687\n')
            f.write('\'NFILES   :\' \'73\'\n')
            f.write(f'{path_to_turbospectrumNLTE}/DATA/Hlinedata\n{path_to_turbospectrumNLTE}DATA/lines/gf0320.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0340.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0350.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0360.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0380.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0400.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0420.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0440.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0460.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0475.list\n{path_to_turbospectrumNLTE}DATA/lines/GES_475_582.list\n{path_to_turbospectrumNLTE}DATA/lines/GES_582_680.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0720.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0760.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0800.list\n{path_to_turbospectrumNLTE}DATA/lines/gf08498.list\n{path_to_turbospectrumNLTE}DATA/lines/GES_8498_8950.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0920.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1000.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1050.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1100.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1200.list\n{path_to_turbospectrumNLTE}DATA/lines/16OH_A-X-bsyn_2000-4100-modif.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_5500.6000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_7000.7500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_9000.9500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_5500.6000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_7000.7500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_9000.9500.list\n{path_to_turbospectrumNLTE}DATA/lines/14NH-AX-Bernath2018.list\n{path_to_turbospectrumNLTE}DATA/lines/12C12C-Brookeetal-Swan.list\n{path_to_turbospectrumNLTE}DATA/lines/12C13C-Rametal-Swan.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_7000.7500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_9000.9500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_5500.6000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_9000.9500.list\n')
            f.write('\'SPHERICAL:\'  \'F\'\n  30\n  300.00\n  15\n  1.30\nEOF\n########################################################################\ndate\n@ k+=1\nend\n')
            f.write('cp {path_to_turbospectrumNLTE}{wdir}/'+f'syntspec/* {outpath}/syntspec\ncp contopac/* {path_to_turbospectrumNLTE}lines/contopac\ncd ..\n')
            #f.write('rm -rf ${wdir}\n')
    else:
        with open(f'synth_script{scriptNumber}', 'w') as f:
            f.write('#!/bin/csh -f\nset wdir = \"outsynth\"\nset NODE = \"`/bin/uname -a|cut -d \' \' -f 2`\"\n set BASDIR = ./\ncd ${BASDIR}\nmkdir ${wdir}\ncd ${wdir}\npwd \nmkdir syntspec\nmkdir contopac\n')
            f.write(f'ln -s {path_to_turbospectrumNLTE}DATA .\nln -s {path_to_turbospectrumNLTE}lines .\nset mpath = {path_to_model}\nset THOME = {path_to_turbospectrumNLTE}\nset MODEL = {modelName}\n\nset k = 1\nwhile ($k < {wl_iterationNumber+1})\nset ord = \"`printf "%02d" $k `\"\n\n')
            wing=limit_width
            for k in range(1, wl_iterationNumber+1):
                if k == 1:
                    wing=0
                else:
                    wing=limit_width
                f.write(f'if ( $k == {k}) then\n set lam_min = {wl_min+1990*(k-1)-wing}\n set lam_max = {min(wl_max-wing, wl_min+1990*(k)-wing)}\nendif\n')
            f.write(f'\n\nset deltalam   = \'{delta_wl:.2f}\'\nset TURBVEL    = \'{TURBVEL:.2f}\'\n')
            f.write('set VT = "`echo ${TURBVEL} | sed  -e \'')
            f.write(r's/\./_/')
            f.write('\'`\"\nset SUFFIX     = N${ord}_vt${VT}.dat\n')
            f.write(f'set result     = {outname}')
            f.write('${k}')
            f.write('\n\n\n')
            f.write('time ${THOME}/exec-gf/babsma_lu <<EOF\n\'LAMBDA_MIN:\'  \'${lam_min}\'\n\'LAMBDA_MAX:\'  \'${lam_max}\'\n\'LAMBDA_STEP:\' \'${deltalam}\'\n\'MODELINPUT:\' \'$mpath/${MODEL}\'\n\'MARCS-FILE:\' \'.false.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n')
            if abundances is None:
                f.write('\'INDIVIDUAL ABUNDANCES:\'   \'91\'\n 2  10.88\n 3  -1.02\n 4  -0.73\n 5   0.76\n 6   6.45\n 7   5.81\n 8   7.11\n 9   2.39\n10   6.40\n11   4.24\n12   5.89\n13   4.41\n14   5.88\n15   3.41\n16   5.51\n17   3.20\n18   4.85\n19   3.06\n20   4.66\n21   1.02\n22   3.28\n23   1.94\n24   3.60\n25   3.45\n26   5.47\n27   2.85\n28   4.17\n29   2.22\n30   2.60\n31   1.05\n32   1.54\n33   0.27\n34   1.31\n35   0.51\n36   1.23\n37   0.33\n38   0.85\n39   0.15\n40   0.52\n41  -0.63\n42  -0.11\n43 -10.02\n44  -0.27\n45  -0.95\n46  -0.38\n47  -0.83\n48  -0.32\n49  -1.27\n50   0.04\n51  -1.02\n52   0.15\n53  -0.48\n54   0.22\n55  -0.95\n56   0.13\n57  -0.86\n58  -0.45\n59  -1.28\n60  -0.58\n61 -10.02\n62  -1.09\n63  -1.53\n64  -0.96\n65  -1.71\n66  -0.91\n67  -1.56\n68  -1.10\n69  -1.91\n70  -1.11\n71  -1.94\n72  -1.18\n73  -2.19\n74  -1.38\n75  -1.77\n76  -0.69\n77  -0.69\n78  -0.41\n79  -1.23\n80  -0.86\n81  -1.26\n82   0.01\n83  -1.38\n84 -10.02\n85 -10.02\n86 -10.02\n87 -10.02\n88 -10.02\n89 -10.02\n90  -1.97\n91 -10.02\n92  -2.57\n\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n\'XIFIX:\' \'T\'\n$TURBVEL\nEOF\n\n\n')
            elif abundances is not None:
                f.write(f'\'INDIVIDUAL ABUNDANCES:\'   \'{len(abundances[0])}\'\n')
                for i in range(len(abundances[0])):
                    if abundances[0][i] <= 9:
                        f.write(f' {abundances[0][i]:<3}')
                    else: 
                        f.write(f'{abundances[0][i]:<3} ')
                    if abundances[0][i] == 2:
                        f.write(f' {abundances[1][i]:.2f}\n')
                    else:
                        if abundances[1][i] > 0:
                            f.write(f'  {abundances[1][i]:.2f}\n')
                        else:
                            f.write(f' {abundances[1][i]:.2f}\n')
                #f.write('\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n\'XIFIX:\' \'T\'\n$TURBVEL\nEOF\n\n\n')
                f.write('\'ISOTOPES : \' \'66\'\n3.006   0.076\n3.007   0.924\n5.010   0.1982\n5.011   0.8018\n6.012   0.990\n6.013   0.010\n7.014   1.000\n19.039  0.93132\n19.040  0.00147\n19.041  0.06721\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n22.046  0.08249\n22.047  0.07437\n22.048  0.7372\n22.049  0.05409\n22.050  0.05185\n28.058  0.680769\n28.060  0.262231\n28.061  0.011399\n28.062  0.036345\n28.064  0.009256\n29.063  0.692\n29.065  0.308\n31.069  0.60108\n31.071  0.39892\n56.134  0.024 \n56.135  0.066 \n56.136  0.078 \n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150  \n62.148  0.112\n62.149  0.138 \n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478 \n63.153  0.522\n78.190  0.00014\n78.192  0.00783\n78.194  0.32967\n78.195  0.33832\n78.196  0.25242\n78.198  0.07163\n80.196  0.0015\n80.198  0.0997\n80.199  0.1687\n80.200  0.2310\n80.201  0.1318\n80.202  0.2986\n80.204  0.0687\n\'XIFIX:\' \'T\'\n$TURBVEL\nEOF\n\n\n')
            f.write('time ${THOME}/exec-gf/bsyn_lu <<EOF\n\'NLTE :\'          \'.false.\'\n\'LAMBDA_MIN:\'     \'${lam_min}\'\n\'LAMBDA_MAX:\'     \'${lam_max}\'\n\'LAMBDA_STEP:\'    \'${deltalam}\'\n\'INTENSITY/FLUX:\' \'Flux\'\n\'ABFIND        :\' \'.false.\'\n\'MODELOPAC:\' \'contopac/${MODEL}opac\'\n\'RESULTFILE :\' \'syntspec/${result}\'\n')
            if abundances is None:
                f.write('\'INDIVIDUAL ABUNDANCES:\'   \'91\'\n 2  10.88\n 3  -1.02\n 4  -0.73\n 5   0.76\n 6   6.45\n 7   5.81\n 8   7.11\n 9   2.39\n10   6.40\n11   4.24\n12   5.89\n13   4.41\n14   5.88\n15   3.41\n16   5.51\n17   3.20\n18   4.85\n19   3.06\n20   4.66\n21   1.02\n22   3.28\n23   1.94\n24   3.60\n25   3.45\n26   5.47\n27   2.85\n28   4.17\n29   2.22\n30   2.60\n31   1.05\n32   1.54\n33   0.27\n34   1.31\n35   0.51\n36   1.23\n37   0.33\n38   0.85\n39   0.15\n40   0.52\n41  -0.63\n42  -0.11\n43 -10.02\n44  -0.27\n45  -0.95\n46  -0.38\n47  -0.83\n48  -0.32\n49  -1.27\n50   0.04\n51  -1.02\n52   0.15\n53  -0.48\n54   0.22\n55  -0.95\n56   0.13\n57  -0.86\n58  -0.45\n59  -1.28\n60  -0.58\n61 -10.02\n62  -1.09\n63  -1.53\n64  -0.96\n65  -1.71\n66  -0.91\n67  -1.56\n68  -1.10\n69  -1.91\n70  -1.11\n71  -1.94\n72  -1.18\n73  -2.19\n74  -1.38\n75  -1.77\n76  -0.69\n77  -0.69\n78  -0.41\n79  -1.23\n80  -0.86\n81  -1.26\n82   0.01\n83  -1.38\n84 -10.02\n85 -10.02\n86 -10.02\n87 -10.02\n88 -10.02\n89 -10.02\n90  -1.97\n91 -10.02\n92  -2.57\n\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n')
            elif abundances is not None:
                f.write(f'\'INDIVIDUAL ABUNDANCES:\'   \'{len(abundances[0])}\'\n')
                for i in range(len(abundances[0])):
                    if abundances[0][i] <= 9:
                        f.write(f' {abundances[0][i]:<3}')
                    else: 
                        f.write(f'{abundances[0][i]:<3} ')
                    if abundances[0][i] == 2:
                        f.write(f' {abundances[1][i]:.2f}\n')
                    else:
                        if abundances[1][i] > 0:
                            f.write(f'  {abundances[1][i]:.2f}\n')
                        else:
                            f.write(f' {abundances[1][i]:.2f}\n')
                #f.write('\'ISOTOPES : \' \'36\'\n3.006   0.076\n3.007   0.924\n6.012   0.990\n6.013   0.010\n7.014   1.000\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n29.063  0.692\n29.065  0.308\n56.134  0.024\n56.135  0.066\n56.136  0.078\n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150\n62.148  0.112\n62.149  0.138\n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478\n63.153  0.522\n')
                f.write('\'ISOTOPES : \' \'66\'\n3.006   0.076\n3.007   0.924\n5.010   0.1982\n5.011   0.8018\n6.012   0.990\n6.013   0.010\n7.014   1.000\n19.039  0.93132\n19.040  0.00147\n19.041  0.06721\n20.040  0.96941\n20.042  0.00647\n20.043  0.00135\n20.044  0.02086\n20.046  0.00004\n20.048  0.00187\n22.046  0.08249\n22.047  0.07437\n22.048  0.7372\n22.049  0.05409\n22.050  0.05185\n28.058  0.680769\n28.060  0.262231\n28.061  0.011399\n28.062  0.036345\n28.064  0.009256\n29.063  0.692\n29.065  0.308\n31.069  0.60108\n31.071  0.39892\n56.134  0.024 \n56.135  0.066 \n56.136  0.078 \n56.137  0.112\n56.138  0.718\n57.139  0.999\n59.141  1.000\n60.142  0.270\n60.143  0.120\n60.144  0.237\n60.145  0.088\n60.146  0.171\n60.148  0.057\n60.150  0.057\n62.144  0.031\n62.147  0.150  \n62.148  0.112\n62.149  0.138 \n62.150  0.075\n62.152  0.267\n62.154  0.227\n63.151  0.478 \n63.153  0.522\n78.190  0.00014\n78.192  0.00783\n78.194  0.32967\n78.195  0.33832\n78.196  0.25242\n78.198  0.07163\n80.196  0.0015\n80.198  0.0997\n80.199  0.1687\n80.200  0.2310\n80.201  0.1318\n80.202  0.2986\n80.204  0.0687\n')
            if long:
                f.write('\'NFILES   :\' \'73\'\n')
                f.write(f'{path_to_turbospectrumNLTE}/DATA/Hlinedata\n{path_to_turbospectrumNLTE}DATA/lines/gf0320.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0340.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0350.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0360.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0380.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0400.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0420.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0440.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0460.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0475.list\n{path_to_turbospectrumNLTE}DATA/lines/GES_475_582.list\n{path_to_turbospectrumNLTE}DATA/lines/GES_582_680.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0720.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0760.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0800.list\n{path_to_turbospectrumNLTE}DATA/lines/gf08498.list\n{path_to_turbospectrumNLTE}DATA/lines/GES_8498_8950.list\n{path_to_turbospectrumNLTE}DATA/lines/gf0920.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1000.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1050.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1100.list\n{path_to_turbospectrumNLTE}DATA/lines/gf1200.list\n{path_to_turbospectrumNLTE}DATA/lines/16OH_A-X-bsyn_2000-4100-modif.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_5500.6000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_7000.7500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_9000.9500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_5500.6000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_7000.7500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_9000.9500.list\n{path_to_turbospectrumNLTE}DATA/lines/14NH-AX-Bernath2018.list\n{path_to_turbospectrumNLTE}DATA/lines/12C12C-Brookeetal-Swan.list\n{path_to_turbospectrumNLTE}DATA/lines/12C13C-Rametal-Swan.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_7000.7500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_9000.9500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_3700.4100.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_4100.4500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_4500.5000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_5000.5500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_5500.6000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_7500.8000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_8000.8500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_8500.9000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_9000.9500.list\n')
            else:
                f.write('\'NFILES   :\' \'14\'\n')
                f.write(f'{path_to_turbospectrumNLTE}/DATA/Hlinedata\n{path_to_turbospectrumNLTE}DATA/lines/GES_582_680.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/12CH-Masseron_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/13CH-Masseron_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/14NH-AX-Bernath2018.list\n{path_to_turbospectrumNLTE}DATA/lines/12C12C-Brookeetal-Swan.list\n{path_to_turbospectrumNLTE}DATA/lines/12C13C-Rametal-Swan.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/C12N14_Brooke_6500.7000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_5500.6000.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_6000.6500.list\n{path_to_turbospectrumNLTE}DATA/lines/C13N14_Sneden_6500.7000.list\n')

            f.write('\'SPHERICAL:\'  \'F\'\n  30\n  300.00\n  15\n  1.30\nEOF\n########################################################################\ndate\n@ k+=1\nend\n')
            f.write('cp {path_to_turbospectrumNLTE}{wdir}/'+f'syntspec/* {outpath}/syntspec\ncp contopac/* {path_to_turbospectrumNLTE}lines/contopac\ncd ..\n')
            #f.write('rm -rf ${wdir}\n')

def crea_istogramma(array, num_bin=10, dimensione=(8, 6), showPlot=True, save=False, outpath=os.getcwd(), outname='histo.pdf'):
    """
    Crea un istogramma a partire da un array di numeri.

    Args:
        array (list o numpy.ndarray): L'array di numeri da rappresentare.
        num_bin (int): Il numero di bin dell'istogramma. Default è 10.
        dimensione (tuple): La dimensione della figura (larghezza, altezza). Default è (8, 6).

    Returns:
        None
    """
    # Controlla che l'array non sia vuoto
    if len(array) == 0:
        print("L'array è vuoto. Fornire un array con almeno un elemento.")
        return

    # Calcola il minimo e il massimo dell'array
    minimo = min(array)
    massimo = max(array)

    # Crea l'istogramma
    plt.figure(figsize=dimensione)
    plt.hist(array, bins=num_bin, range=(minimo, massimo), color='blue', edgecolor='black', alpha=0.7)

    # Aggiungi etichette e titolo
    plt.title("Istogramma")
    plt.xlabel("Valori")
    plt.ylabel("Frequenza")

    # Mostra l'istogramma
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    if save:
        olddir=os.getcwd()
        os.chdir(outpath)
        plt.savefig(outname, format = "pdf", dpi=300)
        os.chdir(olddir)
    if showPlot:
        plt.show()
    plt.clf()


class ParallelRunner:
    """
    Questa classe esegue diversi programmi numerati in parallelo.
    """

    def __init__(self, base_program_path, num_programs):
        """
        Inizializza la classe.

        :param base_program_path: Directory base o prefisso del programma (es: /path/to/abundance).
        :param num_programs: Numero totale di programmi da eseguire (es: abundance1, abundance2, ..., abundanceN).
        """
        self.base_program_path = base_program_path
        self.num_programs = num_programs
        self.processes = []

    def run_parallel(self, additional_args=None):
        """
        Esegue i programmi numerati in parallelo.

        :param additional_args: Argomenti extra da passare ai programmi.
        """
        multiprocessing.freeze_support()
        additional_args = additional_args or []

        for idx in range(1, self.num_programs + 1):
            program_path = f"{self.base_program_path}{idx}"  # Costruisce il nome del programma (es: abundance1)
            command = [program_path, "-t"] + additional_args
            process = multiprocessing.Process(target=self.__run_command, args=(command, idx))
            self.processes.append(process)
            process.start()

        logging.info("Tutti i processi sono stati avviati.")

        # Aspetta che tutti i processi finiscano
        for process in self.processes:
            process.join()

        logging.info("Tutti i processi sono stati completati.")

    @staticmethod
    def __run_command(command, idx):
        """
        Esegue un singolo comando tramite subprocess.

        :param command: Il comando da eseguire (lista).
        :param idx: Identificatore del processo.
        """
        try:
            logging.info("Avvio del processo %d con comando: %s", idx, " ".join(command))
            subprocess.run(command, check=True)
            logging.info("Processo %d completato con successo.", idx)
        except subprocess.CalledProcessError as e:
            logging.error("Errore nel processo %d: %s", idx, e)

BASE_PROGRAM_PATH=f"{path_to_abundance}abundance"
NUM_PROGRAMS=12
# Integrazione nel codice principale
def main_integration():
    """
    Integrazione del ParallelRunner nel codice principale.
    """
    # Configurazione
    BASE_PROGRAM_PATH = f"{path_to_abundance}abundance"  # Sostituisci con il percorso base dei programmi
    NUM_PROGRAMS = 12  # Numero totale di programmi da eseguire

    # Eventuali argomenti aggiuntivi da passare ai programmi
    ADDITIONAL_ARGS = []

    # Logging
    logging.basicConfig(level=logging.INFO)

    # Esegui i programmi in parallelo
    runner = ParallelRunner(BASE_PROGRAM_PATH, NUM_PROGRAMS)
    runner.run_parallel(additional_args=ADDITIONAL_ARGS)

def run_moog_in_directory(directory):
    """
    Esegue il codice MOOGSILENT nella directory specificata utilizzando subprocess.Popen.
    
    :param directory: La directory in cui eseguire il codice.
    :return: Un oggetto Popen per monitorare il processo.
    """
    try:
        # Cambia nella directory specificata
        return subprocess.Popen(
            "/home/starcat8/moog_without_sm/MOOGSILENT",
            cwd=directory,  # Specifica la directory in cui eseguire il comando
            stdout=subprocess.PIPE,  # Cattura l'output standard
            stderr=subprocess.PIPE   # Cattura l'output di errore
        )
    except Exception as e:
        print(f"Errore nell'avviare il processo in {directory}: {e}")
        return None
def lanciaMOOGSILENT(processori):
    '''Dopo aver creato i file batch, lancia il codice MOOGSILENT in parallelo. \n'''
    directories = [path_to_moogBatch+f"runs{i}" for i in range(1, processori+1)]

    # Avvia i processi in parallelo
    processes = []
    for directory in directories:
        process = run_moog_in_directory(directory)
        if process:
            processes.append((directory, process))

    # Attendi che tutti i processi terminino
    for directory, process in processes:
        stdout, stderr = process.communicate()  # Attendi la fine del processo e cattura l'output
        if process.returncode == 0:
            print(f"Completato con successo in {directory}")
        else:
            print(f"Errore in {directory}: {stderr.decode('utf-8')}")





class FileData:
    def __init__(self, data_folder, lineList, ares):
        '''Questa classe aiuta a gestire le line list di un file spettrale. Si crea un oggetto FileData che contiene i dati relativi a tutti i file spettrali presenti in una cartella. \n\nI dati sono organizzati in un dizionario, dove ogni chiave è il nome del file e il valore è un dizionario contenente i parametri T, G, M e le colonne di EW e WL. Il metodo create_final_File() crea un file .csv contenente le larghezze equivalenti ordinate per wl T logg etc...\n Il nome dei file spettrali DEVE essere -m.mm_tttt_g.gg (il meno può non esserci).\n
        \n\nParametri: \n
        \n Data folder: cartella in cui sono contenuti i file spettrali. \n\nlineList: lista delle righe contenute nella linelist.\n
        lineList: nome del file contenente la linelist.\n
        ares: path al file eseguibile ARES.'''
        self.kek = 'kek'
    def leggiFile(self, data_folder, lineList, ares):
        os.chdir(ares+data_folder)
        files = os.listdir()
        pattern = r"(-?\d+\.\d+|\d+)"
        file_data = {}
        ind=0
        for file in files:
            os.chdir(ares)
            matches = re.findall(pattern, file)
            if len(matches) == 3:
                numbers = [float(match) for match in matches]
                file_data[file] = {
                    'id': ind,
                    'T': numbers[1],
                    'G': numbers[2],
                    'M': numbers[0]
                }
                ind += 1
            creaOptFileARES('SYNTHLINES/' + file, readlinedat=lineList, fileout='SYNTHLINES/' + file+'.ew',lambdaf=6800, rejt=0.999, lineresol=0.01, rvmask='0.0', path_to_fitsFiles='/home/starcat8/Scrivania/ANALISIATM/ARES-master1/ARES-master/SYNTHLINES', path_to_ARES='/home/starcat8/Scrivania/ANALISIATM/ARES-master1/ARES-master/', path_to_linemake='/home/starcat8/Scrivania/ANALISIATM/ARES-master1/ARES-master/')
            ! ./ARES
            os.chdir(os.getcwd() + '/SYNTHLINES')
            T = Table.read(file + '.ew', format='ascii')
            EW = T['col5']
            WL = T['col1']
            file_data[file]['EW'] = EW
            file_data[file]['WL'] = WL
        self.file_data = file_data
        self.files=files
    def filter_common_wavelengths(self):
        # Trova le lunghezze d'onda comuni a tutti i file
        common_wavelengths = set(self.file_data[next(iter(self.file_data))]['WL'])
        for file in self.file_data:
            common_wavelengths &= set(self.file_data[file]['WL'])

        # Filtra le righe di WL e EW per mantenere solo quelle comuni
        for file in self.file_data:
            wl = self.file_data[file]['WL']
            ew = self.file_data[file]['EW']
            common_indices = [i for i, w in enumerate(wl) if w in common_wavelengths]
            self.file_data[file]['WL'] = wl[common_indices]
            self.file_data[file]['EW'] = ew[common_indices]
    def create_final_file(self):
        self.filter_common_wavelengths()
        final_file = []
        for i in range(len(dati.file_data[dati.files[25]]['WL'])):
            for file in self.files:
                final_file.append([self.file_data[file]['WL'][i], self.file_data[file]['T'], self.file_data[file]['G'], self.file_data[file]['M'], self.file_data[file]['EW'][i]])
        self.masterdata = final_file
        df = pd.DataFrame(self.masterdata, columns=['WL', 'T', 'G', 'M', 'EW'])
        df_sorted = df.sort_values(by=['WL', 'T', 'G', 'M'])
        with open('masterdata.csv', 'w') as file:
            file.write(df_sorted.to_csv(index=False))
        return df_sorted
    def plotEW(self, T_value=-1, G_value=-5, M_value=5, WL_up=0, WL_low=0, legend=False):
        '''Plotta le larghezze equivalenti per un certo valore di T, G, M e un intervallo di lunghezze d'onda.'''
        os.chdir('/home/starcat8/Scrivania/ANALISIATM/ARES-master1/ARES-master/SYNTHLINES')
        df = pd.read_csv('masterdata.csv')
        df = df[df['WL'] > WL_low]
        df = df[df['WL'] < WL_up]
        if T_value != -1:
            df = df[df['T'] == T_value]
        if G_value != -5:
            df = df[df['G'] == G_value]
        if M_value != 5:
            df = df[df['M'] == M_value]
        if T_value == -1:
            plt.figure(figsize=(10, 6))
            colors = plt.cm.viridis(np.linspace(0, 1, len(df['WL'].unique())))
            for idx, wl in enumerate(df['WL'].unique()):
                WL_value = wl
                filtered_df = df[(df['G'] == G_value) & (df['M'] == M_value) & (df['WL'] == WL_value)]
                filtered_df = filtered_df.sort_values(by='T')
                plt.scatter(filtered_df['T'], filtered_df['EW'], color=colors[idx], label=f'G={G_value}, M={M_value}, WL={WL_value}')
                for i in range(len(filtered_df) - 1):
                    if filtered_df.iloc[i + 1]['T'] > filtered_df.iloc[i]['T']:
                        plt.plot(filtered_df.iloc[i:i + 2]['T'], filtered_df.iloc[i:i + 2]['EW'], color=colors[idx])
            plt.xlabel('T')
            plt.ylabel('EW')
            plt.title(f'Grafico di EW in funzione di T, G={G_value}, M={M_value}')
            if legend:
                plt.legend()
            plt.grid(True)
            plt.show()
        elif G_value == -5:
            plt.figure(figsize=(10, 6))
            colors = plt.cm.viridis(np.linspace(0, 1, len(df['WL'].unique())))
            for idx, wl in enumerate(df['WL'].unique()):
                WL_value = wl
                filtered_df = df[(df['T'] == T_value) & (df['M'] == M_value) & (df['WL'] == WL_value)]
                filtered_df = filtered_df.sort_values(by='T')
                plt.scatter(filtered_df['G'], filtered_df['EW'], color=colors[idx], label=f'G={G_value}, M={M_value}, WL={WL_value}')
                for i in range(len(filtered_df) - 1):
                    if filtered_df.iloc[i + 1]['G'] > filtered_df.iloc[i]['G']:
                        plt.plot(filtered_df.iloc[i:i + 2]['G'], filtered_df.iloc[i:i + 2]['EW'], color=colors[idx])
            plt.xlabel('G')
            plt.ylabel('EW')
            plt.title(f'Grafico di EW in funzione di G, T={T_value}, M={M_value}')
            if legend:
                plt.legend()
            plt.grid(True)
            plt.show()
        elif M_value == 5:
            plt.figure(figsize=(10, 6))
            colors = plt.cm.viridis(np.linspace(0, 1, len(df['WL'].unique())))
            for idx, wl in enumerate(df['WL'].unique()):
                WL_value = wl
                filtered_df = df[(df['T'] == T_value) & (df['G'] == G_value) & (df['WL'] == WL_value)]
                filtered_df = filtered_df.sort_values(by='T')
                plt.scatter(filtered_df['M'], filtered_df['EW'], color=colors[idx], label=f'G={G_value}, M={M_value}, WL={WL_value}')
                for i in range(len(filtered_df) - 1):
                    if filtered_df.iloc[i + 1]['M'] > filtered_df.iloc[i]['M']:
                        plt.plot(filtered_df.iloc[i:i + 2]['M'], filtered_df.iloc[i:i + 2]['EW'], color=colors[idx])
            plt.xlabel('M')
            plt.ylabel('EW')
            plt.title(f'Grafico di EW in funzione di M, T={T_value}, G={G_value}')
            if legend:
                plt.legend()
            plt.grid(True)
            plt.show()
        else:
            print('Lascia un parametro libero!')
    def plotEW1(self, T_value=-1, G_value=-5, M_value=5, WL_up=0, WL_low=0, legend=False, ax=None):
        '''Plotta le larghezze equivalenti per un certo valore di T, G, M e un intervallo di lunghezze d'onda.'''
        os.chdir('/home/starcat8/Scrivania/ANALISIATM/ARES-master1/ARES-master/SYNTHLINES')
        df = pd.read_csv('masterdata.csv')
        self.df = df
        df = df[df['WL'] > WL_low]
        df = df[df['WL'] < WL_up]
        if T_value != -1:
            df = df[df['T'] == T_value]
        if G_value != -5:
            df = df[df['G'] == G_value]
        if M_value != 5:
            df = df[df['M'] == M_value]
        if T_value == -1:
            colors = plt.cm.viridis(np.linspace(0, 1, len(df['WL'].unique())))
            for idx, wl in enumerate(df['WL'].unique()):
                WL_value = wl
                filtered_df = df[(df['G'] == G_value) & (df['M'] == M_value) & (df['WL'] == WL_value)]
                filtered_df = filtered_df.sort_values(by='T')
                ax.scatter(filtered_df['T'], filtered_df['EW'], color=colors[idx], label=f'G={G_value}, M={M_value}, WL={WL_value}')
                for i in range(len(filtered_df) - 1):
                    if filtered_df.iloc[i + 1]['T'] > filtered_df.iloc[i]['T']:
                        ax.plot(filtered_df.iloc[i:i + 2]['T'], filtered_df.iloc[i:i + 2]['EW'], color=colors[idx])
            ax.set_xlabel('T')
            ax.set_ylabel('EW')
            ax.set_title(f'G={G_value}, M={M_value}')
            if legend:
                ax.legend()
            ax.grid(True)
        elif G_value == -5:
            colors = plt.cm.viridis(np.linspace(0, 1, len(df['WL'].unique())))
            for idx, wl in enumerate(df['WL'].unique()):
                WL_value = wl
                filtered_df = df[(df['T'] == T_value) & (df['M'] == M_value) & (df['WL'] == WL_value)]
                filtered_df = filtered_df.sort_values(by='T')
                ax.scatter(filtered_df['G'], filtered_df['EW'], color=colors[idx], label=f'G={G_value}, M={M_value}, WL={WL_value}')
                for i in range(len(filtered_df) - 1):
                    if filtered_df.iloc[i + 1]['G'] > filtered_df.iloc[i]['G']:
                        ax.plot(filtered_df.iloc[i:i + 2]['G'], filtered_df.iloc[i:i + 2]['EW'], color=colors[idx])
            ax.set_xlabel('G')
            ax.set_ylabel('EW')
            ax.set_title(f'T={T_value}, M={M_value}')
            if legend:
                ax.legend()
            ax.grid(True)
        elif M_value == 5:
            colors = plt.cm.viridis(np.linspace(0, 1, len(df['WL'].unique())))
            for idx, wl in enumerate(df['WL'].unique()):
                WL_value = wl
                filtered_df = df[(df['T'] == T_value) & (df['G'] == G_value) & (df['WL'] == WL_value)]
                filtered_df = filtered_df.sort_values(by='T')
                ax.scatter(filtered_df['M'], filtered_df['EW'], color=colors[idx], label=f'G={G_value}, M={M_value}, WL={WL_value}')
                for i in range(len(filtered_df) - 1):
                    if filtered_df.iloc[i + 1]['M'] > filtered_df.iloc[i]['M']:
                        ax.plot(filtered_df.iloc[i:i + 2]['M'], filtered_df.iloc[i:i + 2]['EW'], color=colors[idx])
            ax.set_xlabel('M')
            ax.set_ylabel('EW')
            ax.set_title(f'T={T_value}, G={G_value}')
            if legend:
                ax.legend()
            ax.grid(True)
        else:
            print('Lascia un parametro libero!')
    
class line_List:
    '''Questa classe crea una linelist a partire da un file di input. \n
    listList: file di input contenente la linelist. \n
    ew_low: valore minimo di EW per filtrare le righe. \n
    ew_high: valore massimo di EW per filtrare le righe. \n
    qualityFlag: flag di qualità per filtrare le righe. \n
    '''
    def __init__(self, listList, ew_low, ew_high, qualityFlag):
        self.listList = listList
        self.ew_low = ew_low
        self.ew_high = ew_high
        self.qualityFlag = qualityFlag

#dati = FileData('SYNTHLINES', 'Iron_linesC', '/home/starcat8/Scrivania/ANALISIATM/ARES-master1/ARES-master/')


class Stella:
    def __init__(self, fits_name='', fits_folder='', rad_vel=-20000, t_eff=0, LOGg=0, metal=0, v_turbulence=0, instrument='HARPS-N'):
        """
        Inizializza un oggetto Stella.
        
        :param nome: Nome della stella
        :param temperatura: Temperatura della stella (in Kelvin)
        :param spettro: Lista o array con i dati dello spettro
        """
        if fits_folder == '':
            fits_folder = path_to_fitsFiles
        self.fits_name = fits_name
        print(f"Nome del file FITS: {self.fits_name}, cartella: {fits_folder}")
        self.nome = nome_da_Fits(path_to_fits=fits_folder, nome=self.fits_name, instrument=instrument)
        self.t_eff = t_eff
        self.v_r = rad_vel
        self.logg = LOGg
        self.metal = metal
        self.v_m = v_turbulence
        self.folder = path_to_output + self.nome
        self.fits_folder = fits_folder #path_to_ARES
        self.param_error = 100000
        self.ALL_STARS = []
        crea_cartella(self.folder)
        crea_cartella(self.folder+'/TEMP')
        os.chdir(self.fits_folder)
        self.fits = (fits.open(self.fits_name))
        hdul = fits.open(self.fits_name)
        self.fits_data = hdul[0].data
        self.instrument = instrument
        self.xi = self.fits[0].header.get('CRVAL1')
        self.dxi= self.fits[0].header.get('CDELT1')
        self.xf=self.xi+self.dxi*(self.fits[0].header.get("NAXIS1")-1)
        self.model = ''
        self.gmag=0
        self.zpvals5p=0
        self.zpvals6p=0
        if self.v_r == -20000 and self.instrument == 'HARPS-N':
            self.v_r = hdul[0].header.get('TNG TEL TARG RADVEL')
        if self.instrument == 'FIES':
            self.centroHa()
            self.stima_radvel_Ha()
            self.centroCa()
            self.stima_radvel_Ca()
        self.spettroCaricato = []
    def saveFits(self, new_filename="spettro_corretto.fits"):
        new_header = self.fits[0].header.copy()
        new_header['CRVAL1'] = self.xi
        new_header['CDELT1'] = self.dxi # Nuovo incremento
        if self.instrument == 'HARPS-N':
            new_header['TNG TEL TARG RADVEL'] = self.v_r
        new_hdu = fits.PrimaryHDU(data=self.fits_data, header=new_header)
        new_hdu.writeto(new_filename, overwrite=True)
        self.fits.close()
        print(f"File FITS aggiornato salvato come {new_filename} nella cartella {os.getcwd()}")
        self.fits_name = new_filename
        self.fits = (fits.open(self.fits_name))
        hdul = fits.open(self.fits_name)
        self.fits_data = hdul[0].data
        if self.instrument == 'HARPS-N':
            self.v_r = hdul[0].header.get('TNG TEL TARG RADVEL')
        self.xi = self.fits[0].header.get('CRVAL1')
        self.dxi= self.fits[0].header.get('CDELT1')
        self.xf=self.xi+self.dxi*(self.fits[0].header.get("NAXIS1")-1)
    def EW(self, limits=[10, 180], lineList=lineList):
        preparaAbundance(self.fits_name, eqw_low=limits[0], eqw_high=limits[1], lineList=lineList)
    def creaModel(self, T=-1, logg=10, MH_g=10, microTurb=10, processore=0, T0=0, logg0=0, MH_g0=0, n_istanze=1, outpath='', radSolver = 'spectrum', intpol=False, atlas_cycles=1, outName='', alpha_enhance=0, H=0.92150, He=0.07840, Li=-10.94, Be=-10.64, B=-9.49, C=-3.52, N=-4.12, O=-3.21, F=-7.48, Ne=-3.96, Na=-5.71, Mg=-4.46, Al=-5.57, Si=-4.49, P=-6.59, S=-4.71, Cl=-6.54, Ar=-5.64, K=-6.92, Ca=-5.68, Sc=-8.87, Ti=-7.02, V=-8.04, Cr=-6.37, Mn=-6.65, Fe=-4.54, Co=-7.12, Ni=-5.79, Cu=-7.83, Zn=-7.44, Ga=-9.16, Ge=-8.63, As=-9.67, Se=-8.63, Br=-9.41, Kr=-8.73, Rb=-9.44, Sr=-9.07, Y=-9.80, Zr=-9.44, Nb=-10.62, Mo=-10.12, Tc=-20.00, Ru=-10.20, Rh=-10.92, Pd=-10.35, Ag=-11.10, Cd=-10.27, In=-10.38, Sn=-10.04, Sb=-11.04, Te=-9.80, Io=-10.53, Xe=-9.87, Cs=-10.91, Ba=-9.91, La=-10.87, Ce=-10.46, Pr=-11.33, Nd=-10.54, Pm=-20.00, Sm=-11.03, Eu=-11.53, Gd=-10.92, Tb=-11.69, Dy=-10.90, Ho=-11.78, Er=-11.11, Tm=-12.04, Yb=-10.96, Lu=-11.98, Hf=-11.16, Ta=-12.17, W=-10.93, Re=-11.76, Os=-10.59, Ir=-10.69, Pt=-10.24, Au=-11.03, Hg=-10.91, Tl=-11.14, Pb=-10.09, Bi=-11.33, Po=-20.00, At=-20.00, Rn=-20.00,  Fr=-20.00, Ra=-20.00, Ac=-20.00, Th=-11.95, Pa=-20.00, U=-12.54, Np=-20.00, Pu=-20.00, Am=-20.00, Cm=20.00, Bk=20.00, Cf=20.00, Es=20.00): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
        '''Crea il file ModelEx, dove x è il numero del processore.\n
        Se si usa Turbospectrum il file verrà copiato in path_to_turboModels, se si usa SPECTRUM in path_to_abundance. In ogni caso si avrà una copia anche in path_to_atmosphere + interpol_folder. \n
        T: temperatura (kelvin)\n
        logg: gravità superficiale (dex)\n
        MH_g: [M/H] (dex).\n
        Processore : numero del processore che dovrà usare il modello per il calcolo. \n
        radSolver: \'spectrum\' per usare spectrum, \'turbo\' per usare turbospectrum, \'moog\' per moog \n
        '''
        nome_Stella = self.fits_name
        if T==-1:
            T=self.t_eff
        if logg==10:
            logg=self.logg
        if MH_g ==10:
            MH_g = self.metal
        if microTurb == 10:
            microTurb = self.v_m
        T=round(T, 1)
        logg=round(logg, 3)
        MH_g=round(MH_g, 3)
        microTurb=round(microTurb, 3)
        CHECK = checkExistingAtmo(T, logg, MH_g, microTurb, path='')
        CHECK=False
        if CHECK == False:
            if intpol:
                os.chdir(path_to_interpol)
                #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
                vturb=microTurb
                commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
                os.system(commandInterpol)
                #### FORMATTA KURUCZ ####
                header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"                                                                                   
                header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 {H:.5f} 2 {He:.5f}\n ABUNDANCE CHANGE  3 {Li:.2f}  4 {Be:.2f}  5  {B:.2f}  6  {C:.2f}  7  {N:.2f}  8  {O:.2f}\n ABUNDANCE CHANGE  9  {F:.2f} 10  {Ne:.2f} 11  {Na:.2f} 12  {Mg:.2f} 13  {Al:.2f} 14  {Si:.2f}\n ABUNDANCE CHANGE 15  {P:.2f} 16  {S:.2f} 17  {Cl:.2f} 18  {Ar:.2f} 19  {K:.2f} 20  {Ca:.2f}\n ABUNDANCE CHANGE 21  {Sc:.2f} 22  {Ti:.2f} 23  {V:.2f} 24  {Cr:.2f} 25  {Mn:.2f} 26  {Fe:.2f}\n ABUNDANCE CHANGE 27  {Co:.2f} 28  {Ni:.2f} 29  {Cu:.2f} 30  {Zn:.2f} 31  {Ga:.2f} 32  {Ge:.2f}\n ABUNDANCE CHANGE 33  {As:.2f} 34  {Se:.2f} 35  {Br:.2f} 36  {Kr:.2f} 37  {Rb:.2f} 38  {Sr:.2f}\n ABUNDANCE CHANGE 39  {Y:.2f} 40  {Zr:.2f} 41 {Nb:.2f} 42 {Mo:.2f} 43 {Tc:.2f} 44 {Ru:.2f}\n ABUNDANCE CHANGE 45 {Rh:.2f} 46 {Pd:.2f} 47 {Ag:.2f} 48 {Cd:.2f} 49 {In:.2f} 50 {Sn:.2f}\n ABUNDANCE CHANGE 51 {Sb:.2f} 52  {Te:.2f} 53 {Io:.2f} 54  {Xe:.2f} 55 {Cs:.2f} 56  {Ba:.2f}\n ABUNDANCE CHANGE 57 {La:.2f} 58 {Ce:.2f} 59 {Pr:.2f} 60 {Nd:.2f} 61 {Pm:.2f} 62 {Sm:.2f}\n ABUNDANCE CHANGE 63 {Eu:.2f} 64 {Gd:.2f} 65 {Tb:.2f} 66 {Dy:.2f} 67 {Ho:.2f} 68 {Er:.2f}\n ABUNDANCE CHANGE 69 {Tm:.2f} 70 {Yb:.2f} 71 {Lu:.2f} 72 {Hf:.2f} 73 {Ta:.2f} 74 {W:.2f}\n ABUNDANCE CHANGE 75 {Re:.2f} 76 {Os:.2f} 77 {Ir:.2f} 78 {Pt:.2f} 79 {Au:.2f} 80 {Hg:.2f}\n ABUNDANCE CHANGE 81 {Tl:.2f} 82 {Pb:.2f} 83 {Bi:.2f} 84 {Po:.2f} 85 {At:.2f} 86 {Rn:.2f}\n ABUNDANCE CHANGE 87 {Fr:.2f} 88 {Ra:.2f} 89 {Ac:.2f} 90 {Th:.2f} 91 {Pa:.2f} 92 {U:.2f}\n ABUNDANCE CHANGE 93 {Np:.2f} 94 {Pu:.2f} 95 {Am:.2f} 96 {Cm:.2f} 97 {Bk:.2f} 98 {Cf:.2f}\n ABUNDANCE CHANGE 99 {Es:.2f}\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
                tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"                    
                def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
                    with open(input_file, 'r') as f_in:
                        lines = f_in.readlines()
                    # Remove the first 3 rows and the last 2 rows
                    lines = lines[3:-2]
                    with open(output_file, 'w') as f_out:
                        f_out.write(header1)
                        f_out.write(header2)
                        f_out.writelines(lines)
                        f_out.write(tail)
                read_and_copy_file('modelatmosphere.txt', 'out.txt')
                if radSolver == 'spectrum':
                    if outpath=='':
                        outpath=path_to_abundance
                    if outName=='':
                        comandoATMOS = f"cp {path_to_interpol}out.txt {outpath}/Model{processore}"
                    else:
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}{outName}"
                    os.system(comandoATMOS)
                elif radSolver == 'turbo':
                    if outpath=='':
                        outpath=path_to_turboModels
                    if outName=='':
                        comandoATMOS = f"cp {path_to_interpol}out.txt {outpath}/Model{processore}"
                    else:
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}{outName}"
                    os.system(comandoATMOS)
                elif radSolver == 'moog':
                    if outpath=='':
                        outpath=path_to_moogBatch+f'runs{processore}/atlas9_models'
                    if outName=='':
                        comandoATMOS = f"cp {path_to_interpol}out.txt {outpath}/Model{processore}"
                    else:
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}{outName}"
                    os.system(comandoATMOS)
                    convertATLAS9_to_MOOGcompatible(outpath, f"Model{processore}", f"Model{processore}", T=T, G=logg, M=MH_g, V=microTurb)
                print('AWNAW')
            else:
                vturb=microTurb
                print('USA ATLAS9 per ATMOSFERA') #Da rifare in parallelo. Indi fare copie dei file di atlas lanciare scripti
                os.chdir(path_to_ATLAScom)
                creaATLAS9com(teff=T, logg=logg, metal=MH_g, vturb=vturb, modelName='out.txt', ncycles=atlas_cycles, alpha_enhancement=alpha_enhance)
                print(os.getcwd())
                try:
                    subprocess.run(['bash', '-c', 'source brewed_atlas9.com'], check=True)
                    print("Comando eseguito con successo.")
                except subprocess.CalledProcessError as e:
                    print(f"Errore durante l'esecuzione del comando: {e}")
                nome_file = "nome_del_file.txt"
                # Leggi tutte le righe del file
                with open('out.txt', "r") as file:
                    righe = file.readlines()
                righe[0] = f"TEFF   {T:.0f}  GRAVITY {logg:.2f} LTE\n"
                righe[1] = f"TITLE  [{MH_g:.2f}] VTURB={microTurb}  L/H=1.25 NOVER NEW ODF\n"
                # Scrivi di nuovo tutte le righe nel file
                with open('out.txt', "w") as file:
                    file.writelines(righe)
                if radSolver == 'spectrum':
                    if outpath=='':
                        outpath=path_to_abundance+'/'
                    if outName=='':
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}Model{processore}"
                    else:
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}{outName}"
                    os.system(comandoATMOS)
                elif radSolver == 'turbo':
                    if outpath=='':
                        outpath=path_to_turboModels+'/'
                    if outName=='':
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}Model{processore}"
                    else:
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}{outName}"
                    os.system(comandoATMOS)
                    copia('out.txt', path_to_ATLAScom, outpath, outName)
                elif radSolver == 'moog':
                    if outpath=='':
                        outpath=path_to_moogBatch+f'runs{processore}/atlas9_models/'
                    if outName=='':
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}Model{processore}"
                    else:
                        comandoATMOS = f"cp {path_to_ATLAScom}/out.txt {outpath}{outName}"
                    os.system(comandoATMOS)
                    convertATLAS9_to_MOOGcompatible(outpath, f"Model{processore}", f"Model{processore}", T=T, G=logg, M=MH_g, V=microTurb)
                print('AHOOOY!')
            copia(f"Model{processore}", outpath, path_to_atmosphere + interpol_folder, f"T{T:.0f}_G{logg:.2f}_M{MH_g:.2f}_{vturb:.2f}.model")          
        else:
            print("Atmosfera già esistente!")
            print(path_to_atmosphere + interpol_folder)
            print(outpath)
            copia(CHECK, path_to_atmosphere + interpol_folder, outpath, f"Model{processore}")
            if radSolver == 'moog':
                convertATLAS9_to_MOOGcompatible(outpath, f"Model{processore}", f"Model{processore}", T=T, G=logg, M=MH_g, V=microTurb)
        #preparaAbundance(nome_Stella)
        os.chdir(outpath)
        creaParamfileAbnd1(vturb, processore)
        return processore
    def caricaModel(self, T, G, M, V):
        '''Carica il modello atmosferico creato con creaModel.'''
        os.chdir(path_to_atmosphere + interpol_folder)
        model = f"T{T:.0f}_G{G:.2f}_M{M:.2f}_{V:.2f}.model"
        self.model = path_to_atmosphere + interpol_folder +'/'+ model
        return model
    def synth_creaModel(self, T=-1, logg=10, MH_g=10, microTurb=10, TURBVEL=2, processore=0, T0=0, logg0=0, MH_g0=0, n_istanze=1, outpath=path_to_spectrum, radSolver = 'spectrum', intpol=False, atlas_cycles=1, startwl=4700, endwl=5500, Step=0.01, limit_width=10, absolute_abund=0, path_to_model='', name_model='', H=0.92150, He=0.07840, Li=-10.94, Be=-10.64, B=-9.49, C=-3.52, N=-4.12, O=-3.21, F=-7.48, Ne=-3.96, Na=-5.71, Mg=-4.46, Al=-5.57, Si=-4.49, P=-6.59, S=-4.71, Cl=-6.54, Ar=-5.64, K=-6.92, Ca=-5.68, Sc=-8.87, Ti=-7.02, V=-8.04, Cr=-6.37, Mn=-6.65, Fe=-4.54, Co=-7.12, Ni=-5.79, Cu=-7.83, Zn=-7.44, Ga=-9.16, Ge=-8.63, As=-9.67, Se=-8.63, Br=-9.41, Kr=-8.73, Rb=-9.44, Sr=-9.07, Y=-9.80, Zr=-9.44, Nb=-10.62, Mo=-10.12, Tc=-20.00, Ru=-10.20, Rh=-10.92, Pd=-10.35, Ag=-11.10, Cd=-10.27, In=-10.38, Sn=-10.04, Sb=-11.04, Te=-9.80, Io=-10.53, Xe=-9.87, Cs=-10.91, Ba=-9.91, La=-10.87, Ce=-10.46, Pr=-11.33, Nd=-10.54, Pm=-20.00, Sm=-11.03, Eu=-11.53, Gd=-10.92, Tb=-11.69, Dy=-10.90, Ho=-11.78, Er=-11.11, Tm=-12.04, Yb=-10.96, Lu=-11.98, Hf=-11.16, Ta=-12.17, W=-10.93, Re=-11.76, Os=-10.59, Ir=-10.69, Pt=-10.24, Au=-11.03, Hg=-10.91, Tl=-11.14, Pb=-10.09, Bi=-11.33, Po=-20.00, At=-20.00, Rn=-20.00,  Fr=-20.00, Ra=-20.00, Ac=-20.00, Th=-11.95, Pa=-20.00, U=-12.54, Np=-20.00, Pu=-20.00, Am=-20.00, Cm=20.00, Bk=20.00, Cf=20.00, Es=20.00): #ritorna il nome del file prodotto da abundance con l'atmosfera creata tramite interpolazione
        '''Usare questa funzione per creare i modelli degli spettri sintetici. Inoltre crea il file parameters e il file stdatom che verranno letti da spectrum.\nI metalli vanno espressi in termini del ferro, ovvero come [X/Fe]\nT: temperatura (kelvin)
        logg: gravità superficiale (dex)
        MH_g: [M/H] (dex).\n
        Processore il "numero del processore" che dovrà usare il modello per il calcolo. \n
        Crea il file ModelEx, dove x è il numero del processore.'''
        # solarAbnd = [0.92150, 0.07840, -10.94, -10.64, -9.49, -3.52, -4.12, -3.21, -7.48, -3.96, -5.71, -4.46, -5.57, -4.49, -6.59, -4.71, -6.54, -5.64, -6.92, -5.68, -8.87, -7.02, -8.04, -6.37, -6.65, -4.54, -7.12, -5.79, -7.83, -7.44, -9.16, -8.63, -9.67, -8.63, -9.41, -8.73, -9.44, -9.07, -9.80, -9.44, -10.62, -10.12, -20.00, -10.20, -10.92, -10.35, -11.10, -10.27, -10.38, -10.04, -11.04, -9.80, -10.53, -9.87, -10.91, -9.91, -10.87, -10.46, -11.33, -10.54, -20.00, -11.03, -11.53, -10.92, -11.69, -10.90, -11.78, -11.11, -12.04, -10.96, -11.98, -11.16, -12.17, -10.93, -11.76, -10.59, -10.69, -10.24, -11.03, -10.91, -11.14, -10.09, -11.33, -20.00, -20.00, -20.00, -20.00, -20.00, -20.00, -11.95, -20.00, -12.54]
        # scaled_solarAbnd = []
        # starAbnd = []
        # for i in range(solarAbnd):
        #     if i < 2:
        #         scaled_solarAbnd.append(solarAbnd[i])
        #     else:
        #         scaled_solarAbnd.append(solarAbnd[i]+MH_g)
        # starAbnd = []
        # nome_Stella = self.fits_name
        if T==-1:
            T=self.t_eff
        if logg==10:
            logg=self.logg
        if MH_g ==10:
            MH_g = self.metal
        if microTurb == 10:
            microTurb = self.v_m        
        os.chdir(path_to_interpol)
        #vturb = 0.14 - 0.08 * (MH_g) + 4.9 * T / 10000 - 0.47 * logg
        vturb=microTurb
        if path_to_model=='':
            self.creaModel(T, logg, MH_g, microTurb, outpath=self.folder+'/', radSolver=radSolver, intpol=intpol, atlas_cycles=atlas_cycles, outName='synth.mod',H=H, He=He, Li=Li, Be=Be, B=B, C=C, N=N, O=O, F=F, Ne=Ne, Na=Na, Mg=Mg, Al=Al, Si=Si, P=P, S=S, Cl=Cl, Ar=Ar, K=K, Ca=Ca, Sc=Sc, Ti=Ti, V=V, Cr=Cr, Mn=Mn, Fe=Fe, Co=Co, Ni=Ni, Cu=Cu, Zn=Zn, Ga=Ga, Ge=Ge, As=As, Se=Se, Br=Br, Kr=Kr, Rb=Rb, Sr=Sr, Y=Y, Zr=Zr, Nb=Nb, Mo=Mo, Tc=Tc, Ru=Ru, Rh=Rh, Pd=Pd, Ag=Ag, Cd=Cd, In=In, Sn=Sn, Sb=Sb, Te=Te, Io=Io, Xe=Xe, Cs=Cs, Ba=Ba, La=La, Ce=Ce, Pr=Pr, Nd=Nd, Pm=Pm, Sm=Sm, Eu=Eu, Gd=Gd, Tb=Tb, Dy=Dy, Ho=Ho, Er=Er, Tm=Tm, Yb=Yb, Lu=Lu, Hf=Hf, Ta=Ta, W=W, Re=Re, Os=Os, Ir=Ir, Pt=Pt, Au=Au, Hg=Hg, Tl=Tl, Pb=Pb, Bi=Bi, Po=Po, At=At, Rn=Rn,  Fr=Fr, Ra=Ra, Ac=Ac, Th=Th, Pa=Pa, U=U)
            path_to_model=self.folder+'/'
        else:
            self.creaModel(T, logg, MH_g, microTurb, outpath=path_to_model, radSolver=radSolver, intpol=intpol, atlas_cycles=atlas_cycles, outName='synth.mod',H=H, He=He, Li=Li, Be=Be, B=B, C=C, N=N, O=O, F=F, Ne=Ne, Na=Na, Mg=Mg, Al=Al, Si=Si, P=P, S=S, Cl=Cl, Ar=Ar, K=K, Ca=Ca, Sc=Sc, Ti=Ti, V=V, Cr=Cr, Mn=Mn, Fe=Fe, Co=Co, Ni=Ni, Cu=Cu, Zn=Zn, Ga=Ga, Ge=Ge, As=As, Se=Se, Br=Br, Kr=Kr, Rb=Rb, Sr=Sr, Y=Y, Zr=Zr, Nb=Nb, Mo=Mo, Tc=Tc, Ru=Ru, Rh=Rh, Pd=Pd, Ag=Ag, Cd=Cd, In=In, Sn=Sn, Sb=Sb, Te=Te, Io=Io, Xe=Xe, Cs=Cs, Ba=Ba, La=La, Ce=Ce, Pr=Pr, Nd=Nd, Pm=Pm, Sm=Sm, Eu=Eu, Gd=Gd, Tb=Tb, Dy=Dy, Ho=Ho, Er=Er, Tm=Tm, Yb=Yb, Lu=Lu, Hf=Hf, Ta=Ta, W=W, Re=Re, Os=Os, Ir=Ir, Pt=Pt, Au=Au, Hg=Hg, Tl=Tl, Pb=Pb, Bi=Bi, Po=Po, At=At, Rn=Rn,  Fr=Fr, Ra=Ra, Ac=Ac, Th=Th, Pa=Pa, U=U)
        # if path_to_model == '' or name_model == '':
        #     commandInterpol = f"{path_to_pythonEnv} pykmod.py {T} {logg} {vturb} {MH_g}"
        # os.system(commandInterpol)
        # #### FORMATTA KURUCZ ####
        # header1 = f"TEFF   {T}.  GRAVITY {logg} LTE\nTITLE  [{MH_g}] VTURB={vturb}  L/H=1.25 NOVER NEW ODF\n"                                                                                   
        # header2 = f" OPACITY IFOP 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0\n CONVECTION ON   1.25 TURBULENCE OFF  0.00  0.00  0.00  0.00\nABUNDANCE SCALE   {creaHeader_metal(MH_g):.5f} ABUNDANCE CHANGE 1 {H:.5f} 2 {He:.5f}\n ABUNDANCE CHANGE  3 {Li:.2f}  4 {Be:.2f}  5  {B:.2f}  6  {C:.2f}  7  {N:.2f}  8  {O:.2f}\n ABUNDANCE CHANGE  9  {F:.2f} 10  {Ne:.2f} 11  {Na:.2f} 12  {Mg:.2f} 13  {Al:.2f} 14  {Si:.2f}\n ABUNDANCE CHANGE 15  {P:.2f} 16  {S:.2f} 17  {Cl:.2f} 18  {Ar:.2f} 19  {K:.2f} 20  {Ca:.2f}\n ABUNDANCE CHANGE 21  {Sc:.2f} 22  {Ti:.2f} 23  {V:.2f} 24  {Cr:.2f} 25  {Mn:.2f} 26  {Fe:.2f}\n ABUNDANCE CHANGE 27  {Co:.2f} 28  {Ni:.2f} 29  {Cu:.2f} 30  {Zn:.2f} 31  {Ga:.2f} 32  {Ge:.2f}\n ABUNDANCE CHANGE 33  {As:.2f} 34  {Se:.2f} 35  {Br:.2f} 36  {Kr:.2f} 37  {Rb:.2f} 38  {Sr:.2f}\n ABUNDANCE CHANGE 39  {Y:.2f} 40  {Zr:.2f} 41 {Nb:.2f} 42 {Mo:.2f} 43 {Tc:.2f} 44 {Ru:.2f}\n ABUNDANCE CHANGE 45 {Rh:.2f} 46 {Pd:.2f} 47 {Ag:.2f} 48 {Cd:.2f} 49 {In:.2f} 50 {Sn:.2f}\n ABUNDANCE CHANGE 51 {Sb:.2f} 52  {Te:.2f} 53 {Io:.2f} 54  {Xe:.2f} 55 {Cs:.2f} 56  {Ba:.2f}\n ABUNDANCE CHANGE 57 {La:.2f} 58 {Ce:.2f} 59 {Pr:.2f} 60 {Nd:.2f} 61 {Pm:.2f} 62 {Sm:.2f}\n ABUNDANCE CHANGE 63 {Eu:.2f} 64 {Gd:.2f} 65 {Tb:.2f} 66 {Dy:.2f} 67 {Ho:.2f} 68 {Er:.2f}\n ABUNDANCE CHANGE 69 {Tm:.2f} 70 {Yb:.2f} 71 {Lu:.2f} 72 {Hf:.2f} 73 {Ta:.2f} 74 {W:.2f}\n ABUNDANCE CHANGE 75 {Re:.2f} 76 {Os:.2f} 77 {Ir:.2f} 78 {Pt:.2f} 79 {Au:.2f} 80 {Hg:.2f}\n ABUNDANCE CHANGE 81 {Tl:.2f} 82 {Pb:.2f} 83 {Bi:.2f} 84 {Po:.2f} 85 {At:.2f} 86 {Rn:.2f}\n ABUNDANCE CHANGE 87 {Fr:.2f} 88 {Ra:.2f} 89 {Ac:.2f} 90 {Th:.2f} 91 {Pa:.2f} 92 {U:.2f}\n ABUNDANCE CHANGE 93 {Np:.2f} 94 {Pu:.2f} 95 {Am:.2f} 96 {Cm:.2f} 97 {Bk:.2f} 98 {Cf:.2f}\n ABUNDANCE CHANGE 99 {Es:.2f}\nREAD DECK6 72 RHOX,T,P,XNE,ABROSS,ACCRAD,VTURB, FLXCNV,VCONV,VELSND\n"
        # tail = f"PRADK {getTail(find_closest(T, T_EFF_Nint), find_closest(logg, LOGG_Nint), find_closest(MH_g, MH_Nint))}" + "\nBEGIN                    ITERATION  15 COMPLETED"                    
        # def read_and_copy_file(input_file, output_file, header1=header1, header2=header2, tail=tail):
        #     with open(input_file, 'r') as f_in:
        #         lines = f_in.readlines()
        #     # Remove the first 3 rows and the last 2 rows
        #     lines = lines[3:-2]
        #     with open(output_file, 'w') as f_out:
        #         f_out.write(header1)
        #         f_out.write(header2)
        #         f_out.writelines(lines)
        #         f_out.write(tail)
        # if path_to_model == '' or name_model == '':
        #     read_and_copy_file('modelatmosphere.txt', 'out.txt')
        # if processore == -1:
        #     comandoATMOS = f"cp {path_to_interpol}out.txt {outpath}/Model0"
        # else:
        #     comandoATMOS = f"cp {path_to_interpol}out.txt {outpath}/Model{processore}"
        # os.system(comandoATMOS)
        # #preparaAbundance(nome_Stella)

        if radSolver=='spectrum':
            os.chdir(outpath)
            if processore == -1:
                creaParamfileSpectrum(vturb, f"Model", self.nome, startwl, endwl, Step)
                if path_to_model != '' and name_model != '':
                    copia(name_model, path_to_model, path_to_spectrum, 'Model')
            else:
                creaParamfileSpectrum(vturb, f"Model{processore}", self.nome, startwl, endwl, Step)
                if path_to_model != '' and name_model != '':
                    copia(name_model, path_to_model, path_to_spectrum, f'Model{processore}')
        elif radSolver=='turbo':
            os.chdir(path_to_turbospectrumNLTE)
            crea_synthScriptTurbo(path_to_model=path_to_model, modelName='synth.mod', wl_min=startwl, wl_max=endwl, delta_wl=Step, TURBVEL = TURBVEL, limit_width=limit_width)
        creaChem(H, He, Li, Be, B, C, N, O, F, Ne, Na, Mg, Al, Si, P, S, Cl, Ar, K, Ca, Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, Ga, Ge, As, Se, Br, Kr, Rb, Sr, Y, Zr, Nb, Mo, Tc, Ru, Rh, Pd, Ag, Cd, In, Sn, Sb, Te, Io, Xe, Cs, Ba, La, Ce, Pr, Nd, Pm, Sm, Eu, Gd, Tb, Dy, Ho, Er, Tm, Yb, Lu, Hf, Ta, W, Re, Os, Ir, Pt, Au, Hg, Tl, Pb, Bi, Po, At, Rn,  Fr, Ra, Ac, Th, Pa, U, metal=MH_g)
        #creaChem(metal=MH_g)
        return processore
    def x_axis(self):
        returner = np.zeros(self.fits[0].header.get("NAXIS1"))
        for i in range(self.fits[0].header.get("NAXIS1")):
            returner[i] = self.xi + self.dxi*i
        self.xaxis = returner
        return self.xaxis
    def next_param(self, NLTE=0, searchMatrix=s_Matrix):
        #blocca_stampe()
        temperatu = self.t_eff
        G = self.logg
        M = self.metal
        V = self.v_m
        nome_file = f"HD115575_-1.99_4342_0.63_1.94"
        nome_stella = self.fits_name
        path_to_stella = self.folder
        if NLTE==0:
            temp0 = simpleAnalysisSingle(path_to_stella, nome_file, 26.0)
            temp1 = simpleAnalysisSingle(path_to_stella, nome_file, 26.1)
        elif NLTE == 1:
            temp0 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.0)
            temp1 = NLTE_AnalysisSingle(path_to_stella, nome_file, temperatu, G, M, V, 26.1)
        sgn=1
        g=-1.0
        teff=-1.0
        met=1.0
        DG = 0
        DT = 0
        returner = []
        v=sp.symbols('v')
        # A = np.array([[dsdg, dsdt, dsdm],
        #      [dfdg, dfdt, dfdm],
        #      [dydg, dydt, dydm]])
        #riattiva_stampe()
        #sp.pprint(s_Matrix)
        A = searchMatrix
        AA=A.subs(v, V)
        A_numpy = np.array(AA.tolist(), dtype=float)
        A_inv = np.linalg.inv(A_numpy)
        f=-1*((temp0[0])-(temp1[0]))
        s = -1*(temp0[1])
        m = -1*(temp0[0]-(M))
        ewr = -1*(temp0[3])
        B = np.array([s, f, m, ewr])
        #blocca_stampe()
        if abs(temp0[1]) > 0.0001 or abs((temp0[0])-(temp1[0])) > 0.0003 or abs(M - (temp0[0]))>0.001:
            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
            DT = (np.dot(A_inv, B))[0]
            DG = (np.dot(A_inv, B))[1]
            DM = (np.dot(A_inv, B))[2]
            Dv = (np.dot(A_inv, B))[3]
            print(DT)
            print(DG)
            print(DM)
            print(Dv)
            print(f)
            print(s)
            print(m)
            print(ewr)
            if abs(DG) > 0.5:
                DG=0.5*(DG/abs(DG))
            if abs(DT) > 250:
                DT = 250*(DT/abs(DT))
            if abs(DM) > 0.5:
                DM = 0.5*(DM/abs(DM))
            if abs(Dv) > 0.3:
                Dv = 0.3*(Dv/abs(Dv))
            temperatu=temperatu+DT
            g=G+DG
            met = M+DM
            velturb=V+Dv
            teff= temperatu
        returner.append(teff)
        returner.append(g)
        returner.append(met)
        returner.append(velturb)
        return returner
    def calcola_abundance(processore):
        if __name__ == "__main__":
            main_integration()
    def searchParam2(self, processori=1, NLTE=1, TRIALS = 50, t01=-1, g01=-2, m01=10, v01=-1, t02=-1, g02=-2, m02=10, v02=-1, t03=-1, g03=-2, m03=10, v03=-1, t04=-1, g04=-2, m04=10, v04=-1, t_fixed=0, g_fixed=-1, m_fixed=5, v_fixed=-5, mCheck=3):
        '''Usare un numero di processori divisibile per 4\n NLTE = 0 per modelli LTE, NLTE = 1 per modelli NLTE\n TRIALS = numero di iterazioni\n t0, g0, m0, v0 = condizioni iniziali\n t_fixed, g_fixed, m_fixed, v_fixed = parametri fissati\n mCheck = 3 per modelli con 3 parametri, 4 per modelli con 4 parametri'''
        if processori%4 != 0:
            print("Inserisci un numero di processori divisibile per 4. (e.g. processori=4, processori=12)")
            return 200
        fixd_velturb = True
        CORRECTION = True

        if True: #Setting initial conditions
            if t01==-1:
                t_start1 = int(random.uniform(4000, 4300))
            else:
                t_start1 = t01
            if t02==-1:
                t_start2= int(random.uniform(4300, 4700))
            else:
                t_start2=t02
            if t03==-1:
                t_start3 = int(random.uniform(4700, 5000))
            else:
                t_start3 = t03
            if t04==-1:
                t_start4= int(random.uniform(4300, 4600))
            else:
                t_start4=t04
            
            if g01==-2:
                g_start1= random.uniform(0, 0.7)
            else:
                g_start1 = g01
            if g02==-2:
                g_start2=random.uniform(0.7, 1.45)
            else:
                g_start2=g02
            if g03==-2:
                g_start3= random.uniform(1.45, 2.0)
            else:
                g_start3 = g03
            if g04==-2:
                g_start4=random.uniform(1.0, 1.75)
            else:
                g_start4=g04

            if m01==10:
                m_start1= random.uniform(-2, 0)
            else:
                m_start1 = m01
            if m02==10:
                m_start2 = random.uniform(-2, 0)
            else:
                m_start2=m02
            if m03==10:
                m_start3= random.uniform(-2, 0)
            else:
                m_start3 = m03
            if m04==10:
                m_start4 = random.uniform(-2, 0)
            else:
                m_start4 = m04
            
            if v01==-1:
                v01=vturbo(t_start1, g_start1, m_start1)
            if v02==-1:
                v02=vturbo(t_start2, g_start2, m_start2)
            if v03==-1:
                v03=vturbo(t_start3, g_start3, m_start3)
            if v04==-1:
                v04=vturbo(t_start4, g_start4, m_start4)

        # m_start2=random.uniform(-2, 0)
        # t_start3 = int(random.uniform(4000, 5000))
        # g_start3 = random.uniform(0, 2)
        # m_start3 = random.uniform(-2, 0)
        # t_start4 = int(random.uniform(4000, 5000))
        # g_start4 = random.uniform(0, 2)
        # m_start4 = random.uniform(-2, 0)
        if t_fixed != 0:
            t_start1 = t_fixed
            t_start2 = t_fixed
            t_start3 = t_fixed
            t_start4 = t_fixed
        if g_fixed != -1:
            g_start1 = g_fixed
            g_start2 = g_fixed
            g_start3 = g_fixed
            g_start4 = g_fixed
        if m_fixed != 5:
            m_start1 = m_fixed
            m_start2 = m_fixed
            m_start3 = m_fixed
            m_start4 = m_fixed
        if v_fixed != -5:
            v01 = v_fixed
            v02 = v_fixed
            v03 = v_fixed
            v04 = v_fixed
        self.dopcor()
        self.saveFits()
        delete_files(path_to_abundance)
        #blocca_stampe()
        if True: #Starting block
            crea_ATMinterpol(t_start1, g_start1, m_start1, self.fits_name, instrument=self.instrument)
            crea_ATMinterpol(t_start2, g_start2, m_start2, self.fits_name, instrument=self.instrument)
            crea_ATMinterpol(t_start3, g_start3, m_start3, self.fits_name, instrument=self.instrument)
            crea_ATMinterpol(t_start4, g_start4, m_start4, self.fits_name, instrument=self.instrument)
            search_data1 = []
            search_data2 = []
            search_data3 = []
            search_data4 = []
            master_search_data1 = []
            master_search_data2 = []
            master_search_data3 = []
            master_search_data4 = []
            self.ALL_STARS=[]
            search_data1.append([t_start1, g_start1, m_start1, v01, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
            search_data2.append([t_start2, g_start2, m_start2, vturbo(t_start2, g_start2, m_start2), f"{self.nome}_{m_start2:.2f}_{t_start2:.0f}_{g_start2:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
            search_data3.append([t_start3, g_start3, m_start3, vturbo(t_start3, g_start3, m_start3), f"{self.nome}_{m_start3:.2f}_{t_start3:.0f}_{g_start3:.2f}"])
            search_data4.append([t_start4, g_start4, m_start4, vturbo(t_start4, g_start4, m_start4), f"{self.nome}_{m_start4:.2f}_{t_start4:.0f}_{g_start4:.2f}"])
        os.chdir(path_to_output)
        o = 0
        boundary_check = True
        s=[]
        F=[]
        Y=[]
        S=[]
        crea_cartella(self.folder)
        crea_cartella(self.folder+"/TEMP")
        if True: #help block
            DDT1 = 100
            DDG1 = 0.15
            DDM1 = 0.15
            DDV1 = 0.15
            DDT2 = 100
            DDG2 = 0.15
            DDM2 = 0.15
            DDV2 = 0.15
            DDT3 = 100
            DDG3 = 0.15
            DDM3 = 0.15
            DDV3 = 0.15
            DDT4 = 100
            DDG4 = 0.15
            DDM4 = 0.15
            DDV4 = 0.15
            best1=1
            best2=1
            best3=1
            best4=1
            old_best1=200000
            old_best2=200000
            old_best3=200000
            old_best4=200000
            old_best1_index=-2
            old_best2_index=-2
            old_best3_index=-2
            old_best4_index=-2
        nome_temp=self.nome
        for j in range(TRIALS): ## Numero di iterazioni
            if boundary_check: ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
                if search_data1[0][1] <= 0.0:
                    search_data1[0][1] = 0.5
                if search_data1[0][0] <= 3700:
                    search_data1[0][0] = 3750
                if search_data1[0][1] > 2.8:
                    search_data1[0][1] = 2.5
                if abs(search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])-0.081
                if (search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])+0.081
                
                if search_data2[0][1] <= 0.0:
                    search_data2[0][1] = 0.5
                if search_data2[0][0] <= 3700:
                    search_data2[0][0] = 3750
                if search_data2[0][1] > 2.8:
                    search_data2[0][1] = 2.5
                if abs(search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])-0.081
                if (search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])+0.081
                
                if search_data3[0][1] <= 0.0:
                    search_data3[0][1] = 0.5
                if search_data3[0][0] <= 3700:
                    search_data3[0][0] = 3750
                if search_data3[0][1] > 2.8:
                    search_data3[0][1] = 2.5
                if abs(search_data3[0][3] - vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data3[0][3] = vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])-0.081
                if (search_data3[0][3] - vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data3[0][3] = vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])+0.081
                

                if search_data4[0][1] <= 0.0:
                    search_data4[0][1] = 0.5
                if search_data4[0][0] <= 3700:
                    search_data4[0][0] = 3750
                if search_data4[0][1] > 2.8:
                    search_data4[0][1] = 2.5
                if abs(search_data4[0][3] - vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])) > 0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data4[0][3] = vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])-0.081
                if (search_data4[0][3] - vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])) < -0.1:
                    print("NOOOOOOOOOOOOOOOOO")
                    search_data4[0][3] = vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])+0.081

            for k in range(int(processori/4)): # Crea i modelli da analizzare
                if k == 0:
                    a1=checkExist_newParam("1"+self.nome, search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], self.folder + "/TEMP")
                    search_data1[k][0] = a1[0]
                    search_data1[k][1] = a1[1]
                    search_data1[k][2] = a1[2]
                    search_data1[k][3] = a1[3]
                    if search_data1[k][1]<0:
                        search_data1[k][1]=0.01
                    if search_data1[k][1]>3.0:
                        search_data1[k][1]=2.99
                    
                    if t_fixed != 0:  #vari if se uno dei parametri è da tenere fisso
                        search_data1[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data1[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data1[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data1[k][3] = v_fixed
                    
                    self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], processore=k+1)
                    print(f"{search_data1[k][0]}, {search_data1[k][1]}, {search_data1[k][2]}, {search_data1[k][3]}")
                    a2=checkExist_newParam("2"+self.nome, search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], self.folder + "/TEMP")
                    search_data2[k][0] = a2[0]
                    search_data2[k][1] = a2[1]
                    search_data2[k][2] = a2[2]
                    search_data2[k][3] = a2[3]
                    if search_data2[k][1]<0:
                        search_data2[k][1]=0.01
                    if search_data2[k][1]>3.0:
                        search_data2[k][1]=2.99
                    if t_fixed != 0:
                        search_data2[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data2[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data2[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data2[k][3] = v_fixed
                    self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], processore=k+1+int(processori/4))
                    print(f"{search_data2[k][0]}, {search_data2[k][1]}, {search_data2[k][2]}, {search_data2[k][3]}")
                    a3=checkExist_newParam("3"+self.nome, search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], self.folder + "/TEMP")
                    search_data3[k][0] = a3[0]
                    search_data3[k][1] = a3[1]
                    search_data3[k][2] = a3[2]
                    search_data3[k][3] = a3[3]
                    if search_data3[k][1]<0:
                        search_data3[k][1]=0.01
                    if search_data3[k][1]>3.0:
                        search_data3[k][1]=2.99
                    if t_fixed != 0:
                        search_data3[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data3[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data3[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data3[k][3] = v_fixed
                    self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], processore=k+1+2*int(processori/4))
                    print(f"{search_data3[k][0]}, {search_data3[k][1]}, {search_data3[k][2]}, {search_data3[k][3]}")
                    a4=checkExist_newParam("4"+self.nome, search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], self.folder + "/TEMP")
                    search_data4[k][0] = a4[0]
                    search_data4[k][1] = a4[1]
                    search_data4[k][2] = a4[2]
                    search_data4[k][3] = a4[3]
                    if search_data4[k][1]<0:
                        search_data4[k][1]=0.0
                    if search_data4[k][1]>3.0:
                        search_data4[k][1]=2.99
                    if t_fixed != 0:
                        search_data4[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data4[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data4[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data4[k][3] = v_fixed
                    self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], processore=k+1+3*int(processori/4))
                    print(f"{search_data4[k][0]}, {search_data4[k][1]}, {search_data4[k][2]}, {search_data4[k][3]}")
                else:
                    if abs(DDT1) < 100:
                        DDT1 = 100
                    if abs(DDG1) < 0.25:
                        DDG1=0.25
                    if abs(DDM1) < 0.25:
                        DDM1=0.25
                    if abs(DDV1) < 0.25:
                        DDV1 = 0.25
                    if abs(DDT2) < 100:
                        DDT2 = 100
                    if abs(DDG2) < 0.25:
                        DDG2=0.25
                    if abs(DDM2) < 0.25:
                        DDM2=0.25
                    if abs(DDV2) < 0.25:
                        DDV2 = 0.25
                    if abs(DDT3) < 100:
                        DDT3 = 100
                    if abs(DDG3) < 0.25:
                        DDG3=0.25
                    if abs(DDM3) < 0.25:
                        DDM3=0.25
                    if abs(DDV3) < 0.25:
                        DDV3 = 0.25
                    if abs(DDT4) < 100:
                        DDT4 = 100
                    if abs(DDG4) < 0.25:
                        DDG4=0.25
                    if abs(DDM4) < 0.25:
                        DDM4=0.25
                    if abs(DDV4) < 0.25:
                        DDV4 = 0.25
                    if best1 > 100:
                        best1 = 100
                    search_data1.append([np.random.uniform(search_data1[0][0]-DDT1,search_data1[0][0]+DDT1), np.random.uniform(search_data1[0][1]-DDG1, search_data1[0][1]+DDG1), np.random.uniform(search_data1[0][2]-DDM1, search_data1[0][2]+DDM1), search_data1[0][3]])
                    if search_data1[k][1]<0:
                        search_data1[k][1]=0.01
                    a1=checkExist_newParam("1"+self.nome, search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], self.folder + "/TEMP")
                    search_data1[k][0] = a1[0]
                    search_data1[k][1] = a1[1]
                    search_data1[k][2] = a1[2]
                    search_data1[k][3] = a1[3]

                    if t_fixed != 0:
                        search_data1[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data1[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data1[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data1[k][3] = v_fixed
                    
                    self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], k+1)
                    if best2>100:
                        best2=100
                    search_data2.append([np.random.uniform(search_data2[0][0]-DDT2,search_data2[0][0]+DDT2), np.random.uniform(search_data2[0][1]-DDG2, search_data2[0][1]+DDG2), np.random.uniform(search_data2[0][2]-DDM2, search_data2[0][2]+DDM2), np.random.uniform(search_data2[0][3]+DDV2, search_data2[0][3]-DDV2)])
                    if search_data2[k][1]<0:
                        search_data2[k][1]=0.01
                    a1=checkExist_newParam("2"+self.nome, search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], self.folder + "/TEMP")
                    search_data2[k][0] = a1[0]
                    search_data2[k][1] = a1[1]
                    search_data2[k][2] = a1[2]
                    search_data2[k][3] = a1[3]
                    if t_fixed != 0:
                        search_data2[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data2[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data2[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data2[k][3] = v_fixed
                    self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], k+1+int(processori/4))
                    if best3>100:
                        best3=100
                    search_data3.append([np.random.uniform(search_data3[0][0]-DDT3,search_data3[0][0]+DDT3), np.random.uniform(search_data3[0][1]-DDG3, search_data3[0][1]+DDG3), np.random.uniform(search_data3[0][2]-DDM3, search_data3[0][2]+DDM3), np.random.uniform(search_data3[0][3]+DDV3, search_data3[0][3]-DDV3)])
                    if search_data3[k][1]<0:
                        search_data3[k][1]=0.01
                    a1=checkExist_newParam("3"+self.nome, search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], self.folder + "/TEMP")
                    search_data3[k][0] = a1[0]
                    search_data3[k][1] = a1[1]
                    search_data3[k][2] = a1[2]
                    search_data3[k][3] = a1[3]
                    if t_fixed != 0:
                        search_data3[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data3[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data3[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data3[k][3] = v_fixed
                    self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], k+1+2*int(processori/4))
                    if best4>100:
                        best4=100
                    search_data4.append([np.random.uniform(search_data4[0][0]-DDT2,search_data4[0][0]+DDT4), np.random.uniform(search_data4[0][1]-DDG4, search_data4[0][1]+DDG4), np.random.uniform(search_data4[0][2]-DDM4, search_data4[0][2]+DDM4), np.random.uniform(search_data4[0][3]+DDV4, search_data4[0][3]-DDV4)])
                    if search_data4[k][1]<0:
                        search_data4[k][1]=0.01
                    a1=checkExist_newParam("4"+self.nome, search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], self.folder + "/TEMP")
                    search_data4[k][0] = a1[0]
                    search_data4[k][1] = a1[1]
                    search_data4[k][2] = a1[2]
                    search_data4[k][3] = a1[3]
                    if t_fixed != 0:
                        search_data4[k][0] = t_fixed
                    if g_fixed != -1:
                        search_data4[k][1] = g_fixed
                    if m_fixed != 5:
                        search_data4[k][2] = m_fixed
                    if v_fixed != -5:
                        search_data4[k][3] = v_fixed
                    self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], k+1+3*int(processori/4))
            if __name__ == "__main__": # Calcolo abbondanze
                main_integration()
            copia("stellaFes", path_to_abundance, self.folder, "stellaFes")
            copia("stellaFes", path_to_abundance, self.folder+"/TEMP", "stellaFes")
            best1=10000
            best1_index=-1
            best2=10000
            best2_index=-1
            best3=10000
            best3_index=-1
            best4=10000
            best4_index=-1
            if j == 0: #Valuta atmosfere
                for i in range(int(processori/4)): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                    self.nome = "1" + nome_temp
                    nome_output1 = f"{self.nome}_{search_data1[i][2]:.2f}_{(search_data1[i][0]):.0f}_{search_data1[i][1]:.2f}_{search_data1[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1}", path_to_abundance, self.folder+"/TEMP", nome_output1)
                    if best1 > self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed):
                        best1 = self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        best1_index=i
                    self.nome = "2" + nome_temp
                    nome_output2 = f"{self.nome}_{(search_data2[i][2]):.2f}_{search_data2[i][0]:.0f}_{search_data2[i][1]:.2f}_{search_data2[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output2)
                    if best2 > self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed):
                        best2 = self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        best2_index=i
                    
                    self.nome = "3" + nome_temp
                    nome_output3 = f"{self.nome}_{search_data3[i][2]:.2f}_{(search_data3[i][0]):.0f}_{search_data3[i][1]:.2f}_{search_data3[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+2*int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output3)
                    if best3 > self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed):
                        best3 = self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        best3_index=i
                    self.nome = "4" + nome_temp
                    nome_output4 = f"{self.nome}_{(search_data4[i][2]):.2f}_{search_data4[i][0]:.0f}_{search_data4[i][1]:.2f}_{search_data4[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+3*int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output4)
                    if best4 > self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed):
                        best4 = self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        best4_index=i
            else:
                for i in range(int(processori/4)): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                    self.nome = "1" + nome_temp
                    nome_output1 = f"{self.nome}_{search_data1[i][2]:.2f}_{(search_data1[i][0]):.0f}_{search_data1[i][1]:.2f}_{search_data1[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1}", path_to_abundance, self.folder+"/TEMP", nome_output1)
                    if best1 > self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed): #adv_matrix=A_numpy1, dyn_on=True):
                        best1 = self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)# adv_matrix=A_numpy1, dyn_on=True)
                        best1_index=i
                    self.nome = "2" + nome_temp
                    nome_output2 = f"{self.nome}_{(search_data2[i][2]):.2f}_{search_data2[i][0]:.0f}_{search_data2[i][1]:.2f}_{search_data2[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output2)
                    if best2 > self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed): # adv_matrix=A_numpy2, dyn_on=True):
                        best2 = self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed) # adv_matrix=A_numpy2, dyn_on=True)
                        best2_index=i

                    self.nome = "3" + nome_temp
                    nome_output3 = f"{self.nome}_{search_data3[i][2]:.2f}_{(search_data3[i][0]):.0f}_{search_data3[i][1]:.2f}_{search_data3[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+2*int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output3)
                    if best3 > self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed):# adv_matrix=A_numpy3, dyn_on=True):
                        best3 = self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)# adv_matrix=A_numpy3, dyn_on=True)
                        best3_index=i
                    self.nome = "4" + nome_temp
                    nome_output4 = f"{self.nome}_{(search_data4[i][2]):.2f}_{search_data4[i][0]:.0f}_{search_data4[i][1]:.2f}_{search_data4[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+3*int(processori/4)}", path_to_abundance, self.folder+"/TEMP", nome_output4)
                    if best4 > self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed):# adv_matrix=A_numpy4, dyn_on=True):
                        best4 = self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)# adv_matrix=A_numpy4, dyn_on=True)
                        best4_index=i
            if best1>old_best1 and best1 < 20:
                best1=old_best1
                best1_index=old_best1_index
            if best2>old_best2 and best2 < 20:
                best2=old_best2
                best2_index=old_best2_index
            if best3>old_best3 and best3 < 20:
                best3=old_best3
                best3_index=old_best3_index
            if best4>old_best4 and best4 < 20:
                best4=old_best4
                best4_index=old_best4_index
            
            self.nome=nome_temp
            tt1=search_data1[best1_index][0]
            gg1=search_data1[best1_index][1]
            mm1=search_data1[best1_index][2]
            vv1=search_data1[best1_index][3]
            tt2=search_data2[best2_index][0]
            gg2=search_data2[best2_index][1]
            mm2=search_data2[best2_index][2]
            vv2=search_data2[best2_index][3]
            tt3=search_data3[best3_index][0]
            gg3=search_data3[best3_index][1]
            mm3=search_data3[best3_index][2]
            vv3=search_data3[best3_index][3]
            tt4=search_data4[best4_index][0]
            gg4=search_data4[best4_index][1]
            mm4=search_data4[best4_index][2]
            vv4=search_data4[best4_index][3]
            (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
            #riattiva_stampe()
            if best1 <= mCheck:
                if best2 < best1 and best2<best3 and best2<best4:
                    print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                    self.t_eff = tt2
                    self.logg = gg2
                    self.metal = mm2
                    self.v_m = vv2
                    self.param_error = best2
                    return 0
                elif best3 < best1 and best3 < best2 and best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    return 0
                elif best4 < best1 and best4 < best2 and best4 < best3:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    return 0
                else:
                    print(f"ECCOLA! T: {tt1}\nG: {gg1}\nM: {mm1}\nv: {vv1}\nDelta: {best1}")
                    self.t_eff = tt1
                    self.logg = gg1
                    self.metal = mm1
                    self.v_m = vv1
                    self.param_error = best2
                    return 0
            elif best2 <= mCheck:
                if best2<best3 and best2<best4:
                    print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                    self.t_eff = tt2
                    self.logg = gg2
                    self.metal = mm2
                    self.v_m = vv2
                    self.param_error = best2
                    return 0
                elif best3 < best2 and best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    return 0
                elif best4 < best2 and best4 < best3:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    return 0
            elif best3 <= mCheck:
                if best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    return 0
                else:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    return 0
            elif best4 <= mCheck:
                print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                self.t_eff = tt4
                self.logg = gg4
                self.metal = mm4
                self.v_m = vv4
                self.param_error = best4
                return 0
            #blocca_stampe()
            nome_out1 = f"1{self.nome}_{mm1:.2f}_{tt1:.0f}_{gg1:.2f}_{vv1:.2f}"
            nome_out2 = f"2{self.nome}_{mm2:.2f}_{tt2:.0f}_{gg2:.2f}_{vv2:.2f}"
            nome_out3 = f"3{self.nome}_{mm3:.2f}_{tt3:.0f}_{gg3:.2f}_{vv3:.2f}"
            nome_out4 = f"4{self.nome}_{mm4:.2f}_{tt4:.0f}_{gg4:.2f}_{vv4:.2f}"
            if CORRECTION==True: #Dalla migliore atmosfera calcola i parametri della successiva
                if NLTE==0:
                    temp01 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.0)
                    temp11 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.1)
                    temp02 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.0)
                    temp12 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.1)
                    temp03 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out3, 26.0)
                    temp13 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out3, 26.1)
                    temp04 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out4, 26.0)
                    temp14 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out4, 26.1)
                elif NLTE == 1:
                    temp01 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.0)
                    temp11 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.1)
                    temp02 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.0)
                    temp12 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.1)
                    temp03 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out3, tt3, gg3, mm3, vv3, 26.0)
                    temp13 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out3, tt3, gg3, mm3, vv3, 26.1)
                    temp04 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out4, tt4, gg4, mm4, vv4, 26.0)
                    temp14 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out4, tt4, gg4, mm4, vv4, 26.1)
                master_search_data1.append([[tt1, gg1, mm1, vv1], [(temp01[1]), (temp01[0])-(temp11[0]), temp01[0]-(mm1), (temp01[3])]]) #contengono sia i parametri atmosferici (T, G, M, V) sia i valori da minimizare (s, F, Y, S)
                master_search_data2.append([[tt2, gg2, mm2, vv2], [(temp02[1]), (temp02[0])-(temp12[0]), temp02[0]-(mm2), (temp02[3])]]) #In questi vengono inseriti i migliori parametri per searchBranch.
                master_search_data3.append([[tt3, gg3, mm3, vv3], [(temp03[1]), (temp03[0])-(temp13[0]), temp03[0]-(mm3), (temp03[3])]])
                master_search_data4.append([[tt4, gg4, mm4, vv4], [(temp04[1]), (temp04[0])-(temp14[0]), temp04[0]-(mm4), (temp04[3])]])
                if j==0:
                    FIXD_MAT = 1
                if FIXD_MAT == 1:
                    v1=sp.symbols('v')
                    v2=sp.symbols('v')
                    v3=sp.symbols('v')
                    v4=sp.symbols('v')
                    A1 = s_Matrix
                    A2 = s_Matrix
                    A3 = s_Matrix
                    A4 = s_Matrix
                    AA1=A1.subs(v1, vv1)
                    AA2=A2.subs(v2, vv2)
                    AA3=A3.subs(v3, vv3)
                    AA4=A4.subs(v4, vv4)
                    A_numpy1 = np.array(AA1.tolist(), dtype=float)
                    A_inv1 = np.linalg.inv(A_numpy1)
                    A_numpy2 = np.array(AA2.tolist(), dtype=float)
                    A_inv2 = np.linalg.inv(A_numpy2)
                    A_numpy3 = np.array(AA3.tolist(), dtype=float)
                    A_inv3 = np.linalg.inv(A_numpy3)
                    A_numpy4 = np.array(AA4.tolist(), dtype=float)
                    A_inv4 = np.linalg.inv(A_numpy4)
                    f1=-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])
                    f3=-1*((temp03[0])-(temp13[0]))
                    s3 = -1*(temp03[1])
                    m3 = -1*(temp03[0]-(mm3))
                    ewr3 = -1*(temp03[3])
                    B3 = np.array([s3, f3, m3, ewr3])
                    f4=-1*((temp04[0])-(temp14[0]))
                    s4 = -1*(temp04[1])
                    m4 = -1*(temp04[0]-(mm4))
                    ewr4 = -1*(temp04[3])
                    B4 = np.array([s4, f4, m4, ewr4])
                    #blocca_stampe()
                    if abs(s1)<0.001 and abs(m1) < 0.01:
                        DT1 = 0
                        DM1 = 0
                        Dv1 = 0
                        s1 = 0
                        m1 = 0
                        ewr1 = 0
                        B1 = np.array([s1, f1, m1, ewr1])
                        DM1 = (np.dot(A_inv1, B1), [1])
                    else:
                        if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT1 = (np.dot(A_inv1, B1))[0]
                            DG1 = (np.dot(A_inv1, B1))[1]
                            DM1 = (np.dot(A_inv1, B1))[2]
                            Dv1 = (np.dot(A_inv1, B1))[3]
                            if abs(DG1) > 0.5:
                                DG1=0.5*(DG1/abs(DG1))
                            if abs(DT1) > 250:
                                DT1 = 250*(DT1/abs(DT1))
                            if abs(DM1) > 0.25:
                                DM1 = 0.25*(DM1/abs(DM1))
                            if abs(Dv1) > 0.3:
                                Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(s2)<0.001 and abs(m2) < 0.01:
                        DT2 = 0
                        DM2 = 0
                        Dv2 = 0
                        s2 = 0
                        m2 = 0
                        ewr2 = 0
                        B2 = np.array([s2, f2, m2, ewr2])
                        DM2 = (np.dot(A_inv2, B2), [1])
                    else:
                        if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT2 = (np.dot(A_inv2, B2))[0]
                            DG2 = (np.dot(A_inv2, B2))[1]
                            DM2 = (np.dot(A_inv2, B2))[2]
                            Dv2 = (np.dot(A_inv2, B2))[3]
                            if abs(DG2) > 0.5:
                                DG2=0.5*(DG2/abs(DG2))
                            if abs(DT2) > 250:
                                DT2 = 250*(DT2/abs(DT2))
                            if abs(DM2) > 0.25:
                                DM2 = 0.25*(DM2/abs(DM2))
                            if abs(Dv2) > 0.3:
                                Dv2 = 0.3*(Dv2/abs(Dv2))
                    if abs(s3)<0.001 and abs(m3) < 0.01:
                        DT3 = 0
                        DM3 = 0
                        Dv3 = 0
                        s3 = 0
                        m3 = 0
                        ewr3 = 0
                        B3 = np.array([s3, f3, m3, ewr3])
                        DM3 = (np.dot(A_inv3, B3), [1])
                    else:
                        if abs(temp03[1]) > 0.0001 or abs((temp03[0])-(temp13[0])) > 0.0003 or abs(mm3 - (temp03[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT3 = (np.dot(A_inv3, B3))[0]
                            DG3 = (np.dot(A_inv3, B3))[1]
                            DM3 = (np.dot(A_inv3, B3))[2]
                            Dv3 = (np.dot(A_inv3, B3))[3]
                            if abs(DG3) > 0.5:
                                DG3=0.5*(DG3/abs(DG3))
                            if abs(DT2) > 250:
                                DT3 = 250*(DT3/abs(DT3))
                            if abs(DM3) > 0.25:
                                DM3 = 0.25*(DM3/abs(DM3))
                            if abs(Dv3) > 0.3:
                                Dv3 = 0.3*(Dv3/abs(Dv3))
                    if abs(s4)<0.001 and abs(m4) < 0.01:
                        DT4 = 0
                        DM4 = 0
                        Dv4 = 0
                        s4  = 0
                        m4  = 0
                        ewr4= 0
                        B4 = np.array([s4, f4, m4, ewr4])
                        DM4 = (np.dot(A_inv4, B4), [1])
                    else:
                        if abs(temp04[1]) > 0.0001 or abs((temp04[0])-(temp14[0])) > 0.0003 or abs(mm4 - (temp04[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT4 = (np.dot(A_inv4, B4))[0]
                            DG4 = (np.dot(A_inv4, B4))[1]
                            DM4 = (np.dot(A_inv4, B4))[2]
                            Dv4 = (np.dot(A_inv4, B4))[3]
                            if abs(DG4) > 0.5:
                                DG4=0.5*(DG4/abs(DG4))
                            if abs(DT4) > 250:
                                DT4 = 250*(DT4/abs(DT4))
                            if abs(DM4) > 0.25:
                                DM4 = 0.25*(DM4/abs(DM4))
                            if abs(Dv2) > 0.3:
                                Dv4 = 0.3*(Dv4/abs(Dv4))
                    if t_fixed != 0:
                        DT1=0
                        DT2=0
                        DT3=0
                        DT4=0
                    if g_fixed != -1:
                        DG1=0
                        DG2=0
                        DG3=0
                        DG4=0
                    if m_fixed != 5:
                        DM1=0
                        DM2=0
                        DM3=0
                        DM4=0
                    if v_fixed != -5:
                        Dv1=0
                        Dv2=0
                        Dv3=0
                        Dv4=0
                    FIXD_MAT=0
                else:
                    A_numpy1 = crea_dynMatrix(master_search_data1[j-1][0], master_search_data1[j-1][1], master_search_data1[j][0], master_search_data1[j][1])
                    DDT1 = 2*(master_search_data1[j-1][0][0]-master_search_data1[j][0][0]) #La differenza tra il valore precedente e quello successivo di Teff
                    DDG1 = 2*(master_search_data1[j-1][0][1]-master_search_data1[j][0][1]) #La differenza tra il valore precedente e quello successivo di logg
                    DDM1 = 2*(master_search_data1[j-1][0][2]-master_search_data1[j][0][2]) #La differenza tra il valore precedente e quello successivo di [M/H]
                    DDV1 = 2*(master_search_data1[j-1][0][3]-master_search_data1[j][0][3])
                    A_inv1 = np.linalg.pinv(A_numpy1)
                    f1 =-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])

                    A_numpy2 = crea_dynMatrix(master_search_data2[j-1][0], master_search_data2[j-1][1], master_search_data2[j][0], master_search_data2[j][1])
                    DDT2 = 2*(master_search_data2[j-1][0][0]-master_search_data2[j][0][0])
                    DDG2 = 2*(master_search_data2[j-1][0][1]-master_search_data2[j][0][1])
                    DDM2 = 2*(master_search_data2[j-1][0][2]-master_search_data2[j][0][2])
                    DDV2 = 2*(master_search_data2[j-1][0][3]-master_search_data2[j][0][3])
                    A_inv2 = np.linalg.pinv(A_numpy2)
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])

                    A_numpy3 = crea_dynMatrix(master_search_data3[j-1][0], master_search_data3[j-1][1], master_search_data3[j][0], master_search_data3[j][1])
                    DDT3 = 2*(master_search_data3[j-1][0][0]-master_search_data3[j][0][0])
                    DDG3 = 2*(master_search_data3[j-1][0][1]-master_search_data3[j][0][1])
                    DDM3 = 2*(master_search_data3[j-1][0][2]-master_search_data3[j][0][2])
                    DDV3 = 2*(master_search_data3[j-1][0][3]-master_search_data3[j][0][3])
                    A_inv3 = np.linalg.pinv(A_numpy3)
                    f3 =-1*((temp03[0])-(temp13[0]))
                    s3 = -1*(temp03[1])
                    m3 = -1*(temp03[0]-(mm3))
                    ewr3 = -1*(temp03[3])
                    B3 = np.array([s3, f3, m3, ewr3])

                    A_numpy4 = crea_dynMatrix(master_search_data4[j-1][0], master_search_data4[j-1][1], master_search_data4[j][0], master_search_data4[j][1])
                    DDT4 = 2*(master_search_data4[j-1][0][0]-master_search_data4[j][0][0])
                    DDG4 = 2*(master_search_data4[j-1][0][1]-master_search_data4[j][0][1])
                    DDM4 = 2*(master_search_data4[j-1][0][2]-master_search_data4[j][0][2])
                    DDV4 = 2*(master_search_data4[j-1][0][3]-master_search_data4[j][0][3])
                    A_inv4 = np.linalg.pinv(A_numpy4)
                    f4=-1*((temp04[0])-(temp14[0]))
                    s4 = -1*(temp04[1])
                    m4 = -1*(temp04[0]-(mm4))
                    ewr4 = 0#-1*(temp04[3])
                    B4 = np.array([s4, f4, m4, ewr4])

                    #blocca_stampe()
                    if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT1 = (np.dot(A_inv1, B1))[0]
                        DG1 = (np.dot(A_inv1, B1))[1]
                        DM1 = (np.dot(A_inv1, B1))[2]
                        Dv1 = (np.dot(A_inv1, B1))[3]
                        if abs(DG1) > 0.5:
                            DG1=0.5*(DG1/abs(DG1))
                        if abs(DT1) > 250:
                            DT1 = 250*(DT1/abs(DT1))
                        if abs(DM1) > 0.25:
                            DM1 = 0.25*(DM1/abs(DM1))
                        if abs(Dv1) > 0.3:
                            Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT2 = (np.dot(A_inv2, B2))[0]
                        DG2 = (np.dot(A_inv2, B2))[1]
                        DM2 = (np.dot(A_inv2, B2))[2]
                        Dv2 = (np.dot(A_inv2, B2))[3]
                        if abs(DG2) > 0.5:
                            DG2=0.5*(DG2/abs(DG2))
                        if abs(DT2) > 250:
                            DT2 = 250*(DT2/abs(DT2))
                        if abs(DM2) > 0.25:
                            DM2 = 0.25*(DM2/abs(DM2))
                        if abs(Dv2) > 0.3:
                            Dv2 = 0.3*(Dv2/abs(Dv2))
                    if abs(temp03[1]) > 0.0001 or abs((temp03[0])-(temp13[0])) > 0.0003 or abs(mm3 - (temp03[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT3 = (np.dot(A_inv3, B3))[0]
                        DG3 = (np.dot(A_inv3, B3))[1]
                        DM3 = (np.dot(A_inv3, B3))[2]
                        Dv3 = (np.dot(A_inv3, B3))[3]
                        if abs(DG3) > 0.5:
                            DG3=0.5*(DG3/abs(DG3))
                        if abs(DT3) > 250:
                            DT3 = 250*(DT3/abs(DT3))
                        if abs(DM3) > 0.25:
                            DM3 = 0.25*(DM3/abs(DM3))
                        if abs(Dv3) > 0.3:
                            Dv3 = 0.3*(Dv3/abs(Dv3))
                    if abs(temp04[1]) > 0.0001 or abs((temp04[0])-(temp14[0])) > 0.0003 or abs(mm4 - (temp04[0]))>0.001:
                        # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                        # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                        DT4 = (np.dot(A_inv4, B4))[0]
                        DG4 = (np.dot(A_inv4, B4))[1]
                        DM4 = (np.dot(A_inv4, B4))[2]
                        Dv4 = (np.dot(A_inv4, B4))[3]
                        if abs(DG4) > 0.5:
                            DG4=0.5*(DG4/abs(DG4))
                        if abs(DT4) > 250:
                            DT4 = 250*(DT4/abs(DT4))
                        if abs(DM4) > 0.25:
                            DM4 = 0.25*(DM4/abs(DM4))
                        if abs(Dv4) > 0.3:
                            Dv4 = 0.3*(Dv4/abs(Dv4))
                    if t_fixed != 0:
                        DT1=0
                        DT2=0
                        DT3=0
                        DT4=0
                    if g_fixed != -1:
                        DG1=0
                        DG2=0
                        DG3=0
                        DG4=0
                    if m_fixed != 5:
                        DM1=0
                        DM2=0
                        DM3=0
                        DM4=0
                    if v_fixed != -5:
                        Dv1=0
                        Dv2=0
                        Dv3=0
                        Dv4=0
                temperatu1=tt1+DT1
                G1=gg1+DG1
                velturb1=vv1+Dv1
                teff1= temperatu1
                temperatu2=tt2+DT2
                G2=gg2+DG2
                velturb2=vv2+Dv2
                teff2= temperatu2

                temperatu3=tt3+DT3
                G3=gg3+DG3
                velturb3=vv3+Dv3
                teff3= temperatu3
                temperatu4=tt4+DT4
                G4=gg4+DG4
                velturb4=vv4+Dv4
                teff4= temperatu4
            if fixd_velturb == True:
                velturb1 = vturbo(teff1, G1, mm1+DM1)
                velturb2 = vturbo(teff2, G2, mm2+DM2)
                velturb3 = vturbo(teff3, G3, mm3+DM3)
                velturb4 = vturbo(teff4, G4, mm4+DM4)
            search_data1 = []
            search_data1.append([teff1, G1, mm1+DM1, velturb1])
            search_data2 = []
            search_data2.append([teff2, G2, mm2+DM2, velturb2])
            search_data3 = []
            search_data3.append([teff3, G3, mm3+DM3, velturb3])
            search_data4 = []
            search_data4.append([teff4, G4, mm4+DM4, velturb4])
            old_best1_index=best1_index
            old_best2_index=best2_index
            old_best3_index=best3_index
            old_best4_index=best4_index
            old_best1=best1
            old_best2=best2
            old_best3=best3
            old_best4=best4
        #riattiva_stampe()
        #riattiva_stampe()
        self.nome = "1" + nome_temp
        O1 = self.evaluate_Models(tt1, gg1, mm1, vv1, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
        self.nome = "2" + nome_temp
        O2 = self.evaluate_Models(tt2, gg2, mm2, vv2, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
        self.nome = "3" + nome_temp
        O3 = self.evaluate_Models(tt3, gg3, mm3, vv3, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
        self.nome = "4" + nome_temp
        O4 = self.evaluate_Models(tt4, gg4, mm4, vv4, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
        print(O1)
        print(O2)
        print(O3)
        print(O4)
        self.nome = nome_temp
        print(f"{tt1}, {gg1}, {mm1}. {vv1}")
        print(f"{tt2}, {gg2}, {mm2}. {vv2}")
        vettori_selezionati = []
        for quaterna in self.ALL_STARS: #crea returner
            # Verifica che la quaterna contenga esattamente quattro vettori
            if len(quaterna) == 4:
                # Trova il vettore con il primo elemento più piccolo nella quaterna
                minimo_quaterna = min(quaterna, key=lambda x: x[0])
                vettori_selezionati.append(minimo_quaterna)

        # Trova il vettore con il primo elemento più piccolo tra i selezionati
        vettore_minimo = min(vettori_selezionati, key=lambda x: x[0])
        self.t_eff = vettore_minimo[1]
        self.logg = vettore_minimo[2]
        self.metal = vettore_minimo[3]
        self.v_m = vettore_minimo[4]
        self.param_error = vettore_minimo[0]
        self.stampaStella(self.nome)
        print(f'{self.t_eff}, {self.logg}, {self.metal}, {self.v_m}')
        #sposta()
    def searchParam3(self, processori=1, NLTE=0, TRIALS = 50, mashonkina=True, t01=-1, g01=-2, m01=10, v01=-1, t02=-1, g02=-2, m02=10, v02=-1, t03=-1, g03=-2, m03=10, v03=-1, t04=-1, g04=-2, m04=10, v04=-1, t_fixed=0, g_fixed=-1, m_fixed=5, v_fixed=-5, mCheck=3, radSolver = 'spectrum', atmospheric_model='ATLAS9', boundary_check=True):
        '''Usare un numero di processori divisibile per 4\n
         NLTE = 0 per modelli LTE, NLTE = 1 per modelli NLTE\n
         TRIALS = numero di iterazioni\n
         t0, g0, m0, v0 = condizioni iniziali\n 
         t_fixed, g_fixed, m_fixed, v_fixed = parametri fissati\n 
         mCheck = 3 per modelli con 3 parametri, 4 per modelli con 4 parametri\n
         radSolver = spectrum per utilizzare spectrum, turbo per utilizzare turbospectrum\n
         atmospheric_model = ATLAS9 per calcolare sul momento atmosfere ATLAS9, ATLASint per ottenere le atmosfere ATLAS9 tramite interpolatore'''
        if processori%4 != 0:
            print("Inserisci un numero di processori divisibile per 4. (e.g. processori=4, processori=12)")
            return 200
        fixd_velturb = mashonkina
        CORRECTION = True

        if True: #Setting initial conditions
            if t01==-1:
                t_start1 = int(random.uniform(4000, 4300))
            else:
                t_start1 = t01
            if t02==-1:
                t_start2= int(random.uniform(4300, 4700))
            else:
                t_start2=t02
            if t03==-1:
                t_start3 = int(random.uniform(4700, 5000))
            else:
                t_start3 = t03
            if t04==-1:
                t_start4= int(random.uniform(4300, 4600))
            else:
                t_start4=t04
            
            if g01==-2:
                g_start1= random.uniform(0, 0.7)
            else:
                g_start1 = g01
            if g02==-2:
                g_start2=random.uniform(0.7, 1.45)
            else:
                g_start2=g02
            if g03==-2:
                g_start3= random.uniform(1.45, 2.0)
            else:
                g_start3 = g03
            if g04==-2:
                g_start4=random.uniform(1.0, 1.75)
            else:
                g_start4=g04

            if m01==10:
                m_start1= random.uniform(-2, 0)
            else:
                m_start1 = m01
            if m02==10:
                m_start2 = random.uniform(-2, 0)
            else:
                m_start2=m02
            if m03==10:
                m_start3= random.uniform(-2, 0)
            else:
                m_start3 = m03
            if m04==10:
                m_start4 = random.uniform(-2, 0)
            else:
                m_start4 = m04
            
            if v01==-1:
                v01=vturbo(t_start1, g_start1, m_start1)
            if v02==-1:
                v02=vturbo(t_start2, g_start2, m_start2)
            if v03==-1:
                v03=vturbo(t_start3, g_start3, m_start3)
            if v04==-1:
                v04=vturbo(t_start4, g_start4, m_start4)

        if t_fixed != 0:
            t_start1 = t_fixed
            t_start2 = t_fixed
            t_start3 = t_fixed
            t_start4 = t_fixed
        if g_fixed != -1:
            g_start1 = g_fixed
            g_start2 = g_fixed
            g_start3 = g_fixed
            g_start4 = g_fixed
        if m_fixed != 5:
            m_start1 = m_fixed
            m_start2 = m_fixed
            m_start3 = m_fixed
            m_start4 = m_fixed
        if v_fixed != -5:
            v01 = v_fixed
            v02 = v_fixed
            v03 = v_fixed
            v04 = v_fixed
        self.dopcor()
        self.saveFits()
        delete_files(path_to_abundance)
        delete_allfiles(self.folder+'/TEMP')
        #blocca_stampe()
        if True: #Starting block
            search_data1 = []
            search_data2 = []
            search_data3 = []
            search_data4 = []
            master_search_data1 = []
            master_search_data2 = []
            master_search_data3 = []
            master_search_data4 = []
            self.ALL_STARS=[]
            if t01==t02 and t02==t03 and t03==t04 and g01==g02 and g02==g03 and g03==g04 and m01==m02 and m02==m03 and m03==m04 and v01==v02 and v02==v03 and v03==v04:
                crea_ATMinterpol(t_start1, g_start1, m_start1, self.fits_name, instrument=self.instrument)
                search_data1 = []
                master_search_data1 = []
                self.ALL_STARS=[]
                search_data1.append([t_start1, g_start1, m_start1, v01, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])
                search_data2.append([t_start1, g_start1, m_start1, v01, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])
                search_data3.append([t_start1, g_start1, m_start1, v01, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])
                search_data4.append([t_start1, g_start1, m_start1, v01, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])
            else:
                crea_ATMinterpol(t_start1, g_start1, m_start1, self.fits_name, instrument=self.instrument)
                crea_ATMinterpol(t_start2, g_start2, m_start2, self.fits_name, instrument=self.instrument)
                crea_ATMinterpol(t_start3, g_start3, m_start3, self.fits_name, instrument=self.instrument)
                crea_ATMinterpol(t_start4, g_start4, m_start4, self.fits_name, instrument=self.instrument)
                search_data1.append([t_start1, g_start1, m_start1, v01, f"{self.nome}_{m_start1:.2f}_{t_start1:.0f}_{g_start1:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
                search_data2.append([t_start2, g_start2, m_start2, v02, f"{self.nome}_{m_start2:.2f}_{t_start2:.0f}_{g_start2:.2f}"])   # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
                search_data3.append([t_start3, g_start3, m_start3, v03, f"{self.nome}_{m_start3:.2f}_{t_start3:.0f}_{g_start3:.2f}"])
                search_data4.append([t_start4, g_start4, m_start4, v04, f"{self.nome}_{m_start4:.2f}_{t_start4:.0f}_{g_start4:.2f}"])
        os.chdir(path_to_output)
        o = 0
        s=[]
        F=[]
        Y=[]
        S=[]
        crea_cartella(self.folder)
        crea_cartella(self.folder+"/TEMP")
        if True: #help block
            DDT1 = 100
            DDG1 = 0.15
            DDM1 = 0.15
            DDV1 = 0.15
            DDT2 = 100
            DDG2 = 0.15
            DDM2 = 0.15
            DDV2 = 0.15
            DDT3 = 100
            DDG3 = 0.15
            DDM3 = 0.15
            DDV3 = 0.15
            DDT4 = 100
            DDG4 = 0.15
            DDM4 = 0.15
            DDV4 = 0.15
            best1=1
            best2=1
            best3=1
            best4=1
            old_best1=200000
            old_best2=200000
            old_best3=200000
            old_best4=200000
            old_best1_index=-2
            old_best2_index=-2
            old_best3_index=-2
            old_best4_index=-2
        nome_temp=self.nome

        if radSolver == 'moog':
            creaLineList(fits=self.fits_name, rad_solver='moog', line_list='ESO_FE', instrument='FIES')

        for j in range(TRIALS): ## Numero di iterazioni
            if boundary_check: ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle giganti FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
                if search_data1[0][1] <= 0.0:
                    search_data1[0][1] = 0.5
                if search_data1[0][0] <= 3700:
                    search_data1[0][0] = 3750
                if g_fixed != -1:
                    if search_data1[0][1] > 2.8:
                        search_data1[0][1] = 2.5
                if mashonkina == True:
                    if abs(search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) > 0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])-0.081
                    if (search_data1[0][3] - vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])) < -0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data1[0][3] = vturbo(search_data1[0][0], search_data1[0][1], search_data1[0][2])+0.081
                
                if search_data2[0][1] <= 0.0:
                    search_data2[0][1] = 0.5
                if search_data2[0][0] <= 3700:
                    search_data2[0][0] = 3750
                if g_fixed != -1:
                    if search_data2[0][1] > 2.8:
                        search_data2[0][1] = 2.5
                if mashonkina == True:
                    if abs(search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) > 0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])-0.081
                    if (search_data2[0][3] - vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])) < -0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data2[0][3] = vturbo(search_data2[0][0], search_data2[0][1], search_data2[0][2])+0.081
                
                if search_data3[0][1] <= 0.0:
                    search_data3[0][1] = 0.5
                if search_data3[0][0] <= 3700:
                    search_data3[0][0] = 3750
                if g_fixed != -1:
                    if search_data3[0][1] > 2.8:
                        search_data3[0][1] = 2.5
                if mashonkina == True:
                    if abs(search_data3[0][3] - vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])) > 0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data3[0][3] = vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])-0.081
                    if (search_data3[0][3] - vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])) < -0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data3[0][3] = vturbo(search_data3[0][0], search_data3[0][1], search_data3[0][2])+0.081
                

                if search_data4[0][1] <= 0.0:
                    search_data4[0][1] = 0.5
                if search_data4[0][0] <= 3700:
                    search_data4[0][0] = 3750
                if search_data4[0][1] > 2.8:
                    search_data4[0][1] = 2.5
                if mashonkina == True:
                    if abs(search_data4[0][3] - vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])) > 0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data4[0][3] = vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])-0.081
                    if (search_data4[0][3] - vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])) < -0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data4[0][3] = vturbo(search_data4[0][0], search_data4[0][1], search_data4[0][2])+0.081

            for k in range(int(processori/4)): # Crea i modelli da analizzare
                if k == 0: #Per k = 0 si utilizzano i parametri trovati 
                    if True: #Controlla se l'atmosfera richiesta è già stata creata. In tal caso si cambiano leggermente i parametri
                        a1=checkExist_newParam("1"+self.nome, search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], self.folder + "/TEMP", t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        search_data1[k][0] = a1[0]
                        search_data1[k][1] = a1[1]
                        search_data1[k][2] = a1[2]
                        search_data1[k][3] = a1[3]
                        a2=checkExist_newParam("2"+self.nome, search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], self.folder + "/TEMP", t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        search_data2[k][0] = a2[0]
                        search_data2[k][1] = a2[1]
                        search_data2[k][2] = a2[2]
                        search_data2[k][3] = a2[3]
                        a3=checkExist_newParam("3"+self.nome, search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], self.folder + "/TEMP", t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        search_data3[k][0] = a3[0]
                        search_data3[k][1] = a3[1]
                        search_data3[k][2] = a3[2]
                        search_data3[k][3] = a3[3]
                        a4=checkExist_newParam("4"+self.nome, search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], self.folder + "/TEMP", t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        search_data4[k][0] = a4[0]
                        search_data4[k][1] = a4[1]
                        search_data4[k][2] = a4[2]
                        search_data4[k][3] = a4[3]
                    if True and g_fixed != -1: #Checks sul log(g)
                        if search_data1[k][1]<0:
                            search_data1[k][1]=0.01
                        if search_data1[k][1]>3.0:
                            search_data1[k][1]=2.99
                        if search_data2[k][1]<0:
                            search_data2[k][1]=0.01
                        if search_data2[k][1]>3.0:
                            search_data2[k][1]=2.99
                        
                        if search_data3[k][1]<0:
                            search_data3[k][1]=0.01
                        if search_data3[k][1]>3.0:
                            search_data3[k][1]=2.99

                        if search_data4[k][1]<0:
                            search_data4[k][1]=0.0
                        if search_data4[k][1]>3.0:
                            search_data4[k][1]=2.99
                    if True: #Per parametri fissati
                        if t_fixed != 0:  
                            search_data1[k][0] = t_fixed
                            search_data2[k][0] = t_fixed
                            search_data3[k][0] = t_fixed
                            search_data4[k][0] = t_fixed
                        if g_fixed != -1:
                            search_data1[k][1] = g_fixed
                            search_data2[k][1] = g_fixed
                            search_data3[k][1] = g_fixed
                            search_data4[k][1] = g_fixed
                        if m_fixed != 5:
                            search_data1[k][2] = m_fixed
                            search_data2[k][2] = m_fixed
                            search_data3[k][2] = m_fixed
                            search_data4[k][2] = m_fixed
                        if v_fixed != -5:
                            search_data1[k][3] = v_fixed
                            search_data2[k][3] = v_fixed
                            search_data3[k][3] = v_fixed
                            search_data4[k][3] = v_fixed
                    #Calcola i modelli
                    print('LOLLOSPAMMOX')
                    if atmospheric_model == 'ATLAS9':
                        print('LOLLOSPAMMO')
                        self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], processore=k+1, radSolver=radSolver, intpol=False)
                        print(f"{search_data1[k][0]}, {search_data1[k][1]}, {search_data1[k][2]}, {search_data1[k][3]}")
                        self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], processore=k+1+int(processori/4), radSolver=radSolver, intpol=False)
                        print(f"{search_data2[k][0]}, {search_data2[k][1]}, {search_data2[k][2]}, {search_data2[k][3]}")
                        self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], processore=k+1+2*int(processori/4), radSolver=radSolver, intpol=False)
                        print(f"{search_data3[k][0]}, {search_data3[k][1]}, {search_data3[k][2]}, {search_data3[k][3]}")
                        self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], processore=k+1+3*int(processori/4), radSolver=radSolver, intpol=False)
                        print(f"{search_data4[k][0]}, {search_data4[k][1]}, {search_data4[k][2]}, {search_data4[k][3]}")
                    elif atmospheric_model == 'ATLASint':
                        self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], processore=k+1, radSolver=radSolver, intpol=True)
                        print(f"{search_data1[k][0]}, {search_data1[k][1]}, {search_data1[k][2]}, {search_data1[k][3]}")
                        self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], processore=k+1+int(processori/4), radSolver=radSolver, intpol=True)
                        print(f"{search_data2[k][0]}, {search_data2[k][1]}, {search_data2[k][2]}, {search_data2[k][3]}")
                        self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], processore=k+1+2*int(processori/4), radSolver=radSolver, intpol=True)
                        print(f"{search_data3[k][0]}, {search_data3[k][1]}, {search_data3[k][2]}, {search_data3[k][3]}")
                        self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], processore=k+1+3*int(processori/4), radSolver=radSolver, intpol=True)
                        print(f"{search_data4[k][0]}, {search_data4[k][1]}, {search_data4[k][2]}, {search_data4[k][3]}")
                else: #Per gli altri k la ricerca assume una connotazione randomica
                    if True: #Se l'area della ricerca randomica ha un raggio troppo piccolo lo si fissa a un minimo.
                        if abs(DDT1) < 50:
                            DDT1 = 50
                        if abs(DDG1) < 0.15:
                            DDG1=0.15
                        if abs(DDM1) < 0.15:
                            DDM1=0.15
                        if abs(DDV1) < 0.15:
                            DDV1 = 0.15
                        if abs(DDT2) < 50:
                            DDT2 = 50
                        if abs(DDG2) < 0.15:
                            DDG2=0.15
                        if abs(DDM2) < 0.15:
                            DDM2=0.15
                        if abs(DDV2) < 0.15:
                            DDV2 = 0.15
                        if abs(DDT3) < 100:
                            DDT3 = 100
                        if abs(DDG3) < 0.15:
                            DDG3=0.15
                        if abs(DDM3) < 0.15:
                            DDM3=0.15
                        if abs(DDV3) < 0.15:
                            DDV3 = 0.15
                        if abs(DDT4) < 50:
                            DDT4 = 50
                        if abs(DDG4) < 0.15:
                            DDG4=0.15
                        if abs(DDM4) < 0.15:
                            DDM4=0.15
                        if abs(DDV4) < 0.15:
                            DDV4 = 0.15
                        if t_fixed != 0:
                            DDT1 = 0
                            DDT2 = 0
                            DDT3 = 0
                            DDT4 = 0
                        if g_fixed != -1:
                            DDG1 = 0
                            DDG2 = 0
                            DDG3 = 0
                            DDG4 = 0
                        if m_fixed != 5:
                            DDM1 = 0
                            DDM2 = 0
                            DDM3 = 0
                            DDM4 = 0
                        if v_fixed != -5:
                            DDV1 = 0
                            DDV2 = 0
                            DDV3 = 0
                            DDV4 = 0
                    if best1 > 100:
                        best1 = 100
                    if best2>100:
                        best2=100
                    if best3>100:
                        best3=100
                    if best4>100:
                        best4=100
                    search_data1.append([np.random.uniform(search_data1[0][0]-DDT1,search_data1[0][0]+DDT1), np.random.uniform(search_data1[0][1]-DDG1, search_data1[0][1]+DDG1), np.random.uniform(search_data1[0][2]-DDM1, search_data1[0][2]+DDM1), np.random.uniform(search_data1[0][3]+DDV1, search_data2[0][3]-DDV1)])
                    if search_data1[k][1]<0:
                        search_data1[k][1]=0.01
                    search_data2.append([np.random.uniform(search_data2[0][0]-DDT2,search_data2[0][0]+DDT2), np.random.uniform(search_data2[0][1]-DDG2, search_data2[0][1]+DDG2), np.random.uniform(search_data2[0][2]-DDM2, search_data2[0][2]+DDM2), np.random.uniform(search_data2[0][3]+DDV2, search_data2[0][3]-DDV2)])
                    if search_data2[k][1]<0:
                        search_data2[k][1]=0.01
                    search_data3.append([np.random.uniform(search_data3[0][0]-DDT3,search_data3[0][0]+DDT3), np.random.uniform(search_data3[0][1]-DDG3, search_data3[0][1]+DDG3), np.random.uniform(search_data3[0][2]-DDM3, search_data3[0][2]+DDM3), np.random.uniform(search_data3[0][3]+DDV3, search_data3[0][3]-DDV3)])
                    if search_data3[k][1]<0:
                        search_data3[k][1]=0.01
                    search_data4.append([np.random.uniform(search_data4[0][0]-DDT2,search_data4[0][0]+DDT4), np.random.uniform(search_data4[0][1]-DDG4, search_data4[0][1]+DDG4), np.random.uniform(search_data4[0][2]-DDM4, search_data4[0][2]+DDM4), np.random.uniform(search_data4[0][3]+DDV4, search_data4[0][3]-DDV4)])
                    if search_data4[k][1]<0:
                        search_data4[k][1]=0.01
                    if True: #Controlla se l'atmosfera richiesta è già stata creata. In tal caso si cambiano leggermente i parametri
                        a1=checkExist_newParam("1"+self.nome, search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], self.folder + "/TEMP")
                        search_data1[k][0] = a1[0]
                        search_data1[k][1] = a1[1]
                        search_data1[k][2] = a1[2]
                        search_data1[k][3] = a1[3]
                        a2=checkExist_newParam("2"+self.nome, search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], self.folder + "/TEMP")
                        search_data2[k][0] = a2[0]
                        search_data2[k][1] = a2[1]
                        search_data2[k][2] = a2[2]
                        search_data2[k][3] = a2[3]
                        a3=checkExist_newParam("3"+self.nome, search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], self.folder + "/TEMP")
                        search_data3[k][0] = a3[0]
                        search_data3[k][1] = a3[1]
                        search_data3[k][2] = a3[2]
                        search_data3[k][3] = a3[3]
                        a4=checkExist_newParam("4"+self.nome, search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], self.folder + "/TEMP")
                        search_data4[k][0] = a4[0]
                        search_data4[k][1] = a4[1]
                        search_data4[k][2] = a4[2]
                        search_data4[k][3] = a4[3]

                    print(f'LOLLOSPAMMOX {k}')
                    if atmospheric_model == 'ATLASint':
                        print(f'LOLLOSPAMMO {k}')
                        self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], k+1, radSolver=radSolver, intpol=True)
                        self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], k+1+int(processori/4), radSolver=radSolver, intpol=True)
                        self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], k+1+2*int(processori/4), radSolver=radSolver, intpol=True)
                        self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], k+1+3*int(processori/4), radSolver=radSolver, intpol=True)
                    elif atmospheric_model == 'ATLAS9':
                        self.creaModel(search_data1[k][0], search_data1[k][1], search_data1[k][2], search_data1[k][3], k+1, radSolver=radSolver, intpol=False)
                        self.creaModel(search_data2[k][0], search_data2[k][1], search_data2[k][2], search_data2[k][3], k+1+int(processori/4), radSolver=radSolver, intpol=False)
                        self.creaModel(search_data3[k][0], search_data3[k][1], search_data3[k][2], search_data3[k][3], k+1+2*int(processori/4), radSolver=radSolver, intpol=False)
                        self.creaModel(search_data4[k][0], search_data4[k][1], search_data4[k][2], search_data4[k][3], k+1+3*int(processori/4), radSolver=radSolver, intpol=False)
            if radSolver == 'spectrum':
                if __name__ == "__main__": # Calcolo abbondanze
                    main_integration()
            elif radSolver == 'turbo':
                metalScale = []
                VTURBo = []
                for o in range(processori):
                    if o >= 0 and o < processori//4:
                        metalScale.append(search_data1[o][2])
                        VTURBo.append(search_data1[o][3])
                    if o >= processori//4 and o < 2*(processori//4):
                        metalScale.append(search_data2[o-processori//4][2])
                        VTURBo.append(search_data2[o-processori//4][3])
                    if o >= 2*processori//4 and o < 3*processori//4:
                        metalScale.append(search_data3[o -  2*processori//4][2])
                        VTURBo.append(search_data3[o - 2*processori//4][3])
                    if o >= 3*processori//4 and o < 4*processori//4:
                        metalScale.append(search_data4[o - 3*processori//4][2])
                        VTURBo.append(search_data4[o- 3*processori//4][3])
                    td=os.getcwd()
                    os.chdir(path_to_turboModels)
                    fix_24th_line_35th_char(f'Model{o+1}')
                    os.chdir(td)
                lanciaTurbospectrum2(stellaFITS=self.fits_name, processore=processori, TURBVEL=VTURBo, modelName='Model', outname='PINOLOS', metalScaling=metalScale)
                copia('noPINOLOSpr1pt1', path_to_turboModels, path_to_output, 'stellaFes_turbo')
                copia('noPINOLOSpr2pt1', path_to_turboModels, path_to_output, 'stellaFes_turbo')

                for I in range(processori):
                    turbo_resultCombine(path_to_turboModels, f"PINOLOSpr{I+1}pt", f"PINOLOS{I+1}")
                    turbo_to_spectrum_resultConversion(path_to_turboModels, f'PINOLOS{I+1}')
                for I in range(processori):
                    delete_files_with_prefix(path_to_turboModels, f"PINOLOSpr{I+1}pt")
                    delete_files_with_prefix(path_to_turboModels, f"noPINOLOS{I+1}")
            elif radSolver == 'moog':
                for I in range(processori):
                    creaMOOG_BATCH(I, f'Model{I+1}')
                    lanciaMOOGSILENT(processori=processori)
            copia("stellaFes", path_to_abundance, self.folder, "stellaFes")
            copia("stellaFes", path_to_abundance, self.folder+"/TEMP", "stellaFes")
            copia('noPINOLOSpr2pt1', path_to_turboModels, path_to_output, 'stellaFes_turbo')
            best1=10000
            best1_index=-1
            best2=10000
            best2_index=-1
            best3=10000
            best3_index=-1
            best4=10000
            best4_index=-1
            if radSolver == 'spectrum':
                out_folder = path_to_abundance
                basename = 'PINOLOS'
            elif radSolver == 'turbo':
                basename = 'PINOLOSai'
                out_folder = path_to_turboModels
            elif radSolver == 'moog':
                out_folder = path_to_moogOutput
                for i in range(processori):
                    format_moog_output_to_analysis(path_to_moogBatch+f'runs{i+1}', path_to_moogOutput, 'abfind_eqw2', f'PINOLOS{i+1}')
            if j == 0: #Valuta atmosfere
                for i in range(int(processori/4)): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                    self.nome = "1" + nome_temp
                    nome_output1 = f"{self.nome}_{search_data1[i][2]:.2f}_{(search_data1[i][0]):.0f}_{search_data1[i][1]:.2f}_{search_data1[i][3]:.2f}"
                    if radSolver == 'spectrum' or radSolver == 'turbo':
                        sposta(f"PINOLOS{i+1}", out_folder, self.folder+"/TEMP", nome_output1)
                    elif radSolver == 'moog':
                        #############sposta(f"PINOLOS{i+1}", out_folder, self.folder+"/TEMP", nome_output1)
                        sposta(f"PINOLOS{i+1}", path_to_moogOutput, self.folder+"/TEMP", nome_output1)
                    if best1 > self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver):
                        best1 = self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
                        best1_index=i
                    self.nome = "2" + nome_temp
                    nome_output2 = f"{self.nome}_{(search_data2[i][2]):.2f}_{search_data2[i][0]:.0f}_{search_data2[i][1]:.2f}_{search_data2[i][3]:.2f}"
                    if radSolver == 'spectrum' or radSolver == 'turbo':
                        sposta(f"PINOLOS{i+1+int(processori/4)}", out_folder, self.folder+"/TEMP", nome_output2)
                    elif radSolver == 'moog':
                        sposta(f"PINOLOS{i+1+int(processori/4)}", path_to_moogOutput, self.folder+"/TEMP", nome_output2)
                    if best2 > self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver):
                        best2 = self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
                        best2_index=i
                    self.nome = "3" + nome_temp
                    nome_output3 = f"{self.nome}_{search_data3[i][2]:.2f}_{(search_data3[i][0]):.0f}_{search_data3[i][1]:.2f}_{search_data3[i][3]:.2f}"
                    if radSolver == 'spectrum' or radSolver == 'turbo':
                        sposta(f"PINOLOS{i+1+2*int(processori/4)}", out_folder, self.folder+"/TEMP", nome_output3)
                    elif radSolver == 'moog':
                        sposta(f"PINOLOS{i+1+2*int(processori/4)}", path_to_moogOutput, self.folder+"/TEMP", nome_output3)
                    if best3 > self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver):
                        best3 = self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
                        best3_index=i
                    self.nome = "4" + nome_temp
                    nome_output4 = f"{self.nome}_{(search_data4[i][2]):.2f}_{search_data4[i][0]:.0f}_{search_data4[i][1]:.2f}_{search_data4[i][3]:.2f}"
                    if radSolver == 'spectrum' or radSolver == 'turbo':
                        sposta(f"PINOLOS{i+1+3*int(processori/4)}", out_folder, self.folder+"/TEMP", nome_output4)
                    elif radSolver == 'moog':
                        sposta(f"PINOLOS{i+1+3*int(processori/4)}", path_to_moogOutput, self.folder+"/TEMP", nome_output4)
                    if best4 > self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver):
                        best4 = self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
                        best4_index=i
            else:
                for i in range(int(processori/4)): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                    self.nome = "1" + nome_temp
                    nome_output1 = f"{self.nome}_{search_data1[i][2]:.2f}_{(search_data1[i][0]):.0f}_{search_data1[i][1]:.2f}_{search_data1[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1}", out_folder, self.folder+"/TEMP", nome_output1)
                    if best1 > self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver): #adv_matrix=A_numpy1, dyn_on=True):
                        best1 = self.evaluate_Models(round(search_data1[i][0], 0), round(search_data1[i][1], 2), round(search_data1[i][2], 2), round(search_data1[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)# adv_matrix=A_numpy1, dyn_on=True)
                        best1_index=i
                    self.nome = "2" + nome_temp
                    nome_output2 = f"{self.nome}_{(search_data2[i][2]):.2f}_{search_data2[i][0]:.0f}_{search_data2[i][1]:.2f}_{search_data2[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+int(processori/4)}", out_folder, self.folder+"/TEMP", nome_output2)
                    if best2 > self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver): # adv_matrix=A_numpy2, dyn_on=True):
                        best2 = self.evaluate_Models(round(search_data2[i][0], 0), round(search_data2[i][1], 2), round(search_data2[i][2], 2), round(search_data2[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver) # adv_matrix=A_numpy2, dyn_on=True)
                        best2_index=i

                    self.nome = "3" + nome_temp
                    nome_output3 = f"{self.nome}_{search_data3[i][2]:.2f}_{(search_data3[i][0]):.0f}_{search_data3[i][1]:.2f}_{search_data3[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+2*int(processori/4)}", out_folder, self.folder+"/TEMP", nome_output3)
                    if best3 > self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver):# adv_matrix=A_numpy3, dyn_on=True):
                        best3 = self.evaluate_Models(round(search_data3[i][0], 0), round(search_data3[i][1], 2), round(search_data3[i][2], 2), round(search_data3[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)# adv_matrix=A_numpy3, dyn_on=True)
                        best3_index=i
                    self.nome = "4" + nome_temp
                    nome_output4 = f"{self.nome}_{(search_data4[i][2]):.2f}_{search_data4[i][0]:.0f}_{search_data4[i][1]:.2f}_{search_data4[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1+3*int(processori/4)}", out_folder, self.folder+"/TEMP", nome_output4)
                    if best4 > self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver):# adv_matrix=A_numpy4, dyn_on=True):
                        best4 = self.evaluate_Models(round(search_data4[i][0], 0), round(search_data4[i][1], 2), round(search_data4[i][2], 2), round(search_data4[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)# adv_matrix=A_numpy4, dyn_on=True)
                        best4_index=i
            if best1>old_best1 and best1 < 20:
                best1=old_best1
                best1_index=old_best1_index
            if best2>old_best2 and best2 < 20:
                best2=old_best2
                best2_index=old_best2_index
            if best3>old_best3 and best3 < 20:
                best3=old_best3
                best3_index=old_best3_index
            if best4>old_best4 and best4 < 20:
                best4=old_best4
                best4_index=old_best4_index
            
            self.nome=nome_temp
            tt1=search_data1[best1_index][0]
            gg1=search_data1[best1_index][1]
            mm1=search_data1[best1_index][2]
            vv1=search_data1[best1_index][3]
            tt2=search_data2[best2_index][0]
            gg2=search_data2[best2_index][1]
            mm2=search_data2[best2_index][2]
            vv2=search_data2[best2_index][3]
            tt3=search_data3[best3_index][0]
            gg3=search_data3[best3_index][1]
            mm3=search_data3[best3_index][2]
            vv3=search_data3[best3_index][3]
            tt4=search_data4[best4_index][0]
            gg4=search_data4[best4_index][1]
            mm4=search_data4[best4_index][2]
            vv4=search_data4[best4_index][3]
            #riattiva_stampe()
            if best1 <= mCheck: #Verifica se l'atmosfera è accettabile
                if best2 < best1 and best2<best3 and best2<best4:
                    print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                    self.t_eff = tt2
                    self.logg = gg2
                    self.metal = mm2
                    self.v_m = vv2
                    self.param_error = best2
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
                elif best3 < best1 and best3 < best2 and best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
                elif best4 < best1 and best4 < best2 and best4 < best3:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
                else:
                    print(f"ECCOLA! T: {tt1}\nG: {gg1}\nM: {mm1}\nv: {vv1}\nDelta: {best1}")
                    self.t_eff = tt1
                    self.logg = gg1
                    self.metal = mm1
                    self.v_m = vv1
                    self.param_error = best2
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
            elif best2 <= mCheck:
                if best2<best3 and best2<best4:
                    print(f"ECCOLA! T: {tt2}\nG: {gg2}\nM: {mm2}\nv: {vv2}\nDelta: {best2}")
                    self.t_eff = tt2
                    self.logg = gg2
                    self.metal = mm2
                    self.v_m = vv2
                    self.param_error = best2
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
                elif best3 < best2 and best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
                elif best4 < best2 and best4 < best3:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
            elif best3 <= mCheck:
                if best3 < best4:
                    print(f"ECCOLA! T: {tt3}\nG: {gg3}\nM: {mm3}\nv: {vv3}\nDelta: {best3}")
                    self.t_eff = tt3
                    self.logg = gg3
                    self.metal = mm3
                    self.v_m = vv3
                    self.param_error = best3
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
                else:
                    print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                    self.t_eff = tt4
                    self.logg = gg4
                    self.metal = mm4
                    self.v_m = vv4
                    self.param_error = best4
                    (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                    self.stampaStella(self.nome, radSolver=radSolver)
                    return 0
            elif best4 <= mCheck:
                print(f"ECCOLA! T: {tt4}\nG: {gg4}\nM: {mm4}\nv: {vv4}\nDelta: {best4}")
                self.t_eff = tt4
                self.logg = gg4
                self.metal = mm4
                self.v_m = vv4
                self.param_error = best4
                (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
                self.stampaStella(self.nome, radSolver=radSolver)
                return 0
            #blocca_stampe()
            (self.ALL_STARS).append([[best1, tt1, gg1, mm1, vv1],[best2, tt2, gg2, mm2, vv2],[best3, tt3, gg3, mm3, vv3],[best4, tt4, gg4, mm4, vv4]])
            nome_out1 = f"1{self.nome}_{mm1:.2f}_{tt1:.0f}_{gg1:.2f}_{vv1:.2f}"
            nome_out2 = f"2{self.nome}_{mm2:.2f}_{tt2:.0f}_{gg2:.2f}_{vv2:.2f}"
            nome_out3 = f"3{self.nome}_{mm3:.2f}_{tt3:.0f}_{gg3:.2f}_{vv3:.2f}"
            nome_out4 = f"4{self.nome}_{mm4:.2f}_{tt4:.0f}_{gg4:.2f}_{vv4:.2f}"
            if CORRECTION==True: #Dalla migliore atmosfera calcola i parametri della successiva
                if NLTE==0:
                    temp01 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.0, radSolver=radSolver)
                    temp11 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out1, 26.1, radSolver=radSolver)
                    temp02 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.0, radSolver=radSolver)
                    temp12 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out2, 26.1, radSolver=radSolver)
                    temp03 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out3, 26.0, radSolver=radSolver)
                    temp13 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out3, 26.1, radSolver=radSolver)
                    temp04 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out4, 26.0, radSolver=radSolver)
                    temp14 = simpleAnalysisSingle(self.folder+"/TEMP", nome_out4, 26.1, radSolver=radSolver)
                elif NLTE == 1:
                    temp01 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.0, radSolver=radSolver)
                    temp11 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out1, tt1, gg1, mm1, vv1, 26.1, radSolver=radSolver)
                    temp02 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.0, radSolver=radSolver)
                    temp12 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out2, tt2, gg2, mm2, vv2, 26.1, radSolver=radSolver)
                    temp03 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out3, tt3, gg3, mm3, vv3, 26.0, radSolver=radSolver)
                    temp13 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out3, tt3, gg3, mm3, vv3, 26.1, radSolver=radSolver)
                    temp04 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out4, tt4, gg4, mm4, vv4, 26.0, radSolver=radSolver)
                    temp14 = NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out4, tt4, gg4, mm4, vv4, 26.1, radSolver=radSolver)
                master_search_data1.append([[tt1, gg1, mm1, vv1], [(temp01[1]), (temp01[0])-(temp11[0]), temp01[0]-(mm1), (temp01[3])]]) #contengono sia i parametri atmosferici (T, G, M, V) sia i valori da minimizzare (s, F, Y, S)
                master_search_data2.append([[tt2, gg2, mm2, vv2], [(temp02[1]), (temp02[0])-(temp12[0]), temp02[0]-(mm2), (temp02[3])]]) #In questi vengono inseriti i migliori parametri per searchBranch.
                master_search_data3.append([[tt3, gg3, mm3, vv3], [(temp03[1]), (temp03[0])-(temp13[0]), temp03[0]-(mm3), (temp03[3])]])
                master_search_data4.append([[tt4, gg4, mm4, vv4], [(temp04[1]), (temp04[0])-(temp14[0]), temp04[0]-(mm4), (temp04[3])]])
                if j==0:
                    FIXD_MAT = 1
                if FIXD_MAT == 1:
                    v1=sp.symbols('v')
                    v2=sp.symbols('v')
                    v3=sp.symbols('v')
                    v4=sp.symbols('v')
                    A1 = s_Matrix
                    A2 = s_Matrix
                    A3 = s_Matrix
                    A4 = s_Matrix
                    AA1=A1.subs(v1, vv1)
                    AA2=A2.subs(v2, vv2)
                    AA3=A3.subs(v3, vv3)
                    AA4=A4.subs(v4, vv4)
                    A_numpy1 = np.array(AA1.tolist(), dtype=float)
                    A_inv1 = np.linalg.inv(A_numpy1)
                    A_numpy2 = np.array(AA2.tolist(), dtype=float)
                    A_inv2 = np.linalg.inv(A_numpy2)
                    A_numpy3 = np.array(AA3.tolist(), dtype=float)
                    A_inv3 = np.linalg.inv(A_numpy3)
                    A_numpy4 = np.array(AA4.tolist(), dtype=float)
                    A_inv4 = np.linalg.inv(A_numpy4)
                    f1=-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])
                    f3=-1*((temp03[0])-(temp13[0]))
                    s3 = -1*(temp03[1])
                    m3 = -1*(temp03[0]-(mm3))
                    ewr3 = -1*(temp03[3])
                    B3 = np.array([s3, f3, m3, ewr3])
                    f4=-1*((temp04[0])-(temp14[0]))
                    s4 = -1*(temp04[1])
                    m4 = -1*(temp04[0]-(mm4))
                    ewr4 = -1*(temp04[3])
                    B4 = np.array([s4, f4, m4, ewr4])
                    #blocca_stampe()
                    if abs(s1)<0.001 and abs(m1) < 0.01:
                        DT1 = 0
                        DM1 = 0
                        Dv1 = 0
                        s1 = 0
                        m1 = 0
                        ewr1 = 0
                        B1 = np.array([s1, f1, m1, ewr1])
                        DM1 = (np.dot(A_inv1, B1), [1])
                    else:
                        if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT1 = (np.dot(A_inv1, B1))[0]
                            DG1 = (np.dot(A_inv1, B1))[1]
                            DM1 = (np.dot(A_inv1, B1))[2]
                            Dv1 = (np.dot(A_inv1, B1))[3]
                            if abs(DG1) > 0.5:
                                DG1=0.5*(DG1/abs(DG1))
                            if abs(DT1) > 250:
                                DT1 = 250*(DT1/abs(DT1))
                            if abs(DM1) > 0.25:
                                DM1 = 0.25*(DM1/abs(DM1))
                            if abs(Dv1) > 0.3:
                                Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(s2)<0.001 and abs(m2) < 0.01:
                        DT2 = 0
                        DM2 = 0
                        Dv2 = 0
                        s2 = 0
                        m2 = 0
                        ewr2 = 0
                        B2 = np.array([s2, f2, m2, ewr2])
                        DM2 = (np.dot(A_inv2, B2), [1])
                    else:
                        if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT2 = (np.dot(A_inv2, B2))[0]
                            DG2 = (np.dot(A_inv2, B2))[1]
                            DM2 = (np.dot(A_inv2, B2))[2]
                            Dv2 = (np.dot(A_inv2, B2))[3]
                            if abs(DG2) > 0.5:
                                DG2=0.5*(DG2/abs(DG2))
                            if abs(DT2) > 250:
                                DT2 = 250*(DT2/abs(DT2))
                            if abs(DM2) > 0.25:
                                DM2 = 0.25*(DM2/abs(DM2))
                            if abs(Dv2) > 0.3:
                                Dv2 = 0.3*(Dv2/abs(Dv2))
                    if abs(s3)<0.001 and abs(m3) < 0.01:
                        DT3 = 0
                        DM3 = 0
                        Dv3 = 0
                        s3 = 0
                        m3 = 0
                        ewr3 = 0
                        B3 = np.array([s3, f3, m3, ewr3])
                        DM3 = (np.dot(A_inv3, B3), [1])
                    else:
                        if abs(temp03[1]) > 0.0001 or abs((temp03[0])-(temp13[0])) > 0.0003 or abs(mm3 - (temp03[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT3 = (np.dot(A_inv3, B3))[0]
                            DG3 = (np.dot(A_inv3, B3))[1]
                            DM3 = (np.dot(A_inv3, B3))[2]
                            Dv3 = (np.dot(A_inv3, B3))[3]
                            if abs(DG3) > 0.5:
                                DG3=0.5*(DG3/abs(DG3))
                            if abs(DT2) > 250:
                                DT3 = 250*(DT3/abs(DT3))
                            if abs(DM3) > 0.25:
                                DM3 = 0.25*(DM3/abs(DM3))
                            if abs(Dv3) > 0.3:
                                Dv3 = 0.3*(Dv3/abs(Dv3))
                    if abs(s4)<0.001 and abs(m4) < 0.01:
                        DT4 = 0
                        DM4 = 0
                        Dv4 = 0
                        s4  = 0
                        m4  = 0
                        ewr4= 0
                        B4 = np.array([s4, f4, m4, ewr4])
                        DM4 = (np.dot(A_inv4, B4), [1])
                    else:
                        if abs(temp04[1]) > 0.0001 or abs((temp04[0])-(temp14[0])) > 0.0003 or abs(mm4 - (temp04[0]))>0.001:
                            # DG = -1*temp0[1] / (dsdg + k * dfdt) 
                            # DT = (-1*((temp0[0])-(temp1[0])) - dfdg * (-1)*temp0[1] / (dsdg + k * dfdt) )*dfdt
                            DT4 = (np.dot(A_inv4, B4))[0]
                            DG4 = (np.dot(A_inv4, B4))[1]
                            DM4 = (np.dot(A_inv4, B4))[2]
                            Dv4 = (np.dot(A_inv4, B4))[3]
                            if abs(DG4) > 0.5:
                                DG4=0.5*(DG4/abs(DG4))
                            if abs(DT4) > 250:
                                DT4 = 250*(DT4/abs(DT4))
                            if abs(DM4) > 0.25:
                                DM4 = 0.25*(DM4/abs(DM4))
                            if abs(Dv2) > 0.3:
                                Dv4 = 0.3*(Dv4/abs(Dv4))
                    if t_fixed != 0:
                        DT1=0
                        DT2=0
                        DT3=0
                        DT4=0
                    if g_fixed != -1:
                        DG1=0
                        DG2=0
                        DG3=0
                        DG4=0
                    if m_fixed != 5:
                        DM1=0
                        DM2=0
                        DM3=0
                        DM4=0
                    if v_fixed != -5:
                        Dv1=0
                        Dv2=0
                        Dv3=0
                        Dv4=0
                    FIXD_MAT=0
                else:
                    #Trial i vari DDX prima erano tutti moltiplicati per 2 rhs
                    A_numpy1 = crea_dynMatrix(master_search_data1[j-1][0], master_search_data1[j-1][1], master_search_data1[j][0], master_search_data1[j][1])
                    DDT1 = (master_search_data1[j-1][0][0]-master_search_data1[j][0][0]) #La differenza tra il valore precedente e quello successivo di Teff
                    DDG1 = (master_search_data1[j-1][0][1]-master_search_data1[j][0][1]) #La differenza tra il valore precedente e quello successivo di logg
                    DDM1 = (master_search_data1[j-1][0][2]-master_search_data1[j][0][2]) #La differenza tra il valore precedente e quello successivo di [M/H]
                    DDV1 = (master_search_data1[j-1][0][3]-master_search_data1[j][0][3])
                    # if DDT1 == 0 and DDG1 == 0 and DDM1 == 0 DDV1 == 0:
                    #     DDT1 = (master_search_data1[j-2][0][0]-master_search_data1[j-1][0][0]) #La differenza tra il valore precedente e quello successivo di Teff
                    #     DDG1 = (master_search_data1[j-2][0][1]-master_search_data1[j-1][0][1]) #La differenza tra il valore precedente e quello successivo di logg
                    #     DDM1 = (master_search_data1[j-2][0][2]-master_search_data1[j-1][0][2]) #La differenza tra il valore precedente e quello successivo di [M/H]
                    #     DDV1 = (master_search_data1[j-2][0][3]-master_search_data1[j-1][0][3])
                    #     A_numpy1 = crea_dynMatrix(master_search_data1[j-2][0], master_search_data1[j-2][1], master_search_data1[j-1][0], master_search_data1[j-1][1])
                    A_inv1 = np.linalg.pinv(A_numpy1)
                    f1 =-1*((temp01[0])-(temp11[0]))
                    s1 = -1*(temp01[1])
                    m1 = -1*(temp01[0]-(mm1))
                    ewr1 = -1*(temp01[3])
                    B1 = np.array([s1, f1, m1, ewr1])

                    A_numpy2 = crea_dynMatrix(master_search_data2[j-1][0], master_search_data2[j-1][1], master_search_data2[j][0], master_search_data2[j][1])
                    DDT2 = (master_search_data2[j-1][0][0]-master_search_data2[j][0][0])
                    DDG2 = (master_search_data2[j-1][0][1]-master_search_data2[j][0][1])
                    DDM2 = (master_search_data2[j-1][0][2]-master_search_data2[j][0][2])
                    DDV2 = (master_search_data2[j-1][0][3]-master_search_data2[j][0][3])
                    # if DDT1 == 0 and DDG1 == 0 and DDM1 == 0 DDV1 == 0:
                    #     DDT1 = (master_search_data2[j-2][0][0]-master_search_data2[j-1][0][0]) #La differenza tra il valore precedente e quello successivo di Teff
                    #     DDG1 = (master_search_data2[j-2][0][1]-master_search_data2[j-1][0][1]) #La differenza tra il valore precedente e quello successivo di logg
                    #     DDM1 = (master_search_data2[j-2][0][2]-master_search_data2[j-1][0][2]) #La differenza tra il valore precedente e quello successivo di [M/H]
                    #     DDV1 = (master_search_data2[j-2][0][3]-master_search_data2[j-1][0][3])
                    #     A_numpy2 = crea_dynMatrix(master_search_data2[j-2][0], master_search_data2[j-2][1], master_search_data2[j-1][0], master_search_data2[j-1][1])
                    A_inv2 = np.linalg.pinv(A_numpy2)
                    f2=-1*((temp02[0])-(temp12[0]))
                    s2 = -1*(temp02[1])
                    m2 = -1*(temp02[0]-(mm2))
                    ewr2 = -1*(temp02[3])
                    B2 = np.array([s2, f2, m2, ewr2])

                    A_numpy3 = crea_dynMatrix(master_search_data3[j-1][0], master_search_data3[j-1][1], master_search_data3[j][0], master_search_data3[j][1])
                    DDT3 = (master_search_data3[j-1][0][0]-master_search_data3[j][0][0])
                    DDG3 = (master_search_data3[j-1][0][1]-master_search_data3[j][0][1])
                    DDM3 = (master_search_data3[j-1][0][2]-master_search_data3[j][0][2])
                    DDV3 = (master_search_data3[j-1][0][3]-master_search_data3[j][0][3])
                    # if DDT1 == 0 and DDG1 == 0 and DDM1 == 0 DDV1 == 0:
                    #     DDT1 = (master_search_data3[j-2][0][0]-master_search_data3[j-1][0][0]) #La differenza tra il valore precedente e quello successivo di Teff
                    #     DDG1 = (master_search_data3[j-2][0][1]-master_search_data3[j-1][0][1]) #La differenza tra il valore precedente e quello successivo di logg
                    #     DDM1 = (master_search_data3[j-2][0][2]-master_search_data3[j-1][0][2]) #La differenza tra il valore precedente e quello successivo di [M/H]
                    #     DDV1 = (master_search_data3[j-2][0][3]-master_search_data3[j-1][0][3])
                    #     A_numpy3 = crea_dynMatrix(master_search_data3[j-2][0], master_search_data3[j-2][1], master_search_data3[j-1][0], master_search_data3[j-1][1])
                    A_inv3 = np.linalg.pinv(A_numpy3)
                    f3 =-1*((temp03[0])-(temp13[0]))
                    s3 = -1*(temp03[1])
                    m3 = -1*(temp03[0]-(mm3))
                    ewr3 = -1*(temp03[3])
                    B3 = np.array([s3, f3, m3, ewr3])

                    A_numpy4 = crea_dynMatrix(master_search_data4[j-1][0], master_search_data4[j-1][1], master_search_data4[j][0], master_search_data4[j][1])
                    DDT4 = (master_search_data4[j-1][0][0]-master_search_data4[j][0][0])
                    DDG4 = (master_search_data4[j-1][0][1]-master_search_data4[j][0][1])
                    DDM4 = (master_search_data4[j-1][0][2]-master_search_data4[j][0][2])
                    DDV4 = (master_search_data4[j-1][0][3]-master_search_data4[j][0][3])
                    # if DDT1 == 0 and DDG1 == 0 and DDM1 == 0 DDV1 == 0:
                    #     DDT1 = (master_search_data4[j-2][0][0]-master_search_data4[j-1][0][0]) #La differenza tra il valore precedente e quello successivo di Teff
                    #     DDG1 = (master_search_data4[j-2][0][1]-master_search_data4[j-1][0][1]) #La differenza tra il valore precedente e quello successivo di logg
                    #     DDM1 = (master_search_data4[j-2][0][2]-master_search_data4[j-1][0][2]) #La differenza tra il valore precedente e quello successivo di [M/H]
                    #     DDV1 = (master_search_data4[j-2][0][3]-master_search_data4[j-1][0][3])
                    #     A_numpy4 = crea_dynMatrix(master_search_data4[j-2][0], master_search_data4[j-2][1], master_search_data4[j-1][0], master_search_data4[j-1][1])
                    A_inv4 = np.linalg.pinv(A_numpy4)
                    f4=-1*((temp04[0])-(temp14[0]))
                    s4 = -1*(temp04[1])
                    m4 = -1*(temp04[0]-(mm4))
                    ewr4 = -1*(temp04[3])
                    B4 = np.array([s4, f4, m4, ewr4])

                    #blocca_stampe()
                    if abs(temp01[1]) > 0.0001 or abs((temp01[0])-(temp11[0])) > 0.0003 or abs(mm1 - (temp01[0]))>0.001:
                        DT1 = (np.dot(A_inv1, B1))[0]
                        DG1 = (np.dot(A_inv1, B1))[1]
                        DM1 = (np.dot(A_inv1, B1))[2]
                        Dv1 = (np.dot(A_inv1, B1))[3]
                        if abs(DG1) > 0.5:
                            DG1=0.5*(DG1/abs(DG1))
                        if abs(DT1) > 250:
                            DT1 = 250*(DT1/abs(DT1))
                        if abs(DM1) > 0.25:
                            DM1 = 0.25*(DM1/abs(DM1))
                        if abs(Dv1) > 0.3:
                            Dv1 = 0.3*(Dv1/abs(Dv1))
                    if abs(temp02[1]) > 0.0001 or abs((temp02[0])-(temp12[0])) > 0.0003 or abs(mm2 - (temp02[0]))>0.001:
                        DT2 = (np.dot(A_inv2, B2))[0]
                        DG2 = (np.dot(A_inv2, B2))[1]
                        DM2 = (np.dot(A_inv2, B2))[2]
                        Dv2 = (np.dot(A_inv2, B2))[3]
                        if abs(DG2) > 0.5:
                            DG2=0.5*(DG2/abs(DG2))
                        if abs(DT2) > 250:
                            DT2 = 250*(DT2/abs(DT2))
                        if abs(DM2) > 0.25:
                            DM2 = 0.25*(DM2/abs(DM2))
                        if abs(Dv2) > 0.3:
                            Dv2 = 0.3*(Dv2/abs(Dv2))
                    if abs(temp03[1]) > 0.0001 or abs((temp03[0])-(temp13[0])) > 0.0003 or abs(mm3 - (temp03[0]))>0.001:
                        DT3 = (np.dot(A_inv3, B3))[0]
                        DG3 = (np.dot(A_inv3, B3))[1]
                        DM3 = (np.dot(A_inv3, B3))[2]
                        Dv3 = (np.dot(A_inv3, B3))[3]
                        if abs(DG3) > 0.5:
                            DG3=0.5*(DG3/abs(DG3))
                        if abs(DT3) > 250:
                            DT3 = 250*(DT3/abs(DT3))
                        if abs(DM3) > 0.25:
                            DM3 = 0.25*(DM3/abs(DM3))
                        if abs(Dv3) > 0.3:
                            Dv3 = 0.3*(Dv3/abs(Dv3))
                    if abs(temp04[1]) > 0.0001 or abs((temp04[0])-(temp14[0])) > 0.0003 or abs(mm4 - (temp04[0]))>0.001:
                        DT4 = (np.dot(A_inv4, B4))[0]
                        DG4 = (np.dot(A_inv4, B4))[1]
                        DM4 = (np.dot(A_inv4, B4))[2]
                        Dv4 = (np.dot(A_inv4, B4))[3]
                        if abs(DG4) > 0.5:
                            DG4=0.5*(DG4/abs(DG4))
                        if abs(DT4) > 250:
                            DT4 = 250*(DT4/abs(DT4))
                        if abs(DM4) > 0.25:
                            DM4 = 0.25*(DM4/abs(DM4))
                        if abs(Dv4) > 0.3:
                            Dv4 = 0.3*(Dv4/abs(Dv4))
                    if t_fixed != 0:
                        DT1=0
                        DT2=0
                        DT3=0
                        DT4=0
                    if g_fixed != -1:
                        DG1=0
                        DG2=0
                        DG3=0
                        DG4=0
                    if m_fixed != 5:
                        DM1=0
                        DM2=0
                        DM3=0
                        DM4=0
                    if v_fixed != -5:
                        Dv1=0
                        Dv2=0
                        Dv3=0
                        Dv4=0
                temperatu1=tt1+DT1
                G1=gg1+DG1
                velturb1=vv1+Dv1
                teff1= temperatu1
                temperatu2=tt2+DT2
                G2=gg2+DG2
                velturb2=vv2+Dv2
                teff2= temperatu2

                temperatu3=tt3+DT3
                G3=gg3+DG3
                velturb3=vv3+Dv3
                teff3= temperatu3
                temperatu4=tt4+DT4
                G4=gg4+DG4
                velturb4=vv4+Dv4
                teff4= temperatu4
            if fixd_velturb == True:
                velturb1_t = vturbo(teff1, G1, mm1+DM1)
                velturb2_t = vturbo(teff2, G2, mm2+DM2)
                velturb3_t = vturbo(teff3, G3, mm3+DM3)
                velturb4_t = vturbo(teff4, G4, mm4+DM4)
                if velturb1 > 1.1*velturb1_t:
                    velturb1 = 1.1*velturb1_t
                elif velturb1 < 0.9*velturb1_t:
                    velturb1 = 0.9*velturb1_t
                if velturb2 > 1.1*velturb2_t:
                    velturb2 = 1.1*velturb2_t
                elif velturb2 < 0.9*velturb2_t:
                    velturb2 = 0.9*velturb2_t
                if velturb3 > 1.1*velturb3_t:    
                    velturb3 = 1.1*velturb3_t
                elif velturb3 < 0.9*velturb3_t:
                    velturb3 = 0.9*velturb3_t
                if velturb4 > 1.1*velturb4_t:
                    velturb4 = 1.1*velturb4_t
                elif velturb4 < 0.9*velturb4_t:
                    velturb4 = 0.9*velturb4_t  
            search_data1 = []
            search_data1.append([teff1, G1, mm1+DM1, velturb1])
            search_data2 = []
            search_data2.append([teff2, G2, mm2+DM2, velturb2])
            search_data3 = []
            search_data3.append([teff3, G3, mm3+DM3, velturb3])
            search_data4 = []
            search_data4.append([teff4, G4, mm4+DM4, velturb4])
            old_best1_index=best1_index
            old_best2_index=best2_index
            old_best3_index=best3_index
            old_best4_index=best4_index
            old_best1=best1
            old_best2=best2
            old_best3=best3
            old_best4=best4
        #riattiva_stampe()
        #riattiva_stampe()
        self.nome = "1" + nome_temp
        copia(self.nome, self.folder+'/TEMP', self.folder, self.nome)
        O1 = self.evaluate_Models(tt1, gg1, mm1, vv1, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
        self.nome = "2" + nome_temp
        copia(self.nome, self.folder+'/TEMP', self.folder, self.nome)
        O2 = self.evaluate_Models(tt2, gg2, mm2, vv2, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
        self.nome = "3" + nome_temp
        copia(self.nome, self.folder+'/TEMP', self.folder, self.nome)
        O3 = self.evaluate_Models(tt3, gg3, mm3, vv3, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
        self.nome = "4" + nome_temp
        copia(self.nome, self.folder+'/TEMP', self.folder, self.nome)
        O4 = self.evaluate_Models(tt4, gg4, mm4, vv4, self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed, radSolver=radSolver)
        print(O1)
        print(O2)
        print(O3)
        print(O4)
        self.nome = nome_temp
        print(f"{tt1}, {gg1}, {mm1}. {vv1}")
        print(f"{tt2}, {gg2}, {mm2}. {vv2}")
        vettori_selezionati = []
        for quaterna in self.ALL_STARS: #crea returner
            if len(quaterna) == 4:
                # Trova il vettore con il primo elemento più piccolo nella quaterna
                minimo_quaterna = min(quaterna, key=lambda x: x[0])
                vettori_selezionati.append(minimo_quaterna)

        # Trova il vettore con il primo elemento più piccolo tra i selezionati
        vettore_minimo = min(vettori_selezionati, key=lambda x: x[0])
        if self.param_error > vettore_minimo[0]:
            self.param_error = vettore_minimo[0]
            self.t_eff = vettore_minimo[1]
            self.logg = vettore_minimo[2]
            self.metal = vettore_minimo[3]
            self.v_m = vettore_minimo[4]
        abndfile=[]
        for file in os.listdir(self.folder+"/TEMP"):
            if file.endswith(f'{self.nome}_{self.metal:.2f}_{int(self.t_eff)}_{self.logg:.2f}_{self.v_m:.2f}'):
                abndfile.append(file)
        #self.abndfile = abndfile[0]
        copia(f'T{int(self.t_eff)}_G{(float(self.logg)):.2f}_M{(float(self.metal)):.2f}_{(float(self.v_m)):.2f}.model', path_to_atmosphere+interpol_folder, self.folder, self.nome+f'_T{self.t_eff}_G{self.logg}_M{self.metal}_V{self.v_m}')
        self.model = self.folder + self.nome+f'_T{self.t_eff}_G{self.logg}_M{self.metal}_V{self.v_m}'
        self.stampaStella(self.nome, radSolver=radSolver)
        print()
        sposta('stellaFes_turbo', path_to_output, path_to_output, f'stellaFes_turbo_{self.nome}')
        #sposta()
    def searchParam4(self, processori=1, NLTE=0, TRIALS = 50, mashonkina=True, t01=-1, g01=-2, m01=10, v01=-1, t02=-1, g02=-2, m02=10, v02=-1, t03=-1, g03=-2, m03=10, v03=-1, t04=-1, g04=-2, m04=10, v04=-1, t_fixed=0, g_fixed=-1, m_fixed=5, v_fixed=-5, mCheck=3, radSolver = 'spectrum', atmospheric_model='ATLAS9'):
        '''Ad ogni processore corrisponde un unico branch di ricerca.\n
        Usare un numero di processori divisibile per 4\n
         NLTE = 0 per modelli LTE, NLTE = 1 per modelli NLTE\n
         TRIALS = numero di iterazioni\n
         t0, g0, m0, v0 = condizioni iniziali\n 
         t_fixed, g_fixed, m_fixed, v_fixed = parametri fissati\n 
         mCheck = 3 per modelli con 3 parametri, 4 per modelli con 4 parametri\n
         radSolver = spectrum per utilizzare spectrum, turbo per utilizzare turbospectrum\n
         atmospheric_model = ATLAS9 per calcolare sul momento atmosfere ATLAS9, ATLASint per ottenere le atmosfere ATLAS9 tramite interpolatore'''
        fixd_velturb = mashonkina
        CORRECTION = True
        T_start = np.zeros(processori)
        G_start = np.zeros(processori)
        M_start = np.zeros(processori)
        V_start = np.zeros(processori)
        
        for i in range(processori):
            T_start[i]=4000+(i/processori)*1500
            G_start[i]=0.5+(i/processori)*2.50
            M_start[i]=random.uniform(-2, 0)
            V_start[i]=vturbo(T_start[i], G_start[i], M_start[i])
        if t_fixed != 0:
            for i in range(processori):
                T_start[i]=t_fixed
        if g_fixed != -1:
            for i in range(processori):
                G_start[i]=g_fixed
        if m_fixed != 5:
            for i in range(processori):
                M_start[i]=m_fixed
        if v_fixed != -5:
            for i in range(processori):
                V_start[i]=v_fixed

        self.dopcor()
        self.saveFits()
        delete_files(path_to_abundance)
        #blocca_stampe()
        if True: #Starting block #
            search_data = []
            master_search_data = []
            self.ALL_STARS=[]
            for i in range(processori):
                search_data.append([T_start[i], G_start[i], M_start[i], V_start[i], self.nome+f'_{M_start[i]:.2f}_{T_start[i]:.0f}_{G_start[i]:.2f}'])  # Si specificano le condizioni iniziali della ricerca, nel dubbio si lascino i valori presenti. HARPN0, HARPN1, HARPN2 etc.. sono i nomi delle stelle che ho analizzato.
                self.creaModel(search_data[i][0], search_data[i][0], search_data[i][0], search_data[i][0], processore=i+1, radSolver='spectrum', intpol=True)
                preparaAbundance(self.fits_name, outFes=f'stellaFes{i+1}')
                creaParamfileAbnd1(search_data[i][3], i+1, file_linee=f'stellaFes{i+1}')
            if __name__ == "__main__": # Calcolo abbondanze
                main_integration()
            out_folder = path_to_abundance
            for i in range(processori):
                sposta(f'PINOLOS{i+1}', out_folder, path_to_output, search_data[i][4])
        os.chdir(path_to_output)
        o = 0
        boundary_check = True
        s=[]
        F=[]
        Y=[]
        S=[]
        crea_cartella(self.folder)
        crea_cartella(self.folder+"/TEMP")
        DDT=np.zeros(processori)
        DDG=np.zeros(processori)
        DDM=np.zeros(processori)
        DDV=np.zeros(processori)
        best=np.zeros(processori)
        best_index = np.zeros(processori)
        old_best=np.zeros(processori)
        old_best_index=np.zeros(processori)
        if True: #help block
            for i in range(processori):
                DDT[i]=50
                DDG[i]=0.15
                DDM[i]=0.15
                DDV[i]=0.15
                best[i]=1
                old_best[i]=200000
                old_best_index[i]=-2
        nome_temp=self.nome
        for j in range(TRIALS): ## Numero di iterazioni
            if boundary_check: ### I seguenti if sono stati inseriti per riportare l'analisi entro dei range credibili per stelle giganti FGK metal-poor, è un modo per correggere eventuali divergenze nell'esplorazione dello spazio
                for i in range(processori):
                    if search_data[i][1] <= 0.0:
                        search_data[i][1] = 0.5
                    if search_data[i][0] <= 3700:
                        search_data[i][0] = 3750
                    if search_data[i][1] > 3.2:
                        search_data[i][1] = 3.0
                    if abs(search_data[i][3]-vturbo(search_data[i][0], search_data[i][1], search_data[i][2])) > 0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data[0][3] = vturbo(search_data[i][0], search_data[i][1], search_data[i][2])-0.081
                    if (search_data[0][3] - vturbo(search_data[i][0], search_data[i][1], search_data[i][2])) < -0.1:
                        print("NOOOOOOOOOOOOOOOOO")
                        search_data[0][3] = vturbo(search_data[i][0], search_data[i][1], search_data[i][2])+0.081
            for k in range(processori): # Crea i modelli da analizzare
                if best[i] > 100:
                    best[i] = 100
                if atmospheric_model == 'ATLASint':
                    print(f'LOLLOSPAMMO {k}')
                    self.creaModel(search_data[k][0], search_data[k][1], search_data[k][2], search_data[k][3], k+1, radSolver=radSolver, intpol=True)
            if radSolver == 'spectrum':
                if __name__ == "__main__": # Calcolo abbondanze
                    main_integration()
            elif radSolver == 'turbo':
                metalScale = []
                VTURBo = []
                for o in range(processori):
                    if o >= 0 and o < processori:
                        metalScale.append(search_data[o][2])
                        VTURBo.append(search_data[o][3])
                    td=os.getcwd()
                    os.chdir(path_to_turboModels)
                    fix_24th_line_35th_char(f'Model{o+1}')
                    os.chdir(td)
                lanciaTurbospectrum2(stellaFITS=self.fits_name, processore=processori, TURBVEL=VTURBo, modelName='Model', outname='PINOLOS', metalScaling=metalScale)
                for I in range(processori):
                    turbo_resultCombine(path_to_turboModels, f"PINOLOSpr{I+1}pt", f"PINOLOS{I+1}")
                    turbo_to_spectrum_resultConversion(path_to_turboModels, f'PINOLOS{I+1}')
                for I in range(processori):
                    delete_files_with_prefix(path_to_turboModels, f"PINOLOSpr{I+1}pt")
                    delete_files_with_prefix(path_to_turboModels, f"noPINOLOS{I+1}")
            copia("stellaFes", path_to_abundance, self.folder, "stellaFes")
            copia("stellaFes", path_to_abundance, self.folder+"/TEMP", "stellaFes")
            for i in range(processori):
                best[i]=10000
                best_index[i]=-1
            if radSolver == 'spectrum':
                out_folder = path_to_abundance
                basename = 'PINOLOS'
            elif radSolver == 'turbo':
                basename = 'PINOLOSai'
                out_folder = path_to_turboModels
            if j == 0: #Valuta atmosfere, PRIMA ITERAZIONE
                for i in range(int(processori)): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                    self.nome = f"{i}" + nome_temp
                    nome_output = f"{self.nome}_{search_data[i][2]:.2f}_{(search_data[i][0]):.0f}_{search_data[i][1]:.2f}_{search_data[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1}", out_folder, self.folder+"/TEMP", nome_output)
                    if best[i] > self.evaluate_Models(round(search_data[i][0], 0), round(search_data[i][1], 2), round(search_data[i][2], 2), round(search_data[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed):
                        best[i] = self.evaluate_Models(round(search_data[i][0], 0), round(search_data[i][1], 2), round(search_data[i][2], 2), round(search_data[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)
                        best_index[i]=i
            else: #TUTTE LE ALTRE ITERAZIONI
                for i in range(processori): #Trova la migliore, questo ciclo ritorna la variabile best_index che è l'indice di search_data1[best_index][] contenente i dati dell'atmosfera migliore
                    self.nome = f'{i}' + nome_temp
                    nome_output = f"{self.nome}_{search_data[i][2]:.2f}_{(search_data[i][0]):.0f}_{search_data[i][1]:.2f}_{search_data[i][3]:.2f}"
                    sposta(f"PINOLOS{i+1}", out_folder, self.folder+"/TEMP", nome_output)
                    if best[i] > self.evaluate_Models(round(search_data[i][0], 0), round(search_data[i][1], 2), round(search_data[i][2], 2), round(search_data[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed): #adv_matrix=A_numpy1, dyn_on=True):
                        best[i] = self.evaluate_Models(round(search_data[i][0], 0), round(search_data[i][1], 2), round(search_data[i][2], 2), round(search_data[i][3], 2), self.folder+"/TEMP", NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed)# adv_matrix=A_numpy1, dyn_on=True)
                        best_index[i]=i

            for i in range(processori):
                if best[i]>old_best[i] and best[i] < 20:
                    best[i]=old_best[i]
                    best_index[i]=old_best_index[i]
                
            self.nome=nome_temp
            tt=np.zeros(processori)
            gg=np.zeros(processori)
            mm=np.zeros(processori)
            vv=np.zeros(processori)
            for i in range(processori):
                tt[i]=search_data[i][0]
                gg[i]=search_data[i][1]
                mm[i]=search_data[i][2]
                vv[i]=search_data[i][3]
            print(tt)
            print(gg)
            print(mm)
            print(vv)
            #riattiva_stampe()
            tmp=best[0]
            tmp_index=0
            nome_out=[]
            for i in range(1, processori):
                if best[i] < tmp:
                    tmp=best[i]
                    tmp_index=i
            TEMP_ALLSTARS=[]
            if tmp<mCheck and tmp_index!=-1:
                self.t_eff=tt[tmp_index]
                self.logg=gg[tmp_index]
                self.metal=mm[tmp_index]
                self.v_m=vv[tmp_index]
                self.param_error=best[tmp_index]
                TEMP_ALLSTARS=[]
                for i in range(processori):
                    TEMP_ALLSTARS.append([tt[i], gg[i], mm[i], vv[i]])
                self.ALL_STARS.append(TEMP_ALLSTARS)
                return 0
            for i in range(processori):
                TEMP_ALLSTARS.append([tt[i], gg[i], mm[i], vv[i]])
                nome_out.append(f"{i}{self.nome}_{mm[i]:.2f}_{tt[i]:.0f}_{gg[i]:.2f}_{vv[i]:.2f}")
            self.ALL_STARS.append(TEMP_ALLSTARS)
            if CORRECTION==True: #Dalla migliore atmosfera calcola i parametri della successiva
                temp0=[]
                temp1=[]
                if NLTE==0:
                    for i in range(processori):
                        temp0.append(simpleAnalysisSingle(self.folder+"/TEMP", nome_out[i], 26.0, radSolver=radSolver))
                        temp1.append(simpleAnalysisSingle(self.folder+"/TEMP", nome_out[i], 26.1, radSolver=radSolver))
                elif NLTE == 1:
                    for i in range(processori):
                        temp0.append(NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out[i], tt[i], gg[i], mm[i], vv[i], 26.0, radSolver=radSolver))
                        temp1.append(NLTE_AnalysisSingle(self.folder+"/TEMP", nome_out[i], tt[i], gg[i], mm[i], vv[i], 26.1, radSolver=radSolver))
                TEMP_MASTER_SEARCH_DATA=[]
                for i in range(processori):
                    TEMP_MASTER_SEARCH_DATA.append(([tt[i], gg[i], mm[i], vv[i]], [temp0[i][0], temp0[i][0]-temp1[i][0], temp0[i][0]-mm[i], temp0[i][3]]))  #contengono sia i parametri atmosferici (T, G, M, V) sia i valori da minimizzare (s, F, Y, S)     #In questi vengono inseriti i migliori parametri per searchBranch.
                master_search_data.append(TEMP_MASTER_SEARCH_DATA)
                if j==0:
                    FIXD_MAT = 1

                if FIXD_MAT == 1:
                    v=[]
                    AA=[]
                    A_numpy=[]
                    A_inv=[]
                    A=s_Matrix
                    f=[]
                    s=[]
                    m=[]
                    ewr=[]
                    B=[]
                    DT=[]
                    DG=[]
                    DM=[]
                    Dv=[]
                    for i in range(processori):
                        v.append(sp.symbols('v'))
                        AA.append(A.subs(v[i], vv[i]))
                        A_numpy.append(np.array(AA[i].tolist(), dtype=float))
                        A_inv.append(np.linalg.inv(A_numpy[i]))
                        f.append(-1*((temp0[i][0])-(temp1[i][0])))
                        s.append(-1*temp0[i][1])
                        m.append(-1*(temp0[i][0]-(mm[i])))
                        ewr.append(-1*(temp0[i][3]))
                        B.append(np.array([s[i], f[i], m[i], ewr[i]]))
                        if abs(s[i])<0.001 and abs(m[i]) < 0.01:
                            DT[i] = 0
                            DM[i] = 0
                            Dv[i] = 0
                            s[i] = 0
                            m[i] = 0
                            ewr[i] = 0
                            B[i] = np.array([s[i], f[i], m[i], ewr[i]])
                            DM[i] = (np.dot(A_inv[i], B[i]), [1])
                        else:
                            if abs(temp0[i][1]) > 0.0001 or abs((temp0[i][0])-(temp1[i][0])) > 0.0003 or abs(mm[i] - (temp0[i][0]))>0.001:
                                DT.append((np.dot(A_inv[i], B[i]))[0])
                                DG.append((np.dot(A_inv[i], B[i]))[1])
                                DM.append((np.dot(A_inv[i], B[i]))[2])
                                Dv.append((np.dot(A_inv[i], B[i]))[3])
                                if abs(DG[i]) > 0.5:
                                    DG[i]=0.5*(DG[i]/abs(DG[i]))
                                if abs(DT[i]) > 250:
                                    DT[i] = 250*(DT[i]/abs(DT[i]))
                                if abs(DM[i]) > 0.25:
                                    DM[i] = 0.25*(DM[i]/abs(DM[i]))
                                if abs(Dv[i]) > 0.3:
                                    Dv[i] = 0.3*(Dv[i]/abs(Dv[i]))
                        if t_fixed != 0:
                            DT[i]=0
                        if g_fixed != -1:
                            DG[i]=0
                        if m_fixed != 5:
                            DM[i]=0
                        if v_fixed != -5:
                            Dv[i]=0
                        FIXD_MAT=0
                else:
                    #Trial i vari DDX prima erano tutti moltiplicati per 2 rhs
                    v=[]
                    AA=[]
                    A_numpy=[]
                    A_inv=[]
                    #A=s_Matrix
                    f=[]
                    s=[]
                    m=[]
                    ewr=[]
                    B=[]
                    DT=[]
                    DG=[]
                    DM=[]
                    Dv=[]
                    for i in range(processori):
                        A_numpy.append(crea_dynMatrix(master_search_data[j-1][i][0], master_search_data[j-1][i][1], master_search_data[j][i][0], master_search_data[j][i][1]))
                        DDT[i] = (master_search_data[j-1][i][0][0]-master_search_data[j][i][0][0]) #La differenza tra il valore precedente e quello successivo di Teff
                        DDG[i] = (master_search_data[j-1][i][0][1]-master_search_data[j][i][0][1]) #La differenza tra il valore precedente e quello successivo di logg
                        DDM[i] = (master_search_data[j-1][i][0][2]-master_search_data[j][i][0][2]) #La differenza tra il valore precedente e quello successivo di [M/H]
                        DDV[i] = (master_search_data[j-1][i][0][3]-master_search_data[j][i][0][3])
                        #v.append(sp.symbols('v'))
                        #AA.append(A.subs(v[i], vv[i]))
                        #A_numpy.append(np.array(AA[i].tolist(), dtype=float))
                        A_inv.append(np.linalg.pinv(A_numpy[i]))
                        f.append(-1*((temp0[i][0])-(temp1[i][0])))
                        s.append(-1*temp0[i][1])
                        m.append(-1*(temp0[i][0]-(mm[i])))
                        ewr.append(-1*(temp0[i][3]))
                        B.append(np.array([s[i], f[i], m[i], ewr[i]]))

                        if abs(temp0[i][1]) > 0.0001 or abs((temp0[i][0])-(temp1[i][0])) > 0.0003 or abs(mm[i] - (temp0[i][0]))>0.001:
                            DT.append((np.dot(A_inv[i], B[i]))[0])
                            DG.append((np.dot(A_inv[i], B[i]))[1])
                            DM.append((np.dot(A_inv[i], B[i]))[2])
                            Dv.append((np.dot(A_inv[i], B[i]))[3])
                            if abs(DG[i]) > 0.5:
                                DG[i]=0.5*(DG[i]/abs(DG[i]))
                            if abs(DT[i]) > 250:
                                DT[i] = 250*(DT[i]/abs(DT[i]))
                            if abs(DM[i]) > 0.25:
                                DM[i] = 0.25*(DM[i]/abs(DM[i]))
                            if abs(Dv[i]) > 0.3:
                                Dv[i] = 0.3*(Dv[i]/abs(Dv[i]))
                        if t_fixed != 0:
                            DT[i]=0
                        if g_fixed != -1:
                            DG[i]=0
                        if m_fixed != 5:
                            DM[i]=0
                        if v_fixed != -5:
                            Dv[i]=0

                TEMPERATU=[]
                GRAV=[]
                VELTURB=[]
                VELTURB_t=[]
                search_data = []
                for i in range(processori):
                    TEMPERATU.append(tt[i]+DT[i])
                    GRAV.append(gg[i]+DG[i])
                    VELTURB.append(vv[i]+Dv[i])
                    VELTURB_t.append(vturbo(TEMPERATU[i], GRAV[i], mm[i]+DM[i]))
                    if fixd_velturb == True:
                        if VELTURB[i]>1.1*VELTURB_t[i]:
                            VELTURB[i]=1.1*VELTURB_t[i]
                        elif VELTURB[i]<0.9*VELTURB_t[i]:
                            VELTURB[i]=0.9*VELTURB_t[i]
                    search_data.append([TEMPERATU[i], GRAV[i], mm[i]+DM[i], VELTURB[i]])
                    old_best_index[i]=best_index[i]
                    old_best[i]=best[i]
        #riattiva_stampe()
        #riattiva_stampe()
        O=[]
        for i in range(processori):
            self.nome = f"{i}" + nome_temp
            O.append(self.evaluate_Models(tt[i], gg[i], mm[i], vv[i], self.folder+'/TEMP', NLTE=NLTE, t_fixed=t_fixed, g_fixed=g_fixed, m_fixed=m_fixed, v_fixed=v_fixed))

        self.nome = nome_temp
        vettori_selezionati = []
        for quaterna in self.ALL_STARS: #crea returner
            if len(quaterna) == processori:
                # Trova il vettore con il primo elemento più piccolo nella quaterna
                minimo_quaterna = min(quaterna, key=lambda x: x[0])
                vettori_selezionati.append(minimo_quaterna)

        # Trova il vettore con il primo elemento più piccolo tra i selezionati
        vettore_minimo = min(vettori_selezionati, key=lambda x: x[0])
        if self.param_error > vettore_minimo[0]:
            self.param_error = vettore_minimo[0]
            self.t_eff = vettore_minimo[1]
            self.logg = vettore_minimo[2]
            self.metal = vettore_minimo[3]
            self.v_m = vettore_minimo[4]
        copia(f'T{int(self.t_eff)}_G{(float(self.logg)):.2f}_M{(float(self.metal)):.2f}_{(float(self.v_m)):.2f}.model', path_to_atmosphere+interpol_folder, self.folder, self.nome+f'_T{self.t_eff}_G{self.logg}_M{self.metal}_V{self.v_m}')
        self.model = self.folder + self.nome+f'_T{self.t_eff}_G{self.logg}_M{self.metal}_V{self.v_m}'
        self.stampaStella(self.nome)
        print()
        #sposta()
    def dopcor(self):
        '''Corregge il doppler shift del file fits'''
        # if self.v_r == -20000 and self.instrument=='FIES':
        #     self.stima_radvel_Ha()
        #     print("Prima stima da fit gaussiano sulla riga H-alpha...")
        #     return 0
        
        self.xi= corrDoppler(self.xi, self.v_r)
        self.dxi= corrDoppler(self.dxi, self.v_r)
        self.xf=self.xi+self.dxi*(self.fits[0].header.get("NAXIS1")-1)
        self.v_r=0
        print("Prima correzione doppler...")
        # if self.instrument=='FIES':
        #     self.stima_radvel_Ca()
        #     print("Velocità radiale stimata da fit gaussiano sulla riga 8542 del tripletto del calcio!")
        #     self.xi= corrDoppler(self.xi, self.v_r)
        #     self.dxi= corrDoppler(self.dxi, self.v_r)
        #     self.xf=self.xi+self.dxi*(self.fits[0].header.get("NAXIS1")-1)
        #     self.v_r=0
        # print("Correzione doppler!")
    def spectrum(self, showplot=True, FITS=True):
        '''Carica i dati spettrali da file .fits. Restituisce una lista contenente due array, uno delle X con le wl, uno delle Y con il flusso'''
        Xi=self.xi
        Xf=self.xf
        os.chdir(self.fits_folder)
        file_fits = self.fits_name
        hdul = fits.open(file_fits)

        # Stampa informazioni sul file FITS
        hdul.info()

        print(f"{len(hdul[0].data)} {type(hdul[0].data)}")

        Y = hdul[0].data
        X = np.linspace(Xi, Xf, len(Y))
        if showplot:
            plt.figure(figsize=(10, 6))
            plt.plot(X, Y, label="Spettro HARPS-N")
            plt.xlabel("Lunghezza d'onda (Å)")
            plt.ylabel("Flusso")
            plt.title("Spettro Stellare - HARPS-N")
            plt.legend()
            plt.grid()
            plt.show()
        self.xax = X
        self.ydata = Y
        return [X, Y]
    def synth(self, T=-10, G=-10, M=-10, Vturb=-10, TURBVEL=2, cpu=-1, startwl=4700, endwl=5500, vsini=0, darklimb=0.6, Step=0.01, computeModel=0, output_folder='', path_to_model='', name_model='', radSolver='spectrum', limit_width=10, H=0.92150, He=0.07840, Li=-10.94, Be=-10.64, B=-9.49, C=-3.52, N=-4.12, O=-3.21, F=-7.48, Ne=-3.96, Na=-5.71, Mg=-4.46, Al=-5.57, Si=-4.49, P=-6.59, S=-4.71, Cl=-6.54, Ar=-5.64, K=-6.92, Ca=-5.68, Sc=-8.87, Ti=-7.02, V=-8.04, Cr=-6.37, Mn=-6.65, Fe=-4.54, Co=-7.12, Ni=-5.79, Cu=-7.83, Zn=-7.44, Ga=-9.16, Ge=-8.63, As=-9.67, Se=-8.63, Br=-9.41, Kr=-8.73, Rb=-9.44, Sr=-9.07, Y=-9.80, Zr=-9.44, Nb=-10.62, Mo=-10.12, Tc=-20.00, Ru=-10.20, Rh=-10.92, Pd=-10.35, Ag=-11.10, Cd=-10.27, In=-10.38, Sn=-10.04, Sb=-11.04, Te=-9.80, Io=-10.53, Xe=-9.87, Cs=-10.91, Ba=-9.91, La=-10.87, Ce=-10.46, Pr=-11.33, Nd=-10.54, Pm=-20.00, Sm=-11.03, Eu=-11.53, Gd=-10.92, Tb=-11.69, Dy=-10.90, Ho=-11.78, Er=-11.11, Tm=-12.04, Yb=-10.96, Lu=-11.98, Hf=-11.16, Ta=-12.17, W=-10.93, Re=-11.76, Os=-10.59, Ir=-10.69, Pt=-10.24, Au=-11.03, Hg=-10.91, Tl=-11.14, Pb=-10.09, Bi=-11.33, Po=-20.00, At=-20.00, Rn=-20.00,  Fr=-20.00, Ra=-20.00, Ac=-20.00, Th=-11.95, Pa=-20.00, U=-12.54):
        '''Crea uno spettro sintetico. Specifica le caratteristiche del modello atosferico, intervallo da sintetizzare, correzioni vsini. \n
        Per utilizzare un modello già esistente specifica SIA il path che il nome del modello in path_to_model e name_model, e.g. path_to_model = 'home/user/models/', name_model = 'model'.\n
        E' possibile specificare l'abbondanza chimica di ogni elemento tramite la sigla dell'elemento, e.g. Na=5.00 per impostare l'abbondanza di sodio a 5. Si tenga presente che il modo in cui spectrum tratta le abbondanze dei metalli è tale che per cui queste vengono scalate secondo il valore di metallicità dell'atmosfera. Questo significa che se si specifica un valore di sodio di Na=5 e si utilizza un modello atmosferico con [M/H]=-1 il valore effettivo di sodio sarà di Na=4. I valori di default sono quelli utilizzati da pykmod nella creazione delle atmosfere (ATLAS9).\n
        T: Temperatura dell'atmosfera in Kelvin.\n
        G: Logaritmo della gravità dell'atmosfera.\n
        M: Metallicità dell'atmosfera.\n
        Vturb: Velocità di turbolenza dell'atmosfera.\n
        cpu: Specifica il numero di processore da utilizzare per la creazione del modello atmosferico. Se cpu=-1 viene utilizzato un solo processore.\n
        startwl: Lunghezza d'onda di inizio dell'intervallo da sintetizzare.\n
        endwl: Lunghezza d'onda di fine dell'intervallo da sintetizzare.\n
        vsini: Velocità di rotazione del profilo di velocità.\n
        darklimb: Limite scuro del profilo di velocità.\n
        Step: Passo di sintesi dello spettro.\n
        computeModel: Se 0 viene creato un nuovo modello atmosferico, se 1 viene utilizzato un modello già esistente.\n
        output_folder: Specifica la cartella di output per il file sintetico.\n
        path_to_model: Specifica il path del modello atmosferico da utilizzare.\n
        name_model: Specifica il nome del modello atmosferico da utilizzare.\n
        H: Abbondanza di idrogeno.\n
        He: Abbondanza di elio.\n
        Li: Abbondanza di litio.\n
        ... etc ...\n'''
        if T==-10:
            T=self.t_eff
        if G==-10:
            G=self.logg
        if M ==-10:
            M=self.metal
        if Vturb == -10:
            Vturb=self.v_m
        print(computeModel)
        if computeModel == 0:
            print('Creando modello atmosferico..')
            self.synth_creaModel(T, G, M, Vturb, TURBVEL=TURBVEL, outpath=path_to_spectrum, processore=cpu, Step=Step, radSolver=radSolver, startwl=startwl, endwl=endwl, path_to_model='', name_model='', limit_width=limit_width, H=H, He=He, Li=Li, Be=Be, B=B, C=C, N=N, O=O, F=F, Ne=Ne, Na=Na, Mg=Mg, Al=Al, Si=Si, P=P, S=S, Cl=Cl, Ar=Ar, K=K, Ca=Ca, Sc=Sc, Ti=Ti, V=V, Cr=Cr, Mn=Mn, Fe=Fe, Co=Co, Ni=Ni, Cu=Cu, Zn=Zn, Ga=Ga, Ge=Ge, As=As, Se=Se, Br=Br, Kr=Kr, Rb=Rb, Sr=Sr, Y=Y, Zr=Zr, Nb=Nb, Mo=Mo, Tc=Tc, Ru=Ru, Rh=Rh, Pd=Pd, Ag=Ag, Cd=Cd, In=In, Sn=Sn, Sb=Sb, Te=Te, Io=Io, Xe=Xe, Cs=Cs, Ba=Ba, La=La, Ce=Ce, Pr=Pr, Nd=Nd, Pm=Pm, Sm=Sm, Eu=Eu, Gd=Gd, Tb=Tb, Dy=Dy, Ho=Ho, Er=Er, Tm=Tm, Yb=Yb, Lu=Lu, Hf=Hf, Ta=Ta, W=W, Re=Re, Os=Os, Ir=Ir, Pt=Pt, Au=Au, Hg=Hg, Tl=Tl, Pb=Pb, Bi=Bi, Po=Po, At=At, Rn=Rn,  Fr=Fr, Ra=Ra, Ac=Ac, Th=Th, Pa=Pa, U=U)
        else:
            if radSolver=='spectrum':
                if cpu==-1:
                    creaParamfileSpectrum(Vturb, f"Model", self.nome, startwl, endwl, Step, numero_spectrum=cpu)
                    if path_to_model != '' and name_model != '':
                        copia(name_model, path_to_model, path_to_spectrum, 'Model')
                else:
                    creaParamfileSpectrum(Vturb, f"Model", self.nome, startwl, endwl, Step, numero_spectrum=cpu)
                    if path_to_model != '' and name_model != '':
                        copia(name_model, path_to_model, path_to_spectrum, f'Model{cpu}')
            elif radSolver=='turbo':
                crea_synthScriptTurbo(path_to_model=path_to_model, modelName=name_model, wl_min=startwl, wl_max=endwl, delta_wl=Step, TURBVEL = TURBVEL, limit_width=limit_width)
        if output_folder=='':
            output_folder = self.folder
        else:
            output_folder = self.folder + output_folder
        if radSolver == 'spectrum':
            os.chdir(path_to_spectrum)
            spct_path = "./spectrum"
            process1 = subprocess.Popen([spct_path, '-t'])
            try:
                process1.wait()
            except AttributeError:
                print('Keep going...')
            try:
                process1.terminate()
            except AttributeError:
                print('Keep going...')
            copia(self.nome + '.synstar', path_to_spectrum, output_folder, self.nome + f'{startwl}{endwl}.synstar')
            os.chdir(path_to_spectrum)
            spct_path = "./avsini"
            args = [spct_path, f'{self.nome}.synstar', f'{self.nome}broad.synstar', f"{vsini}", f"{darklimb}", f"{Step}"]
            process1 = subprocess.run(args)
            try:
                process1.wait()
            except AttributeError:
                print('Keep going...')
            try:
                process1.terminate()
            except AttributeError:
                print('Keep going...')
            copia(self.nome + 'broad.synstar', path_to_spectrum, output_folder, self.nome + f'{startwl}{endwl}broad.synstar')    
            self.synstar_name = self.nome + f'{startwl}{endwl}.synstar'
            self.synstar_name_avsini = self.nome + f'{startwl}{endwl}broad.synstar'
            return [self.synstar_name, self.synstar_name_avsini]
        elif radSolver == 'turbo':
            os.chdir(path_to_turbospectrumNLTE)
            cmd='source synth_script'
            subprocess.run(['csh', '-c', cmd])
            self.synstar_name = f'{T}_{G}_{M}_{Vturb}.synstar'
            copia('outsynth/syntspec/star', path_to_turbospectrumNLTE, self.folder, f'{T}_{G}_{M}_{Vturb}.synstar')
            return [self.synstar_name]
    def avsini(self, path='', file='', vsini=0, darklimb=0.6, Step=0.01):
        '''Applica la correzione vsini utilizzando avsini (spectrum)'''
        if path == '':
            path = self.folder
        if file == '':
            file = self.nome+'.synstar'
        os.chdir(path_to_spectrum)
        spct_path = "./avsini"
        copia(file, path, path_to_spectrum, file)
        args = [spct_path, file, f'{self.nome}broad.synstar', f"{vsini}", f"{darklimb}", f"{Step}"]
        process1 = subprocess.run(args)
        try:
            process1.wait()
        except AttributeError:
            print('Keep going...')
        try:
            process1.terminate()
        except AttributeError:
            print('Keep going...')
        copia(self.nome + 'broad.synstar', path_to_spectrum, self.folder, self.nome + 'broad.synstar')
        os.chdir(self.folder)
    def crea_log(self, FOLDER='', NLTE = 0):
        '''Crea una file di testo log.txt contenente i principali risultati dell'analisi atmosferica.
        folderLog2 stampa anche le quantità necessarie alla determinazione della microturbolenza. 
        Restituisce una lista con i parametri delle atmosfere vincenti.\n
        mode (1, 2 o 3) varia a seconda del tipo di parsing del nome dei file che vogliamo analizzare.
        1 per i file che iniziano con 'STELLA', 2 per quelli che iniziano con 'AH_STELLA'. \n mode=3 è per trattare un nome qualsiasi, in questo caso si DEVE specificare anche il nome tramite la variabile 'baseNome', eg. "HARPN1.fits_INT <--- per ora non implementato, solo per uso manuale (cioè modifica la funzione sparseFile3 se vuoi cambiare)'''
        if FOLDER=='':
            path = self.folder + "/TEMP"
        else:
            path = FOLDER
        old_dir = os.getcwd()
        os.chdir(path)
        a = os.listdir()
        b = []
        returner = np.zeros(9)
        baseNome = str(self.nome)+"_"
        for i in range(len(a)):
            if a[i] != "log.txt" and a[i] != "exit" and a[i] != 'stellaFes':
                b.append(a[i]) #in b i nomi delle stelle
        H = np.zeros((len(b), 9)) #H[0]=slope elemento1, H[1]=abbondanza elemento1, H[2]=abbondanza elemento2, H[3]=temperatura, H[4]=logg, H[5]=metallicità come letta sul file, H[6]=std(FeI), H[7]=std(FeII), H[8]=slope(EWR-A(Fe))
        min = 10
        j=0
        with open("log.txt", 'w') as f:
            f.write("    T      G      M    vturb    FeI        FeII           FeI-M/H        FeI-FeII       s(FeI)    EWR\n")
            for i in range(len(b)):
                print(os.getcwd())
                H[i][3] = (sparseFileName4(path, b[i], name_suffix=baseNome))[1]
                H[i][4] = (sparseFileName4(path, b[i], name_suffix=baseNome))[2]
                H[i][5] = (sparseFileName4(path, b[i], name_suffix=baseNome))[0]
                vturbos = (sparseFileName4(path, b[i], name_suffix=baseNome))[3]
                if NLTE == 0:
                    tempD = simpleAnalysisSingle(path, b[i], 26.1)
                    tempS = simpleAnalysisSingle(path, b[i], 26.0)
                elif NLTE == 1:
                    tempD = NLTE_AnalysisSingle(path, b[i], H[i][3],  H[i][4],  H[i][5], vturbos, 26.1)
                    tempS = NLTE_AnalysisSingle(path, b[i], H[i][3],  H[i][4],  H[i][5], vturbos, 26.0)
                H[i][0] = tempS[1]
                H[i][1] = float(tempS[0])
                H[i][2] = float(tempD[0])
                H[i][6] = tempS[2]
                H[i][7] = tempD[2]
                f.write(f"{H[i][3]:.0f}   {H[i][4]:.2f}   {H[i][5]:.2f}   {vturbos:.2f}    {H[i][1]:.3}+-{H[i][6]:.3f}   {H[i][2]:.3}+-{H[i][7]:.3f}   {H[i][1]-H[i][5]:.4f}         {H[i][1]-H[i][2]:.3f}          {H[i][0]:.4f}        {tempS[3]:.5f}\n")
        winners = []
        for i in range(len(b)): #erano 0.6                            0.21                    0.11
            if (abs(H[i][1]-H[i][5]) < 0.17) and (abs(H[i][1]-H[i][2])<0.01) and (abs(H[i][0])<0.005):
                winners.append([H[i][3], H[i][4], H[i][6], 100])
        ds=0.002
        dg=0.04
        dm=0.08
        q=1
        for u in range(100):
            for i in range(len(b)):
                if len(winners) < 4:
                    if (abs(H[i][1]-H[i][5])<0.1+dm) and (abs(H[i][1]-H[i][2])<0.01+dg) and (abs(H[i][0])<0.005+ds):
                        if 100-5*q <= -700:
                            winners.append([H[i][3], H[i][4], H[i][5], 10/q])
                        else:
                            winners.append([H[i][3], H[i][4], H[i][5], 100/q])#-5*(q)])
                q=q+1
                ds=ds+0.001
                dg=dg+0.005
                dm=dm+0.01
        os.chdir(old_dir)
        return winners
    def evaluate_Models(self, t=-1, g=10, M=10, V=-1, folder=' ', eigen_weights=None, NLTE = 1, nom_file='', adv_matrix=s_Matrix, dyn_on=False, t_fixed=0, g_fixed=-1, m_fixed=5, v_fixed=-5, radSolver='spectrum'):
        '''Restituisce il valore che stima la qualità dell'atmosfera. I valori fixed non fanno parte della valutazione. Se hanno i valori default non vengono considerati. \n
        NLTE = 0 per atmosfere LTE, NLTE = 1 per atmosfere NLTE. \n
        Se nom_file è specificato, il programma cerca il file di log con quel nome. \n
        Se folder è specificato, il programma cerca il file di log in quella cartella. \n'''
        if folder == ' ':
            folder = self.folder + "/TEMP"
        if M == 10:
            M = self.metal
        if dyn_on:
            A_numpy = adv_matrix
        else:
            A = adv_matrix
            v=sp.symbols('v')
            AA=A.subs(v, V)
            A_numpy = np.array(AA.tolist(), dtype=float)
        #print(A_numpy)
        try:
            A_inv = np.linalg.pinv(A_numpy)
        except np.linalg.LinAlgError:
            print(f"\n\nSingular matrix, {A_inv}\n\n")
        if nom_file == '':
            if NLTE == 0 or M >= 0.0:
                DATI = simpleAnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}", radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}", 26.1, radSolver=radSolver)
            elif NLTE == 1 and M<0:
                DATI = NLTE_AnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}", t, g, M, V)
                DATI2 = NLTE_AnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}", t, g, M, V, 26.1)
            elif NLTE == 1 and M>0:
                DATI = simpleAnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}", radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(folder, self.nome+f"_{M:.2f}_{t:.0f}_{g:.2f}_{V:.2f}", 26.1, radSolver=radSolver)
        else:
            if NLTE == 0 or M >= 0.0:
                DATI = simpleAnalysisSingle(folder, nom_file, radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(folder, nom_file, 26.1, radSolver=radSolver)
            elif NLTE == 1 and M<0:
                DATI = NLTE_AnalysisSingle(folder, nom_file, t, g, M, V)
                DATI2 = NLTE_AnalysisSingle(folder, nom_file, t, g, M, V, 26.1)
            elif NLTE == 1 and M>0:
                DATI = simpleAnalysisSingle(folder, nom_file, radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(folder, nom_file, 26.1, radSolver=radSolver)
        s = -1*DATI[1]
        f=-0.0
        m = -0.0
        ewr = -0.0
        A1 = np.array([s, f, m, ewr])
        s = 0.00
        f=-1*(DATI[0]-DATI2[0])
        m = -0.0
        ewr = -0.0
        A2 = np.array([s, f, m, ewr])
        s = 0.00
        f=-0.0
        m = -1*(DATI[0]-M)
        ewr = -0.0
        A3 = np.array([s, f, m, ewr])
        s = 0.0
        f=-0.0
        m = -0.0
        ewr = -1*DATI[3]
        A4 = np.array([s, f, m, ewr])
        if eigen_weights==None:
            eigen_weights = [30, 0.1, 0.082, 0.02]
        if True:
            DT1 = abs((np.dot(A_inv, A1))[0])
            DG1 = abs((np.dot(A_inv, A1))[1])
            DM1 = abs((np.dot(A_inv, A1))[2])
            Dv1 = abs((np.dot(A_inv, A1))[3])
            DT2 = abs((np.dot(A_inv, A2))[0])
            DG2 = abs((np.dot(A_inv, A2))[1])
            DM2 = abs((np.dot(A_inv, A2))[2])
            Dv2 = abs((np.dot(A_inv, A2))[3])
            DT3 = abs((np.dot(A_inv, A3))[0])
            DG3 = abs((np.dot(A_inv, A3))[1])
            DM3 = abs((np.dot(A_inv, A3))[2])
            Dv3 = abs((np.dot(A_inv, A3))[3])
            DT4 = abs((np.dot(A_inv, A4))[0])
            DG4 = abs((np.dot(A_inv, A4))[1])
            DM4 = abs((np.dot(A_inv, A4))[2])
            Dv4 = abs((np.dot(A_inv, A4))[3])
            # print(f"DT1: {DT1}\nDG1: {DG1}\nDM1: {DM1}\nDV1: {Dv1}\n\nDT2: {DT2}\nDG2: {DG2}\nDM2: {DM2}\nDV2: {Dv2}\n\nDT3: {DT3}\nDG3: {DG3}\nDM3: {DM3}\nDV3: {Dv3}\n\nDT4: {DT4}\nDG4: {DG4}\nDM4: {DM4}\nDV4: {Dv4}")
            DT=(DT1+DT2+DT3+DT4)/(eigen_weights[0])
            DG=(DG1+DG2+DG3+DG4)/(eigen_weights[1])
            DM=(DM1+DM2+DM3+DM4)/(eigen_weights[2])
            Dv=(Dv1+Dv2+Dv3+Dv4)/(eigen_weights[3])
        #print(f"DT: {DT}\nDG: {DG}\nDM: {DM}\nDV: {Dv}")
        # print(f"DT: {DT}\nDG: {DG}\nDM: {DM}\nDV: {Dv}")
        if t_fixed != 0:
            DT = 0
            DG = DG * 4/3
            DM = DM * 4/3
        if g_fixed != -1:
            DG = 0
            DT = DT * 4/3
            DM = DM * 4/3
        if m_fixed != 5:
            DM = 0
            DG = DG * 4/3
            DT = DT * 4/3
        if v_fixed != 5:
            DM = 0
            DG = DG * 4/3
            DT = DT * 4/3
        # print(DT+DG+DM+Dv)
        if DT+DG+DM+Dv>2:
            return DT+DG+DM+Dv
        else:
            return DATI[2]
    def stampaStella(self, eh, NLTE=False, radSolver='spectrum'):
        '''Crea il file .star contenente le informazioni della stella che stai analizzando.'''
        if NLTE:
            try:
                DATI = NLTE_AnalysisSingle(self.folder + "/TEMP", "1"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2), rad)
                DATI2 = NLTE_AnalysisSingle(self.folder + "/TEMP", "1"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2), 26.1)
            except FileNotFoundError:
                aaa=True
            try:
                DATI = NLTE_AnalysisSingle(self.folder + "/TEMP", "2"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2))
                DATI2 = NLTE_AnalysisSingle(self.folder + "/TEMP", "2"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2), 26.1)
            except FileNotFoundError:
                aaa=True
            try:
                DATI = NLTE_AnalysisSingle(self.folder + "/TEMP", "3"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2))
                DATI2 = NLTE_AnalysisSingle(self.folder + "/TEMP", "3"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2), 26.1)
            except FileNotFoundError:
                aaa=True
            try:
                DATI = NLTE_AnalysisSingle(self.folder + "/TEMP", "4"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2))
                DATI2 = NLTE_AnalysisSingle(self.folder + "/TEMP", "4"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2), 26.1)
            except FileNotFoundError:
                aaa=True
        else:
            try: 
                DATI = simpleAnalysisSingle(self.folder + "/TEMP", "1"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(self.folder + "/TEMP", "1"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", 26.1, radSolver=radSolver)
            except FileNotFoundError:
                print(f"File 1{self.nome}_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f} non trovato")
                aaa=True
            try:
                DATI = simpleAnalysisSingle(self.folder + "/TEMP", "2"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(self.folder + "/TEMP", "2"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", 26.1, radSolver=radSolver)
            except FileNotFoundError:
                print(f"File 2{self.nome}_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f} non trovato")
                aaa=True
            try:
                DATI = simpleAnalysisSingle(self.folder + "/TEMP", "3"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(self.folder + "/TEMP", "3"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", 26.1, radSolver=radSolver)
                print("Beccato!")
            except FileNotFoundError:
                print(f"File 3{self.nome}_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f} non trovato")
                aaa=True
            try:
                DATI = simpleAnalysisSingle(self.folder + "/TEMP", "4"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", radSolver=radSolver)
                DATI2 = simpleAnalysisSingle(self.folder + "/TEMP", "4"+self.nome+f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", radSolver=radSolver)
            except FileNotFoundError:
                print(f"File 4{self.nome}_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f} non trovato")
                aaa=True

        s = -1*DATI[1]
        f=float(-1*(DATI[0]-DATI2[0]))
        m = float(-1*(DATI[0]-self.metal))
        m_error = DATI[2]
        self.m_error = m_error
        ewr = -1*DATI[3]
        os.chdir(self.folder)
        with open(self.folder + f"{eh}.star", 'w') as file:
            file.write(f"Nome: {self.nome}\n")
            file.write(f"T_eff: {self.t_eff}K\n")
            file.write(f"log(g): {self.logg}dex\n")
            file.write(f"[Fe/H]: {self.metal} +- {self.m_error}dex\n")
            file.write(f"v_microturbulence: {self.v_m} km/s\n")
            file.write(f"Error: {self.param_error}\n")
            file.write(f"s: {s}\n")
            file.write(f"F: {f}\n")
            file.write(f"Y: {m}\n")
            file.write(f"S: {ewr}\n")
            file.write("\n \n")
            file.write("ALL STARS: \n")
            for i in range(len(self.ALL_STARS)):
                file.write(f"{self.ALL_STARS[i]}\n")
        os.chdir(self.folder+'/TEMP')
        # tab=Table.read(self.abndfile, format='ascii')
        # dati=tab['col6']
        # crea_istogramma(dati, num_bin=len(dati)/10, showPlot=False, save=True)

    def stampaStella1(self, eh, NLTE=False):
        '''Crea il file .star contenente le informazioni della stella che stai analizzando.'''
        DATI = None
        DATI2 = None
        aaa = False

        prefixes = ["1", "2", "3", "4"]
        for prefix in prefixes:
            nome_file=prefix + self.nome + f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}"
            print(nome_file)
            print(self.folder + "/TEMP")
            try:
                if NLTE==False:
                    DATI = simpleAnalysisSingle(self.folder + "/TEMP", prefix + self.nome + f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}")
                    DATI2 = simpleAnalysisSingle(self.folder + "/TEMP", prefix + self.nome + f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", 26.1)
                    nome_file=prefix + self.nome + f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}"
                    print(nome_file)
                else:
                    DATI = NLTE_AnalysisSingle(self.folder + "/TEMP", prefix + self.nome + f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2))
                    DATI2 = NLTE_AnalysisSingle(self.folder + "/TEMP", prefix + self.nome + f"_{self.metal:.2f}_{self.t_eff:.0f}_{self.logg:.2f}_{self.v_m:.2f}", round(self.t_eff, 0), round(self.logg, 2), round(self.metal, 2), round(self.v_m, 2), 26.1)
                print(f"File trovato con prefisso {prefix}!")
                break  # Esci dal ciclo se i file sono trovati
            except FileNotFoundError:
                print(f"File con prefisso {prefix} non trovato")
                aaa = True

        s = -1 * DATI[1]
        f = float(-1 * (DATI[0] - DATI2[0]))
        m = float(-1 * (DATI[0] - self.metal))
        ewr = -1 * DATI[3]
        with open(self.folder + f"{eh}.star", 'w') as file:
            file.write(f"Nome: {self.nome}\n")
            file.write(f"T_eff: {self.t_eff}K\n")
            file.write(f"log(g): {self.logg}dex\n")
            file.write(f"[Fe/H]: {self.metal}dex\n")
            file.write(f"v_microturbulence: {self.v_m} km/s\n")
            file.write(f"Error: {self.param_error}\n")
            file.write(f"s: {s}\n")
            file.write(f"F: {f}\n")
            file.write(f"Y: {m}\n")
            file.write(f"S: {ewr}\n")
            file.write("\n \n")
            file.write("ALL STARS: \n")
            for i in range(len(self.ALL_STARS)):
                file.write(f"{self.ALL_STARS[i]}\n")
    def radvel_Ha(self, Ha=0):
        if Ha == 0:
            print("Inserire la lunghezza d'onda della riga Halpha!")
            return 0   
        else:
            self.v_r = (Ha - 6562.81)/6562.81*LIGHTSPEED
    def radvel_Ca(self, Ca=0):
        if Ca == 0:
            print("Inserire la lunghezza d'onda della riga 8542 dela calcio!")
            return 0
        else:
            self.v_r = (Ca - 8542.089)/8542.089*LIGHTSPEED
    def normalize_spectrum(self, X, Y, N_avepoint=800, window_size=15, cosmic_cut=550):
        '''Normalizza uno spettro, fornito specificando l'array delle wl e l'array del flusso (NB devono essere della stessa dimensione). \n\nIl modo in cui viene normalizzato prevede diverse operazioni di filtraggio dove, considerando una finestra si calcola da prima la media del flusso, successivamente si impostano le wl a cui corrisponde un flusso inferiore alla media il valore della media stessa. Si itera il processo. In questo modo si rimuovono le righe di assorbimento.\n Dopo di che si escludono con dei tagli i raggi cosmici.\n
          Si opera dunque una media mobile sui restanti punti.'''
        Xfinaltemp=np.zeros(len(X)%N_avepoint)
        Yfinaltemp=np.zeros(len(X)%N_avepoint)
        x1max=np.zeros(int((len(X)-len(X)%N_avepoint)/N_avepoint+1))
        y1max=np.zeros(int((len(X)-len(X)%N_avepoint)/N_avepoint+1))
        x2max=np.zeros(int((len(X)-len(X)%N_avepoint)/N_avepoint+1))
        y2max=np.zeros(int((len(X)-len(X)%N_avepoint)/N_avepoint+1))
        for j in range(int((len(X)-len(X)%N_avepoint)/N_avepoint)-1):
            X1temp=np.zeros(N_avepoint)
            Y1temp=np.zeros(N_avepoint)
            X2temp=np.zeros(N_avepoint)
            Y2temp=np.zeros(N_avepoint)
            for i in range(N_avepoint):
                X1temp[i]=X[i+j*N_avepoint]
                Y1temp[i]=Y[i+j*N_avepoint]
                X2temp[i]=X[i+N_avepoint+j*N_avepoint]
                Y2temp[i]=Y[i+N_avepoint+j*N_avepoint]
            Y1ave = np.mean(Y1temp)
            Y2ave = np.mean(Y2temp)
            for lowcut in range(70):
                for i in range(N_avepoint):
                    if Y1temp[i] < Y1ave:
                        Y1temp[i]=Y1ave
                    if Y2temp[i] < Y2ave:
                        Y2temp[i]=Y2ave
                Y1ave = np.mean(Y1temp)
                Y2ave = np.mean(Y2temp)
            lim1 = ((Y1temp ) < Y1ave+ cosmic_cut)
            lim2 = ((Y2temp ) < Y2ave+ cosmic_cut)
            X1temp = X1temp[lim1]
            Y1temp = Y1temp[lim1]
            X2temp = X2temp[lim2]
            Y2temp = Y2temp[lim2]
            x1max[j] = X1temp[np.argmin(Y1temp)]
            y1max[j] = np.min(Y1temp)
            x2max[j] = X2temp[np.argmin(Y2temp)]
            y2max[j] = np.min(Y2temp)
            # print(f"{Y1ave}    aaoo {Y2ave}")
            # print(f'x: {x1max[j]} , y: {y1max[j]}\n')
            # print(f'x: {x2max[j]} , y: {y2max[j]}\n')
        interpol_function = moving_average(x1max, y1max, window_size=window_size)
        y_normalized = np.zeros(len(X))
        self.normalized_spectrum = np.zeros(len(X))
        for i in range(len(X)):
            y_interpol = interpol_function(X[i])
            self.normalized_spectrum[i] = Y[i]/y_interpol+0.02
            # for i in range(N_avepoint):
            #     if i != 0:
            #         if y1max[j] < Y1temp[i]:# and (Y1temp[i]-750<y1max[j]) and abs(Y1temp[i]-Y1ave+200)<350:
            #             y1max[j] = Y1temp[i]
            #             x1max[j] = X1temp[i]
            #         else:
            #             y1max[j] = Y1ave+200
            #         if y2max[j] < Y2temp[i]:# and (Y2temp[i]-750<y2max[j]) and abs(Y2temp[i]-Y2ave+200)<350:
            #             y2max[j] = Y2temp[i]
            #             x2max[j] = X2temp[i]
            #         else:
            #             y2max[j] = Y2ave+200
            #     else:
            #         y1max[j] = Y1temp[i]
            #         x1max[j] = X1temp[i]
            #         y2max[j] = Y2temp[i]
            #         x2max[j] = X2temp[i]
        return [x1max, y1max, x2max, y2max]
    def linelist(self):
        '''Crea self.lineList, una lista contenente in [0] tutte le righe lette dalla lineList "stellaFes" e il rispettivo codice della specie. \n\nRestituisce la lista di tutte le righe contenute nella linelist "stellaFes"'''
        preparaAbundance(self.fits_name)
        tab = Table.read('stellaFes', format='ascii')
        righe = tab['col1']
        species = tab['col2']
        self.lineList = [righe, species]
        return righe
    def compareLine(self, x1, y1, step, x2, y2, central_wl):
        '''Si fornisca due spettri (x: wl, y: flux), restituisce: \n- lo scarto quadratico medio \n- lo scarto medio\n- la wl della linea considerata. \n\nNB: i due spettri devono avere la stessa lunghezza d'onda.'''
        llim=0.1
        rlim=0.1
        points=int((rlim+llim)/step)+1
        print(points)
        res = 0
        res0 = 0
        for i in range(points):
            Y1 = np.interp(llim+i*step, x1, y1)
            Y2 = np.interp(llim+i*step, x2, y2)
            res0 = res0 + (Y1-Y2)/(points)
            res = res + (Y1-Y2)**2/(points-1)
            print(f"Y1: {Y1}\nY2: {Y2}\nres: {res}")
        return [res, res0, central_wl]
    def caricaSpettro(self, nomeFile):
        '''Carica uno spettro nella lista self.spettroCaricato[[wl0, flux0, nomefile0],[wl1, flux1, nomefile1],...] \nnomeFile: path completo del file (oppure dentro la cartella in cui si è)'''
        tab=Table.read(nomeFile, format='ascii')
        wl=tab['col1']
        flux=tab['col2']
        self.spettroCaricato.append([wl, flux, nomeFile])
        self.loadedxax = wl
        self.loadedydata = flux
        return([wl, flux, nomeFile])
    def computeResidue(self, path_to_data='', spec_file='', prefix='', suffix="broad.synstar"):
        '''Questa funzione computa i residui tramite il modulo .compareLine a partire dai file di testo contenenti i dati spettrali di ogni singola riga. Aggrega tutte le linee'''
        if path_to_data=='':
            path_to_data=self.folder
        if spec_file=='':
            print("Specifica un file spettrale!")
        os.chdir(path_to_data)
        wl=self.lineList[0]
        residue=[]
        for i in range(len(star1.lineList[0])):
            print(i)
            A=star1.caricaSpettro(f'{prefix}{wl[i]-2}{wl[i]+2}{suffix}')
            residue.append(star1.compareLine(M[0], star1.normalized_spectrum, 0.01, A[0], A[1], 4817.778))
        star1.residue=residue
        return residue
    def centroHa(self, showplot=False, synthspectra=False):
        if synthspectra:
            x = self.loadedxax[(self.loadedxax >= 6550) & (self.loadedxax <= 6570.7)]
            y = self.loadedydata[(self.loadedxax >= 6550) & (self.loadedxax <= 6570.7)]
        else:
            self.spectrum(showplot=False)
            x_restricted = self.xax[(self.xax >= 6550) & (self.xax <= 6570.7)]
            y_restricted = self.ydata[(self.xax >= 6550) & (self.xax <= 6570.7)]
            M = self.normalize_spectrum(x_restricted, y_restricted)
            x=M[0]
            y=M[1]
            # Trovare il minimo assoluto
        min_index = np.argmin(y_restricted)
        x_min = x_restricted[min_index]
        y_min = y_restricted[min_index]
        # Fit di una gaussiana nei pressi del minimo
        initial_guess = [y_min, x_min, 1.0, y_restricted[0]]  # [A, mu, sigma] iniziali
        try:
            popt, pcov = curve_fit(gaussiana, x_restricted, y_restricted, p0=initial_guess, maxfev=500000)
            A, mu, sigma, offs = popt
            print(f"Parametri della gaussiana: A = {A}, mu = {mu}, sigma = {sigma}")
            # Grafico
            if showplot:
                plt.figure(figsize=(8, 5))
                plt.plot(x, y, label="Dati originali", alpha=0.7)
                plt.plot(x_restricted, y_restricted, label="Intervallo ristretto", color="orange", alpha=0.7)
                plt.plot(x_restricted, gaussiana(x_restricted, *popt), label="Fit Gaussiano", color="red")
                plt.scatter(x_min, y_min, color="black", label="Minimo", zorder=5)
                plt.xlabel("Lunghezza d'onda")
                plt.ylabel("Intensità")
                plt.legend()
                plt.title("Fit Gaussiano attorno al minimo")
                plt.show()
        except RuntimeError as e:
            print(f"Impossibile effettuare il fit: {e}")
        self.Ha_center = mu
        return mu
    def centroCa(self, showplot=False):
        self.spectrum(showplot=False)
        x_restricted = self.xax[(self.xax >= 8537) & (self.xax <= 8547)]
        y_restricted = self.ydata[(self.xax >= 8537) & (self.xax <= 8547)]
        M = self.normalize_spectrum(x_restricted, y_restricted)
        x = M[0]
        y = M[1]
        # Trovare il minimo assoluto
        min_index = np.argmin(y_restricted)
        x_min = x_restricted[min_index]
        y_min = y_restricted[min_index]
        # Fit di una Voigtiana nei pressi del minimo
        initial_guess = [y_min, x_min, 0.10, 0.10, y_restricted[0]]  # [A, mu, sigma, gamma, off] iniziali
        try:
            popt, pcov = curve_fit(voigt, x_restricted, y_restricted, p0=initial_guess, maxfev=500000)
            A, mu, sigma, gamma, offs = popt
            print(f"Parametri della Voigtiana: A = {A}, mu = {mu}, sigma = {sigma}, gamma = {gamma}, offs = {offs}")
            # Grafico
            if showplot:
                plt.figure(figsize=(8, 5))
                plt.plot(x, y, label="Dati originali", alpha=0.7)
                plt.plot(x_restricted, y_restricted, label="Intervallo ristretto", color="orange", alpha=0.7)
                plt.plot(x_restricted, voigt(x_restricted, *popt), label="Fit Voigtiano", color="red")
                plt.scatter(x_min, y_min, color="black", label="Minimo", zorder=5)
                plt.xlabel("Lunghezza d'onda")
                plt.ylabel("Intensità")
                plt.legend()
                plt.title("Fit Voigtiano attorno al minimo")
                plt.show()
        except RuntimeError as e:
            print(f"Impossibile effettuare il fit: {e}")
        self.Ca_center = mu
        return mu
    def stima_radvel_Ha(self):
        vv=self.centroHa()
        self.radvel_Ha(self.Ha_center)
        print(f'v_r: {self.v_r}')
    def stima_radvel_Ca(self):
        vv=self.centroCa()
        self.radvel_Ca(self.Ca_center)
        print(f'v_r: {self.v_r}')
    def fitLine(self, showplot=True, center=6562.8, wl_range=4, func='voigt', param=None, synthspectra=False):
        '''Fitta una funzione Voigtiana o Gaussiana attorno al minimo di una riga. \n\nshowplot: se True mostra il plot del fit. \n\ncenter: lunghezza d'onda centrale della riga. \n\nwl_range: range di lunghezza d'onda attorno al minimo. \n\nfunc: 'voigt' o 'gauss' per selezionare la funzione da fittare. \n\nparam: lista di parametri iniziali per il fit.'''
        if synthspectra:
            x = self.loadedxax[(self.loadedxax >= center - wl_range) & (self.loadedxax <= center + wl_range)]
            y = self.loadedydata[(self.loadedxax >= center - wl_range) & (self.loadedxax <= center + wl_range)]
            x_restricted = self.loadedxax[(self.loadedxax >= center - wl_range) & (self.loadedxax <= center + wl_range)]
            y_restricted = self.loadedydata[(self.loadedxax >= center - wl_range) & (self.loadedxax <= center + wl_range)]
        else:
            self.spectrum(showplot=False)
            x_restricted = self.xax[(self.xax >= center - wl_range) & (self.xax <= center + wl_range)]
            y_restricted = self.ydata[(self.xax >= center - wl_range) & (self.xax <= center + wl_range)]
            M = self.normalize_spectrum(x_restricted, y_restricted)
            x = M[0]
            y = M[1]
        # Trovare il minimo assoluto
        min_index = np.argmin(y_restricted)
        x_min = x_restricted[min_index]
        y_min = y_restricted[min_index]
        # Fit di una Voigtiana nei pressi del minimo
        if func == 'voigt':
            if param == None:
                initial_guess = [y_min, x_min, 0.30, 0.0, y_restricted[0]]  # [A, mu, sigma, gamma, off] iniziali
            elif len(param) == 5:
                initial_guess = param
            else:
                print("Parametri non validi")
                return 0
            try:
                popt, pcov = curve_fit(voigt, x_restricted, y_restricted, p0=initial_guess, maxfev=500000)
                A, mu, sigma, gamma, offs = popt
                print(f"Parametri della Voigtiana: A = {A}, mu = {mu}, sigma = {sigma}, gamma = {gamma}, offs = {offs}")
                # Grafico
                if showplot:
                    plt.figure(figsize=(8, 5))
                    plt.plot(x, y, label="Dati originali", alpha=0.7)
                    plt.plot(x_restricted, y_restricted, label="Intervallo ristretto", color="orange", alpha=0.7)
                    plt.plot(x_restricted, voigt(x_restricted, *popt), label="Fit Voigtiano", color="red")
                    plt.scatter(x_min, y_min, color="black", label="Minimo", zorder=5)
                    plt.xlabel("Lunghezza d'onda")
                    plt.ylabel("Intensità")
                    plt.legend()
                    plt.title("Fit Voigtiano attorno al minimo")
                    plt.show()
            except RuntimeError as e:
                print(f"Impossibile effettuare il fit: {e}")
            self.lastline = [A, mu, sigma, gamma, offs]
        elif func == 'gauss':
            if param == None:
                initial_guess = [y_min, x_min, 1.0, y_restricted[0]]  # [A, mu, sigma] iniziali
            elif len(param) == 4:
                initial_guess = param
            else:
                print("Parametri non validi")
                return 0
            try:
                popt, pcov = curve_fit(gaussiana, x_restricted, y_restricted, p0=initial_guess, maxfev=500000)
                A, mu, sigma, offs = popt
                print(f"Parametri della gaussiana: A = {A}, mu = {mu}, sigma = {sigma}")
                # Grafico
                plt.figure(figsize=(8, 5))
                plt.plot(x, y, label="Dati originali", alpha=0.7)
                plt.plot(x_restricted, y_restricted, label="Intervallo ristretto", color="orange", alpha=0.7)
                plt.plot(x_restricted, gaussiana(x_restricted, *popt), label="Fit Gaussiano", color="red")
                plt.scatter(x_min, y_min, color="black", label="Minimo", zorder=5)
                plt.xlabel("Lunghezza d'onda")
                plt.ylabel("Intensità")
                plt.legend()
                plt.title("Fit Gaussiano attorno al minimo")
                plt.show()
            except RuntimeError as e:
                print(f"Impossibile effettuare il fit: {e}")
            self.lastline = [A, mu, sigma, offs]
        print(f"Param fit: {popt}, Cov fit: {pcov}")
        return [popt, pcov]
    def integrateVoigt(self, voigt_params=[0.5, 5014.7, 1, 0.1, 0.5], wl_range=0.0):
        '''Integra una funzione Voigtiana data la lista di parametri [A, mu, sigma, gamma] e il range di lunghezza d'onda.'''
        A, mu, sigma, gamma, off = voigt_params
        if wl_range == 0.0:
            def equation(x):
                return voigt_upright(x, A, mu, sigma, gamma) - 0.01 * A
            x0 = mu + sigma
            x_solution1 = fsolve(equation, x0)
            x0 = mu - sigma
            x_solution2 = fsolve(equation, x0)
            integral = quad(voigt_upright, x_solution1, x_solution2, args=(A, mu, sigma, gamma))
        else:
            integral = quad(voigt_upright, mu-1.5*sigma, mu+1.5*sigma, args=(A, mu, sigma, gamma))
        print(f"Integrale Voigt: {integral}")
        return integral
    def lineEW(self, line, showplot=True, wl_range=0.0):
        '''Calcola l'equivalent width di una riga data la lista di parametri Voigt [A, mu, sigma, gamma] e il range di lunghezza d'onda.'''
        voigt_params=[0.5, line, 1, 0.1, 0.5]
        self.fitLine(showplot=showplot, center=line, wl_range=0.2, func='voigt', param=voigt_params, synthspectra=True)
        integral = self.integrateVoigt(self.lastline, wl_range)
        EW = integral[0]
        return EW*1000
    def display_ModelProfiles(self, model1=''):
        if model1 == '':
            model1 = star1.model
        if model1 == '':
            print('Errore: devi inserire almeno un modello!')
            return 0
        model2=path_to_atmosphere+'MH-2.0/atmoT45001.5.txt'
        two_model=False
        # Lettura modelli
        skip_lines = 23
        num_lines = 72
        def read_Model(filename):
            data = np.loadtxt(filename, skiprows=skip_lines, max_rows=num_lines, 
                            usecols=range(8), dtype=float)
            return data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4], data[:, 5], data[:, 6]
        solo_teff = 'N'
        crea_ps = 'N'  # 'Y'
        fileps = 'test.eps'

        model_AT9_1 = model1
        model_AT9_2 = model2
        # Lettura file

        RHOX_AT9_1, T_AT9_1, P_AT9_1, XNE_AT9_1, ABROSS_AT9_1, ACCRAD_AT9_1, VTURB_AT9_1 = read_Model(model_AT9_1)
        RHOX_AT9_2, T_AT9_2, P_AT9_2, XNE_AT9_2, ABROSS_AT9_2, ACCRAD_AT9_2, VTURB_AT9_2 = read_Model(model_AT9_2)

        # Calcoli di massa e tau
        m_AT9_1 = np.zeros(num_lines)                  #
        m_AT9_1[0] = RHOX_AT9_1[0]                     #
        m_AT9_1[1:] = np.diff(RHOX_AT9_1)              #
        tau_AT9_1 = np.cumsum(m_AT9_1 * ABROSS_AT9_1)  #

        m_AT9_2 = np.zeros(num_lines)
        m_AT9_2[0] = RHOX_AT9_2[0]
        m_AT9_2[1:] = np.diff(RHOX_AT9_2)
        tau_AT9_2 = np.cumsum(m_AT9_2 * ABROSS_AT9_2)

        # Setup grafico
        fig, axes = plt.subplots(3, 2, figsize=(12, 18))
        if model2 != '':
            xmin, xmax = np.log10(min(tau_AT9_1.min(), tau_AT9_2.min())), np.log10(max(tau_AT9_1.max(), tau_AT9_2.max()))
        else:
            xmin, xmax = np.log10(tau_AT9_1.min()), np.log10(tau_AT9_1.max())

        # Funzione per generare i plot
        def plot_graph(ax, x1, y1, x2, y2, xlabel, ylabel, two_model=two_model):
            ax.plot(np.log10(x1), y1, label='Model 1')
            if two_model:
                ax.plot(np.log10(x2), y2, label='Model 2', color='r')
            ax.set_xlabel(xlabel)
            ax.set_ylabel(ylabel)
            ax.legend()

        plot_graph(axes[0, 0], tau_AT9_1, T_AT9_1, tau_AT9_2, T_AT9_2, 'log(Tau_ross)', 'T(K)')
        plot_graph(axes[0, 1], tau_AT9_1, np.log10(P_AT9_1), tau_AT9_2, np.log10(P_AT9_2), 'log(Tau_ross)', 'log(Pgas)')
        plot_graph(axes[1, 0], tau_AT9_1, np.log10(RHOX_AT9_1), tau_AT9_2, np.log10(RHOX_AT9_2), 'log(Tau_ross)', 'log(RHOX)')
        plot_graph(axes[1, 1], tau_AT9_1, np.log10(XNE_AT9_1), tau_AT9_2, np.log10(XNE_AT9_2), 'log(Tau_ross)', 'log(XNE)')
        plot_graph(axes[2, 0], tau_AT9_1, np.log10(ABROSS_AT9_1), tau_AT9_2, np.log10(ABROSS_AT9_2), 'log(Tau_ross)', 'log(ABROSS)')
        plot_graph(axes[2, 1], tau_AT9_1, np.log10(ACCRAD_AT9_1), tau_AT9_2, np.log10(ACCRAD_AT9_2), 'log(Tau_ross)', 'log(ACCRAD)')

        plt.tight_layout()

        # Salvataggio o visualizzazione
        if crea_ps == 'Y':
            plt.savefig(fileps, format='eps')
        else:
            plt.show()
    def compare_ModelProfiles(self, model1='', model2='', two_model=False, nomeModello1 = '', nomeModello2 = ''):
        two_model=True
        if model2 == '':
            print('Errore: devi inserire due modelli! eg. model2="/home/starcat8/Scrivania/ANALISIATM/ATMOSFERE/interpolated_atmospheres/T4500_G1.50_M-1.70_2.00.model"')
            return 0
        # Lettura modelli
        skip_lines = 23
        num_lines = 72
        def read_Model(filename):
            data = np.loadtxt(filename, skiprows=skip_lines, max_rows=num_lines, 
                            usecols=range(8), dtype=float)
            return data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4], data[:, 5], data[:, 6]
        solo_teff = 'N'
        crea_ps = 'N'  # 'Y'
        fileps = 'test.eps'
        if model1 == '':
            model1 = star1.model
        model_AT9_1 = model1
        model_AT9_2 = model2
        # Lettura file

        RHOX_AT9_1, T_AT9_1, P_AT9_1, XNE_AT9_1, ABROSS_AT9_1, ACCRAD_AT9_1, VTURB_AT9_1 = read_Model(model_AT9_1)
        RHOX_AT9_2, T_AT9_2, P_AT9_2, XNE_AT9_2, ABROSS_AT9_2, ACCRAD_AT9_2, VTURB_AT9_2 = read_Model(model_AT9_2)

        # Calcoli di massa e tau
        m_AT9_1 = np.zeros(num_lines)                  #
        m_AT9_1[0] = RHOX_AT9_1[0]                     #
        m_AT9_1[1:] = np.diff(RHOX_AT9_1)              #
        tau_AT9_1 = np.cumsum(m_AT9_1 * ABROSS_AT9_1)  #

        m_AT9_2 = np.zeros(num_lines)
        m_AT9_2[0] = RHOX_AT9_2[0]
        m_AT9_2[1:] = np.diff(RHOX_AT9_2)
        tau_AT9_2 = np.cumsum(m_AT9_2 * ABROSS_AT9_2)

        # Setup grafico
        fig, axes = plt.subplots(3, 2, figsize=(12, 18))
        if model2 != '':
            xmin, xmax = np.log10(min(tau_AT9_1.min(), tau_AT9_2.min())), np.log10(max(tau_AT9_1.max(), tau_AT9_2.max()))
        else:
            xmin, xmax = np.log10(tau_AT9_1.min()), np.log10(tau_AT9_1.max())

        # Funzione per generare i plot
        def plot_graph(ax, x1, y1, x2, y2, xlabel, ylabel, two_model=two_model):
            ax.plot(np.log10(x1), y1, label=f'Model 1 - {nomeModello1}')
            if two_model:
                ax.plot(np.log10(x2), y2, label=f'Model 2- {nomeModello2}', color='r')
            ax.set_xlabel(xlabel)
            ax.set_ylabel(ylabel)
            ax.legend()

        plot_graph(axes[0, 0], tau_AT9_1, T_AT9_1, tau_AT9_2, T_AT9_2, 'log(Tau_ross)', 'T(K)')
        plot_graph(axes[0, 1], tau_AT9_1, np.log10(P_AT9_1), tau_AT9_2, np.log10(P_AT9_2), 'log(Tau_ross)', 'log(Pgas)')
        plot_graph(axes[1, 0], tau_AT9_1, np.log10(RHOX_AT9_1), tau_AT9_2, np.log10(RHOX_AT9_2), 'log(Tau_ross)', 'log(RHOX)')
        plot_graph(axes[1, 1], tau_AT9_1, np.log10(XNE_AT9_1), tau_AT9_2, np.log10(XNE_AT9_2), 'log(Tau_ross)', 'log(XNE)')
        plot_graph(axes[2, 0], tau_AT9_1, np.log10(ABROSS_AT9_1), tau_AT9_2, np.log10(ABROSS_AT9_2), 'log(Tau_ross)', 'log(ABROSS)')
        plot_graph(axes[2, 1], tau_AT9_1, np.log10(ACCRAD_AT9_1), tau_AT9_2, np.log10(ACCRAD_AT9_2), 'log(Tau_ross)', 'log(ACCRAD)')

        plt.tight_layout()

        # Salvataggio o visualizzazione
        if crea_ps == 'Y':
            plt.savefig(fileps, format='eps')
        else:
            plt.show()
    def compare_ManyModelProfiles(self, models=[], model_names=[], LEGEND=0):
        """
        Confronta un numero arbitrario di modelli atmosferici.
        
        :param models: Lista dei percorsi ai file dei modelli da confrontare.
        :param model_names: Lista dei nomi dei modelli da utilizzare nelle legende.
        """
        if len(models) < 2:
            print('Errore: devi inserire almeno due modelli!')
            return 0

        # Lettura modelli
        skip_lines = 23
        num_lines = 72

        def read_Model(filename):
            data = np.loadtxt(filename, skiprows=skip_lines, max_rows=num_lines, 
                            usecols=range(8), dtype=float)
            return data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4], data[:, 5], data[:, 6]

        # Lettura dei dati per tutti i modelli
        model_data = []
        for model in models:
            model_data.append(read_Model(model))

        # Calcoli di massa e tau per tutti i modelli
        tau_data = []
        for data in model_data:
            RHOX, _, _, _, ABROSS, _, _ = data
            m = np.zeros(num_lines)
            m[0] = RHOX[0]
            m[1:] = np.diff(RHOX)
            tau = np.cumsum(m * ABROSS)
            tau_data.append(tau)

        # Setup grafico
        fig, axes = plt.subplots(3, 2, figsize=(12, 18))

        # Determina i limiti degli assi
        tau_min = min([tau.min() for tau in tau_data])
        tau_max = max([tau.max() for tau in tau_data])
        xmin, xmax = np.log10(tau_min), np.log10(tau_max)

        # Funzione per generare i plot
        def plot_graph(ax, x_data, y_data_list, xlabel, ylabel, model_names):
            for i, y_data in enumerate(y_data_list):
                ax.plot(np.log10(x_data[i]), y_data, label=f'Model {i + 1} - {model_names[i]}')
            ax.set_xlabel(xlabel)
            ax.set_ylabel(ylabel)
            if LEGEND==1:
                ax.legend()

        # Genera i plot per ogni parametro
        plot_graph(axes[0, 0], tau_data, [data[1] for data in model_data], 'log(Tau_ross)', 'T(K)', model_names)
        plot_graph(axes[0, 1], tau_data, [np.log10(data[2]) for data in model_data], 'log(Tau_ross)', 'log(Pgas)', model_names)
        plot_graph(axes[1, 0], tau_data, [np.log10(data[0]) for data in model_data], 'log(Tau_ross)', 'log(RHOX)', model_names)
        plot_graph(axes[1, 1], tau_data, [np.log10(data[3]) for data in model_data], 'log(Tau_ross)', 'log(XNE)', model_names)
        plot_graph(axes[2, 0], tau_data, [np.log10(data[4]) for data in model_data], 'log(Tau_ross)', 'log(ABROSS)', model_names)
        plot_graph(axes[2, 1], tau_data, [np.log10(data[5]) for data in model_data], 'log(Tau_ross)', 'log(ACCRAD)', model_names)

        plt.tight_layout()
        plt.show()
    def ironFromEW(self, T, logg, m, vturb):
        '''Deriva abbondanze da EW con Turbospectrum'''
        self.creaModel(T=T, logg=logg, MH_g=m[0], microTurb=vturb[0], processore=1, radSolver='turbo')
        lanciaTurbospectrum2(stellaFITS=self.fits_name, processore=1, TURBVEL=vturb, modelName='Model', outname='PINOLOS', metalScaling=m)
        for I in range(1):
            turbo_resultCombine(path_to_turboModels, f"PINOLOSpr1pt", f"PINOLOS1")
            turbo_to_spectrum_resultConversion(path_to_turboModels, f'PINOLOS1')
        for I in range(1):
            delete_files_with_prefix(path_to_turboModels, f"PINOLOSpr1pt")
            delete_files_with_prefix(path_to_turboModels, f"noPINOLOS1")
        sposta('PINOLOS1', path_to_turboModels, self.folder, self.nome+'Fe.iron')
        return self.nome +'Fe.iron'
    def load_Gaiaedr3_table(self, table_path, retrieve_Param=''):
        '''Read .csv tables from gaiaedr3.gaia_source'''
        self.Tabledata = pd.read_csv('aiaii.csv')
        self.gmag = self.Tabledata['phot_g_mean_mag'].values
        self.nueffused = self.Tabledata['nu_eff_used_in_astrometry'].values
        self.psc = self.Tabledata['pseudocolour'].values
        self.ecl_lat = self.Tabledata['ecl_lat'].values
        self.soltype = self.Tabledata['astrometric_params_solved'].values
        self.ra=self.Tabledata['ra'].values
        self.dec=self.Tabledata['dec'].values
        self.nome=self.Tabledata['target_id'].values
        self.parallax=self.Tabledata['parallax'].values
        self.parallax_error=self.Tabledata['parallax_error'].values
        self.source_id=self.Tabledata['source_id'].values
        self.BP_RP = self.Tabledata['bp_rp'].values
        return self.Tabledata[retrieve_Param].values
    def parallax_zero_point(self, table_path=''):
        '''Compute the parallax zero-point according to Lindegren 2021 procedures. It may uses a 5 parameters astrometric solution or a 6 parameters one. These parameters are found in the gaiaedr3.gaia_source catalogue. \n
        Feed a .csv table containing a subset of the gaia3dr2.gaia_source catalogue.'''
        if table_path=='':
            if self.gmag!=0:
                if self.soltype>3:
                    fivep = (self.soltype==31)
                    sixp = (self.soltype==95)
                    if self.soltype[fivep]==31:
                        c=SkyCoord(self.ra, self.dec, unit='deg')
                        l=c.galactic.l.degree
                        b=c.galactic.b.degree
                        self.zpvals5p = zpt.get_zpt(self.gmag[fivep], self.nueffused[fivep], self.psc[fivep], self.ecl_lat[fivep], self.soltype[fivep])
                        print(f'{self.zpvals5p} ' + f'  {self.ra}  {self.dec}           {l}           {b}    {1/(self.parallax-self.zpvals5p)}          {1/((self.parallax+self.parallax_error)-self.zpvals5p)}         {1/((self.parallax-self.parallax_error)-self.zpvals5p)}       {self.nome}')
                    if self.soltype[fivep]==95:
                        c=SkyCoord(self.ra, self.dec, unit='deg')
                        l=c.galactic.l.degree
                        b=c.galactic.b.degree
                        self.zpvals6p = zpt.get_zpt(self.gmag[sixp], self.nueffused[sixp], self.psc[sixp], self.ecl_lat[sixp], self.soltype[sixp])
                        print(f'{self.zpvals6p} ' + f'  {self.ra}  {self.dec}           {l}           {b}    {1/(self.parallax-self.zpvals6p)}          {1/((self.parallax+self.parallax_error)-self.zpvals6p)}         {1/((self.parallax-self.parallax_error)-self.zpvals6p)}       {self.nome}')
                else:
                    print('Wrong soltype! Needed a 5p or 6p astrometric solution for this method!')
                    return 0
            else:
                print('No table loaded! Either call the load_Gaiaedr3_table() function OR specify the table path through the table_path variable in this function.')
        else:
            self.load_Gaiaedr3_table(table_path=table_path)
            self.parallax_zero_point(table_path='')
        return 1
    def photometric_paramSearch(self, table_path, metal_guess):
        '''Obtain the atmospherical parameters for the star using the MINCE way, ie using astrometric data from the GaiaDR3 release (Lombardo+2021).\n
          Derive Teff and log(g) from magnitudes and distances.'''
        if self.zpvals5p!=0:
            self.zp=self.zpvals5p
        elif self.zpvals6p!=0:
            self.zp=self.zpvals6p
        else:
            self.parallax_zero_point(table_path=table_path)
            self.photometric_paramSearch()
        
        crea_photometric_parameters_inputFile(self.source_id, self.ra, self.dec, self.parallax, self.gmag, self.BP_RP, self.zp, self.A0, metal_guess)
        table=Table.read('fort.7', format='ascii')
        self.teff_phot=table['met']
    def normalize(self, x=[0], y=[-1], cutoff_frequency1=0.001, cutoff_frequency2=0.01, cutoff_frequency3=0.001, cutoff_frequency4=1, iter=15, wl_i=None, wl_f = None, showPlot=True):
        '''Normalize spectral data contained in the relative \'Stella\' class object \'fits_name\'.\n
        Creates the object Stella.continuum which is a 2d list, [wavelenght, continuum_intensity].\n
        Yields [wavelength, normalized_spectrum].\n
        ----------------------\n
        Variables:\n
        - cutoff_frequency1: value of the cutoff for the first smoothing (float)\n
        - cutoff_frequency2: value of the cutoff for the second smoothing (float)\n
        - cutoff_frequency3: value of the cutoff for the smoothing during iterations (float)\n
        - iter: number of iterations (int)\n
        - wl_i: initial wavelenght\n
        - wl_f: final wavelength\n 
        In order obtained the normalized spectrum a high-frequency (low wavelength) cut is implemented.\n
        - 1) a narrower filter smoothes the noise oscillations and narrower lines (this usually also removes cosmic rays).\n
        - 2) the new smoothed spectrum S_smooth will be consistently below the continuum as all the absortpion lines still hold effects on the opacity. Hence all the spectral points S_obs(wl) in the observed spectrum will be changed as such: S_new(w) = max(S_obs(w), S_smooth(w))\n
        - 3) Repeat point 1) and 2) for \'iter\' amount to times \n
        - 4) '''
        if wl_i == None:
            wl_i=self.xi
        if wl_f == None:
            wl_f=self.xf
        os.chdir(path_to_fitsFiles)
        a=[]
        if x[0] == 0 or y[0] == -1:
            a=self.spectrum(showplot=False)
            original=a
            x=a[0]
            y=a[1]
            print('AHI!')
            print(type(x))
            print('AHI!')
        else:
            print('OHI!')
            print(type(x))
            print('OHI!')
        a.append(x)
        a.append(y)
        original=a
        print(f'{a[0]}   {len(a[0])}')
        print(f'{a[1]}   {len(a[1])}')
        sel=((a[0] > wl_i) & (a[0] < wl_f))
        a[0]=(a[0])[sel]
        a[1]=(a[1])[sel]
        A=np.zeros(2*len(a[0]))
        B=np.zeros(2*len(a[0]))
        for i in range(len(a[0])):
            A[i]=a[0][i+1-len(a[1])] - (wl_f - wl_i)
            B[i]=a[1][-i-1]
            A[i+len(a[0])]=a[0][i]
            B[i+len(a[0])]=a[1][i]
        a[0] = A
        a[1] = B
        wavelength=a[0]
        amplitude=a[1]
        print(len(wavelength))
        print(len(amplitude))
        wavelength_uniform = np.linspace(wavelength.min(), wavelength.max(), len(wavelength))
        interp_func = interp1d(wavelength, amplitude, kind='linear')
        amplitude_uniform = interp_func(wavelength_uniform)

        # 2. Trasformata di Fourier
        fft_amplitude = fft(amplitude_uniform)
        frequencies = fftfreq(len(wavelength_uniform), d=(wavelength_uniform[1] - wavelength_uniform[0]))

        # 3. Filtraggio: rimuovi alte frequenze
        cutoff_frequency = cutoff_frequency1  # Soglia per il filtro passa-basso (da regolare)
        fft_amplitude_filtered = fft_amplitude.copy()
        fft_amplitude_filtered[np.abs(frequencies) > cutoff_frequency] = 0

        # 4. Trasformata inversa di Fourier
        amplitude_filtered = np.real(ifft(fft_amplitude_filtered))

        risultato = confronta_array(a[1], amplitude_filtered)

        wavelength=a[0]
        amplitude=risultato
        wavelength_uniform = np.linspace(wavelength.min(), wavelength.max(), len(wavelength))
        interp_func = interp1d(wavelength, amplitude, kind='cubic')
        amplitude_uniform = interp_func(wavelength_uniform)

        # 2. Trasformata di Fourier
        fft_amplitude = fft(amplitude_uniform)
        frequencies = fftfreq(len(wavelength_uniform), d=(wavelength_uniform[1] - wavelength_uniform[0]))

        # 3. Filtraggio: rimuovi alte frequenze
        cutoff_frequency = cutoff_frequency2  # Soglia per il filtro passa-basso (da regolare)
        fft_amplitude_filtered = fft_amplitude.copy()
        fft_amplitude_filtered[np.abs(frequencies) > cutoff_frequency] = 0

        # 4. Trasformata inversa di Fourier
        amplitude_filtered = np.real(ifft(fft_amplitude_filtered))

        for i in range(iter):
            risultato = confronta_array(a[1], amplitude_filtered)

            wavelength=a[0]
            amplitude=risultato
            wavelength_uniform = np.linspace(wavelength.min(), wavelength.max(), len(wavelength))
            interp_func = interp1d(wavelength, amplitude, kind='cubic')
            amplitude_uniform = interp_func(wavelength_uniform)

            # 2. Trasformata di Fourier
            fft_amplitude = fft(amplitude_uniform)
            frequencies = fftfreq(len(wavelength_uniform), d=(wavelength_uniform[1] - wavelength_uniform[0]))

            # 3. Filtraggio: rimuovi alte frequenze
            cutoff_frequency = cutoff_frequency3  # Soglia per il filtro passa-basso (da regolare)
            fft_amplitude_filtered = fft_amplitude.copy()
            fft_amplitude_filtered[np.abs(frequencies) > cutoff_frequency] = 0

            # 4. Trasformata inversa di Fourier
            amplitude_filtered = np.real(ifft(fft_amplitude_filtered))

        
        #Rimuovi specchio

        A=np.zeros(len(a[0]))
        B=np.zeros(len(a[0]))
        for i in range(int(len(a[0])/2),(len(a[0]))):
            A[i]=a[0][i]
            B[i]=a[1][i]
        
        a[0]=A
        a[1]=B
        a_new=np.zeros(len(a[0]))
        # 5. Visualizzazione
        # interpol_function=moving_average(a[0], a[1], 50)
        # while i < (len(a[0])):
        #     a_new[i]=interpol_function(a[0][i])
        #     if i%1000==0:
        #         print(f'{(0.5*20*i/len(a[0]))}%')
            #i+=25
        if showPlot:
            plt.figure(figsize=(10, 6))
            plt.plot(a[0], original[1], label='Spettro originale', alpha=0.7)
            plt.plot(a[0], amplitude_filtered, label='Spettro filtrato (corpo nero)', linewidth=2)
            plt.xlabel('Lunghezza d\'onda')
            plt.ylabel('Ampiezza')
            plt.legend()
            plt.title('Filtraggio del rumore e isolamento del corpo nero')
            plt.show()
        self.continuum = [wavelength_uniform, amplitude_filtered]
        norm_spectrum = a[1]/amplitude_filtered
        return [wavelength_uniform, norm_spectrum]
    def synth_linelist(self, linelist, T=-10, G=-10, M=-10, Vturb=-10, TURBVEL=2, radSolver='turbo', H=0.92150, He=0.07840, Li=-10.94, Be=-10.64, B=-9.49, C=-3.52, N=-4.12, O=-3.21, F=-7.48, Ne=-3.96, Na=-5.71, Mg=-4.46, Al=-5.57, Si=-4.49, P=-6.59, S=-4.71, Cl=-6.54, Ar=-5.64, K=-6.92, Ca=-5.68, Sc=-8.87, Ti=-7.02, V=-8.04, Cr=-6.37, Mn=-6.65, Fe=-4.54, Co=-7.12, Ni=-5.79, Cu=-7.83, Zn=-7.44, Ga=-9.16, Ge=-8.63, As=-9.67, Se=-8.63, Br=-9.41, Kr=-8.73, Rb=-9.44, Sr=-9.07, Y=-9.80, Zr=-9.44, Nb=-10.62, Mo=-10.12, Tc=-20.00, Ru=-10.20, Rh=-10.92, Pd=-10.35, Ag=-11.10, Cd=-10.27, In=-10.38, Sn=-10.04, Sb=-11.04, Te=-9.80, Io=-10.53, Xe=-9.87, Cs=-10.91, Ba=-9.91, La=-10.87, Ce=-10.46, Pr=-11.33, Nd=-10.54, Pm=-20.00, Sm=-11.03, Eu=-11.53, Gd=-10.92, Tb=-11.69, Dy=-10.90, Ho=-11.78, Er=-11.11, Tm=-12.04, Yb=-10.96, Lu=-11.98, Hf=-11.16, Ta=-12.17, W=-10.93, Re=-11.76, Os=-10.59, Ir=-10.69, Pt=-10.24, Au=-11.03, Hg=-10.91, Tl=-11.14, Pb=-10.09, Bi=-11.33, Po=-20.00, At=-20.00, Rn=-20.00,  Fr=-20.00, Ra=-20.00, Ac=-20.00, Th=-11.95, Pa=-20.00, U=-12.54):
        '''Crea spettri sintetici attorno alle righe specificate in \'linelist\'.\n
        - linelist: lista contenente le lunghezze d'onda di interesse'''
        nomi_files=[]
        if T==-10:
            T=self.t_eff
        if G==-10:
            G=self.logg
        if M ==-10:
            M=self.metal
        if Vturb == -10:
            Vturb=self.v_m
        for i in range(len(linelist)):
            self.synth(T=T, G=G, M=M, Vturb=Vturb, TURBVEL=TURBVEL, startwl=linelist[i]-2.5, endwl=linelist[i]+2.5, radSolver=radSolver, limit_width=1, H=H, He=He, Li=Li, Be=Be, B=B, C=C, N=N, O=O, F=F, Ne=Ne, Na=Na, Mg=Mg, Al=Al, Si=Si, P=P, S=S, Cl=Cl, Ar=Ar, K=K, Ca=Ca, Sc=Sc, Ti=Ti, V=V, Cr=Cr, Mn=Mn, Fe=Fe, Co=Co, Ni=Ni, Cu=Cu, Zn=Zn, Ga=Ga, Ge=Ge, As=As, Se=Se, Br=Br, Kr=Kr, Rb=Rb, Sr=Sr, Y=Y, Zr=Zr, Nb=Nb, Mo=Mo, Tc=Tc, Ru=Ru, Rh=Rh, Pd=Pd, Ag=Ag, Cd=Cd, In=In, Sn=Sn, Sb=Sb, Te=Te, Io=Io, Xe=Xe, Cs=Cs, Ba=Ba, La=La, Ce=Ce, Pr=Pr, Nd=Nd, Pm=Pm, Sm=Sm, Eu=Eu, Gd=Gd, Tb=Tb, Dy=Dy, Ho=Ho, Er=Er, Tm=Tm, Yb=Yb, Lu=Lu, Hf=Hf, Ta=Ta, W=W, Re=Re, Os=Os, Ir=Ir, Pt=Pt, Au=Au, Hg=Hg, Tl=Tl, Pb=Pb, Bi=Bi, Po=Po, At=At, Rn=Rn,  Fr=Fr, Ra=Ra, Ac=Ac, Th=Th, Pa=Pa, U=U)
            nomi_files.append(self.synstar_name+f'{i}')
            sposta(self.synstar_name, self.folder, self.folder, nomi_files[i])
        for i in range(len(linelist)):
            combine_files(nomi_files, 'lines.synstar', self.folder)

def crea_refabu(data, index=0, instrument='FIES'):
    '''Crea il file refabu.txt a partire da un file di dati contenente le informazioni necessarie. \n
    - data: lista di liste contenente i dati da scrivere nel file. Il formato è tipo \'notData\' o \'snotData\' \n
    - index: indice della riga da cui iniziare a leggere i dati (default=0).'''

    i=index
    OOO=index
    s=Stella(data[i][5], instrument=instrument)
    alp= [-0.4, -0.2, 0, 0.2, 0.4]
    find_closest(data[i][6], alp)
    dalp= data[i][6]-find_closest(data[i][6], alp)
    s.creaModel(
                data[i][1],
                data[i][2],
                data[i][3],
                data[i][4],
                #notData[O+mrbubz][4]+0.5,
                processore=1,
                radSolver='turbo',
                outpath='/home/starcat8/Documenti/',
                outName='fort.11',
                alpha_enhance=data[i][6]
                )
    !cd /home/starcat8/Documenti && echo 2 | ./abuxturbo.exe
    os.chdir('/home/starcat8/Documenti/')
    filame = 'fort.11'  # Replace with your actual filename
    # Open the file and read the 5th row (index 4)
    val1=0.0
    val2=0.0
    with open(filame, 'r') as f:
        lines = f.readlines()
        if len(lines) >= 5:
            line = lines[4] 
            val1_str = line[45:52]
            val2_str = line[55:62]
            
            val1 = float(val1_str)
            val2 = float(val2_str)
    T = Table.read('fort.7', format='ascii')


    os.chdir('/home/starcat8/Documenti/MyGisFOS/mygisfos_lapo/dogrid/SINT_FEROS2/')
    Y=T['col2']
    x=T['col1']

    print(len(Y))

    with open('refabu.txt', mode='w') as f:
        for i in range(len(Y)+1):
            if i == 0:
                f.write(f' 1   {val1:.5f}\n')
            elif i == 1:
                f.write(f' 2   {val2:.5f}\n')
            elif i>1 and x[i-1]%2!=0:
                f.write(f'{x[i-1]}   ')
                f.write(f'{(Y[i-1]-data[OOO][3]):.2f}\n')
            elif i>1 and x[i-1]%2==0 and x[i-1] < 23:
                f.write(f'{x[i-1]}   ')
                f.write(f'{(dalp+Y[i-1]-data[OOO][3]):.2f}\n')
            elif i>1 and x[i-1]%2==0 and x[i-1] >= 23:
                f.write(f'{x[i-1]}   ')
                f.write(f'{(Y[i-1]-data[OOO][3]):.2f}\n')



In [ ]:
os.chdir('/home/starcat8/Scrivania/sysp/syntspec')
FILES = sorted(f for f in os.listdir() if f.endswith('.tlmva1'))
i=0
macroT = [5, 6, 7, 8, 9]
for file in range(len(FILES)):
    for j in range(len(macroT)):
        fout = f'ai_{i}_{j}'
        copia(file, '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/syntspec', fout)
        broaden_withMIDAS(fout, macroT[j], -1, '/home/starcat8/Scrivania/sysp/syntspec')
        formatta(fout + '.dat')
        
        sposta(fout + '.dat_', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', file.replace('.tlmva1', f"_{macroT[j]:.1f}.sd"))
        sposta(fout + '.dat', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', 'dump2')
        sposta(fout + '.tbl', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', 'dump')
    i+=1

In [ ]:
from multiprocessing import Process
import os

macroT = [5, 6, 7, 7.5, 8, 9]

def worker(file, j, idx):
    fout = f'ai_{idx:02d}_{j}'
    
    copia(file, '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/syntspec', fout)
    broaden_withMIDAS(fout, macroT[j], idx, '/home/starcat8/Scrivania/sysp/syntspec')
    formatta(fout + '.dat')
    
    sposta(fout + '.dat_', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi',
           file.replace('.tlmva1', f"_{macroT[j]:.1f}.sd"))
    sposta(fout + '.dat', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', 'dump2')
    #sposta(fout + '.tbl', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', 'dump')

# Inizializzazione
idx = 0
os.chdir('/home/starcat8/Scrivania/sysp/syntspec')
FILES = sorted(f for f in os.listdir() if f.endswith('.tlmva1'))
FILES2 = os.listdir('/home/starcat8/Scrivania/sysp/ehi')

def already_done(file):
    for mt in macroT:
        broadened_name = file.replace('.tlmva1', f"_{mt:.1f}.sd")
        if broadened_name not in FILES2:
            return False
    return True

# Processamento in batch da 2 file × 6 processi ciascuno
for i in range(0, len(FILES), 2):
    file_batch = FILES[i:i+2]
    processes = []
    for file in file_batch:
        if already_done(file):
            print(f"🟡 Skipping {file} (already done)")
            continue

        for j in range(len(macroT)):
            p = Process(target=worker, args=(file, j, idx%80))
            p.start()
            processes.append(p)
            idx += 1
            #os.wait()
            time.sleep(0.03)
    # Aspetta tutti i processi avviati per questi 1–2 file
    print(f"{1.0*i/len(FILES)*100:.2f}% done")
    for p in processes:
        p.join()


In [ ]:
from multiprocessing import Process

macroT = [5, 6, 7, 7.5, 8, 9]

def worker(file, j, idx):
    #macroT = 6 + 0.5 * j
    fout = f'ai_{idx:02d}_{j}'
    
    copia(file, '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/syntspec', fout)
    broaden_withMIDAS(fout, macroT[j], idx, '/home/starcat8/Scrivania/sysp/syntspec')
    formatta(fout+'.dat')
    sposta(fout+'.dat_', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', file.replace('.tlmva1', f"_{macroT[j]:.1f}.sd"))  # Correggi il path se è incompleto
    sposta(fout+'.dat', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', 'dump2')  # Correggi il path se è incompleto
    sposta(fout+'.tbl', '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', 'dump')  # Correggi il path se è incompleto

idx = 0  # identificativo univoco per ogni processo
batch = []  # batch di processi attivi

os.chdir('/home/starcat8/Scrivania/sysp/syntspec')
FILES=os.listdir()
FILES2=os.listdir('/home/starcat8/Scrivania/sysp/ehi')

# # Cicla sui file due alla volta
for i in range(0, len(FILES), 2):
    file_batch = FILES[i:i+2]  # prende 1 o 2 file
    O=0
    for file in file_batch:
        for j in range(6):
            p = Process(target=worker, args=(file, j, idx%12))
            p.start()
            batch.append(p)
            idx += 1
        O+=1


    # Attende che i 12 processi (o meno se alla fine) siano completati
    for proc in batch:
        proc.join()
    batch = []

In [ ]:
from multiprocessing import Process
import os

def skip_file(file, broadened_files):
    """Restituisce True se tutti e 6 i broadened file per questo file sono già presenti."""
    for j in range(6):
        macroT = 6 + 0.5 * j
        broadened_name = file.replace('.tlmva1', f"_{macroT:.1f}.sd")
        if broadened_name not in broadened_files:
            return False
    return True


def worker(file, j, idx):
    macroT = 6 + 0.5 * j
    fout = f'ai_{idx:02d}_{j}'
    
    copia(file, '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/syntspec', fout)
    broaden_withMIDAS(fout, macroT, idx)
    sposta(fout, '/home/starcat8/Scrivania/sysp/syntspec', '/home/starcat8/Scrivania/sysp/ehi', file.replace('.tlmva1', f"_{macroT:.1f}.sd"))


idx = 0
batch = []

os.chdir('/home/starcat8/Scrivania/sysp/syntspec')
FILES = os.listdir()
FILES2 = os.listdir('/home/starcat8/Scrivania/sysp/ehi')

# Cicla due file alla volta
for i in range(0, len(FILES), 2):
    file_batch = FILES[i:i+2]

    for file in file_batch:
        if not file.endswith('.tlmva1'):
            continue  # ignora altri tipi di file

        if skip_file(file, FILES2):
            print(f"🟡 Skip: {file} già elaborato.")
            continue

        for j in range(6):
            p = Process(target=worker, args=(file, j, idx%6))
            p.start()
            batch.append(p)
            idx += 1

    # Aspetta che i 12 processi (o meno) finiscano
    for proc in batch:
        proc.join()
    batch = []
    print(f'{1.0*i/len(FILES)*100:.2f}% completato.')

In [ ]:
#RUMORE GAUSSIANO
import numpy as np
import glob
import os
from joblib import Parallel, delayed
from tqdm import tqdm

# Parametri
SNR = 300
N_JOBS = 12
SD_DIR = "/home/starcat8/Scrivania/sysp/ehi/"
OUT_DIR = "/home/starcat8/Scrivania/sysp/spettri_noisy_npy/"

def process_file(sd_path, snr=100, out_dir="spettri_noisy_npy/"):
    try:
        # Controlla se il file contiene almeno un asterisco
        with open(sd_path, 'r') as f_check:
            for i, line in enumerate(f_check, 1):
                if '*' in line:
                    print(f"[⏭️  Skip] File saltato per presenza di asterisco: {sd_path}, riga {i}")
                    return False

        # Se nessun asterisco, leggi i dati da colonna 2 e 3
        data = np.loadtxt(sd_path, usecols=(1, 2), dtype=np.float32)

        wavelengths = data[:, 0]
        intensities = data[:, 1]

        # Aggiungi rumore gaussiano
        sigma = intensities / snr
        noise = np.random.normal(0, sigma)
        noisy_intensities = intensities + noise

        # Combina wavelength + intensità rumorosa
        combined = np.column_stack((wavelengths, noisy_intensities))

        # Salva file
        base = os.path.basename(sd_path).replace(".sd", "_noisy.npy")
        out_path = os.path.join(out_dir, base)
        np.save(out_path, combined)

        return True

    except Exception as e:
        print(f"[❌ ERRORE] {sd_path}: {e}")
        return False


os.makedirs(OUT_DIR, exist_ok=True)

all_sd_files = glob.glob(os.path.join(SD_DIR, "*.sd"))

results = Parallel(n_jobs=12)(
    delayed(process_file)(f, snr=SNR, out_dir=OUT_DIR)
    for f in tqdm(all_sd_files, desc="Processamento", unit="file")
)



In [ ]:
import os
import numpy as np
import re

def parse_filename_npy(filename):
    """
    Estrae Teff, logg, [Fe/H], vturb, macroturbolenza dal nome del file.
    Ignora l'alpha enhancement (sempre -0.20).
    Formato atteso:
    TTTT_logg_FeH_vturb_-0.20_macroturb_noisy.npy
    Esempio:
    4000_0.20_0.05_1.00_-0.20_6.0_noisy.npy
    """
    pattern = r"(\d{4})_([+-]?\d\.\d{2})_([+-]?\d\.\d{2})_([+-]?\d\.\d{2})_-0\.20_([+-]?\d+\.?\d*)_noisy\.npy"
    match = re.match(pattern, filename)
    if not match:
        raise ValueError(f"Impossibile interpretare: {filename}")
    Teff = int(match.group(1))
    logg = float(match.group(2))
    FeH = float(match.group(3))
    vturb = float(match.group(4))
    macroturb = float(match.group(5))
    return Teff, logg, FeH, vturb, macroturb

def combine_spectra_npy(input_dir, output_file):
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with open(output_file, 'w') as master:
        for filename in sorted(os.listdir(input_dir)):
            if not filename.endswith('_noisy.npy'):
                continue
            try:
                Teff, logg, FeH, vturb, macroturb = parse_filename_npy(filename)
            except ValueError as e:
                print(f"[Warning] {e}")
                continue

            filepath = os.path.join(input_dir, filename)
            spectrum = np.load(filepath)

            master.write("# START_SPECTRUM\n")
            master.write(f"# Teff: {Teff} K\n")
            master.write(f"# logg: {logg:.2f} dex\n")
            master.write(f"# [Fe/H]: {FeH:.2f} dex\n")
            master.write(f"# vturb: {vturb:.2f} km/s\n")
            master.write(f"# macroturb: {macroturb:.2f} km/s\n")
            master.write(f"# FILENAME: {filename}\n")

            for row in spectrum:
                master.write(f"{row[0]:.6f} {row[1]:.6f}\n")

            master.write("# END_SPECTRUM\n\n")

    print("✅ File combinato creato in:", output_file)

# Esempio d’uso
input_dir = "/home/starcat8/Scrivania/sysp/spettri_noisy_npy/"        # cartella con i .npy rumorosi
output_file = "/home/starcat8/Scrivania/sysp/master_de.puppa"  # file di output combinato

combine_spectra_npy(input_dir, output_file)



In [ ]:
import numpy as np
import os
import csv
import re

def parse_filename_npy(filename):
    """
    Estrae Teff, logg, FeH, vturb, macroturb da filename tipo:
    '4000_0.20_0.05_1.00_-0.20_6.0_noisy.npy'
    Ignora alpha enhancement (-0.20)
    """
    pattern = r"(\d{4})_([0-9.]+)_([+-]?[0-9.]+)_([0-9.]+)_-0\.20_([0-9.]+)_noisy\.npy"
    match = re.match(pattern, filename)
    if not match:
        raise ValueError(f"Filename non conforme: {filename}")
    Teff = int(match.group(1))
    logg = float(match.group(2))
    FeH = float(match.group(3))
    vturb = float(match.group(4))
    macroturb = float(match.group(5))
    return Teff, logg, FeH, vturb, macroturb

def combine_spectra_npy_to_npz(input_dir, output_npz):
    spectra_dict = {}
    metadata = []

    for filename in sorted(os.listdir(input_dir)):
        if not filename.endswith('_noisy.npy'):
            continue
        try:
            Teff, logg, FeH, vturb, macroturb = parse_filename_npy(filename)
        except ValueError as e:
            print(f"[Warning] {e}")
            continue

        filepath = os.path.join(input_dir, filename)
        spectrum = np.load(filepath)

        key = os.path.splitext(filename)[0]

        spectra_dict[key] = spectrum
        metadata.append([key, Teff, logg, FeH, vturb, macroturb])

    # Salva tutti gli spettri compressi in un unico file npz
    np.savez_compressed(output_npz, **spectra_dict)
    print(f"✅ File compresso salvato in: {output_npz}")

    # Salva metadati in CSV
    meta_csv = output_npz.replace('.npz', '_metadata.csv')
    with open(meta_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['key', 'Teff', 'logg', 'FeH', 'vturb', 'macroturb'])
        writer.writerows(metadata)
    print(f"✅ Metadati salvati in: {meta_csv}")

if __name__ == "__main__":
    input_dir = "/home/starcat8/Scrivania/sysp/spettri_noisy_npy/"        # cartella con i .npy rumorosi
    output_file = "/home/starcat8/Scrivania/sysp/master_de.puppa"  # file di output combinato


    combine_spectra_npy_to_npz(input_dir, output_file)


In [ ]:
import numpy as np
import os
import csv
import re

def parse_filename_npy(filename):
    pattern = r"(\d{4})_([0-9.]+)_([+-]?[0-9.]+)_([0-9.]+)_-0\.20_([0-9.]+)_noisy\.npy"
    match = re.match(pattern, filename)
    if not match:
        raise ValueError(f"Filename non conforme: {filename}")
    Teff = int(match.group(1))
    logg = float(match.group(2))
    FeH = float(match.group(3))
    vturb = float(match.group(4))
    macroturb = float(match.group(5))
    return Teff, logg, FeH, vturb, macroturb

def combine_spectra_with_wavelength(input_dir, output_prefix):
    filenames = sorted([
        f for f in os.listdir(input_dir)
        if f.endswith('_noisy.npy')
    ])
    n_spectra = len(filenames)
    if n_spectra == 0:
        raise ValueError("Nessuno spettro trovato!")

    # Carica uno spettro per determinare lunghezza
    test_spec = np.load(os.path.join(input_dir, filenames[0]))
    if test_spec.ndim != 2 or test_spec.shape[1] != 2:
        raise ValueError("Gli spettri devono avere 2 colonne (wavelength, intensity)")

    n_pixels = test_spec.shape[0]

    # Prealloca array su disco
    wavelengths = np.memmap(f"{output_prefix}_wavelengths.npy", dtype='float32', mode='w+', shape=(n_spectra, n_pixels))
    intensities = np.memmap(f"{output_prefix}_intensities.npy", dtype='float32', mode='w+', shape=(n_spectra, n_pixels))
    print(f"🔧 Allocati array {n_spectra} x {n_pixels} (≈ {(wavelengths.nbytes + intensities.nbytes) / 1e9:.2f} GB)")

    # CSV metadati
    with open(f"{output_prefix}_metadata.csv", 'w', newline='') as fcsv:
        writer = csv.writer(fcsv)
        writer.writerow(['index', 'filename', 'Teff', 'logg', 'FeH', 'vturb', 'macroturb'])

        for i, filename in enumerate(filenames):
            try:
                Teff, logg, FeH, vturb, macroturb = parse_filename_npy(filename)
                spectrum = np.load(os.path.join(input_dir, filename))

                if spectrum.shape != (n_pixels, 2):
                    raise ValueError(f"Forma inattesa: {spectrum.shape}")

                wavelengths[i] = spectrum[:, 0]
                intensities[i] = spectrum[:, 1]

                writer.writerow([i, filename, Teff, logg, FeH, vturb, macroturb])
            except Exception as e:
                print(f"[Errore] {filename}: {e}")
                continue

    del wavelengths
    del intensities
    print(f"✅ Salvati:")
    print(f"  - Lunghezze d'onda in: {output_prefix}_wavelengths.npy")
    print(f"  - Intensità in:       {output_prefix}_intensities.npy")
    print(f"  - Metadati in:        {output_prefix}_metadata.csv")

# === USO ===
if __name__ == "__main__":
    input_dir = "/home/starcat8/Scrivania/sysp/spettri_noisy_npy/"
    output_prefix = "/home/starcat8/Scrivania/sysp/master_de"

    combine_spectra_with_wavelength(input_dir, output_prefix)


In [ ]:
import numpy as np
import os
import csv
import re
import glob
from joblib import Parallel, delayed
from tqdm import tqdm


def process_file(sd_path, snr=100, out_dir="spettri_noisy_npy/"):
    try:
        # Controlla se il file contiene almeno un asterisco
        with open(sd_path, 'r') as f_check:
            for i, line in enumerate(f_check, 1):
                if '*' in line:
                    print(f"[⏭️  Skip] File saltato per presenza di asterisco: {sd_path}, riga {i}")
                    return False

        # Se nessun asterisco, leggi i dati da colonna 2 e 3
        data = np.loadtxt(sd_path, usecols=(1, 2), dtype=np.float32)

        wavelengths = data[:, 0]
        intensities = data[:, 1]

        # Aggiungi rumore gaussiano
        sigma = intensities / snr
        noise = np.random.normal(0, sigma)
        noisy_intensities = intensities + noise

        # Combina wavelength + intensità rumorosa
        combined = np.column_stack((wavelengths, noisy_intensities))

        # Salva file
        base = os.path.basename(sd_path).replace(".sd", "_noisy.npy")
        out_path = os.path.join(out_dir, base)
        np.save(out_path, combined)

        return True

    except Exception as e:
        print(f"[❌ ERRORE] {sd_path}: {e}")
        return False
def parse_filename_npy(filename):
    pattern = r"(\d{4})_([0-9.]+)_([+-]?[0-9.]+)_([0-9.]+)_-0\.20_([0-9.]+)_noisy\.npy"
    match = re.match(pattern, filename)
    if not match:
        raise ValueError(f"Filename non conforme: {filename}")
    Teff = int(match.group(1))
    logg = float(match.group(2))
    FeH = float(match.group(3))
    vturb = float(match.group(4))
    macroturb = float(match.group(5))
    return Teff, logg, FeH, vturb, macroturb

def combine_spectra_with_wavelength(input_dir, output_prefix):
    filenames = sorted([
        f for f in os.listdir(input_dir)
        if f.endswith('_noisy.npy')
    ])
    n_spectra = len(filenames)
    if n_spectra == 0:
        raise ValueError("Nessuno spettro trovato!")

    # Carica uno spettro per determinare lunghezza
    test_spec = np.load(os.path.join(input_dir, filenames[0]))
    if test_spec.ndim != 2 or test_spec.shape[1] != 2:
        raise ValueError("Gli spettri devono avere 2 colonne (wavelength, intensity)")

    n_pixels = test_spec.shape[0]

    # Prealloca array su disco
    wavelengths = np.memmap(f"{output_prefix}_wavelengths.npy", dtype='float32', mode='w+', shape=(n_spectra, n_pixels))
    intensities = np.memmap(f"{output_prefix}_intensities.npy", dtype='float32', mode='w+', shape=(n_spectra, n_pixels))
    print(f"🔧 Allocati array {n_spectra} x {n_pixels} (≈ {(wavelengths.nbytes + intensities.nbytes) / 1e9:.2f} GB)")

    # CSV metadati
    with open(f"{output_prefix}_metadata.csv", 'w', newline='') as fcsv:
        writer = csv.writer(fcsv)
        writer.writerow(['index', 'filename', 'Teff', 'logg', 'FeH', 'vturb', 'macroturb'])

        for i, filename in enumerate(filenames):
            try:
                Teff, logg, FeH, vturb, macroturb = parse_filename_npy(filename)
                spectrum = np.load(os.path.join(input_dir, filename))

                if spectrum.shape != (n_pixels, 2):
                    raise ValueError(f"Forma inattesa: {spectrum.shape}")

                wavelengths[i] = spectrum[:, 0]
                intensities[i] = spectrum[:, 1]

                writer.writerow([i, filename, Teff, logg, FeH, vturb, macroturb])
            except Exception as e:
                print(f"[Errore] {filename}: {e}")
                continue

    del wavelengths
    del intensities
    print(f"✅ Salvati:")
    print(f"  - Lunghezze d'onda in: {output_prefix}_wavelengths.npy")
    print(f"  - Intensità in:       {output_prefix}_intensities.npy")
    print(f"  - Metadati in:        {output_prefix}_metadata.csv")

# Parametri
SNR = 120
N_JOBS = 12
SD_DIR = "/home/starcat8/Scrivania/sysp/ehi/"
OUT_DIR = "/home/starcat8/Scrivania/sysp/spettri_noisy_npy/"


os.makedirs(OUT_DIR, exist_ok=True)

all_sd_files = glob.glob(os.path.join(SD_DIR, "*.sd"))

results = Parallel(n_jobs=12)(
    delayed(process_file)(f, snr=SNR, out_dir=OUT_DIR)
    for f in tqdm(all_sd_files, desc="Processamento", unit="file")
)


# === USO ===
if __name__ == "__main__":
    input_dir = "/home/starcat8/Scrivania/sysp/spettri_noisy_npy/"
    output_prefix = "/home/starcat8/Scrivania/sysp/master_de"

    combine_spectra_with_wavelength(input_dir, output_prefix)


In [ ]:
import numpy as np
import pandas as pd

metadata = pd.read_csv("/home/starcat8/Scrivania/sysp/master_de1_metadata.csv")
n_spectra = metadata.shape[0]
n_pixels = 1431  # o ricavalo da file

wavelengths = np.memmap("/home/starcat8/Scrivania/sysp/master_de1_wavelengths.npy", dtype='float32', mode='r', shape=(n_spectra, n_pixels))
intensities = np.memmap("/home/starcat8/Scrivania/sysp/master_de1_intensities.npy", dtype='float32', mode='r', shape=(n_spectra, n_pixels))


In [ ]:
STELLOZ = Stella('BD+07_5118.fits', instrument='FIES')
STELLOZ.searchParam3(TRIALS=25, radSolver='turbo', processori=12)

Processo ['csh', 'Turbo2_EW1.csh'] terminato con codice 0
Output:
/home/starcat8/Documenti/Turbospectrum2019-master/DATA1
mer 13 ago 2025, 22:05:22, CEST

 ***********************
 * EQWIDT version 19.1 *
 ***********************

 PURE-LTE:.false.
 LAMBDA_MIN:4261.675
 LAMBDA_MAX:8013.944
 LAMBDA_STEP:0.005
 INTENSITY/FLUX:Flux
 COS(THETA)    :1.00
 ABFIND        :.true.
 MODELOPAC:contopac/Model1opac
 RESULTFILE :PINOLOSpr1pt1
 METALLICITY:-1.5766861476563319
 ALPHA/Fe   :0.1
 HELIUM     :0.00
 R-PROCESS  :0.00
 S-PROCESS  :0.00
 INDIVIDUAL ABUNDANCES:1
           6   7.30000019    
 ISOTOPES :11
 INCLUDE FOLLOWING ATOMS:10
 MOLECULES :/home/starcat8/Documenti/Turbospectrum2019-master/DATA1/LISTE_molecules_all_v12.1.dat
 NFILES   :1
 SPHERICAL:F
 LOGFILE:outdata/blistbb.dat
 MULTIDUMP:F
 Following atomic species included in molecular equilibrium:
 H  He Li Be B  C  N  O  F  Ne 
 Na Mg Al Si P  S  Cl Ar K  Ca 
 Sc Ti V  Cr Mn Fe Co Ni Cu Zn 
 Ga Ge As Se Br Kr Rb Sr Y  Zr 
 Nb Mo Tc R

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

# === CONFIG ===
BASE_DIR = "/home/starcat8/Scrivania/sysp/"
SET_NAMES = ["master_de1", "master_de2"]#, "master_de3"]#, "master_de4", "master_de5"]
PIXELS = 1431  # numero fisso di pixel per spettro
N_CORES = 12
FEATURES = []
TARGETS = []

print("📦 Caricamento dei dati...")

for name in tqdm(SET_NAMES, desc="Caricamento set"):
    meta_path = os.path.join(BASE_DIR, f"{name}_metadata.csv")
    wave_path = os.path.join(BASE_DIR, f"{name}_wavelengths.npy")
    intens_path = os.path.join(BASE_DIR, f"{name}_intensities.npy")

    # Caricamento metadata
    metadata = pd.read_csv(meta_path)
    n_spectra = metadata.shape[0]

    # Caricamento memmap
    wavelengths = np.memmap(wave_path, dtype='float32', mode='r', shape=(n_spectra, PIXELS))
    intensities = np.memmap(intens_path, dtype='float32', mode='r', shape=(n_spectra, PIXELS))

    # Costruzione feature vector [w1, w2, ..., wn, i1, i2, ..., in]
    combined_features = np.hstack([wavelengths, intensities])
    FEATURES.append(combined_features)

    # Estrazione target: Teff, logg, FeH, vturb, macroturb
    target = metadata[['Teff', 'logg', 'FeH', 'vturb', 'macroturb']].to_numpy()
    TARGETS.append(target)

# === Unione di tutti i set ===
print("🧩 Combinazione dei dati...")
X = np.vstack(FEATURES)
y = np.vstack(TARGETS)

# === Split train/val ===
print("✂️ Suddivisione in training e validation set (90% / 10%)...")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

# === Addestramento Random Forest ===
print(f"🌲 Addestramento Random Forest con {N_CORES} core...")
rf = RandomForestRegressor(
    n_estimators=100,
    n_jobs=N_CORES,
    random_state=42,
    verbose=1
)
rf.fit(X_train, y_train)

# === Valutazione ===
print("📊 Valutazione...")
preds = rf.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, preds))
print(f"✅ RMSE globale: {rmse:.2f}")

# === Salvataggio modello ===
MODEL_PATH = os.path.join(BASE_DIR, "random_forest_model.joblib")
joblib.dump(rf, MODEL_PATH)
print(f"💾 Modello salvato in: {MODEL_PATH}")


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import joblib

# === CONFIG ===
BASE_DIR = "/home/starcat8/Scrivania/sysp/"
SET_NAMES = ["master_de1", "master_de2", "master_de3"]#, "master_de4", "master_de5"]
PIXELS = 1431
TEST_SIZE = 0.10
RANDOM_STATE = 42
MODEL_PATH = os.path.join(BASE_DIR, "histboost_multi_model.joblib")

X_list = []
y_list = []

print("📦 Caricamento dati con memmap...")

for name in tqdm(SET_NAMES, desc="Caricamento dataset"):
    meta_path = os.path.join(BASE_DIR, f"{name}_metadata.csv")
    wave_path = os.path.join(BASE_DIR, f"{name}_wavelengths.npy")
    intens_path = os.path.join(BASE_DIR, f"{name}_intensities.npy")

    metadata = pd.read_csv(meta_path)
    n_spectra = metadata.shape[0]

    # Usa memmap per non caricare tutto in RAM
    wavelengths = np.memmap(wave_path, dtype='float32', mode='r', shape=(n_spectra, PIXELS))
    intensities = np.memmap(intens_path, dtype='float32', mode='r', shape=(n_spectra, PIXELS))

    # Combina feature: [w1, ..., wn, i1, ..., in]
    features = np.hstack([wavelengths, intensities])
    X_list.append(features)

    # Target: Teff, logg, FeH, vturb, macroturb
    y_list.append(metadata[['Teff', 'logg', 'FeH', 'vturb', 'macroturb']].to_numpy())

# === Unione e split ===
print("🧩 Combinazione dei dati...")
X = np.vstack(X_list)
y = np.vstack(y_list)

print("✂️ Suddivisione in training e validation set (90% / 10%)...")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# === Addestramento ===
print("🌲 Addestramento modello multivariato con HistGradientBoostingRegressor...")

# Funzione wrapper per tqdm con fit
class TQDMFit(MultiOutputRegressor):
    def fit(self, X, y):
        self.estimators_ = []
        with tqdm(total=y.shape[1], desc="Training targets", unit="target") as pbar:
            for i in range(y.shape[1]):
                estimator = self._make_estimator()
                estimator.fit(X, y[:, i])
                self.estimators_.append(estimator)
                pbar.update(1)
        return self

# Inizializza il modello base
base_model = HistGradientBoostingRegressor(
    loss='squared_error',
    learning_rate=0.1,
    max_iter=300,
    max_depth=7,
    early_stopping=True,
    random_state=RANDOM_STATE,
    verbose=0
)

# Wrappa con MultiOutput + tqdm
model = TQDMFit(base_model, n_jobs=-1)
model.fit(X_train, y_train)

# === Valutazione ===
print("📊 Valutazione sul validation set...")
preds = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, preds))
print(f"✅ RMSE globale: {rmse:.2f}")

# === Salvataggio ===
joblib.dump(model, MODEL_PATH)
print(f"💾 Modello salvato in: {MODEL_PATH}")


In [ ]:
############################## CREA SINGLE-MODEL GRIDS PER MYGISFOS ##########################################


METALi = [-1.0, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1.0]
METAL=-1
KKK=0
mrbubz=0
ranger=0
suppa=0
while suppa < 2:
    METAL=-1
    KKK=0
    mrbubz=0
    ranger=0
    if suppa == 0:
        for K in range(1):
            ueee=0
            if K == 0:
                mrbubz = 12
                ranger=1
                ueee=0
            elif K == 1:
                mrbubz = 5
                ranger=5
                ueee=0
            elif K == 2:
                mrbubz = 10
                ranger=4
                ueee=6
            
            for kop in range(ueee, 11):
                processi = []
                METAL = METALi[kop]
                # Loop su 12 oggetti (modifica l'intervallo secondo necessità)
                for O in range(ranger):
                    s = Stella(notData[O+mrbubz][5], instrument='FIES')
                    #s = Stella(notData[len(notData)-1][5], instrument='FIES')
                    nomeout = s.nome.replace(' ', '_')

                    s.creaModel(
                        notData[O+mrbubz][1],
                        notData[O+mrbubz][2],
                        notData[O+mrbubz][3],
                        notData[O+mrbubz][4],
                        #notData[O+mrbubz][4]+0.5,
                        processore=1,
                        radSolver='turbo',
                        outpath='/home/starcat8/Documenti/',
                        outName='fort.11',
                        alpha_enhance=notData[O+mrbubz][6]
                    )

                    os.chdir('/home/starcat8/Documenti/MyGisFOS/test/best/')
                    aoaos = abbondanzeTurbo(notData[O+mrbubz][6], s.nome + f'_{notData[O+mrbubz][7]}.out', metal_singleMode=METAL)
                    #aoaos = abbondanzeTurbo(notData[len(notData)-1][6], s.nome + f'_{notData[len(notData)-1][7]}.out', metal_singleMode=METAL)
                    files = glob.glob(nomeout + '*')

                    with open('/home/starcat8/Documenti/MyGisFOS/test/best/' + files[0], 'r') as f:
                        righe = f.readlines()
                    quarta_riga = righe[3]
                    valore_str = quarta_riga[46:52]
                    valore_float = float(valore_str)
                    print(valore_float)
                    copia('fort.11', '/home/starcat8/Documenti/', '/home/starcat8/Documenti/', f'fort.11{s.nome}')
                    # Ogni script avrà un nome diverso
                    script_name = f'synth_script{O+mrbubz}'
                    crea_synthScriptTurbo(
                        '/home/starcat8/Documenti/',
                        f'fort.11{s.nome}',
                        outname=s.nome+f'{kop}_{O+mrbubz}_',
                        wl_min=3700,
                        wl_max=8850,
                        TURBVEL=notData[O+mrbubz][4],
                        abundances=[aoaos[0][0], aoaos[0][1]],
                        scriptNumber=O+mrbubz  # Assicurati che questa funzione accetti un parametro per il nome dello script
                    )
                    olddir=os.getcwd()
                    os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
                    
                    # Lancio in parallelo
                    p = subprocess.Popen(['csh', '-c', f'source {script_name}'])
                    processi.append(p)
                    os.chdir(olddir)

                # Aspetta che tutti i processi finiscano
                for p in processi:
                    p.wait()
    elif suppa == 1:
        for K in range(2):
            ueee=0
            if K == 0:
                mrbubz = 0
                ranger=5
                ueee=0
            elif K == 1:
                mrbubz = 5
                ranger=4
            elif K == 2:
                mrbubz = 11
                ranger=4
            for kop in range(ueee, 11):
                processi = []
                METAL = METALi[kop]
                # Loop su 12 oggetti (modifica l'intervallo secondo necessità)
                for O in range(ranger):
                    s = Stella(snotData[O+mrbubz][5], instrument='HARPS-N')
                    nomeout = s.nome.replace(' ', '_')

                    s.creaModel(
                        snotData[O+mrbubz][1],
                        snotData[O+mrbubz][2],
                        snotData[O+mrbubz][3],
                        snotData[O+mrbubz][4],
                        #notData[O+mrbubz][4]+0.5,
                        processore=1,
                        radSolver='turbo',
                        outpath='/home/starcat8/Documenti/',
                        outName='fort.11',
                        alpha_enhance=snotData[O+mrbubz][6]
                    )

                    os.chdir('/home/starcat8/Documenti/MyGisFOS/test/best/')
                    aoaos = abbondanzeTurbo(snotData[O+mrbubz][6], s.nome + f'_{75}.out', metal_singleMode=METAL)
                    files = glob.glob(nomeout + '*')

                    with open('/home/starcat8/Documenti/MyGisFOS/test/best/' + files[0], 'r') as f:
                        righe = f.readlines()
                    quarta_riga = righe[3]
                    valore_str = quarta_riga[46:52]
                    valore_float = float(valore_str)
                    print(valore_float)
                    copia('fort.11', '/home/starcat8/Documenti/', '/home/starcat8/Documenti/', f'fort.11{s.nome}')
                    # Ogni script avrà un nome diverso
                    script_name = f'synth_script{O+mrbubz}'
                    crea_synthScriptTurbo(
                        '/home/starcat8/Documenti/',
                        f'fort.11{s.nome}',
                        outname=s.nome+f'{kop}_{O+mrbubz}_',
                        wl_min=3700,
                        wl_max=8850,
                        TURBVEL=snotData[O+mrbubz][4],
                        abundances=[aoaos[0][0], aoaos[0][1]],
                        scriptNumber=O+mrbubz  # Assicurati che questa funzione accetti un parametro per il nome dello script
                    )
                    olddir=os.getcwd()
                    os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
                    
                    # Lancio in parallelo
                    p = subprocess.Popen(['csh', '-c', f'source {script_name}'])
                    processi.append(p)
                    os.chdir(olddir)

                # Aspetta che tutti i processi finiscano
                for p in processi:
                    p.wait()

    METAL=-1
    KKK=0
    mrbubz=0
    ranger=0
    suppa += 23

In [ ]:
############# CREA SYNTHETIC DATASET FOR MACHINE LEARNING ##############

T_range=50
logg_range=10
mh_range=20
vmicro_range=4
vmacro_range=1#da rifare fino a 5!
alpha_range=0
grido=np.zeros((5, 5))
s = Stella(notData[0][5], instrument='FIES')

lines=[ [5701.3   , 5701.8],
        [5752.68  , 5753.44],
        [6419.28 ,  6421.925],
        [6624.5  ,  6625.5],
        [6430    ,  6433] ]

for i in range(4, T_range+1):
    mh1=0
    vm1=0
    log1=0
    if i == 11:
        mh1=6
        vm1=4
    for iii in range(mh1, mh_range+1):
        for iv in range(vm1, vmicro_range+1):
            processi = []
            for ii in range(log1, logg_range+1):
                for v in range(vmacro_range):
                    for vi in range(alpha_range+1):
                        s.creaModel(
                            4000+20*(i),
                            0.2+0.3*(ii),
                            0.2-0.15*(iii),
                            1+0.4*(iv),

                            processore=iv+vi,
                            radSolver='turbo',
                            outpath='/home/starcat8/Documenti/',
                            outName='fort.11',
                            alpha_enhance=0
                        )
                        copia('fort.11', '/home/starcat8/Documenti/', '/home/starcat8/Documenti/sbundance/ANALISIATM/ATMOSFERE', f'fort.11{s.nome}_{4000+20*(i)}_{(0.2+0.3*(ii)):.2f}_{(0.2-0.15*(iii)):.2f}_{(1+0.4*(iv)):.2f}_{(-0.2+0.2*(vi)):.2f}')

                        #for vii in range(len(lines)):
                        script_name = f'synth_script{vi+iv}'
                        crea_synthScriptTurbo(
                            '/home/starcat8/Documenti/sbundance/ANALISIATM/ATMOSFERE',
                            f'fort.11{s.nome}_{4000+20*(i)}_{(0.2+0.3*(ii)):.2f}_{(0.2-0.15*(iii)):.2f}_{(1+0.4*(iv)):.2f}_{(-0.2+0.2*(vi)):.2f}',
                            outname=f'{4000+20*(i)}_{(0.2+0.3*(ii)):.2f}_{(0.2-0.15*(iii)):.2f}_{(1+0.4*(iv)):.2f}_{(-0.2+0.2*(vi)):.2f}.tlmva',
                            wl_min=6419,
                            wl_max=6433,
                            TURBVEL=1+0.4*(iv),
                            limit_width=0.3,
                            abundances=[[26], [0.2-0.15*(iii)]],
                            outpath='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ml',
                            scriptNumber=iv+vi,  # Assicurati che questa funzione accetti un parametro per il nome dello script
                            long=False
                        )
                        olddir=os.getcwd()
                        os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
                        p = subprocess.Popen(['csh', '-c', f'source {script_name}'])
                        processi.append(p)
                        os.chdir(olddir)

            for p in processi:
                p.wait()
                        


In [ ]:
############# CREA SYNTHETIC DATASET FOR MACHINE LEARNING ##############



import subprocess
from itertools import product
from concurrent.futures import ThreadPoolExecutor



T_range = 50
logg_range = 10
mh_range = 20
vmicro_range = 5
vmacro_range = 1  # da rifare fino a 5!
alpha_range = 2

lines = [
    [5701.3, 5701.8],
    [5752.68, 5753.44],
    [6419.28, 6421.925],
    [6624.5, 6625.5],
    [6430, 6433]
]

def run_synth(i, ii, iii, iv, v, vi, s, lines):
    T = 4000 + 20 * i
    logg = 0.2 + 0.3 * ii
    mh = 0.2 - 0.15 * iii
    vmicro = 1 + 0.4 * iv
    alpha = 0.0 + 0.2 * vi

    s.creaModel(
        T, logg, mh, vmicro,
        processore=1,
        radSolver='turbo',
        outpath='/home/starcat8/Documenti/',
        outName='fort.11',
        alpha_enhance=alpha
    )
    copia('fort.11', '/home/starcat8/Documenti/', '/home/starcat8/Documenti/', f'fort.11{s.nome}')

    processes = []
    for vii, (wl_min, wl_max) in enumerate(lines):
        script_name = f'synth_script_{i}_{ii}_{iii}_{iv}_{v}_{vi}_{vii}'
        crea_synthScriptTurbo(
            '/home/starcat8/Documenti/',
            f'fort.11{s.nome}',
            outname=f'{T}_{logg}_{mh}_{vmicro}_{alpha}.tlmva',
            outpath='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ml',
            wl_min=wl_min,
            wl_max=wl_max,
            TURBVEL=vmicro,
            abundances=[26, mh],
            scriptNumber=vii * vi
        )
        olddir = os.getcwd()
        os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
        p = subprocess.Popen(['csh', '-c', f'source {script_name}'])
        processes.append(p)
        os.chdir(olddir)
    for p in processes:
        p.wait()

s = Stella(notData[0][5], instrument='FIES')

# Use ThreadPoolExecutor for parallel execution (or ProcessPoolExecutor if CPU-bound)
with ThreadPoolExecutor(max_workers=12) as executor:
    for args in product(
        range(T_range + 1),
        range(logg_range + 1),
        range(mh_range + 1),
        range(vmicro_range + 1),
        range(vmacro_range),
        range(alpha_range + 1)
    ):
        executor.submit(run_synth, *args, s, lines)

In [ ]:
import glob
inizio=1
for O in range(inizio, inizio+1):
    if O != 15 and O != 16:
        s=Stella(notData[O][5], instrument='FIES')
        nomeout=s.nome.replace(' ', '_')
        s.creaModel(notData[O][1], notData[O][2], notData[O][3], notData[O][4],processore=1, radSolver='turbo', outpath='/home/starcat8/Documenti/', outName='fort.11', alpha_enhance=notData[O][6])
        os.chdir('/home/starcat8/Documenti/MyGisFOS/test/best/')
        aoaos=abbondanzeTurbo(notData[O][6], s.nome+f'_{notData[O][7]}.out')
        files=glob.glob(nomeout+'*')
        with open('/home/starcat8/Documenti/MyGisFOS/test/best/'+files[0], 'r') as f:
            righe = f.readlines()
            # Prende la quarta riga (indice 3)
        quarta_riga = righe[3]
        # Prende i caratteri dal 47 al 52 (in Python: 46:52)
        valore_str = quarta_riga[46:52]
        valore_float = float(valore_str)
        print(valore_float)
        crea_synthScriptTurbo('/home/starcat8/Documenti/', 'fort.11', outname=s.nome+'ba', wl_min=6630, wl_max=6660, TURBVEL=notData[O][4], abundances=[aoaos[0][0], aoaos[0][1]])
        os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
        copia(s.nome+'ba', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba')
        subprocess.run(['csh', '-c', 'source synth_script'])

In [ ]:
# O=0
S=[]
#p=8
for O in range(len(notData)):

    s=Stella(notData[O][5], instrument='FIES')
    S.append(s.nome)
from io import StringIO
#print(S[p])

In [ ]:
###CREA SYNTH SPECTRA FROM MYGISFOS RESULTS, WORKING FOR NOT (notData) STARS###
OOOOO=0
for PP in range(0, 1): #PP seleziona la stella
    if PP != 2 and PP != 3:
        path='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/'
        os.chdir(path)
        O=PP
        o=PP
        import glob
        copia('abundances.out', f'/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[PP]}', '/home/starcat8/Documenti/MyGisFOS/test/best/', 'abundances.out')
        broaden_withMIDAS(S[O]+'2', 1.0*(notData[O][7])/10)
        a=formatta(S[O]+'2'+'.dat')
        broaden_withMIDAS(S[O]+'1', 1.0*(notData[O][7])/10)
        b=formatta(S[O]+'1'+'.dat')
        combina_spettri([a, b], S[O]+'.dat', output_folder=path)

        print('\n\n')
        print(f'/hhome/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}')
        print(S[O])
        plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=5)
        #plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}_mac_{notData[O-o][7]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)

        inizio=O
        s=None
        for O in range(inizio, inizio+1): 
            if O != 15: #and O != 16:
                s=Stella(notData[O][5], instrument='FIES')
                nomeout=s.nome.replace(' ', '_')
                s.creaModel(notData[O][1], notData[O][2], notData[O][3], notData[O][4],processore=1, radSolver='turbo', outpath='/home/starcat8/Documenti/', outName='fort.11', alpha_enhance=notData[O][6])
                os.chdir('/home/starcat8/Documenti/MyGisFOS/test/best/')
                aoaos=abbondanzeTurbo(notData[O][6], 'abundances.out', bario=+0.31)#s.nome+f'_{notData[O][7]}.out', europio=0.00001, zirconio=0.632) #aoaos contiene le abbondanze in formato [elemento, log(abbondanza)] queste verranno poi fornite allo script di sintesi
                europio=0
                files=glob.glob(nomeout+'*')
                with open('/home/starcat8/Documenti/MyGisFOS/test/best/'+files[0], 'r') as f:
                    righe = f.readlines()
                    # Prende la quarta riga (indice 3)
                quarta_riga = righe[3]
                # Prende i caratteri dal 47 al 52 (in Python: 46:52)
                valore_str = quarta_riga[46:52]
                valore_float = float(valore_str)
                print(valore_float)
                crea_synthScriptTurbo('/home/starcat8/Documenti/', 'fort.11', outname=s.nome+'ba', wl_min=6650, wl_max=6750, TURBVEL=notData[O][4], abundances=[aoaos[0][0], aoaos[0][1]])
                os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
                copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
                subprocess.run(['csh', '-c', 'source synth_script'])
                print(aoaos)

        #s.nome='BD+17_2086'
        copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
        broaden_withMIDAS(s.nome+'ba1', 7.5)#1.0*(notData[O][7])/10)
        os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/')
        a=formatta(s.nome+'ba1'+'.dat')
        ST=Table.read(s.nome+'ba1.dat_', format='ascii')
        copia(s.nome+'ba1.dat_', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/', s.nome+'1.spectrum')
        wl=ST['col2']
        inte=ST['col3']
        xx= np.zeros(len(wl))
        X= np.asarray(wl)
        for i in range(len(wl)):
            xx[i] = X[i]*1/(-2.15/LIGHTSPEED +1)
        # X=np.asarray(wl)
        # xx= np.zeros(len(wl))
        # for i in range(len(wl)):
        #     xx[i] = X[i]*1/(-2.15/LIGHTSPEED +1)
        # for i in range(len(inte)):
        #     inte[i]=inte[i]#*1.05019878296
        #     inte[i]=inte[i]/(1+(1.0146-1)*(wl[i]-4690)/(2000))
        
        plt.plot(xx, inte)#*1.05019878296)
        plt.show()
        cartella = '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops'
        elimina_file_cartella(cartella)







# ###NOT###

# for PP in range(16, 17):
#     if PP != 2 and PP != 3 and OOOOO!=12:
#         path='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/'
#         os.chdir(path)
#         O=PP
#         o=PP
#         import glob
#         copia('abundances.out', f'/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[PP]}', '/home/starcat8/Documenti/MyGisFOS/test/best/', 'abundances.out')
#         broaden_withMIDAS(S[O]+'2', 1.0*(notData[O][7])/10)
#         a=formatta(S[O]+'2'+'.dat')
#         broaden_withMIDAS(S[O]+'1', 1.0*(notData[O][7])/10)
#         b=formatta(S[O]+'1'+'.dat')
#         combina_spettri([a, b], S[O]+'.dat', output_folder=path)

#         print('\n\n')
#         print(f'/hhome/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}')
#         print(S[O])
#         plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)
#         #plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}_mac_{notData[O-o][7]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)

#         inizio=O
#         s=None
#         for O in range(inizio, inizio+1):
#             if O != 15:# and O != 16:
#                 s=Stella(notData[O][5], instrument='FIES')
#                 nomeout=s.nome.replace(' ', '_')
#                 s.creaModel(notData[O][1], notData[O][2], notData[O][3], notData[O][4],processore=1, radSolver='turbo', outpath='/home/starcat8/Documenti/', outName='fort.11', alpha_enhance=notData[O][6])
#                 os.chdir('/home/starcat8/Documenti/MyGisFOS/test/best/')
#                 aoaos=abbondanzeTurbo(notData[O][6], 'abundances.out')#s.nome+f'_{notData[O][7]}.out', europio=0.00001, zirconio=0.632)
#                 europio=0
#                 files=glob.glob(nomeout+'*')
#                 with open('/home/starcat8/Documenti/MyGisFOS/test/best/'+files[0], 'r') as f:
#                     righe = f.readlines()
#                     # Prende la quarta riga (indice 3)
#                 quarta_riga = righe[3]
#                 # Prende i caratteri dal 47 al 52 (in Python: 46:52)
#                 valore_str = quarta_riga[46:52]
#                 valore_float = float(valore_str)
#                 print(valore_float)
#                 crea_synthScriptTurbo('/home/starcat8/Documenti/', 'fort.11', outname=s.nome+'ba', wl_min=6540, wl_max=7830, TURBVEL=notData[O][4], abundances=[aoaos[0][0], aoaos[0][1]])
#                 os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
#                 copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
#                 subprocess.run(['csh', '-c', 'source synth_script'])
#                 print(aoaos)

#         #s.nome='BD+17_2086'
#         copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
#         broaden_withMIDAS(s.nome+'ba1', 1.0*(notData[O][7])/10)
#         os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/')
#         a=formatta(s.nome+'ba1'+'.dat')
#         ST=Table.read(s.nome+'ba1.dat_', format='ascii')
#         copia(s.nome+'ba1.dat_', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/', s.nome+'2.spectrum')
#         wl=ST['col2']
#         inte=ST['col3']
#         # for i in range(len(inte)):
#         #     inte[i]=inte[i]/0.96#*1.05019878296
#         #     inte[i]=inte[i]/(1+(1.0146-1)*(wl[i]-4690)/(2000))
#         plt.plot(wl, inte)#*1.05019878296)
#         plt.show()
#         cartella = '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops'
#         elimina_file_cartella(cartella)

In [ ]:
xx= np.zeros(len(wl))
X= np.asarray(wl)
for i in range(len(wl)):
    xx[i] = X[i]*1/(-2.15/LIGHTSPEED +1)

In [ ]:
xx

In [ ]:
    ######CREA SYNTH SPECTRA FROM MYGISFOS RESULTS, WORKING FOR HARPS-N (snotData) STARS###SNOT###

for PP in range(len(snotData)):
    path='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/'
    os.chdir(path)
    O=PP
    o=PP
    import glob
    copia('abundances.out', f'/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[PP]}Copia', '/home/starcat8/Documenti/MyGisFOS/test/best1/', 'abundances.out')
    broaden_withMIDAS(S[O]+'2', 1.0*(snotData[O][7])/10)
    a=formatta(S[O]+'2'+'.dat')
    broaden_withMIDAS(S[O]+'1', 1.0*(snotData[O][7])/10)
    b=formatta(S[O]+'1'+'.dat')
    combina_spettri([a, b], S[O]+'.dat', output_folder=path)

    print('\n\n')
    print(f'/hhome/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}')
    print(S[O])
    plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)
    #plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}_mac_{notData[O-o][7]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)

    inizio=O
    s=None
    for O in range(inizio, inizio+1):
        if O != 15 and O != 16:
            s=Stella(snotData[O][5], instrument='HARPS-N')
            nomeout=s.nome.replace(' ', '_')
            s.creaModel(snotData[O][1], snotData[O][2], snotData[O][3], snotData[O][4],processore=1, radSolver='turbo', outpath='/home/starcat8/Documenti/', outName='fort.11', alpha_enhance=snotData[O][6])
            os.chdir('/home/starcat8/Documenti/MyGisFOS/test/best1/')
            aoaos=abbondanzeTurbo(snotData[O][6], 'abundances.out', ossigeno=0)#s.nome+f'_{notData[O][7]}.out', europio=0.00001, zirconio=0.632)
            europio=0
            files=glob.glob(nomeout+'*')
            with open('/home/starcat8/Documenti/MyGisFOS/test/best/'+files[0], 'r') as f:
                righe = f.readlines()
                # Prende la quarta riga (indice 3)
            quarta_riga = righe[3]
            # Prende i caratteri dal 47 al 52 (in Python: 46:52)
            valore_str = quarta_riga[46:52]
            valore_float = float(valore_str)
            print(valore_float)
            crea_synthScriptTurbo('/home/starcat8/Documenti/', 'fort.11', outname=s.nome+'ba', wl_min=4450, wl_max=6440, TURBVEL=snotData[O][4], abundances=[aoaos[0][0], aoaos[0][1]])
            os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
            copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
            subprocess.run(['csh', '-c', 'source synth_script'])
            print(aoaos)

    #s.nome='BD+17_2086'
    copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
    broaden_withMIDAS(s.nome+'ba1', 1.0*(snotData[O][7])/10)
    os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/')
    a=formatta(s.nome+'ba1'+'.dat')
    ST=Table.read(s.nome+'ba1.dat_', format='ascii')
    copia(s.nome+'ba1.dat_', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/', s.nome+'1.spectrum')
    wl=ST['col2']
    inte=ST['col3']
    for i in range(len(inte)):
        inte[i]=inte[i]
    #     inte[i]=inte[i]/(1+(1.0146-1)*(wl[i]-4690)/(2000))
    plt.plot(wl, inte, color='red')#*1.05019878296)
    plt.show()
    cartella = '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops'
    elimina_file_cartella(cartella)

    ###SNOT###

# for PP in range(len(snotData)):
#     path='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/'
#     os.chdir(path)
#     O=PP
#     o=PP
#     import glob
#     copia('abundances.out', f'/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/{S[PP]}(Copia)', '/home/starcat8/Documenti/MyGisFOS/test/best1/', 'abundances.out')
#     broaden_withMIDAS(S[O]+'2', 1.0*(snotData[O][7])/10)
#     a=formatta(S[O]+'2'+'.dat')
#     broaden_withMIDAS(S[O]+'1', 1.0*(snotData[O][7])/10)
#     b=formatta(S[O]+'1'+'.dat')
#     combina_spettri([a, b], S[O]+'.dat', output_folder=path)

#     print('\n\n')
#     print(f'/hhome/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}')
#     print(S[O])
#     plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)
#     #plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}_mac_{notData[O-o][7]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)

#     inizio=O
#     s=None
#     for O in range(inizio, inizio+1):
#         if O != 15 and O != 16:
#             s=Stella(snotData[O][5], instrument='HARPS-N')
#             nomeout=s.nome.replace(' ', '_')
#             s.creaModel(snotData[O][1], snotData[O][2], snotData[O][3]+0.1, snotData[O][4],processore=1, radSolver='turbo', outpath='/home/starcat8/Documenti/', outName='fort.11', alpha_enhance=snotData[O][6])
#             os.chdir('/home/starcat8/Documenti/MyGisFOS/test/best1/')
#             aoaos=abbondanzeTurbo(snotData[O][6], 'abundances.out', ossigeno=0)#s.nome+f'_{notData[O][7]}.out', europio=0.00001, zirconio=0.632)
#             europio=0
#             files=glob.glob(nomeout+'*')
#             with open('/home/starcat8/Documenti/MyGisFOS/test/best/'+files[0], 'r') as f:
#                 righe = f.readlines()
#                 # Prende la quarta riga (indice 3)
#             quarta_riga = righe[3]
#             # Prende i caratteri dal 47 al 52 (in Python: 46:52)
#             valore_str = quarta_riga[46:52]
#             valore_float = float(valore_str)
#             print(valore_float)
#             crea_synthScriptTurbo('/home/starcat8/Documenti/', 'fort.11', outname=s.nome+'ba', wl_min=6440, wl_max=7840, TURBVEL=snotData[O][4], abundances=[aoaos[0][0], aoaos[0][1]])
#             os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master')
#             copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
#             subprocess.run(['csh', '-c', 'source synth_script'])
#             print(aoaos)

#     #s.nome='BD+17_2086'
#     copia(s.nome+'ba1', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops', s.nome+'ba1')
#     broaden_withMIDAS(s.nome+'ba1', 1.0*(snotData[O][7])/10)
#     os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/')
#     a=formatta(s.nome+'ba1'+'.dat')
#     ST=Table.read(s.nome+'ba1.dat_', format='ascii')
#     copia(s.nome+'ba1.dat_', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/', '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/', s.nome+'.spectrum')
#     wl=ST['col2']
#     inte=ST['col3']
#     for i in range(len(inte)):
#         inte[i]=inte[i]/(0.9571)
#     #     inte[i]=inte[i]/(1+(1.0146-1)*(wl[i]-4690)/(2000))
#     plt.plot(wl-0.03, inte, color='red')#*1.05019878296)
#     plt.show()
#     cartella = '/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops'
#     elimina_file_cartella(cartella)



In [ ]:
O=16
path='/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/ops/'
plot_mygisfos(f'/home/starcat8/Documenti/MyGisFOS/test/STAR_{S[O]}/', S[O]+'_'+S[O], datibooni=S[O], path_datibooni=path, coeff=[0, 1], region=4)
os.chdir('/home/starcat8/Documenti/Turbospectrum_NLTE-master/outsynth/syntspec/')
ST=Table.read(S[O]+'1.spectrum', format='ascii')
wl=ST['col2']
inte=ST['col3']
# for i in range(len(inte)):
#     inte[i]=inte[i]/(0.98+0.0000060804*(wl[i]-4450))/1.003
plt.plot(wl, inte)
plt.show()

In [ ]:
### search for parameters ###
# In nomi_fileFits are stored the fits files names. This snippet will go through all of the ones indexed by the 'i' in the first for loop. It will make a first run exploring a wider range of the parameters, then it will make a second run with a more precise range of the parameters. The results will be stored in a .star file with the name of the star spelled twice.
star1 = []
for i in range(len(otherMinceData)):
    indice_stella = i+9
    star1.append(Stella(otherMinceData[indice_stella], instrument="HARPS-N"))
    print(f"Stella {i} caricata - {star1[i].nome}")
    if i<4 or i > 7:
        #star1[i].v_r=0
        star1[i].searchParam3(processori=12, NLTE=0, TRIALS=12, radSolver='spectrum', atmospheric_model='ATLASint')
        #star1[i].stampaStella1(star1[i].nome, NLTE=False)
        with open(star1[i].nome+'param.txt', 'w') as file:
            file.write(f'T: {star1[i].t_eff}\nlogg: {star1[i].logg}\n[M/H]: {star1[i].metal}\nV: {star1[i].v_m}')
        star1[i].param_error=100
        star1[i].searchParam3(processori=12, NLTE=0, TRIALS=2, 
                            t01=star1[i].t_eff, g01=star1[i].logg, m01=star1[i].metal, v01=star1[i].v_m,
                            t02=star1[i].t_eff, g02=star1[i].logg, m02=star1[i].metal, v02=star1[i].v_m,
                            t03=star1[i].t_eff, g03=star1[i].logg, m03=star1[i].metal, v03=star1[i].v_m, 
                            t04=star1[i].t_eff, g04=star1[i].logg, m04=star1[i].metal, v04=star1[i].v_m, 
                            mCheck=1, radSolver='turbo', atmospheric_model='ATLASint')
        star1[i].searchParam3(processori=12, NLTE=0, TRIALS=2, 
                            t01=star1[i].t_eff, g01=star1[i].logg, m01=star1[i].metal, v01=star1[i].v_m,
                            t02=star1[i].t_eff, g02=star1[i].logg, m02=star1[i].metal, v02=star1[i].v_m,
                            t03=star1[i].t_eff, g03=star1[i].logg, m03=star1[i].metal, v03=star1[i].v_m, 
                            t04=star1[i].t_eff, g04=star1[i].logg, m04=star1[i].metal, v04=star1[i].v_m, 
                            g_fixed=star1[i].logg, m_fixed=star1[i].metal, v_fixed=star1[i].v_m, mCheck=1, radSolver='turbo', atmospheric_model='ATLASint')
        star1[i].searchParam3(processori=12, NLTE=0, TRIALS=2, 
                            t01=star1[i].t_eff, g01=star1[i].logg, m01=star1[i].metal, v01=star1[i].v_m,
                            t02=star1[i].t_eff, g02=star1[i].logg, m02=star1[i].metal, v02=star1[i].v_m,
                            t03=star1[i].t_eff, g03=star1[i].logg, m03=star1[i].metal, v03=star1[i].v_m, 
                            t04=star1[i].t_eff, g04=star1[i].logg, m04=star1[i].metal, v04=star1[i].v_m, 
                            t_fixed=star1[i].t_eff, v_fixed=star1[i].v_m, mCheck=1, radSolver='turbo', atmospheric_model='ATLASint')
        star1[i].searchParam3(processori=12, NLTE=0, TRIALS=2, 
                                t01=star1[i].t_eff, g01=star1[i].logg, m01=star1[i].metal, v01=star1[i].v_m,
                                t02=star1[i].t_eff, g02=star1[i].logg, m02=star1[i].metal, v02=star1[i].v_m,
                                t03=star1[i].t_eff, g03=star1[i].logg, m03=star1[i].metal, v03=star1[i].v_m, 
                                t04=star1[i].t_eff, g04=star1[i].logg, m04=star1[i].metal, v04=star1[i].v_m, 
                                g_fixed=star1[i].logg, m_fixed=star1[i].metal, v_fixed=star1[i].v_m, mCheck=1, radSolver='turbo', atmospheric_model='ATLASint')

